# Model Selection

In this notebook we provide several options for shape of the q(i) function:

- [linear](#one)
- [sigmoidal](#two)
- [low_high_low](#three)
- [bell shape](#four)

creating respective model with less parameters.

Parameter space of selected model is either refined or sampled in case of larger models.

There are four things to setup in this notebook:
1. `agents_quantities` - set of agents number to be considered - set numbers for which you want to run analysis,
2. `alpha` and `n_samples`  in `check_deeper(thresh,prop,data,alpha,n_samples,n,epsilon,cov,silent,version)` to set the margin of the data  -  `margin(alpha, n_samples, data)`,
3. `n`, `epsilon`, `cov` in `check_deeper(thresh,prop,data,alpha,n_samples,n,epsilon,cov,silent,version)` - thresholds used in parameter space division.

moreover each model contains paramters constraining the shape of the selected curve.

## IMPORTS, PATHS, and SETTINGS

if a package fails to load due to it is not installed please uncomment and use following code with respective package name

In [ ]:
#import sys
#!{sys.executable} -m pip install <package_name>

In [ ]:
import struct
print("You are running "+ str(struct.calcsize("P") * 8)+"bit Python, please verify that installed z3 is compatible")

In [ ]:
import os,sys,time,socket

In [ ]:
cwd = os.getcwd()
os.chdir("..")
sys.path.append(os.getcwd())
os.chdir(cwd)

In [ ]:
os.getcwd()

Set agents quantities to be examined

In [ ]:
agents_quantities = [2,3,5,10,20]

## 1. LOADING OF PARAMETER SYNTHESIS RESULTS AND DATA

Now we load list of propbabilities for respective BSCC and expected number of successes.

Here we provide first optimisation by factorising the polynomials received as an output from PRISM.

In [ ]:
from src.load import get_f, get_rewards
sys.setrecursionlimit(15000)

Recalling N is number of agents, and f_N(i) is probability function of having i successes,

Load f_N, a map N-> array of f(i)s:

## Load polynomials for **two-param** models:

### Factorised version:

In [ ]:
start_time = time.time()
f = get_f("./asyn*[0-9].txt", "storm", False, agents_quantities)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

### Not factorised (expanded version):

In [ ]:
start_time = time.time()
f_expanded = get_f("./asyn*[0-9].txt", "prism", False, agents_quantities)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

### Now load the expected number of sucesses in reward section:

In [ ]:
start_time = time.time()
rewards = get_rewards("./asyn*[0-9].txt", "prism", True, agents_quantities)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
start_time = time.time()
rewards_expanded = get_rewards("./sem*[0-9].txt", "prism", False, agents_quantities)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
start_time = time.time()
rewards_storm = get_rewards("./asyn*[0-9]_moments*.txt", "storm", True, agents_quantities)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

## Load polynomials for **three-param** models.

### Models with linear function:

In [ ]:
# start_time = time.time()
# f_lin = get_f("./lin*[0-9].txt", "prism", True)
# print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
# start_time = time.time()
# rewards_lin = get_rewards("./lin*[0-9]_moment*.txt", "prism", True)
# print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

## Load polynomials for **multi-param** models.

### Synchronous multiparam models

This may take a while to factorise for f_multiparam_syn[10], so I will load precomputed version for you

In [ ]:
# start_time = time.time()
# f_multiparam_syn = get_f("./multiparam_syn*[0-9].txt",True)
# print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run"))

In [ ]:
f_multiparam_syn = get_f("./multiparam_syn*_[0-9].txt", "prism", True, agents_quantities)
f_multiparam_syn[10] = [0,0,0,0,0,0,0,0,0,0,0]
f_multiparam_syn[10][0] = '(p - 1)**10'
f_multiparam_syn[10][1] = '10*p*(p - 1)**9*(q1 - 1)**9'
f_multiparam_syn[10][2] = '-45*p*(p - 1)**8*(2*p*q1**9 - 16*p*q1**8 + 56*p*q1**7 - 112*p*q1**6 + 140*p*q1**5 - 112*p*q1**4 + 56*p*q1**3 - 16*p*q1**2 + 2*p*q1 - p*q2**8 + 8*p*q2**7 - 28*p*q2**6 + 56*p*q2**5 - 70*p*q2**4 + 56*p*q2**3 - 28*p*q2**2 + 8*p*q2 - p - 2*q1**9 + 16*q1**8 - 56*q1**7 + 112*q1**6 - 140*q1**5 + 112*q1**4 - 56*q1**3 + 16*q1**2 - 2*q1)'
f_multiparam_syn[10][3] = '120*p*(p - 1)**7*(3*p**2*q1**9 - 21*p**2*q1**8 + 63*p**2*q1**7 - 105*p**2*q1**6 + 105*p**2*q1**5 - 63*p**2*q1**4 + 21*p**2*q1**3 - 3*p**2*q1**2 - 3*p**2*q2**8 + 21*p**2*q2**7 - 63*p**2*q2**6 + 105*p**2*q2**5 - 105*p**2*q2**4 + 63*p**2*q2**3 - 21*p**2*q2**2 + 3*p**2*q2 + p**2*q3**7 - 7*p**2*q3**6 + 21*p**2*q3**5 - 35*p**2*q3**4 + 35*p**2*q3**3 - 21*p**2*q3**2 + 7*p**2*q3 - p**2 - 6*p*q1**9 + 42*p*q1**8 - 126*p*q1**7 + 210*p*q1**6 - 210*p*q1**5 + 126*p*q1**4 - 42*p*q1**3 + 6*p*q1**2 + 3*p*q2**8 - 21*p*q2**7 + 63*p*q2**6 - 105*p*q2**5 + 105*p*q2**4 - 63*p*q2**3 + 21*p*q2**2 - 3*p*q2 + 3*q1**9 - 21*q1**8 + 63*q1**7 - 105*q1**6 + 105*q1**5 - 63*q1**4 + 21*q1**3 - 3*q1**2)'
f_multiparam_syn[10][4] = '-210*p*(p - 1)**6*(4*p**3*q1**9 - 24*p**3*q1**8 + 60*p**3*q1**7 - 80*p**3*q1**6 + 60*p**3*q1**5 - 24*p**3*q1**4 + 4*p**3*q1**3 - 6*p**3*q2**8 + 36*p**3*q2**7 - 90*p**3*q2**6 + 120*p**3*q2**5 - 90*p**3*q2**4 + 36*p**3*q2**3 - 6*p**3*q2**2 + 4*p**3*q3**7 - 24*p**3*q3**6 + 60*p**3*q3**5 - 80*p**3*q3**4 + 60*p**3*q3**3 - 24*p**3*q3**2 + 4*p**3*q3 - p**3*q4**6 + 6*p**3*q4**5 - 15*p**3*q4**4 + 20*p**3*q4**3 - 15*p**3*q4**2 + 6*p**3*q4 - p**3 - 12*p**2*q1**9 + 72*p**2*q1**8 - 180*p**2*q1**7 + 240*p**2*q1**6 - 180*p**2*q1**5 + 72*p**2*q1**4 - 12*p**2*q1**3 + 12*p**2*q2**8 - 72*p**2*q2**7 + 180*p**2*q2**6 - 240*p**2*q2**5 + 180*p**2*q2**4 - 72*p**2*q2**3 + 12*p**2*q2**2 - 4*p**2*q3**7 + 24*p**2*q3**6 - 60*p**2*q3**5 + 80*p**2*q3**4 - 60*p**2*q3**3 + 24*p**2*q3**2 - 4*p**2*q3 + 12*p*q1**9 - 72*p*q1**8 + 180*p*q1**7 - 240*p*q1**6 + 180*p*q1**5 - 72*p*q1**4 + 12*p*q1**3 - 6*p*q2**8 + 36*p*q2**7 - 90*p*q2**6 + 120*p*q2**5 - 90*p*q2**4 + 36*p*q2**3 - 6*p*q2**2 - 4*q1**9 + 24*q1**8 - 60*q1**7 + 80*q1**6 - 60*q1**5 + 24*q1**4 - 4*q1**3)'
f_multiparam_syn[10][5] = '252*p*(p - 1)**5*(5*p**4*q1**9 - 25*p**4*q1**8 + 50*p**4*q1**7 - 50*p**4*q1**6 + 25*p**4*q1**5 - 5*p**4*q1**4 - 10*p**4*q2**8 + 50*p**4*q2**7 - 100*p**4*q2**6 + 100*p**4*q2**5 - 50*p**4*q2**4 + 10*p**4*q2**3 + 10*p**4*q3**7 - 50*p**4*q3**6 + 100*p**4*q3**5 - 100*p**4*q3**4 + 50*p**4*q3**3 - 10*p**4*q3**2 - 5*p**4*q4**6 + 25*p**4*q4**5 - 50*p**4*q4**4 + 50*p**4*q4**3 - 25*p**4*q4**2 + 5*p**4*q4 + p**4*q5**5 - 5*p**4*q5**4 + 10*p**4*q5**3 - 10*p**4*q5**2 + 5*p**4*q5 - p**4 - 20*p**3*q1**9 + 100*p**3*q1**8 - 200*p**3*q1**7 + 200*p**3*q1**6 - 100*p**3*q1**5 + 20*p**3*q1**4 + 30*p**3*q2**8 - 150*p**3*q2**7 + 300*p**3*q2**6 - 300*p**3*q2**5 + 150*p**3*q2**4 - 30*p**3*q2**3 - 20*p**3*q3**7 + 100*p**3*q3**6 - 200*p**3*q3**5 + 200*p**3*q3**4 - 100*p**3*q3**3 + 20*p**3*q3**2 + 5*p**3*q4**6 - 25*p**3*q4**5 + 50*p**3*q4**4 - 50*p**3*q4**3 + 25*p**3*q4**2 - 5*p**3*q4 + 30*p**2*q1**9 - 150*p**2*q1**8 + 300*p**2*q1**7 - 300*p**2*q1**6 + 150*p**2*q1**5 - 30*p**2*q1**4 - 30*p**2*q2**8 + 150*p**2*q2**7 - 300*p**2*q2**6 + 300*p**2*q2**5 - 150*p**2*q2**4 + 30*p**2*q2**3 + 10*p**2*q3**7 - 50*p**2*q3**6 + 100*p**2*q3**5 - 100*p**2*q3**4 + 50*p**2*q3**3 - 10*p**2*q3**2 - 20*p*q1**9 + 100*p*q1**8 - 200*p*q1**7 + 200*p*q1**6 - 100*p*q1**5 + 20*p*q1**4 + 10*p*q2**8 - 50*p*q2**7 + 100*p*q2**6 - 100*p*q2**5 + 50*p*q2**4 - 10*p*q2**3 + 5*q1**9 - 25*q1**8 + 50*q1**7 - 50*q1**6 + 25*q1**5 - 5*q1**4)'
f_multiparam_syn[10][6] = '-210*p*(p - 1)**4*(6*p**5*q1**9 - 24*p**5*q1**8 + 36*p**5*q1**7 - 24*p**5*q1**6 + 6*p**5*q1**5 - 15*p**5*q2**8 + 60*p**5*q2**7 - 90*p**5*q2**6 + 60*p**5*q2**5 - 15*p**5*q2**4 + 20*p**5*q3**7 - 80*p**5*q3**6 + 120*p**5*q3**5 - 80*p**5*q3**4 + 20*p**5*q3**3 - 15*p**5*q4**6 + 60*p**5*q4**5 - 90*p**5*q4**4 + 60*p**5*q4**3 - 15*p**5*q4**2 + 6*p**5*q5**5 - 24*p**5*q5**4 + 36*p**5*q5**3 - 24*p**5*q5**2 + 6*p**5*q5 - p**5*q6**4 + 4*p**5*q6**3 - 6*p**5*q6**2 + 4*p**5*q6 - p**5 - 30*p**4*q1**9 + 120*p**4*q1**8 - 180*p**4*q1**7 + 120*p**4*q1**6 - 30*p**4*q1**5 + 60*p**4*q2**8 - 240*p**4*q2**7 + 360*p**4*q2**6 - 240*p**4*q2**5 + 60*p**4*q2**4 - 60*p**4*q3**7 + 240*p**4*q3**6 - 360*p**4*q3**5 + 240*p**4*q3**4 - 60*p**4*q3**3 + 30*p**4*q4**6 - 120*p**4*q4**5 + 180*p**4*q4**4 - 120*p**4*q4**3 + 30*p**4*q4**2 - 6*p**4*q5**5 + 24*p**4*q5**4 - 36*p**4*q5**3 + 24*p**4*q5**2 - 6*p**4*q5 + 60*p**3*q1**9 - 240*p**3*q1**8 + 360*p**3*q1**7 - 240*p**3*q1**6 + 60*p**3*q1**5 - 90*p**3*q2**8 + 360*p**3*q2**7 - 540*p**3*q2**6 + 360*p**3*q2**5 - 90*p**3*q2**4 + 60*p**3*q3**7 - 240*p**3*q3**6 + 360*p**3*q3**5 - 240*p**3*q3**4 + 60*p**3*q3**3 - 15*p**3*q4**6 + 60*p**3*q4**5 - 90*p**3*q4**4 + 60*p**3*q4**3 - 15*p**3*q4**2 - 60*p**2*q1**9 + 240*p**2*q1**8 - 360*p**2*q1**7 + 240*p**2*q1**6 - 60*p**2*q1**5 + 60*p**2*q2**8 - 240*p**2*q2**7 + 360*p**2*q2**6 - 240*p**2*q2**5 + 60*p**2*q2**4 - 20*p**2*q3**7 + 80*p**2*q3**6 - 120*p**2*q3**5 + 80*p**2*q3**4 - 20*p**2*q3**3 + 30*p*q1**9 - 120*p*q1**8 + 180*p*q1**7 - 120*p*q1**6 + 30*p*q1**5 - 15*p*q2**8 + 60*p*q2**7 - 90*p*q2**6 + 60*p*q2**5 - 15*p*q2**4 - 6*q1**9 + 24*q1**8 - 36*q1**7 + 24*q1**6 - 6*q1**5)'
f_multiparam_syn[10][7] = '120*p*(p - 1)**3*(7*p**6*q1**9 - 21*p**6*q1**8 + 21*p**6*q1**7 - 7*p**6*q1**6 - 21*p**6*q2**8 + 63*p**6*q2**7 - 63*p**6*q2**6 + 21*p**6*q2**5 + 35*p**6*q3**7 - 105*p**6*q3**6 + 105*p**6*q3**5 - 35*p**6*q3**4 - 35*p**6*q4**6 + 105*p**6*q4**5 - 105*p**6*q4**4 + 35*p**6*q4**3 + 21*p**6*q5**5 - 63*p**6*q5**4 + 63*p**6*q5**3 - 21*p**6*q5**2 - 7*p**6*q6**4 + 21*p**6*q6**3 - 21*p**6*q6**2 + 7*p**6*q6 + p**6*q7**3 - 3*p**6*q7**2 + 3*p**6*q7 - p**6 - 42*p**5*q1**9 + 126*p**5*q1**8 - 126*p**5*q1**7 + 42*p**5*q1**6 + 105*p**5*q2**8 - 315*p**5*q2**7 + 315*p**5*q2**6 - 105*p**5*q2**5 - 140*p**5*q3**7 + 420*p**5*q3**6 - 420*p**5*q3**5 + 140*p**5*q3**4 + 105*p**5*q4**6 - 315*p**5*q4**5 + 315*p**5*q4**4 - 105*p**5*q4**3 - 42*p**5*q5**5 + 126*p**5*q5**4 - 126*p**5*q5**3 + 42*p**5*q5**2 + 7*p**5*q6**4 - 21*p**5*q6**3 + 21*p**5*q6**2 - 7*p**5*q6 + 105*p**4*q1**9 - 315*p**4*q1**8 + 315*p**4*q1**7 - 105*p**4*q1**6 - 210*p**4*q2**8 + 630*p**4*q2**7 - 630*p**4*q2**6 + 210*p**4*q2**5 + 210*p**4*q3**7 - 630*p**4*q3**6 + 630*p**4*q3**5 - 210*p**4*q3**4 - 105*p**4*q4**6 + 315*p**4*q4**5 - 315*p**4*q4**4 + 105*p**4*q4**3 + 21*p**4*q5**5 - 63*p**4*q5**4 + 63*p**4*q5**3 - 21*p**4*q5**2 - 140*p**3*q1**9 + 420*p**3*q1**8 - 420*p**3*q1**7 + 140*p**3*q1**6 + 210*p**3*q2**8 - 630*p**3*q2**7 + 630*p**3*q2**6 - 210*p**3*q2**5 - 140*p**3*q3**7 + 420*p**3*q3**6 - 420*p**3*q3**5 + 140*p**3*q3**4 + 35*p**3*q4**6 - 105*p**3*q4**5 + 105*p**3*q4**4 - 35*p**3*q4**3 + 105*p**2*q1**9 - 315*p**2*q1**8 + 315*p**2*q1**7 - 105*p**2*q1**6 - 105*p**2*q2**8 + 315*p**2*q2**7 - 315*p**2*q2**6 + 105*p**2*q2**5 + 35*p**2*q3**7 - 105*p**2*q3**6 + 105*p**2*q3**5 - 35*p**2*q3**4 - 42*p*q1**9 + 126*p*q1**8 - 126*p*q1**7 + 42*p*q1**6 + 21*p*q2**8 - 63*p*q2**7 + 63*p*q2**6 - 21*p*q2**5 + 7*q1**9 - 21*q1**8 + 21*q1**7 - 7*q1**6)'
f_multiparam_syn[10][8] = '-45*p*(p - 1)**2*(8*p**7*q1**9 - 16*p**7*q1**8 + 8*p**7*q1**7 - 28*p**7*q2**8 + 56*p**7*q2**7 - 28*p**7*q2**6 + 56*p**7*q3**7 - 112*p**7*q3**6 + 56*p**7*q3**5 - 70*p**7*q4**6 + 140*p**7*q4**5 - 70*p**7*q4**4 + 56*p**7*q5**5 - 112*p**7*q5**4 + 56*p**7*q5**3 - 28*p**7*q6**4 + 56*p**7*q6**3 - 28*p**7*q6**2 + 8*p**7*q7**3 - 16*p**7*q7**2 + 8*p**7*q7 - p**7*q8**2 + 2*p**7*q8 - p**7 - 56*p**6*q1**9 + 112*p**6*q1**8 - 56*p**6*q1**7 + 168*p**6*q2**8 - 336*p**6*q2**7 + 168*p**6*q2**6 - 280*p**6*q3**7 + 560*p**6*q3**6 - 280*p**6*q3**5 + 280*p**6*q4**6 - 560*p**6*q4**5 + 280*p**6*q4**4 - 168*p**6*q5**5 + 336*p**6*q5**4 - 168*p**6*q5**3 + 56*p**6*q6**4 - 112*p**6*q6**3 + 56*p**6*q6**2 - 8*p**6*q7**3 + 16*p**6*q7**2 - 8*p**6*q7 + 168*p**5*q1**9 - 336*p**5*q1**8 + 168*p**5*q1**7 - 420*p**5*q2**8 + 840*p**5*q2**7 - 420*p**5*q2**6 + 560*p**5*q3**7 - 1120*p**5*q3**6 + 560*p**5*q3**5 - 420*p**5*q4**6 + 840*p**5*q4**5 - 420*p**5*q4**4 + 168*p**5*q5**5 - 336*p**5*q5**4 + 168*p**5*q5**3 - 28*p**5*q6**4 + 56*p**5*q6**3 - 28*p**5*q6**2 - 280*p**4*q1**9 + 560*p**4*q1**8 - 280*p**4*q1**7 + 560*p**4*q2**8 - 1120*p**4*q2**7 + 560*p**4*q2**6 - 560*p**4*q3**7 + 1120*p**4*q3**6 - 560*p**4*q3**5 + 280*p**4*q4**6 - 560*p**4*q4**5 + 280*p**4*q4**4 - 56*p**4*q5**5 + 112*p**4*q5**4 - 56*p**4*q5**3 + 280*p**3*q1**9 - 560*p**3*q1**8 + 280*p**3*q1**7 - 420*p**3*q2**8 + 840*p**3*q2**7 - 420*p**3*q2**6 + 280*p**3*q3**7 - 560*p**3*q3**6 + 280*p**3*q3**5 - 70*p**3*q4**6 + 140*p**3*q4**5 - 70*p**3*q4**4 - 168*p**2*q1**9 + 336*p**2*q1**8 - 168*p**2*q1**7 + 168*p**2*q2**8 - 336*p**2*q2**7 + 168*p**2*q2**6 - 56*p**2*q3**7 + 112*p**2*q3**6 - 56*p**2*q3**5 + 56*p*q1**9 - 112*p*q1**8 + 56*p*q1**7 - 28*p*q2**8 + 56*p*q2**7 - 28*p*q2**6 - 8*q1**9 + 16*q1**8 - 8*q1**7)'
f_multiparam_syn[10][9] = '10*p*(p - 1)*(9*p**8*q1**9 - 9*p**8*q1**8 - 36*p**8*q2**8 + 36*p**8*q2**7 + 84*p**8*q3**7 - 84*p**8*q3**6 - 126*p**8*q4**6 + 126*p**8*q4**5 + 126*p**8*q5**5 - 126*p**8*q5**4 - 84*p**8*q6**4 + 84*p**8*q6**3 + 36*p**8*q7**3 - 36*p**8*q7**2 - 9*p**8*q8**2 + 9*p**8*q8 + p**8*q9 - p**8 - 72*p**7*q1**9 + 72*p**7*q1**8 + 252*p**7*q2**8 - 252*p**7*q2**7 - 504*p**7*q3**7 + 504*p**7*q3**6 + 630*p**7*q4**6 - 630*p**7*q4**5 - 504*p**7*q5**5 + 504*p**7*q5**4 + 252*p**7*q6**4 - 252*p**7*q6**3 - 72*p**7*q7**3 + 72*p**7*q7**2 + 9*p**7*q8**2 - 9*p**7*q8 + 252*p**6*q1**9 - 252*p**6*q1**8 - 756*p**6*q2**8 + 756*p**6*q2**7 + 1260*p**6*q3**7 - 1260*p**6*q3**6 - 1260*p**6*q4**6 + 1260*p**6*q4**5 + 756*p**6*q5**5 - 756*p**6*q5**4 - 252*p**6*q6**4 + 252*p**6*q6**3 + 36*p**6*q7**3 - 36*p**6*q7**2 - 504*p**5*q1**9 + 504*p**5*q1**8 + 1260*p**5*q2**8 - 1260*p**5*q2**7 - 1680*p**5*q3**7 + 1680*p**5*q3**6 + 1260*p**5*q4**6 - 1260*p**5*q4**5 - 504*p**5*q5**5 + 504*p**5*q5**4 + 84*p**5*q6**4 - 84*p**5*q6**3 + 630*p**4*q1**9 - 630*p**4*q1**8 - 1260*p**4*q2**8 + 1260*p**4*q2**7 + 1260*p**4*q3**7 - 1260*p**4*q3**6 - 630*p**4*q4**6 + 630*p**4*q4**5 + 126*p**4*q5**5 - 126*p**4*q5**4 - 504*p**3*q1**9 + 504*p**3*q1**8 + 756*p**3*q2**8 - 756*p**3*q2**7 - 504*p**3*q3**7 + 504*p**3*q3**6 + 126*p**3*q4**6 - 126*p**3*q4**5 + 252*p**2*q1**9 - 252*p**2*q1**8 - 252*p**2*q2**8 + 252*p**2*q2**7 + 84*p**2*q3**7 - 84*p**2*q3**6 - 72*p*q1**9 + 72*p*q1**8 + 36*p*q2**8 - 36*p*q2**7 + 9*q1**9 - 9*q1**8)'
f_multiparam_syn[10][10] = '-p*(10*p**9*q1**9 - 45*p**9*q2**8 + 120*p**9*q3**7 - 210*p**9*q4**6 + 252*p**9*q5**5 - 210*p**9*q6**4 + 120*p**9*q7**3 - 45*p**9*q8**2 + 10*p**9*q9 - p**9 - 90*p**8*q1**9 + 360*p**8*q2**8 - 840*p**8*q3**7 + 1260*p**8*q4**6 - 1260*p**8*q5**5 + 840*p**8*q6**4 - 360*p**8*q7**3 + 90*p**8*q8**2 - 10*p**8*q9 + 360*p**7*q1**9 - 1260*p**7*q2**8 + 2520*p**7*q3**7 - 3150*p**7*q4**6 + 2520*p**7*q5**5 - 1260*p**7*q6**4 + 360*p**7*q7**3 - 45*p**7*q8**2 - 840*p**6*q1**9 + 2520*p**6*q2**8 - 4200*p**6*q3**7 + 4200*p**6*q4**6 - 2520*p**6*q5**5 + 840*p**6*q6**4 - 120*p**6*q7**3 + 1260*p**5*q1**9 - 3150*p**5*q2**8 + 4200*p**5*q3**7 - 3150*p**5*q4**6 + 1260*p**5*q5**5 - 210*p**5*q6**4 - 1260*p**4*q1**9 + 2520*p**4*q2**8 - 2520*p**4*q3**7 + 1260*p**4*q4**6 - 252*p**4*q5**5 + 840*p**3*q1**9 - 1260*p**3*q2**8 + 840*p**3*q3**7 - 210*p**3*q4**6 - 360*p**2*q1**9 + 360*p**2*q2**8 - 120*p**2*q3**7 + 90*p*q1**9 - 45*p*q2**8 - 10*q1**9)'

In [ ]:
start_time = time.time()
yy = get_f("./multiparam_syn*20.txt", "storm", True, agents_quantities)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
start_time = time.time()
f_multiparam_syn.update(get_f("./multiparam_syn*20.txt", "prism", False, agents_quantities))
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
start_time = time.time()
rewards_multiparam_syn = get_rewards("./multiparam_syn*[0-9]_moments*.txt", "prism", False, agents_quantities)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

### Semisynchronous multiparam models

This may take a while to factorise for f_multiparam_syn[10], so I will load precomputed version for you

In [ ]:
# sys.setrecursionlimit(15000)
# start_time = time.time()
# f_multiparam = get_f("./multiparam_se*[0-9].txt", "prism", True)
# print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
f_multiparam_semisyn = get_f("./multiparam_se*_[0-9].txt", "prism", True, agents_quantities)
f_multiparam_semisyn[10] = [0,0,0,0,0,0,0,0,0,0,0]
f_multiparam_semisyn[10][0] = '(p - 1)**10'
f_multiparam_semisyn[10][1] = '10*p*(p - 1)**9*(q1 - 1)**9'
f_multiparam_semisyn[10][2] = '-5*p*(p - 1)**8*(2*p*q1**9 + 2*p*q1**8*q2 - 18*p*q1**8 + 2*p*q1**7*q2**2 - 18*p*q1**7*q2 + 72*p*q1**7 + 2*p*q1**6*q2**3 - 18*p*q1**6*q2**2 + 72*p*q1**6*q2 - 168*p*q1**6 + 2*p*q1**5*q2**4 - 18*p*q1**5*q2**3 + 72*p*q1**5*q2**2 - 168*p*q1**5*q2 + 252*p*q1**5 + 2*p*q1**4*q2**5 - 18*p*q1**4*q2**4 + 72*p*q1**4*q2**3 - 168*p*q1**4*q2**2 + 252*p*q1**4*q2 - 252*p*q1**4 + 2*p*q1**3*q2**6 - 18*p*q1**3*q2**5 + 72*p*q1**3*q2**4 - 168*p*q1**3*q2**3 + 252*p*q1**3*q2**2 - 252*p*q1**3*q2 + 168*p*q1**3 + 2*p*q1**2*q2**7 - 18*p*q1**2*q2**6 + 72*p*q1**2*q2**5 - 168*p*q1**2*q2**4 + 252*p*q1**2*q2**3 - 252*p*q1**2*q2**2 + 168*p*q1**2*q2 - 72*p*q1**2 + 2*p*q1*q2**8 - 18*p*q1*q2**7 + 72*p*q1*q2**6 - 168*p*q1*q2**5 + 252*p*q1*q2**4 - 252*p*q1*q2**3 + 168*p*q1*q2**2 - 72*p*q1*q2 + 18*p*q1 - 9*p*q2**8 + 72*p*q2**7 - 252*p*q2**6 + 504*p*q2**5 - 630*p*q2**4 + 504*p*q2**3 - 252*p*q2**2 + 72*p*q2 - 9*p - 2*q1**9 - 2*q1**8*q2 + 18*q1**8 - 2*q1**7*q2**2 + 18*q1**7*q2 - 72*q1**7 - 2*q1**6*q2**3 + 18*q1**6*q2**2 - 72*q1**6*q2 + 168*q1**6 - 2*q1**5*q2**4 + 18*q1**5*q2**3 - 72*q1**5*q2**2 + 168*q1**5*q2 - 252*q1**5 - 2*q1**4*q2**5 + 18*q1**4*q2**4 - 72*q1**4*q2**3 + 168*q1**4*q2**2 - 252*q1**4*q2 + 252*q1**4 - 2*q1**3*q2**6 + 18*q1**3*q2**5 - 72*q1**3*q2**4 + 168*q1**3*q2**3 - 252*q1**3*q2**2 + 252*q1**3*q2 - 168*q1**3 - 2*q1**2*q2**7 + 18*q1**2*q2**6 - 72*q1**2*q2**5 + 168*q1**2*q2**4 - 252*q1**2*q2**3 + 252*q1**2*q2**2 - 168*q1**2*q2 + 72*q1**2 - 2*q1*q2**8 + 18*q1*q2**7 - 72*q1*q2**6 + 168*q1*q2**5 - 252*q1*q2**4 + 252*q1*q2**3 - 168*q1*q2**2 + 72*q1*q2 - 18*q1)'
f_multiparam_semisyn[10][3] = '5*p*(p - 1)**7*(2*p**2*q1**8*q2 + 2*p**2*q1**7*q2**2 + 2*p**2*q1**7*q2*q3 - 18*p**2*q1**7*q2 + 2*p**2*q1**6*q2**3 + 2*p**2*q1**6*q2**2*q3 - 18*p**2*q1**6*q2**2 + 2*p**2*q1**6*q2*q3**2 - 18*p**2*q1**6*q2*q3 + 72*p**2*q1**6*q2 + 2*p**2*q1**5*q2**4 + 2*p**2*q1**5*q2**3*q3 - 18*p**2*q1**5*q2**3 + 2*p**2*q1**5*q2**2*q3**2 - 18*p**2*q1**5*q2**2*q3 + 72*p**2*q1**5*q2**2 + 2*p**2*q1**5*q2*q3**3 - 18*p**2*q1**5*q2*q3**2 + 72*p**2*q1**5*q2*q3 - 168*p**2*q1**5*q2 + 2*p**2*q1**4*q2**5 + 2*p**2*q1**4*q2**4*q3 - 18*p**2*q1**4*q2**4 + 2*p**2*q1**4*q2**3*q3**2 - 18*p**2*q1**4*q2**3*q3 + 72*p**2*q1**4*q2**3 + 2*p**2*q1**4*q2**2*q3**3 - 18*p**2*q1**4*q2**2*q3**2 + 72*p**2*q1**4*q2**2*q3 - 168*p**2*q1**4*q2**2 + 2*p**2*q1**4*q2*q3**4 - 18*p**2*q1**4*q2*q3**3 + 72*p**2*q1**4*q2*q3**2 - 168*p**2*q1**4*q2*q3 + 252*p**2*q1**4*q2 + 2*p**2*q1**3*q2**6 + 2*p**2*q1**3*q2**5*q3 - 18*p**2*q1**3*q2**5 + 2*p**2*q1**3*q2**4*q3**2 - 18*p**2*q1**3*q2**4*q3 + 72*p**2*q1**3*q2**4 + 2*p**2*q1**3*q2**3*q3**3 - 18*p**2*q1**3*q2**3*q3**2 + 72*p**2*q1**3*q2**3*q3 - 168*p**2*q1**3*q2**3 + 2*p**2*q1**3*q2**2*q3**4 - 18*p**2*q1**3*q2**2*q3**3 + 72*p**2*q1**3*q2**2*q3**2 - 168*p**2*q1**3*q2**2*q3 + 252*p**2*q1**3*q2**2 + 2*p**2*q1**3*q2*q3**5 - 18*p**2*q1**3*q2*q3**4 + 72*p**2*q1**3*q2*q3**3 - 168*p**2*q1**3*q2*q3**2 + 252*p**2*q1**3*q2*q3 - 252*p**2*q1**3*q2 + 2*p**2*q1**2*q2**7 + 2*p**2*q1**2*q2**6*q3 - 18*p**2*q1**2*q2**6 + 2*p**2*q1**2*q2**5*q3**2 - 18*p**2*q1**2*q2**5*q3 + 72*p**2*q1**2*q2**5 + 2*p**2*q1**2*q2**4*q3**3 - 18*p**2*q1**2*q2**4*q3**2 + 72*p**2*q1**2*q2**4*q3 - 168*p**2*q1**2*q2**4 + 2*p**2*q1**2*q2**3*q3**4 - 18*p**2*q1**2*q2**3*q3**3 + 72*p**2*q1**2*q2**3*q3**2 - 168*p**2*q1**2*q2**3*q3 + 252*p**2*q1**2*q2**3 + 2*p**2*q1**2*q2**2*q3**5 - 18*p**2*q1**2*q2**2*q3**4 + 72*p**2*q1**2*q2**2*q3**3 - 168*p**2*q1**2*q2**2*q3**2 + 252*p**2*q1**2*q2**2*q3 - 252*p**2*q1**2*q2**2 + 2*p**2*q1**2*q2*q3**6 - 18*p**2*q1**2*q2*q3**5 + 72*p**2*q1**2*q2*q3**4 - 168*p**2*q1**2*q2*q3**3 + 252*p**2*q1**2*q2*q3**2 - 252*p**2*q1**2*q2*q3 + 168*p**2*q1**2*q2 + 2*p**2*q1*q2**8 + 2*p**2*q1*q2**7*q3 - 18*p**2*q1*q2**7 + 2*p**2*q1*q2**6*q3**2 - 18*p**2*q1*q2**6*q3 + 72*p**2*q1*q2**6 + 2*p**2*q1*q2**5*q3**3 - 18*p**2*q1*q2**5*q3**2 + 72*p**2*q1*q2**5*q3 - 168*p**2*q1*q2**5 + 2*p**2*q1*q2**4*q3**4 - 18*p**2*q1*q2**4*q3**3 + 72*p**2*q1*q2**4*q3**2 - 168*p**2*q1*q2**4*q3 + 252*p**2*q1*q2**4 + 2*p**2*q1*q2**3*q3**5 - 18*p**2*q1*q2**3*q3**4 + 72*p**2*q1*q2**3*q3**3 - 168*p**2*q1*q2**3*q3**2 + 252*p**2*q1*q2**3*q3 - 252*p**2*q1*q2**3 + 2*p**2*q1*q2**2*q3**6 - 18*p**2*q1*q2**2*q3**5 + 72*p**2*q1*q2**2*q3**4 - 168*p**2*q1*q2**2*q3**3 + 252*p**2*q1*q2**2*q3**2 - 252*p**2*q1*q2**2*q3 + 168*p**2*q1*q2**2 + 2*p**2*q1*q2*q3**7 - 18*p**2*q1*q2*q3**6 + 72*p**2*q1*q2*q3**5 - 168*p**2*q1*q2*q3**4 + 252*p**2*q1*q2*q3**3 - 252*p**2*q1*q2*q3**2 + 168*p**2*q1*q2*q3 - 72*p**2*q1*q2 - 9*p**2*q2**8 - 9*p**2*q2**7*q3 + 72*p**2*q2**7 - 9*p**2*q2**6*q3**2 + 72*p**2*q2**6*q3 - 252*p**2*q2**6 - 9*p**2*q2**5*q3**3 + 72*p**2*q2**5*q3**2 - 252*p**2*q2**5*q3 + 504*p**2*q2**5 - 9*p**2*q2**4*q3**4 + 72*p**2*q2**4*q3**3 - 252*p**2*q2**4*q3**2 + 504*p**2*q2**4*q3 - 630*p**2*q2**4 - 9*p**2*q2**3*q3**5 + 72*p**2*q2**3*q3**4 - 252*p**2*q2**3*q3**3 + 504*p**2*q2**3*q3**2 - 630*p**2*q2**3*q3 + 504*p**2*q2**3 - 9*p**2*q2**2*q3**6 + 72*p**2*q2**2*q3**5 - 252*p**2*q2**2*q3**4 + 504*p**2*q2**2*q3**3 - 630*p**2*q2**2*q3**2 + 504*p**2*q2**2*q3 - 252*p**2*q2**2 - 9*p**2*q2*q3**7 + 72*p**2*q2*q3**6 - 252*p**2*q2*q3**5 + 504*p**2*q2*q3**4 - 630*p**2*q2*q3**3 + 504*p**2*q2*q3**2 - 252*p**2*q2*q3 + 72*p**2*q2 + 24*p**2*q3**7 - 168*p**2*q3**6 + 504*p**2*q3**5 - 840*p**2*q3**4 + 840*p**2*q3**3 - 504*p**2*q3**2 + 168*p**2*q3 - 24*p**2 - 4*p*q1**8*q2 - 4*p*q1**7*q2**2 - 4*p*q1**7*q2*q3 + 36*p*q1**7*q2 - 4*p*q1**6*q2**3 - 4*p*q1**6*q2**2*q3 + 36*p*q1**6*q2**2 - 4*p*q1**6*q2*q3**2 + 36*p*q1**6*q2*q3 - 144*p*q1**6*q2 - 4*p*q1**5*q2**4 - 4*p*q1**5*q2**3*q3 + 36*p*q1**5*q2**3 - 4*p*q1**5*q2**2*q3**2 + 36*p*q1**5*q2**2*q3 - 144*p*q1**5*q2**2 - 4*p*q1**5*q2*q3**3 + 36*p*q1**5*q2*q3**2 - 144*p*q1**5*q2*q3 + 336*p*q1**5*q2 - 4*p*q1**4*q2**5 - 4*p*q1**4*q2**4*q3 + 36*p*q1**4*q2**4 - 4*p*q1**4*q2**3*q3**2 + 36*p*q1**4*q2**3*q3 - 144*p*q1**4*q2**3 - 4*p*q1**4*q2**2*q3**3 + 36*p*q1**4*q2**2*q3**2 - 144*p*q1**4*q2**2*q3 + 336*p*q1**4*q2**2 - 4*p*q1**4*q2*q3**4 + 36*p*q1**4*q2*q3**3 - 144*p*q1**4*q2*q3**2 + 336*p*q1**4*q2*q3 - 504*p*q1**4*q2 - 4*p*q1**3*q2**6 - 4*p*q1**3*q2**5*q3 + 36*p*q1**3*q2**5 - 4*p*q1**3*q2**4*q3**2 + 36*p*q1**3*q2**4*q3 - 144*p*q1**3*q2**4 - 4*p*q1**3*q2**3*q3**3 + 36*p*q1**3*q2**3*q3**2 - 144*p*q1**3*q2**3*q3 + 336*p*q1**3*q2**3 - 4*p*q1**3*q2**2*q3**4 + 36*p*q1**3*q2**2*q3**3 - 144*p*q1**3*q2**2*q3**2 + 336*p*q1**3*q2**2*q3 - 504*p*q1**3*q2**2 - 4*p*q1**3*q2*q3**5 + 36*p*q1**3*q2*q3**4 - 144*p*q1**3*q2*q3**3 + 336*p*q1**3*q2*q3**2 - 504*p*q1**3*q2*q3 + 504*p*q1**3*q2 - 4*p*q1**2*q2**7 - 4*p*q1**2*q2**6*q3 + 36*p*q1**2*q2**6 - 4*p*q1**2*q2**5*q3**2 + 36*p*q1**2*q2**5*q3 - 144*p*q1**2*q2**5 - 4*p*q1**2*q2**4*q3**3 + 36*p*q1**2*q2**4*q3**2 - 144*p*q1**2*q2**4*q3 + 336*p*q1**2*q2**4 - 4*p*q1**2*q2**3*q3**4 + 36*p*q1**2*q2**3*q3**3 - 144*p*q1**2*q2**3*q3**2 + 336*p*q1**2*q2**3*q3 - 504*p*q1**2*q2**3 - 4*p*q1**2*q2**2*q3**5 + 36*p*q1**2*q2**2*q3**4 - 144*p*q1**2*q2**2*q3**3 + 336*p*q1**2*q2**2*q3**2 - 504*p*q1**2*q2**2*q3 + 504*p*q1**2*q2**2 - 4*p*q1**2*q2*q3**6 + 36*p*q1**2*q2*q3**5 - 144*p*q1**2*q2*q3**4 + 336*p*q1**2*q2*q3**3 - 504*p*q1**2*q2*q3**2 + 504*p*q1**2*q2*q3 - 336*p*q1**2*q2 - 4*p*q1*q2**8 - 4*p*q1*q2**7*q3 + 36*p*q1*q2**7 - 4*p*q1*q2**6*q3**2 + 36*p*q1*q2**6*q3 - 144*p*q1*q2**6 - 4*p*q1*q2**5*q3**3 + 36*p*q1*q2**5*q3**2 - 144*p*q1*q2**5*q3 + 336*p*q1*q2**5 - 4*p*q1*q2**4*q3**4 + 36*p*q1*q2**4*q3**3 - 144*p*q1*q2**4*q3**2 + 336*p*q1*q2**4*q3 - 504*p*q1*q2**4 - 4*p*q1*q2**3*q3**5 + 36*p*q1*q2**3*q3**4 - 144*p*q1*q2**3*q3**3 + 336*p*q1*q2**3*q3**2 - 504*p*q1*q2**3*q3 + 504*p*q1*q2**3 - 4*p*q1*q2**2*q3**6 + 36*p*q1*q2**2*q3**5 - 144*p*q1*q2**2*q3**4 + 336*p*q1*q2**2*q3**3 - 504*p*q1*q2**2*q3**2 + 504*p*q1*q2**2*q3 - 336*p*q1*q2**2 - 4*p*q1*q2*q3**7 + 36*p*q1*q2*q3**6 - 144*p*q1*q2*q3**5 + 336*p*q1*q2*q3**4 - 504*p*q1*q2*q3**3 + 504*p*q1*q2*q3**2 - 336*p*q1*q2*q3 + 144*p*q1*q2 + 9*p*q2**8 + 9*p*q2**7*q3 - 72*p*q2**7 + 9*p*q2**6*q3**2 - 72*p*q2**6*q3 + 252*p*q2**6 + 9*p*q2**5*q3**3 - 72*p*q2**5*q3**2 + 252*p*q2**5*q3 - 504*p*q2**5 + 9*p*q2**4*q3**4 - 72*p*q2**4*q3**3 + 252*p*q2**4*q3**2 - 504*p*q2**4*q3 + 630*p*q2**4 + 9*p*q2**3*q3**5 - 72*p*q2**3*q3**4 + 252*p*q2**3*q3**3 - 504*p*q2**3*q3**2 + 630*p*q2**3*q3 - 504*p*q2**3 + 9*p*q2**2*q3**6 - 72*p*q2**2*q3**5 + 252*p*q2**2*q3**4 - 504*p*q2**2*q3**3 + 630*p*q2**2*q3**2 - 504*p*q2**2*q3 + 252*p*q2**2 + 9*p*q2*q3**7 - 72*p*q2*q3**6 + 252*p*q2*q3**5 - 504*p*q2*q3**4 + 630*p*q2*q3**3 - 504*p*q2*q3**2 + 252*p*q2*q3 - 72*p*q2 + 2*q1**8*q2 + 2*q1**7*q2**2 + 2*q1**7*q2*q3 - 18*q1**7*q2 + 2*q1**6*q2**3 + 2*q1**6*q2**2*q3 - 18*q1**6*q2**2 + 2*q1**6*q2*q3**2 - 18*q1**6*q2*q3 + 72*q1**6*q2 + 2*q1**5*q2**4 + 2*q1**5*q2**3*q3 - 18*q1**5*q2**3 + 2*q1**5*q2**2*q3**2 - 18*q1**5*q2**2*q3 + 72*q1**5*q2**2 + 2*q1**5*q2*q3**3 - 18*q1**5*q2*q3**2 + 72*q1**5*q2*q3 - 168*q1**5*q2 + 2*q1**4*q2**5 + 2*q1**4*q2**4*q3 - 18*q1**4*q2**4 + 2*q1**4*q2**3*q3**2 - 18*q1**4*q2**3*q3 + 72*q1**4*q2**3 + 2*q1**4*q2**2*q3**3 - 18*q1**4*q2**2*q3**2 + 72*q1**4*q2**2*q3 - 168*q1**4*q2**2 + 2*q1**4*q2*q3**4 - 18*q1**4*q2*q3**3 + 72*q1**4*q2*q3**2 - 168*q1**4*q2*q3 + 252*q1**4*q2 + 2*q1**3*q2**6 + 2*q1**3*q2**5*q3 - 18*q1**3*q2**5 + 2*q1**3*q2**4*q3**2 - 18*q1**3*q2**4*q3 + 72*q1**3*q2**4 + 2*q1**3*q2**3*q3**3 - 18*q1**3*q2**3*q3**2 + 72*q1**3*q2**3*q3 - 168*q1**3*q2**3 + 2*q1**3*q2**2*q3**4 - 18*q1**3*q2**2*q3**3 + 72*q1**3*q2**2*q3**2 - 168*q1**3*q2**2*q3 + 252*q1**3*q2**2 + 2*q1**3*q2*q3**5 - 18*q1**3*q2*q3**4 + 72*q1**3*q2*q3**3 - 168*q1**3*q2*q3**2 + 252*q1**3*q2*q3 - 252*q1**3*q2 + 2*q1**2*q2**7 + 2*q1**2*q2**6*q3 - 18*q1**2*q2**6 + 2*q1**2*q2**5*q3**2 - 18*q1**2*q2**5*q3 + 72*q1**2*q2**5 + 2*q1**2*q2**4*q3**3 - 18*q1**2*q2**4*q3**2 + 72*q1**2*q2**4*q3 - 168*q1**2*q2**4 + 2*q1**2*q2**3*q3**4 - 18*q1**2*q2**3*q3**3 + 72*q1**2*q2**3*q3**2 - 168*q1**2*q2**3*q3 + 252*q1**2*q2**3 + 2*q1**2*q2**2*q3**5 - 18*q1**2*q2**2*q3**4 + 72*q1**2*q2**2*q3**3 - 168*q1**2*q2**2*q3**2 + 252*q1**2*q2**2*q3 - 252*q1**2*q2**2 + 2*q1**2*q2*q3**6 - 18*q1**2*q2*q3**5 + 72*q1**2*q2*q3**4 - 168*q1**2*q2*q3**3 + 252*q1**2*q2*q3**2 - 252*q1**2*q2*q3 + 168*q1**2*q2 + 2*q1*q2**8 + 2*q1*q2**7*q3 - 18*q1*q2**7 + 2*q1*q2**6*q3**2 - 18*q1*q2**6*q3 + 72*q1*q2**6 + 2*q1*q2**5*q3**3 - 18*q1*q2**5*q3**2 + 72*q1*q2**5*q3 - 168*q1*q2**5 + 2*q1*q2**4*q3**4 - 18*q1*q2**4*q3**3 + 72*q1*q2**4*q3**2 - 168*q1*q2**4*q3 + 252*q1*q2**4 + 2*q1*q2**3*q3**5 - 18*q1*q2**3*q3**4 + 72*q1*q2**3*q3**3 - 168*q1*q2**3*q3**2 + 252*q1*q2**3*q3 - 252*q1*q2**3 + 2*q1*q2**2*q3**6 - 18*q1*q2**2*q3**5 + 72*q1*q2**2*q3**4 - 168*q1*q2**2*q3**3 + 252*q1*q2**2*q3**2 - 252*q1*q2**2*q3 + 168*q1*q2**2 + 2*q1*q2*q3**7 - 18*q1*q2*q3**6 + 72*q1*q2*q3**5 - 168*q1*q2*q3**4 + 252*q1*q2*q3**3 - 252*q1*q2*q3**2 + 168*q1*q2*q3 - 72*q1*q2)'
f_multiparam_semisyn[10][4] = '-5*p*(p - 1)**6*(2*p**3*q1**7*q2*q3 + 2*p**3*q1**6*q2**2*q3 + 2*p**3*q1**6*q2*q3**2 + 2*p**3*q1**6*q2*q3*q4 - 18*p**3*q1**6*q2*q3 + 2*p**3*q1**5*q2**3*q3 + 2*p**3*q1**5*q2**2*q3**2 + 2*p**3*q1**5*q2**2*q3*q4 - 18*p**3*q1**5*q2**2*q3 + 2*p**3*q1**5*q2*q3**3 + 2*p**3*q1**5*q2*q3**2*q4 - 18*p**3*q1**5*q2*q3**2 + 2*p**3*q1**5*q2*q3*q4**2 - 18*p**3*q1**5*q2*q3*q4 + 72*p**3*q1**5*q2*q3 + 2*p**3*q1**4*q2**4*q3 + 2*p**3*q1**4*q2**3*q3**2 + 2*p**3*q1**4*q2**3*q3*q4 - 18*p**3*q1**4*q2**3*q3 + 2*p**3*q1**4*q2**2*q3**3 + 2*p**3*q1**4*q2**2*q3**2*q4 - 18*p**3*q1**4*q2**2*q3**2 + 2*p**3*q1**4*q2**2*q3*q4**2 - 18*p**3*q1**4*q2**2*q3*q4 + 72*p**3*q1**4*q2**2*q3 + 2*p**3*q1**4*q2*q3**4 + 2*p**3*q1**4*q2*q3**3*q4 - 18*p**3*q1**4*q2*q3**3 + 2*p**3*q1**4*q2*q3**2*q4**2 - 18*p**3*q1**4*q2*q3**2*q4 + 72*p**3*q1**4*q2*q3**2 + 2*p**3*q1**4*q2*q3*q4**3 - 18*p**3*q1**4*q2*q3*q4**2 + 72*p**3*q1**4*q2*q3*q4 - 168*p**3*q1**4*q2*q3 + 2*p**3*q1**3*q2**5*q3 + 2*p**3*q1**3*q2**4*q3**2 + 2*p**3*q1**3*q2**4*q3*q4 - 18*p**3*q1**3*q2**4*q3 + 2*p**3*q1**3*q2**3*q3**3 + 2*p**3*q1**3*q2**3*q3**2*q4 - 18*p**3*q1**3*q2**3*q3**2 + 2*p**3*q1**3*q2**3*q3*q4**2 - 18*p**3*q1**3*q2**3*q3*q4 + 72*p**3*q1**3*q2**3*q3 + 2*p**3*q1**3*q2**2*q3**4 + 2*p**3*q1**3*q2**2*q3**3*q4 - 18*p**3*q1**3*q2**2*q3**3 + 2*p**3*q1**3*q2**2*q3**2*q4**2 - 18*p**3*q1**3*q2**2*q3**2*q4 + 72*p**3*q1**3*q2**2*q3**2 + 2*p**3*q1**3*q2**2*q3*q4**3 - 18*p**3*q1**3*q2**2*q3*q4**2 + 72*p**3*q1**3*q2**2*q3*q4 - 168*p**3*q1**3*q2**2*q3 + 2*p**3*q1**3*q2*q3**5 + 2*p**3*q1**3*q2*q3**4*q4 - 18*p**3*q1**3*q2*q3**4 + 2*p**3*q1**3*q2*q3**3*q4**2 - 18*p**3*q1**3*q2*q3**3*q4 + 72*p**3*q1**3*q2*q3**3 + 2*p**3*q1**3*q2*q3**2*q4**3 - 18*p**3*q1**3*q2*q3**2*q4**2 + 72*p**3*q1**3*q2*q3**2*q4 - 168*p**3*q1**3*q2*q3**2 + 2*p**3*q1**3*q2*q3*q4**4 - 18*p**3*q1**3*q2*q3*q4**3 + 72*p**3*q1**3*q2*q3*q4**2 - 168*p**3*q1**3*q2*q3*q4 + 252*p**3*q1**3*q2*q3 + 2*p**3*q1**2*q2**6*q3 + 2*p**3*q1**2*q2**5*q3**2 + 2*p**3*q1**2*q2**5*q3*q4 - 18*p**3*q1**2*q2**5*q3 + 2*p**3*q1**2*q2**4*q3**3 + 2*p**3*q1**2*q2**4*q3**2*q4 - 18*p**3*q1**2*q2**4*q3**2 + 2*p**3*q1**2*q2**4*q3*q4**2 - 18*p**3*q1**2*q2**4*q3*q4 + 72*p**3*q1**2*q2**4*q3 + 2*p**3*q1**2*q2**3*q3**4 + 2*p**3*q1**2*q2**3*q3**3*q4 - 18*p**3*q1**2*q2**3*q3**3 + 2*p**3*q1**2*q2**3*q3**2*q4**2 - 18*p**3*q1**2*q2**3*q3**2*q4 + 72*p**3*q1**2*q2**3*q3**2 + 2*p**3*q1**2*q2**3*q3*q4**3 - 18*p**3*q1**2*q2**3*q3*q4**2 + 72*p**3*q1**2*q2**3*q3*q4 - 168*p**3*q1**2*q2**3*q3 + 2*p**3*q1**2*q2**2*q3**5 + 2*p**3*q1**2*q2**2*q3**4*q4 - 18*p**3*q1**2*q2**2*q3**4 + 2*p**3*q1**2*q2**2*q3**3*q4**2 - 18*p**3*q1**2*q2**2*q3**3*q4 + 72*p**3*q1**2*q2**2*q3**3 + 2*p**3*q1**2*q2**2*q3**2*q4**3 - 18*p**3*q1**2*q2**2*q3**2*q4**2 + 72*p**3*q1**2*q2**2*q3**2*q4 - 168*p**3*q1**2*q2**2*q3**2 + 2*p**3*q1**2*q2**2*q3*q4**4 - 18*p**3*q1**2*q2**2*q3*q4**3 + 72*p**3*q1**2*q2**2*q3*q4**2 - 168*p**3*q1**2*q2**2*q3*q4 + 252*p**3*q1**2*q2**2*q3 + 2*p**3*q1**2*q2*q3**6 + 2*p**3*q1**2*q2*q3**5*q4 - 18*p**3*q1**2*q2*q3**5 + 2*p**3*q1**2*q2*q3**4*q4**2 - 18*p**3*q1**2*q2*q3**4*q4 + 72*p**3*q1**2*q2*q3**4 + 2*p**3*q1**2*q2*q3**3*q4**3 - 18*p**3*q1**2*q2*q3**3*q4**2 + 72*p**3*q1**2*q2*q3**3*q4 - 168*p**3*q1**2*q2*q3**3 + 2*p**3*q1**2*q2*q3**2*q4**4 - 18*p**3*q1**2*q2*q3**2*q4**3 + 72*p**3*q1**2*q2*q3**2*q4**2 - 168*p**3*q1**2*q2*q3**2*q4 + 252*p**3*q1**2*q2*q3**2 + 2*p**3*q1**2*q2*q3*q4**5 - 18*p**3*q1**2*q2*q3*q4**4 + 72*p**3*q1**2*q2*q3*q4**3 - 168*p**3*q1**2*q2*q3*q4**2 + 252*p**3*q1**2*q2*q3*q4 - 252*p**3*q1**2*q2*q3 + 2*p**3*q1*q2**7*q3 + 2*p**3*q1*q2**6*q3**2 + 2*p**3*q1*q2**6*q3*q4 - 18*p**3*q1*q2**6*q3 + 2*p**3*q1*q2**5*q3**3 + 2*p**3*q1*q2**5*q3**2*q4 - 18*p**3*q1*q2**5*q3**2 + 2*p**3*q1*q2**5*q3*q4**2 - 18*p**3*q1*q2**5*q3*q4 + 72*p**3*q1*q2**5*q3 + 2*p**3*q1*q2**4*q3**4 + 2*p**3*q1*q2**4*q3**3*q4 - 18*p**3*q1*q2**4*q3**3 + 2*p**3*q1*q2**4*q3**2*q4**2 - 18*p**3*q1*q2**4*q3**2*q4 + 72*p**3*q1*q2**4*q3**2 + 2*p**3*q1*q2**4*q3*q4**3 - 18*p**3*q1*q2**4*q3*q4**2 + 72*p**3*q1*q2**4*q3*q4 - 168*p**3*q1*q2**4*q3 + 2*p**3*q1*q2**3*q3**5 + 2*p**3*q1*q2**3*q3**4*q4 - 18*p**3*q1*q2**3*q3**4 + 2*p**3*q1*q2**3*q3**3*q4**2 - 18*p**3*q1*q2**3*q3**3*q4 + 72*p**3*q1*q2**3*q3**3 + 2*p**3*q1*q2**3*q3**2*q4**3 - 18*p**3*q1*q2**3*q3**2*q4**2 + 72*p**3*q1*q2**3*q3**2*q4 - 168*p**3*q1*q2**3*q3**2 + 2*p**3*q1*q2**3*q3*q4**4 - 18*p**3*q1*q2**3*q3*q4**3 + 72*p**3*q1*q2**3*q3*q4**2 - 168*p**3*q1*q2**3*q3*q4 + 252*p**3*q1*q2**3*q3 + 2*p**3*q1*q2**2*q3**6 + 2*p**3*q1*q2**2*q3**5*q4 - 18*p**3*q1*q2**2*q3**5 + 2*p**3*q1*q2**2*q3**4*q4**2 - 18*p**3*q1*q2**2*q3**4*q4 + 72*p**3*q1*q2**2*q3**4 + 2*p**3*q1*q2**2*q3**3*q4**3 - 18*p**3*q1*q2**2*q3**3*q4**2 + 72*p**3*q1*q2**2*q3**3*q4 - 168*p**3*q1*q2**2*q3**3 + 2*p**3*q1*q2**2*q3**2*q4**4 - 18*p**3*q1*q2**2*q3**2*q4**3 + 72*p**3*q1*q2**2*q3**2*q4**2 - 168*p**3*q1*q2**2*q3**2*q4 + 252*p**3*q1*q2**2*q3**2 + 2*p**3*q1*q2**2*q3*q4**5 - 18*p**3*q1*q2**2*q3*q4**4 + 72*p**3*q1*q2**2*q3*q4**3 - 168*p**3*q1*q2**2*q3*q4**2 + 252*p**3*q1*q2**2*q3*q4 - 252*p**3*q1*q2**2*q3 + 2*p**3*q1*q2*q3**7 + 2*p**3*q1*q2*q3**6*q4 - 18*p**3*q1*q2*q3**6 + 2*p**3*q1*q2*q3**5*q4**2 - 18*p**3*q1*q2*q3**5*q4 + 72*p**3*q1*q2*q3**5 + 2*p**3*q1*q2*q3**4*q4**3 - 18*p**3*q1*q2*q3**4*q4**2 + 72*p**3*q1*q2*q3**4*q4 - 168*p**3*q1*q2*q3**4 + 2*p**3*q1*q2*q3**3*q4**4 - 18*p**3*q1*q2*q3**3*q4**3 + 72*p**3*q1*q2*q3**3*q4**2 - 168*p**3*q1*q2*q3**3*q4 + 252*p**3*q1*q2*q3**3 + 2*p**3*q1*q2*q3**2*q4**5 - 18*p**3*q1*q2*q3**2*q4**4 + 72*p**3*q1*q2*q3**2*q4**3 - 168*p**3*q1*q2*q3**2*q4**2 + 252*p**3*q1*q2*q3**2*q4 - 252*p**3*q1*q2*q3**2 + 2*p**3*q1*q2*q3*q4**6 - 18*p**3*q1*q2*q3*q4**5 + 72*p**3*q1*q2*q3*q4**4 - 168*p**3*q1*q2*q3*q4**3 + 252*p**3*q1*q2*q3*q4**2 - 252*p**3*q1*q2*q3*q4 + 168*p**3*q1*q2*q3 - 9*p**3*q2**7*q3 - 9*p**3*q2**6*q3**2 - 9*p**3*q2**6*q3*q4 + 72*p**3*q2**6*q3 - 9*p**3*q2**5*q3**3 - 9*p**3*q2**5*q3**2*q4 + 72*p**3*q2**5*q3**2 - 9*p**3*q2**5*q3*q4**2 + 72*p**3*q2**5*q3*q4 - 252*p**3*q2**5*q3 - 9*p**3*q2**4*q3**4 - 9*p**3*q2**4*q3**3*q4 + 72*p**3*q2**4*q3**3 - 9*p**3*q2**4*q3**2*q4**2 + 72*p**3*q2**4*q3**2*q4 - 252*p**3*q2**4*q3**2 - 9*p**3*q2**4*q3*q4**3 + 72*p**3*q2**4*q3*q4**2 - 252*p**3*q2**4*q3*q4 + 504*p**3*q2**4*q3 - 9*p**3*q2**3*q3**5 - 9*p**3*q2**3*q3**4*q4 + 72*p**3*q2**3*q3**4 - 9*p**3*q2**3*q3**3*q4**2 + 72*p**3*q2**3*q3**3*q4 - 252*p**3*q2**3*q3**3 - 9*p**3*q2**3*q3**2*q4**3 + 72*p**3*q2**3*q3**2*q4**2 - 252*p**3*q2**3*q3**2*q4 + 504*p**3*q2**3*q3**2 - 9*p**3*q2**3*q3*q4**4 + 72*p**3*q2**3*q3*q4**3 - 252*p**3*q2**3*q3*q4**2 + 504*p**3*q2**3*q3*q4 - 630*p**3*q2**3*q3 - 9*p**3*q2**2*q3**6 - 9*p**3*q2**2*q3**5*q4 + 72*p**3*q2**2*q3**5 - 9*p**3*q2**2*q3**4*q4**2 + 72*p**3*q2**2*q3**4*q4 - 252*p**3*q2**2*q3**4 - 9*p**3*q2**2*q3**3*q4**3 + 72*p**3*q2**2*q3**3*q4**2 - 252*p**3*q2**2*q3**3*q4 + 504*p**3*q2**2*q3**3 - 9*p**3*q2**2*q3**2*q4**4 + 72*p**3*q2**2*q3**2*q4**3 - 252*p**3*q2**2*q3**2*q4**2 + 504*p**3*q2**2*q3**2*q4 - 630*p**3*q2**2*q3**2 - 9*p**3*q2**2*q3*q4**5 + 72*p**3*q2**2*q3*q4**4 - 252*p**3*q2**2*q3*q4**3 + 504*p**3*q2**2*q3*q4**2 - 630*p**3*q2**2*q3*q4 + 504*p**3*q2**2*q3 - 9*p**3*q2*q3**7 - 9*p**3*q2*q3**6*q4 + 72*p**3*q2*q3**6 - 9*p**3*q2*q3**5*q4**2 + 72*p**3*q2*q3**5*q4 - 252*p**3*q2*q3**5 - 9*p**3*q2*q3**4*q4**3 + 72*p**3*q2*q3**4*q4**2 - 252*p**3*q2*q3**4*q4 + 504*p**3*q2*q3**4 - 9*p**3*q2*q3**3*q4**4 + 72*p**3*q2*q3**3*q4**3 - 252*p**3*q2*q3**3*q4**2 + 504*p**3*q2*q3**3*q4 - 630*p**3*q2*q3**3 - 9*p**3*q2*q3**2*q4**5 + 72*p**3*q2*q3**2*q4**4 - 252*p**3*q2*q3**2*q4**3 + 504*p**3*q2*q3**2*q4**2 - 630*p**3*q2*q3**2*q4 + 504*p**3*q2*q3**2 - 9*p**3*q2*q3*q4**6 + 72*p**3*q2*q3*q4**5 - 252*p**3*q2*q3*q4**4 + 504*p**3*q2*q3*q4**3 - 630*p**3*q2*q3*q4**2 + 504*p**3*q2*q3*q4 - 252*p**3*q2*q3 + 24*p**3*q3**7 + 24*p**3*q3**6*q4 - 168*p**3*q3**6 + 24*p**3*q3**5*q4**2 - 168*p**3*q3**5*q4 + 504*p**3*q3**5 + 24*p**3*q3**4*q4**3 - 168*p**3*q3**4*q4**2 + 504*p**3*q3**4*q4 - 840*p**3*q3**4 + 24*p**3*q3**3*q4**4 - 168*p**3*q3**3*q4**3 + 504*p**3*q3**3*q4**2 - 840*p**3*q3**3*q4 + 840*p**3*q3**3 + 24*p**3*q3**2*q4**5 - 168*p**3*q3**2*q4**4 + 504*p**3*q3**2*q4**3 - 840*p**3*q3**2*q4**2 + 840*p**3*q3**2*q4 - 504*p**3*q3**2 + 24*p**3*q3*q4**6 - 168*p**3*q3*q4**5 + 504*p**3*q3*q4**4 - 840*p**3*q3*q4**3 + 840*p**3*q3*q4**2 - 504*p**3*q3*q4 + 168*p**3*q3 - 42*p**3*q4**6 + 252*p**3*q4**5 - 630*p**3*q4**4 + 840*p**3*q4**3 - 630*p**3*q4**2 + 252*p**3*q4 - 42*p**3 - 6*p**2*q1**7*q2*q3 - 6*p**2*q1**6*q2**2*q3 - 6*p**2*q1**6*q2*q3**2 - 6*p**2*q1**6*q2*q3*q4 + 54*p**2*q1**6*q2*q3 - 6*p**2*q1**5*q2**3*q3 - 6*p**2*q1**5*q2**2*q3**2 - 6*p**2*q1**5*q2**2*q3*q4 + 54*p**2*q1**5*q2**2*q3 - 6*p**2*q1**5*q2*q3**3 - 6*p**2*q1**5*q2*q3**2*q4 + 54*p**2*q1**5*q2*q3**2 - 6*p**2*q1**5*q2*q3*q4**2 + 54*p**2*q1**5*q2*q3*q4 - 216*p**2*q1**5*q2*q3 - 6*p**2*q1**4*q2**4*q3 - 6*p**2*q1**4*q2**3*q3**2 - 6*p**2*q1**4*q2**3*q3*q4 + 54*p**2*q1**4*q2**3*q3 - 6*p**2*q1**4*q2**2*q3**3 - 6*p**2*q1**4*q2**2*q3**2*q4 + 54*p**2*q1**4*q2**2*q3**2 - 6*p**2*q1**4*q2**2*q3*q4**2 + 54*p**2*q1**4*q2**2*q3*q4 - 216*p**2*q1**4*q2**2*q3 - 6*p**2*q1**4*q2*q3**4 - 6*p**2*q1**4*q2*q3**3*q4 + 54*p**2*q1**4*q2*q3**3 - 6*p**2*q1**4*q2*q3**2*q4**2 + 54*p**2*q1**4*q2*q3**2*q4 - 216*p**2*q1**4*q2*q3**2 - 6*p**2*q1**4*q2*q3*q4**3 + 54*p**2*q1**4*q2*q3*q4**2 - 216*p**2*q1**4*q2*q3*q4 + 504*p**2*q1**4*q2*q3 - 6*p**2*q1**3*q2**5*q3 - 6*p**2*q1**3*q2**4*q3**2 - 6*p**2*q1**3*q2**4*q3*q4 + 54*p**2*q1**3*q2**4*q3 - 6*p**2*q1**3*q2**3*q3**3 - 6*p**2*q1**3*q2**3*q3**2*q4 + 54*p**2*q1**3*q2**3*q3**2 - 6*p**2*q1**3*q2**3*q3*q4**2 + 54*p**2*q1**3*q2**3*q3*q4 - 216*p**2*q1**3*q2**3*q3 - 6*p**2*q1**3*q2**2*q3**4 - 6*p**2*q1**3*q2**2*q3**3*q4 + 54*p**2*q1**3*q2**2*q3**3 - 6*p**2*q1**3*q2**2*q3**2*q4**2 + 54*p**2*q1**3*q2**2*q3**2*q4 - 216*p**2*q1**3*q2**2*q3**2 - 6*p**2*q1**3*q2**2*q3*q4**3 + 54*p**2*q1**3*q2**2*q3*q4**2 - 216*p**2*q1**3*q2**2*q3*q4 + 504*p**2*q1**3*q2**2*q3 - 6*p**2*q1**3*q2*q3**5 - 6*p**2*q1**3*q2*q3**4*q4 + 54*p**2*q1**3*q2*q3**4 - 6*p**2*q1**3*q2*q3**3*q4**2 + 54*p**2*q1**3*q2*q3**3*q4 - 216*p**2*q1**3*q2*q3**3 - 6*p**2*q1**3*q2*q3**2*q4**3 + 54*p**2*q1**3*q2*q3**2*q4**2 - 216*p**2*q1**3*q2*q3**2*q4 + 504*p**2*q1**3*q2*q3**2 - 6*p**2*q1**3*q2*q3*q4**4 + 54*p**2*q1**3*q2*q3*q4**3 - 216*p**2*q1**3*q2*q3*q4**2 + 504*p**2*q1**3*q2*q3*q4 - 756*p**2*q1**3*q2*q3 - 6*p**2*q1**2*q2**6*q3 - 6*p**2*q1**2*q2**5*q3**2 - 6*p**2*q1**2*q2**5*q3*q4 + 54*p**2*q1**2*q2**5*q3 - 6*p**2*q1**2*q2**4*q3**3 - 6*p**2*q1**2*q2**4*q3**2*q4 + 54*p**2*q1**2*q2**4*q3**2 - 6*p**2*q1**2*q2**4*q3*q4**2 + 54*p**2*q1**2*q2**4*q3*q4 - 216*p**2*q1**2*q2**4*q3 - 6*p**2*q1**2*q2**3*q3**4 - 6*p**2*q1**2*q2**3*q3**3*q4 + 54*p**2*q1**2*q2**3*q3**3 - 6*p**2*q1**2*q2**3*q3**2*q4**2 + 54*p**2*q1**2*q2**3*q3**2*q4 - 216*p**2*q1**2*q2**3*q3**2 - 6*p**2*q1**2*q2**3*q3*q4**3 + 54*p**2*q1**2*q2**3*q3*q4**2 - 216*p**2*q1**2*q2**3*q3*q4 + 504*p**2*q1**2*q2**3*q3 - 6*p**2*q1**2*q2**2*q3**5 - 6*p**2*q1**2*q2**2*q3**4*q4 + 54*p**2*q1**2*q2**2*q3**4 - 6*p**2*q1**2*q2**2*q3**3*q4**2 + 54*p**2*q1**2*q2**2*q3**3*q4 - 216*p**2*q1**2*q2**2*q3**3 - 6*p**2*q1**2*q2**2*q3**2*q4**3 + 54*p**2*q1**2*q2**2*q3**2*q4**2 - 216*p**2*q1**2*q2**2*q3**2*q4 + 504*p**2*q1**2*q2**2*q3**2 - 6*p**2*q1**2*q2**2*q3*q4**4 + 54*p**2*q1**2*q2**2*q3*q4**3 - 216*p**2*q1**2*q2**2*q3*q4**2 + 504*p**2*q1**2*q2**2*q3*q4 - 756*p**2*q1**2*q2**2*q3 - 6*p**2*q1**2*q2*q3**6 - 6*p**2*q1**2*q2*q3**5*q4 + 54*p**2*q1**2*q2*q3**5 - 6*p**2*q1**2*q2*q3**4*q4**2 + 54*p**2*q1**2*q2*q3**4*q4 - 216*p**2*q1**2*q2*q3**4 - 6*p**2*q1**2*q2*q3**3*q4**3 + 54*p**2*q1**2*q2*q3**3*q4**2 - 216*p**2*q1**2*q2*q3**3*q4 + 504*p**2*q1**2*q2*q3**3 - 6*p**2*q1**2*q2*q3**2*q4**4 + 54*p**2*q1**2*q2*q3**2*q4**3 - 216*p**2*q1**2*q2*q3**2*q4**2 + 504*p**2*q1**2*q2*q3**2*q4 - 756*p**2*q1**2*q2*q3**2 - 6*p**2*q1**2*q2*q3*q4**5 + 54*p**2*q1**2*q2*q3*q4**4 - 216*p**2*q1**2*q2*q3*q4**3 + 504*p**2*q1**2*q2*q3*q4**2 - 756*p**2*q1**2*q2*q3*q4 + 756*p**2*q1**2*q2*q3 - 6*p**2*q1*q2**7*q3 - 6*p**2*q1*q2**6*q3**2 - 6*p**2*q1*q2**6*q3*q4 + 54*p**2*q1*q2**6*q3 - 6*p**2*q1*q2**5*q3**3 - 6*p**2*q1*q2**5*q3**2*q4 + 54*p**2*q1*q2**5*q3**2 - 6*p**2*q1*q2**5*q3*q4**2 + 54*p**2*q1*q2**5*q3*q4 - 216*p**2*q1*q2**5*q3 - 6*p**2*q1*q2**4*q3**4 - 6*p**2*q1*q2**4*q3**3*q4 + 54*p**2*q1*q2**4*q3**3 - 6*p**2*q1*q2**4*q3**2*q4**2 + 54*p**2*q1*q2**4*q3**2*q4 - 216*p**2*q1*q2**4*q3**2 - 6*p**2*q1*q2**4*q3*q4**3 + 54*p**2*q1*q2**4*q3*q4**2 - 216*p**2*q1*q2**4*q3*q4 + 504*p**2*q1*q2**4*q3 - 6*p**2*q1*q2**3*q3**5 - 6*p**2*q1*q2**3*q3**4*q4 + 54*p**2*q1*q2**3*q3**4 - 6*p**2*q1*q2**3*q3**3*q4**2 + 54*p**2*q1*q2**3*q3**3*q4 - 216*p**2*q1*q2**3*q3**3 - 6*p**2*q1*q2**3*q3**2*q4**3 + 54*p**2*q1*q2**3*q3**2*q4**2 - 216*p**2*q1*q2**3*q3**2*q4 + 504*p**2*q1*q2**3*q3**2 - 6*p**2*q1*q2**3*q3*q4**4 + 54*p**2*q1*q2**3*q3*q4**3 - 216*p**2*q1*q2**3*q3*q4**2 + 504*p**2*q1*q2**3*q3*q4 - 756*p**2*q1*q2**3*q3 - 6*p**2*q1*q2**2*q3**6 - 6*p**2*q1*q2**2*q3**5*q4 + 54*p**2*q1*q2**2*q3**5 - 6*p**2*q1*q2**2*q3**4*q4**2 + 54*p**2*q1*q2**2*q3**4*q4 - 216*p**2*q1*q2**2*q3**4 - 6*p**2*q1*q2**2*q3**3*q4**3 + 54*p**2*q1*q2**2*q3**3*q4**2 - 216*p**2*q1*q2**2*q3**3*q4 + 504*p**2*q1*q2**2*q3**3 - 6*p**2*q1*q2**2*q3**2*q4**4 + 54*p**2*q1*q2**2*q3**2*q4**3 - 216*p**2*q1*q2**2*q3**2*q4**2 + 504*p**2*q1*q2**2*q3**2*q4 - 756*p**2*q1*q2**2*q3**2 - 6*p**2*q1*q2**2*q3*q4**5 + 54*p**2*q1*q2**2*q3*q4**4 - 216*p**2*q1*q2**2*q3*q4**3 + 504*p**2*q1*q2**2*q3*q4**2 - 756*p**2*q1*q2**2*q3*q4 + 756*p**2*q1*q2**2*q3 - 6*p**2*q1*q2*q3**7 - 6*p**2*q1*q2*q3**6*q4 + 54*p**2*q1*q2*q3**6 - 6*p**2*q1*q2*q3**5*q4**2 + 54*p**2*q1*q2*q3**5*q4 - 216*p**2*q1*q2*q3**5 - 6*p**2*q1*q2*q3**4*q4**3 + 54*p**2*q1*q2*q3**4*q4**2 - 216*p**2*q1*q2*q3**4*q4 + 504*p**2*q1*q2*q3**4 - 6*p**2*q1*q2*q3**3*q4**4 + 54*p**2*q1*q2*q3**3*q4**3 - 216*p**2*q1*q2*q3**3*q4**2 + 504*p**2*q1*q2*q3**3*q4 - 756*p**2*q1*q2*q3**3 - 6*p**2*q1*q2*q3**2*q4**5 + 54*p**2*q1*q2*q3**2*q4**4 - 216*p**2*q1*q2*q3**2*q4**3 + 504*p**2*q1*q2*q3**2*q4**2 - 756*p**2*q1*q2*q3**2*q4 + 756*p**2*q1*q2*q3**2 - 6*p**2*q1*q2*q3*q4**6 + 54*p**2*q1*q2*q3*q4**5 - 216*p**2*q1*q2*q3*q4**4 + 504*p**2*q1*q2*q3*q4**3 - 756*p**2*q1*q2*q3*q4**2 + 756*p**2*q1*q2*q3*q4 - 504*p**2*q1*q2*q3 + 18*p**2*q2**7*q3 + 18*p**2*q2**6*q3**2 + 18*p**2*q2**6*q3*q4 - 144*p**2*q2**6*q3 + 18*p**2*q2**5*q3**3 + 18*p**2*q2**5*q3**2*q4 - 144*p**2*q2**5*q3**2 + 18*p**2*q2**5*q3*q4**2 - 144*p**2*q2**5*q3*q4 + 504*p**2*q2**5*q3 + 18*p**2*q2**4*q3**4 + 18*p**2*q2**4*q3**3*q4 - 144*p**2*q2**4*q3**3 + 18*p**2*q2**4*q3**2*q4**2 - 144*p**2*q2**4*q3**2*q4 + 504*p**2*q2**4*q3**2 + 18*p**2*q2**4*q3*q4**3 - 144*p**2*q2**4*q3*q4**2 + 504*p**2*q2**4*q3*q4 - 1008*p**2*q2**4*q3 + 18*p**2*q2**3*q3**5 + 18*p**2*q2**3*q3**4*q4 - 144*p**2*q2**3*q3**4 + 18*p**2*q2**3*q3**3*q4**2 - 144*p**2*q2**3*q3**3*q4 + 504*p**2*q2**3*q3**3 + 18*p**2*q2**3*q3**2*q4**3 - 144*p**2*q2**3*q3**2*q4**2 + 504*p**2*q2**3*q3**2*q4 - 1008*p**2*q2**3*q3**2 + 18*p**2*q2**3*q3*q4**4 - 144*p**2*q2**3*q3*q4**3 + 504*p**2*q2**3*q3*q4**2 - 1008*p**2*q2**3*q3*q4 + 1260*p**2*q2**3*q3 + 18*p**2*q2**2*q3**6 + 18*p**2*q2**2*q3**5*q4 - 144*p**2*q2**2*q3**5 + 18*p**2*q2**2*q3**4*q4**2 - 144*p**2*q2**2*q3**4*q4 + 504*p**2*q2**2*q3**4 + 18*p**2*q2**2*q3**3*q4**3 - 144*p**2*q2**2*q3**3*q4**2 + 504*p**2*q2**2*q3**3*q4 - 1008*p**2*q2**2*q3**3 + 18*p**2*q2**2*q3**2*q4**4 - 144*p**2*q2**2*q3**2*q4**3 + 504*p**2*q2**2*q3**2*q4**2 - 1008*p**2*q2**2*q3**2*q4 + 1260*p**2*q2**2*q3**2 + 18*p**2*q2**2*q3*q4**5 - 144*p**2*q2**2*q3*q4**4 + 504*p**2*q2**2*q3*q4**3 - 1008*p**2*q2**2*q3*q4**2 + 1260*p**2*q2**2*q3*q4 - 1008*p**2*q2**2*q3 + 18*p**2*q2*q3**7 + 18*p**2*q2*q3**6*q4 - 144*p**2*q2*q3**6 + 18*p**2*q2*q3**5*q4**2 - 144*p**2*q2*q3**5*q4 + 504*p**2*q2*q3**5 + 18*p**2*q2*q3**4*q4**3 - 144*p**2*q2*q3**4*q4**2 + 504*p**2*q2*q3**4*q4 - 1008*p**2*q2*q3**4 + 18*p**2*q2*q3**3*q4**4 - 144*p**2*q2*q3**3*q4**3 + 504*p**2*q2*q3**3*q4**2 - 1008*p**2*q2*q3**3*q4 + 1260*p**2*q2*q3**3 + 18*p**2*q2*q3**2*q4**5 - 144*p**2*q2*q3**2*q4**4 + 504*p**2*q2*q3**2*q4**3 - 1008*p**2*q2*q3**2*q4**2 + 1260*p**2*q2*q3**2*q4 - 1008*p**2*q2*q3**2 + 18*p**2*q2*q3*q4**6 - 144*p**2*q2*q3*q4**5 + 504*p**2*q2*q3*q4**4 - 1008*p**2*q2*q3*q4**3 + 1260*p**2*q2*q3*q4**2 - 1008*p**2*q2*q3*q4 + 504*p**2*q2*q3 - 24*p**2*q3**7 - 24*p**2*q3**6*q4 + 168*p**2*q3**6 - 24*p**2*q3**5*q4**2 + 168*p**2*q3**5*q4 - 504*p**2*q3**5 - 24*p**2*q3**4*q4**3 + 168*p**2*q3**4*q4**2 - 504*p**2*q3**4*q4 + 840*p**2*q3**4 - 24*p**2*q3**3*q4**4 + 168*p**2*q3**3*q4**3 - 504*p**2*q3**3*q4**2 + 840*p**2*q3**3*q4 - 840*p**2*q3**3 - 24*p**2*q3**2*q4**5 + 168*p**2*q3**2*q4**4 - 504*p**2*q3**2*q4**3 + 840*p**2*q3**2*q4**2 - 840*p**2*q3**2*q4 + 504*p**2*q3**2 - 24*p**2*q3*q4**6 + 168*p**2*q3*q4**5 - 504*p**2*q3*q4**4 + 840*p**2*q3*q4**3 - 840*p**2*q3*q4**2 + 504*p**2*q3*q4 - 168*p**2*q3 + 6*p*q1**7*q2*q3 + 6*p*q1**6*q2**2*q3 + 6*p*q1**6*q2*q3**2 + 6*p*q1**6*q2*q3*q4 - 54*p*q1**6*q2*q3 + 6*p*q1**5*q2**3*q3 + 6*p*q1**5*q2**2*q3**2 + 6*p*q1**5*q2**2*q3*q4 - 54*p*q1**5*q2**2*q3 + 6*p*q1**5*q2*q3**3 + 6*p*q1**5*q2*q3**2*q4 - 54*p*q1**5*q2*q3**2 + 6*p*q1**5*q2*q3*q4**2 - 54*p*q1**5*q2*q3*q4 + 216*p*q1**5*q2*q3 + 6*p*q1**4*q2**4*q3 + 6*p*q1**4*q2**3*q3**2 + 6*p*q1**4*q2**3*q3*q4 - 54*p*q1**4*q2**3*q3 + 6*p*q1**4*q2**2*q3**3 + 6*p*q1**4*q2**2*q3**2*q4 - 54*p*q1**4*q2**2*q3**2 + 6*p*q1**4*q2**2*q3*q4**2 - 54*p*q1**4*q2**2*q3*q4 + 216*p*q1**4*q2**2*q3 + 6*p*q1**4*q2*q3**4 + 6*p*q1**4*q2*q3**3*q4 - 54*p*q1**4*q2*q3**3 + 6*p*q1**4*q2*q3**2*q4**2 - 54*p*q1**4*q2*q3**2*q4 + 216*p*q1**4*q2*q3**2 + 6*p*q1**4*q2*q3*q4**3 - 54*p*q1**4*q2*q3*q4**2 + 216*p*q1**4*q2*q3*q4 - 504*p*q1**4*q2*q3 + 6*p*q1**3*q2**5*q3 + 6*p*q1**3*q2**4*q3**2 + 6*p*q1**3*q2**4*q3*q4 - 54*p*q1**3*q2**4*q3 + 6*p*q1**3*q2**3*q3**3 + 6*p*q1**3*q2**3*q3**2*q4 - 54*p*q1**3*q2**3*q3**2 + 6*p*q1**3*q2**3*q3*q4**2 - 54*p*q1**3*q2**3*q3*q4 + 216*p*q1**3*q2**3*q3 + 6*p*q1**3*q2**2*q3**4 + 6*p*q1**3*q2**2*q3**3*q4 - 54*p*q1**3*q2**2*q3**3 + 6*p*q1**3*q2**2*q3**2*q4**2 - 54*p*q1**3*q2**2*q3**2*q4 + 216*p*q1**3*q2**2*q3**2 + 6*p*q1**3*q2**2*q3*q4**3 - 54*p*q1**3*q2**2*q3*q4**2 + 216*p*q1**3*q2**2*q3*q4 - 504*p*q1**3*q2**2*q3 + 6*p*q1**3*q2*q3**5 + 6*p*q1**3*q2*q3**4*q4 - 54*p*q1**3*q2*q3**4 + 6*p*q1**3*q2*q3**3*q4**2 - 54*p*q1**3*q2*q3**3*q4 + 216*p*q1**3*q2*q3**3 + 6*p*q1**3*q2*q3**2*q4**3 - 54*p*q1**3*q2*q3**2*q4**2 + 216*p*q1**3*q2*q3**2*q4 - 504*p*q1**3*q2*q3**2 + 6*p*q1**3*q2*q3*q4**4 - 54*p*q1**3*q2*q3*q4**3 + 216*p*q1**3*q2*q3*q4**2 - 504*p*q1**3*q2*q3*q4 + 756*p*q1**3*q2*q3 + 6*p*q1**2*q2**6*q3 + 6*p*q1**2*q2**5*q3**2 + 6*p*q1**2*q2**5*q3*q4 - 54*p*q1**2*q2**5*q3 + 6*p*q1**2*q2**4*q3**3 + 6*p*q1**2*q2**4*q3**2*q4 - 54*p*q1**2*q2**4*q3**2 + 6*p*q1**2*q2**4*q3*q4**2 - 54*p*q1**2*q2**4*q3*q4 + 216*p*q1**2*q2**4*q3 + 6*p*q1**2*q2**3*q3**4 + 6*p*q1**2*q2**3*q3**3*q4 - 54*p*q1**2*q2**3*q3**3 + 6*p*q1**2*q2**3*q3**2*q4**2 - 54*p*q1**2*q2**3*q3**2*q4 + 216*p*q1**2*q2**3*q3**2 + 6*p*q1**2*q2**3*q3*q4**3 - 54*p*q1**2*q2**3*q3*q4**2 + 216*p*q1**2*q2**3*q3*q4 - 504*p*q1**2*q2**3*q3 + 6*p*q1**2*q2**2*q3**5 + 6*p*q1**2*q2**2*q3**4*q4 - 54*p*q1**2*q2**2*q3**4 + 6*p*q1**2*q2**2*q3**3*q4**2 - 54*p*q1**2*q2**2*q3**3*q4 + 216*p*q1**2*q2**2*q3**3 + 6*p*q1**2*q2**2*q3**2*q4**3 - 54*p*q1**2*q2**2*q3**2*q4**2 + 216*p*q1**2*q2**2*q3**2*q4 - 504*p*q1**2*q2**2*q3**2 + 6*p*q1**2*q2**2*q3*q4**4 - 54*p*q1**2*q2**2*q3*q4**3 + 216*p*q1**2*q2**2*q3*q4**2 - 504*p*q1**2*q2**2*q3*q4 + 756*p*q1**2*q2**2*q3 + 6*p*q1**2*q2*q3**6 + 6*p*q1**2*q2*q3**5*q4 - 54*p*q1**2*q2*q3**5 + 6*p*q1**2*q2*q3**4*q4**2 - 54*p*q1**2*q2*q3**4*q4 + 216*p*q1**2*q2*q3**4 + 6*p*q1**2*q2*q3**3*q4**3 - 54*p*q1**2*q2*q3**3*q4**2 + 216*p*q1**2*q2*q3**3*q4 - 504*p*q1**2*q2*q3**3 + 6*p*q1**2*q2*q3**2*q4**4 - 54*p*q1**2*q2*q3**2*q4**3 + 216*p*q1**2*q2*q3**2*q4**2 - 504*p*q1**2*q2*q3**2*q4 + 756*p*q1**2*q2*q3**2 + 6*p*q1**2*q2*q3*q4**5 - 54*p*q1**2*q2*q3*q4**4 + 216*p*q1**2*q2*q3*q4**3 - 504*p*q1**2*q2*q3*q4**2 + 756*p*q1**2*q2*q3*q4 - 756*p*q1**2*q2*q3 + 6*p*q1*q2**7*q3 + 6*p*q1*q2**6*q3**2 + 6*p*q1*q2**6*q3*q4 - 54*p*q1*q2**6*q3 + 6*p*q1*q2**5*q3**3 + 6*p*q1*q2**5*q3**2*q4 - 54*p*q1*q2**5*q3**2 + 6*p*q1*q2**5*q3*q4**2 - 54*p*q1*q2**5*q3*q4 + 216*p*q1*q2**5*q3 + 6*p*q1*q2**4*q3**4 + 6*p*q1*q2**4*q3**3*q4 - 54*p*q1*q2**4*q3**3 + 6*p*q1*q2**4*q3**2*q4**2 - 54*p*q1*q2**4*q3**2*q4 + 216*p*q1*q2**4*q3**2 + 6*p*q1*q2**4*q3*q4**3 - 54*p*q1*q2**4*q3*q4**2 + 216*p*q1*q2**4*q3*q4 - 504*p*q1*q2**4*q3 + 6*p*q1*q2**3*q3**5 + 6*p*q1*q2**3*q3**4*q4 - 54*p*q1*q2**3*q3**4 + 6*p*q1*q2**3*q3**3*q4**2 - 54*p*q1*q2**3*q3**3*q4 + 216*p*q1*q2**3*q3**3 + 6*p*q1*q2**3*q3**2*q4**3 - 54*p*q1*q2**3*q3**2*q4**2 + 216*p*q1*q2**3*q3**2*q4 - 504*p*q1*q2**3*q3**2 + 6*p*q1*q2**3*q3*q4**4 - 54*p*q1*q2**3*q3*q4**3 + 216*p*q1*q2**3*q3*q4**2 - 504*p*q1*q2**3*q3*q4 + 756*p*q1*q2**3*q3 + 6*p*q1*q2**2*q3**6 + 6*p*q1*q2**2*q3**5*q4 - 54*p*q1*q2**2*q3**5 + 6*p*q1*q2**2*q3**4*q4**2 - 54*p*q1*q2**2*q3**4*q4 + 216*p*q1*q2**2*q3**4 + 6*p*q1*q2**2*q3**3*q4**3 - 54*p*q1*q2**2*q3**3*q4**2 + 216*p*q1*q2**2*q3**3*q4 - 504*p*q1*q2**2*q3**3 + 6*p*q1*q2**2*q3**2*q4**4 - 54*p*q1*q2**2*q3**2*q4**3 + 216*p*q1*q2**2*q3**2*q4**2 - 504*p*q1*q2**2*q3**2*q4 + 756*p*q1*q2**2*q3**2 + 6*p*q1*q2**2*q3*q4**5 - 54*p*q1*q2**2*q3*q4**4 + 216*p*q1*q2**2*q3*q4**3 - 504*p*q1*q2**2*q3*q4**2 + 756*p*q1*q2**2*q3*q4 - 756*p*q1*q2**2*q3 + 6*p*q1*q2*q3**7 + 6*p*q1*q2*q3**6*q4 - 54*p*q1*q2*q3**6 + 6*p*q1*q2*q3**5*q4**2 - 54*p*q1*q2*q3**5*q4 + 216*p*q1*q2*q3**5 + 6*p*q1*q2*q3**4*q4**3 - 54*p*q1*q2*q3**4*q4**2 + 216*p*q1*q2*q3**4*q4 - 504*p*q1*q2*q3**4 + 6*p*q1*q2*q3**3*q4**4 - 54*p*q1*q2*q3**3*q4**3 + 216*p*q1*q2*q3**3*q4**2 - 504*p*q1*q2*q3**3*q4 + 756*p*q1*q2*q3**3 + 6*p*q1*q2*q3**2*q4**5 - 54*p*q1*q2*q3**2*q4**4 + 216*p*q1*q2*q3**2*q4**3 - 504*p*q1*q2*q3**2*q4**2 + 756*p*q1*q2*q3**2*q4 - 756*p*q1*q2*q3**2 + 6*p*q1*q2*q3*q4**6 - 54*p*q1*q2*q3*q4**5 + 216*p*q1*q2*q3*q4**4 - 504*p*q1*q2*q3*q4**3 + 756*p*q1*q2*q3*q4**2 - 756*p*q1*q2*q3*q4 + 504*p*q1*q2*q3 - 9*p*q2**7*q3 - 9*p*q2**6*q3**2 - 9*p*q2**6*q3*q4 + 72*p*q2**6*q3 - 9*p*q2**5*q3**3 - 9*p*q2**5*q3**2*q4 + 72*p*q2**5*q3**2 - 9*p*q2**5*q3*q4**2 + 72*p*q2**5*q3*q4 - 252*p*q2**5*q3 - 9*p*q2**4*q3**4 - 9*p*q2**4*q3**3*q4 + 72*p*q2**4*q3**3 - 9*p*q2**4*q3**2*q4**2 + 72*p*q2**4*q3**2*q4 - 252*p*q2**4*q3**2 - 9*p*q2**4*q3*q4**3 + 72*p*q2**4*q3*q4**2 - 252*p*q2**4*q3*q4 + 504*p*q2**4*q3 - 9*p*q2**3*q3**5 - 9*p*q2**3*q3**4*q4 + 72*p*q2**3*q3**4 - 9*p*q2**3*q3**3*q4**2 + 72*p*q2**3*q3**3*q4 - 252*p*q2**3*q3**3 - 9*p*q2**3*q3**2*q4**3 + 72*p*q2**3*q3**2*q4**2 - 252*p*q2**3*q3**2*q4 + 504*p*q2**3*q3**2 - 9*p*q2**3*q3*q4**4 + 72*p*q2**3*q3*q4**3 - 252*p*q2**3*q3*q4**2 + 504*p*q2**3*q3*q4 - 630*p*q2**3*q3 - 9*p*q2**2*q3**6 - 9*p*q2**2*q3**5*q4 + 72*p*q2**2*q3**5 - 9*p*q2**2*q3**4*q4**2 + 72*p*q2**2*q3**4*q4 - 252*p*q2**2*q3**4 - 9*p*q2**2*q3**3*q4**3 + 72*p*q2**2*q3**3*q4**2 - 252*p*q2**2*q3**3*q4 + 504*p*q2**2*q3**3 - 9*p*q2**2*q3**2*q4**4 + 72*p*q2**2*q3**2*q4**3 - 252*p*q2**2*q3**2*q4**2 + 504*p*q2**2*q3**2*q4 - 630*p*q2**2*q3**2 - 9*p*q2**2*q3*q4**5 + 72*p*q2**2*q3*q4**4 - 252*p*q2**2*q3*q4**3 + 504*p*q2**2*q3*q4**2 - 630*p*q2**2*q3*q4 + 504*p*q2**2*q3 - 9*p*q2*q3**7 - 9*p*q2*q3**6*q4 + 72*p*q2*q3**6 - 9*p*q2*q3**5*q4**2 + 72*p*q2*q3**5*q4 - 252*p*q2*q3**5 - 9*p*q2*q3**4*q4**3 + 72*p*q2*q3**4*q4**2 - 252*p*q2*q3**4*q4 + 504*p*q2*q3**4 - 9*p*q2*q3**3*q4**4 + 72*p*q2*q3**3*q4**3 - 252*p*q2*q3**3*q4**2 + 504*p*q2*q3**3*q4 - 630*p*q2*q3**3 - 9*p*q2*q3**2*q4**5 + 72*p*q2*q3**2*q4**4 - 252*p*q2*q3**2*q4**3 + 504*p*q2*q3**2*q4**2 - 630*p*q2*q3**2*q4 + 504*p*q2*q3**2 - 9*p*q2*q3*q4**6 + 72*p*q2*q3*q4**5 - 252*p*q2*q3*q4**4 + 504*p*q2*q3*q4**3 - 630*p*q2*q3*q4**2 + 504*p*q2*q3*q4 - 252*p*q2*q3 - 2*q1**7*q2*q3 - 2*q1**6*q2**2*q3 - 2*q1**6*q2*q3**2 - 2*q1**6*q2*q3*q4 + 18*q1**6*q2*q3 - 2*q1**5*q2**3*q3 - 2*q1**5*q2**2*q3**2 - 2*q1**5*q2**2*q3*q4 + 18*q1**5*q2**2*q3 - 2*q1**5*q2*q3**3 - 2*q1**5*q2*q3**2*q4 + 18*q1**5*q2*q3**2 - 2*q1**5*q2*q3*q4**2 + 18*q1**5*q2*q3*q4 - 72*q1**5*q2*q3 - 2*q1**4*q2**4*q3 - 2*q1**4*q2**3*q3**2 - 2*q1**4*q2**3*q3*q4 + 18*q1**4*q2**3*q3 - 2*q1**4*q2**2*q3**3 - 2*q1**4*q2**2*q3**2*q4 + 18*q1**4*q2**2*q3**2 - 2*q1**4*q2**2*q3*q4**2 + 18*q1**4*q2**2*q3*q4 - 72*q1**4*q2**2*q3 - 2*q1**4*q2*q3**4 - 2*q1**4*q2*q3**3*q4 + 18*q1**4*q2*q3**3 - 2*q1**4*q2*q3**2*q4**2 + 18*q1**4*q2*q3**2*q4 - 72*q1**4*q2*q3**2 - 2*q1**4*q2*q3*q4**3 + 18*q1**4*q2*q3*q4**2 - 72*q1**4*q2*q3*q4 + 168*q1**4*q2*q3 - 2*q1**3*q2**5*q3 - 2*q1**3*q2**4*q3**2 - 2*q1**3*q2**4*q3*q4 + 18*q1**3*q2**4*q3 - 2*q1**3*q2**3*q3**3 - 2*q1**3*q2**3*q3**2*q4 + 18*q1**3*q2**3*q3**2 - 2*q1**3*q2**3*q3*q4**2 + 18*q1**3*q2**3*q3*q4 - 72*q1**3*q2**3*q3 - 2*q1**3*q2**2*q3**4 - 2*q1**3*q2**2*q3**3*q4 + 18*q1**3*q2**2*q3**3 - 2*q1**3*q2**2*q3**2*q4**2 + 18*q1**3*q2**2*q3**2*q4 - 72*q1**3*q2**2*q3**2 - 2*q1**3*q2**2*q3*q4**3 + 18*q1**3*q2**2*q3*q4**2 - 72*q1**3*q2**2*q3*q4 + 168*q1**3*q2**2*q3 - 2*q1**3*q2*q3**5 - 2*q1**3*q2*q3**4*q4 + 18*q1**3*q2*q3**4 - 2*q1**3*q2*q3**3*q4**2 + 18*q1**3*q2*q3**3*q4 - 72*q1**3*q2*q3**3 - 2*q1**3*q2*q3**2*q4**3 + 18*q1**3*q2*q3**2*q4**2 - 72*q1**3*q2*q3**2*q4 + 168*q1**3*q2*q3**2 - 2*q1**3*q2*q3*q4**4 + 18*q1**3*q2*q3*q4**3 - 72*q1**3*q2*q3*q4**2 + 168*q1**3*q2*q3*q4 - 252*q1**3*q2*q3 - 2*q1**2*q2**6*q3 - 2*q1**2*q2**5*q3**2 - 2*q1**2*q2**5*q3*q4 + 18*q1**2*q2**5*q3 - 2*q1**2*q2**4*q3**3 - 2*q1**2*q2**4*q3**2*q4 + 18*q1**2*q2**4*q3**2 - 2*q1**2*q2**4*q3*q4**2 + 18*q1**2*q2**4*q3*q4 - 72*q1**2*q2**4*q3 - 2*q1**2*q2**3*q3**4 - 2*q1**2*q2**3*q3**3*q4 + 18*q1**2*q2**3*q3**3 - 2*q1**2*q2**3*q3**2*q4**2 + 18*q1**2*q2**3*q3**2*q4 - 72*q1**2*q2**3*q3**2 - 2*q1**2*q2**3*q3*q4**3 + 18*q1**2*q2**3*q3*q4**2 - 72*q1**2*q2**3*q3*q4 + 168*q1**2*q2**3*q3 - 2*q1**2*q2**2*q3**5 - 2*q1**2*q2**2*q3**4*q4 + 18*q1**2*q2**2*q3**4 - 2*q1**2*q2**2*q3**3*q4**2 + 18*q1**2*q2**2*q3**3*q4 - 72*q1**2*q2**2*q3**3 - 2*q1**2*q2**2*q3**2*q4**3 + 18*q1**2*q2**2*q3**2*q4**2 - 72*q1**2*q2**2*q3**2*q4 + 168*q1**2*q2**2*q3**2 - 2*q1**2*q2**2*q3*q4**4 + 18*q1**2*q2**2*q3*q4**3 - 72*q1**2*q2**2*q3*q4**2 + 168*q1**2*q2**2*q3*q4 - 252*q1**2*q2**2*q3 - 2*q1**2*q2*q3**6 - 2*q1**2*q2*q3**5*q4 + 18*q1**2*q2*q3**5 - 2*q1**2*q2*q3**4*q4**2 + 18*q1**2*q2*q3**4*q4 - 72*q1**2*q2*q3**4 - 2*q1**2*q2*q3**3*q4**3 + 18*q1**2*q2*q3**3*q4**2 - 72*q1**2*q2*q3**3*q4 + 168*q1**2*q2*q3**3 - 2*q1**2*q2*q3**2*q4**4 + 18*q1**2*q2*q3**2*q4**3 - 72*q1**2*q2*q3**2*q4**2 + 168*q1**2*q2*q3**2*q4 - 252*q1**2*q2*q3**2 - 2*q1**2*q2*q3*q4**5 + 18*q1**2*q2*q3*q4**4 - 72*q1**2*q2*q3*q4**3 + 168*q1**2*q2*q3*q4**2 - 252*q1**2*q2*q3*q4 + 252*q1**2*q2*q3 - 2*q1*q2**7*q3 - 2*q1*q2**6*q3**2 - 2*q1*q2**6*q3*q4 + 18*q1*q2**6*q3 - 2*q1*q2**5*q3**3 - 2*q1*q2**5*q3**2*q4 + 18*q1*q2**5*q3**2 - 2*q1*q2**5*q3*q4**2 + 18*q1*q2**5*q3*q4 - 72*q1*q2**5*q3 - 2*q1*q2**4*q3**4 - 2*q1*q2**4*q3**3*q4 + 18*q1*q2**4*q3**3 - 2*q1*q2**4*q3**2*q4**2 + 18*q1*q2**4*q3**2*q4 - 72*q1*q2**4*q3**2 - 2*q1*q2**4*q3*q4**3 + 18*q1*q2**4*q3*q4**2 - 72*q1*q2**4*q3*q4 + 168*q1*q2**4*q3 - 2*q1*q2**3*q3**5 - 2*q1*q2**3*q3**4*q4 + 18*q1*q2**3*q3**4 - 2*q1*q2**3*q3**3*q4**2 + 18*q1*q2**3*q3**3*q4 - 72*q1*q2**3*q3**3 - 2*q1*q2**3*q3**2*q4**3 + 18*q1*q2**3*q3**2*q4**2 - 72*q1*q2**3*q3**2*q4 + 168*q1*q2**3*q3**2 - 2*q1*q2**3*q3*q4**4 + 18*q1*q2**3*q3*q4**3 - 72*q1*q2**3*q3*q4**2 + 168*q1*q2**3*q3*q4 - 252*q1*q2**3*q3 - 2*q1*q2**2*q3**6 - 2*q1*q2**2*q3**5*q4 + 18*q1*q2**2*q3**5 - 2*q1*q2**2*q3**4*q4**2 + 18*q1*q2**2*q3**4*q4 - 72*q1*q2**2*q3**4 - 2*q1*q2**2*q3**3*q4**3 + 18*q1*q2**2*q3**3*q4**2 - 72*q1*q2**2*q3**3*q4 + 168*q1*q2**2*q3**3 - 2*q1*q2**2*q3**2*q4**4 + 18*q1*q2**2*q3**2*q4**3 - 72*q1*q2**2*q3**2*q4**2 + 168*q1*q2**2*q3**2*q4 - 252*q1*q2**2*q3**2 - 2*q1*q2**2*q3*q4**5 + 18*q1*q2**2*q3*q4**4 - 72*q1*q2**2*q3*q4**3 + 168*q1*q2**2*q3*q4**2 - 252*q1*q2**2*q3*q4 + 252*q1*q2**2*q3 - 2*q1*q2*q3**7 - 2*q1*q2*q3**6*q4 + 18*q1*q2*q3**6 - 2*q1*q2*q3**5*q4**2 + 18*q1*q2*q3**5*q4 - 72*q1*q2*q3**5 - 2*q1*q2*q3**4*q4**3 + 18*q1*q2*q3**4*q4**2 - 72*q1*q2*q3**4*q4 + 168*q1*q2*q3**4 - 2*q1*q2*q3**3*q4**4 + 18*q1*q2*q3**3*q4**3 - 72*q1*q2*q3**3*q4**2 + 168*q1*q2*q3**3*q4 - 252*q1*q2*q3**3 - 2*q1*q2*q3**2*q4**5 + 18*q1*q2*q3**2*q4**4 - 72*q1*q2*q3**2*q4**3 + 168*q1*q2*q3**2*q4**2 - 252*q1*q2*q3**2*q4 + 252*q1*q2*q3**2 - 2*q1*q2*q3*q4**6 + 18*q1*q2*q3*q4**5 - 72*q1*q2*q3*q4**4 + 168*q1*q2*q3*q4**3 - 252*q1*q2*q3*q4**2 + 252*q1*q2*q3*q4 - 168*q1*q2*q3)'
f_multiparam_semisyn[10][5] = 'p*(p - 1)**5*(10*p**4*q1**6*q2*q3*q4 + 10*p**4*q1**5*q2**2*q3*q4 + 10*p**4*q1**5*q2*q3**2*q4 + 10*p**4*q1**5*q2*q3*q4**2 + 10*p**4*q1**5*q2*q3*q4*q5 - 90*p**4*q1**5*q2*q3*q4 + 10*p**4*q1**4*q2**3*q3*q4 + 10*p**4*q1**4*q2**2*q3**2*q4 + 10*p**4*q1**4*q2**2*q3*q4**2 + 10*p**4*q1**4*q2**2*q3*q4*q5 - 90*p**4*q1**4*q2**2*q3*q4 + 10*p**4*q1**4*q2*q3**3*q4 + 10*p**4*q1**4*q2*q3**2*q4**2 + 10*p**4*q1**4*q2*q3**2*q4*q5 - 90*p**4*q1**4*q2*q3**2*q4 + 10*p**4*q1**4*q2*q3*q4**3 + 10*p**4*q1**4*q2*q3*q4**2*q5 - 90*p**4*q1**4*q2*q3*q4**2 + 10*p**4*q1**4*q2*q3*q4*q5**2 - 90*p**4*q1**4*q2*q3*q4*q5 + 360*p**4*q1**4*q2*q3*q4 + 10*p**4*q1**3*q2**4*q3*q4 + 10*p**4*q1**3*q2**3*q3**2*q4 + 10*p**4*q1**3*q2**3*q3*q4**2 + 10*p**4*q1**3*q2**3*q3*q4*q5 - 90*p**4*q1**3*q2**3*q3*q4 + 10*p**4*q1**3*q2**2*q3**3*q4 + 10*p**4*q1**3*q2**2*q3**2*q4**2 + 10*p**4*q1**3*q2**2*q3**2*q4*q5 - 90*p**4*q1**3*q2**2*q3**2*q4 + 10*p**4*q1**3*q2**2*q3*q4**3 + 10*p**4*q1**3*q2**2*q3*q4**2*q5 - 90*p**4*q1**3*q2**2*q3*q4**2 + 10*p**4*q1**3*q2**2*q3*q4*q5**2 - 90*p**4*q1**3*q2**2*q3*q4*q5 + 360*p**4*q1**3*q2**2*q3*q4 + 10*p**4*q1**3*q2*q3**4*q4 + 10*p**4*q1**3*q2*q3**3*q4**2 + 10*p**4*q1**3*q2*q3**3*q4*q5 - 90*p**4*q1**3*q2*q3**3*q4 + 10*p**4*q1**3*q2*q3**2*q4**3 + 10*p**4*q1**3*q2*q3**2*q4**2*q5 - 90*p**4*q1**3*q2*q3**2*q4**2 + 10*p**4*q1**3*q2*q3**2*q4*q5**2 - 90*p**4*q1**3*q2*q3**2*q4*q5 + 360*p**4*q1**3*q2*q3**2*q4 + 10*p**4*q1**3*q2*q3*q4**4 + 10*p**4*q1**3*q2*q3*q4**3*q5 - 90*p**4*q1**3*q2*q3*q4**3 + 10*p**4*q1**3*q2*q3*q4**2*q5**2 - 90*p**4*q1**3*q2*q3*q4**2*q5 + 360*p**4*q1**3*q2*q3*q4**2 + 10*p**4*q1**3*q2*q3*q4*q5**3 - 90*p**4*q1**3*q2*q3*q4*q5**2 + 360*p**4*q1**3*q2*q3*q4*q5 - 840*p**4*q1**3*q2*q3*q4 + 10*p**4*q1**2*q2**5*q3*q4 + 10*p**4*q1**2*q2**4*q3**2*q4 + 10*p**4*q1**2*q2**4*q3*q4**2 + 10*p**4*q1**2*q2**4*q3*q4*q5 - 90*p**4*q1**2*q2**4*q3*q4 + 10*p**4*q1**2*q2**3*q3**3*q4 + 10*p**4*q1**2*q2**3*q3**2*q4**2 + 10*p**4*q1**2*q2**3*q3**2*q4*q5 - 90*p**4*q1**2*q2**3*q3**2*q4 + 10*p**4*q1**2*q2**3*q3*q4**3 + 10*p**4*q1**2*q2**3*q3*q4**2*q5 - 90*p**4*q1**2*q2**3*q3*q4**2 + 10*p**4*q1**2*q2**3*q3*q4*q5**2 - 90*p**4*q1**2*q2**3*q3*q4*q5 + 360*p**4*q1**2*q2**3*q3*q4 + 10*p**4*q1**2*q2**2*q3**4*q4 + 10*p**4*q1**2*q2**2*q3**3*q4**2 + 10*p**4*q1**2*q2**2*q3**3*q4*q5 - 90*p**4*q1**2*q2**2*q3**3*q4 + 10*p**4*q1**2*q2**2*q3**2*q4**3 + 10*p**4*q1**2*q2**2*q3**2*q4**2*q5 - 90*p**4*q1**2*q2**2*q3**2*q4**2 + 10*p**4*q1**2*q2**2*q3**2*q4*q5**2 - 90*p**4*q1**2*q2**2*q3**2*q4*q5 + 360*p**4*q1**2*q2**2*q3**2*q4 + 10*p**4*q1**2*q2**2*q3*q4**4 + 10*p**4*q1**2*q2**2*q3*q4**3*q5 - 90*p**4*q1**2*q2**2*q3*q4**3 + 10*p**4*q1**2*q2**2*q3*q4**2*q5**2 - 90*p**4*q1**2*q2**2*q3*q4**2*q5 + 360*p**4*q1**2*q2**2*q3*q4**2 + 10*p**4*q1**2*q2**2*q3*q4*q5**3 - 90*p**4*q1**2*q2**2*q3*q4*q5**2 + 360*p**4*q1**2*q2**2*q3*q4*q5 - 840*p**4*q1**2*q2**2*q3*q4 + 10*p**4*q1**2*q2*q3**5*q4 + 10*p**4*q1**2*q2*q3**4*q4**2 + 10*p**4*q1**2*q2*q3**4*q4*q5 - 90*p**4*q1**2*q2*q3**4*q4 + 10*p**4*q1**2*q2*q3**3*q4**3 + 10*p**4*q1**2*q2*q3**3*q4**2*q5 - 90*p**4*q1**2*q2*q3**3*q4**2 + 10*p**4*q1**2*q2*q3**3*q4*q5**2 - 90*p**4*q1**2*q2*q3**3*q4*q5 + 360*p**4*q1**2*q2*q3**3*q4 + 10*p**4*q1**2*q2*q3**2*q4**4 + 10*p**4*q1**2*q2*q3**2*q4**3*q5 - 90*p**4*q1**2*q2*q3**2*q4**3 + 10*p**4*q1**2*q2*q3**2*q4**2*q5**2 - 90*p**4*q1**2*q2*q3**2*q4**2*q5 + 360*p**4*q1**2*q2*q3**2*q4**2 + 10*p**4*q1**2*q2*q3**2*q4*q5**3 - 90*p**4*q1**2*q2*q3**2*q4*q5**2 + 360*p**4*q1**2*q2*q3**2*q4*q5 - 840*p**4*q1**2*q2*q3**2*q4 + 10*p**4*q1**2*q2*q3*q4**5 + 10*p**4*q1**2*q2*q3*q4**4*q5 - 90*p**4*q1**2*q2*q3*q4**4 + 10*p**4*q1**2*q2*q3*q4**3*q5**2 - 90*p**4*q1**2*q2*q3*q4**3*q5 + 360*p**4*q1**2*q2*q3*q4**3 + 10*p**4*q1**2*q2*q3*q4**2*q5**3 - 90*p**4*q1**2*q2*q3*q4**2*q5**2 + 360*p**4*q1**2*q2*q3*q4**2*q5 - 840*p**4*q1**2*q2*q3*q4**2 + 10*p**4*q1**2*q2*q3*q4*q5**4 - 90*p**4*q1**2*q2*q3*q4*q5**3 + 360*p**4*q1**2*q2*q3*q4*q5**2 - 840*p**4*q1**2*q2*q3*q4*q5 + 1260*p**4*q1**2*q2*q3*q4 + 10*p**4*q1*q2**6*q3*q4 + 10*p**4*q1*q2**5*q3**2*q4 + 10*p**4*q1*q2**5*q3*q4**2 + 10*p**4*q1*q2**5*q3*q4*q5 - 90*p**4*q1*q2**5*q3*q4 + 10*p**4*q1*q2**4*q3**3*q4 + 10*p**4*q1*q2**4*q3**2*q4**2 + 10*p**4*q1*q2**4*q3**2*q4*q5 - 90*p**4*q1*q2**4*q3**2*q4 + 10*p**4*q1*q2**4*q3*q4**3 + 10*p**4*q1*q2**4*q3*q4**2*q5 - 90*p**4*q1*q2**4*q3*q4**2 + 10*p**4*q1*q2**4*q3*q4*q5**2 - 90*p**4*q1*q2**4*q3*q4*q5 + 360*p**4*q1*q2**4*q3*q4 + 10*p**4*q1*q2**3*q3**4*q4 + 10*p**4*q1*q2**3*q3**3*q4**2 + 10*p**4*q1*q2**3*q3**3*q4*q5 - 90*p**4*q1*q2**3*q3**3*q4 + 10*p**4*q1*q2**3*q3**2*q4**3 + 10*p**4*q1*q2**3*q3**2*q4**2*q5 - 90*p**4*q1*q2**3*q3**2*q4**2 + 10*p**4*q1*q2**3*q3**2*q4*q5**2 - 90*p**4*q1*q2**3*q3**2*q4*q5 + 360*p**4*q1*q2**3*q3**2*q4 + 10*p**4*q1*q2**3*q3*q4**4 + 10*p**4*q1*q2**3*q3*q4**3*q5 - 90*p**4*q1*q2**3*q3*q4**3 + 10*p**4*q1*q2**3*q3*q4**2*q5**2 - 90*p**4*q1*q2**3*q3*q4**2*q5 + 360*p**4*q1*q2**3*q3*q4**2 + 10*p**4*q1*q2**3*q3*q4*q5**3 - 90*p**4*q1*q2**3*q3*q4*q5**2 + 360*p**4*q1*q2**3*q3*q4*q5 - 840*p**4*q1*q2**3*q3*q4 + 10*p**4*q1*q2**2*q3**5*q4 + 10*p**4*q1*q2**2*q3**4*q4**2 + 10*p**4*q1*q2**2*q3**4*q4*q5 - 90*p**4*q1*q2**2*q3**4*q4 + 10*p**4*q1*q2**2*q3**3*q4**3 + 10*p**4*q1*q2**2*q3**3*q4**2*q5 - 90*p**4*q1*q2**2*q3**3*q4**2 + 10*p**4*q1*q2**2*q3**3*q4*q5**2 - 90*p**4*q1*q2**2*q3**3*q4*q5 + 360*p**4*q1*q2**2*q3**3*q4 + 10*p**4*q1*q2**2*q3**2*q4**4 + 10*p**4*q1*q2**2*q3**2*q4**3*q5 - 90*p**4*q1*q2**2*q3**2*q4**3 + 10*p**4*q1*q2**2*q3**2*q4**2*q5**2 - 90*p**4*q1*q2**2*q3**2*q4**2*q5 + 360*p**4*q1*q2**2*q3**2*q4**2 + 10*p**4*q1*q2**2*q3**2*q4*q5**3 - 90*p**4*q1*q2**2*q3**2*q4*q5**2 + 360*p**4*q1*q2**2*q3**2*q4*q5 - 840*p**4*q1*q2**2*q3**2*q4 + 10*p**4*q1*q2**2*q3*q4**5 + 10*p**4*q1*q2**2*q3*q4**4*q5 - 90*p**4*q1*q2**2*q3*q4**4 + 10*p**4*q1*q2**2*q3*q4**3*q5**2 - 90*p**4*q1*q2**2*q3*q4**3*q5 + 360*p**4*q1*q2**2*q3*q4**3 + 10*p**4*q1*q2**2*q3*q4**2*q5**3 - 90*p**4*q1*q2**2*q3*q4**2*q5**2 + 360*p**4*q1*q2**2*q3*q4**2*q5 - 840*p**4*q1*q2**2*q3*q4**2 + 10*p**4*q1*q2**2*q3*q4*q5**4 - 90*p**4*q1*q2**2*q3*q4*q5**3 + 360*p**4*q1*q2**2*q3*q4*q5**2 - 840*p**4*q1*q2**2*q3*q4*q5 + 1260*p**4*q1*q2**2*q3*q4 + 10*p**4*q1*q2*q3**6*q4 + 10*p**4*q1*q2*q3**5*q4**2 + 10*p**4*q1*q2*q3**5*q4*q5 - 90*p**4*q1*q2*q3**5*q4 + 10*p**4*q1*q2*q3**4*q4**3 + 10*p**4*q1*q2*q3**4*q4**2*q5 - 90*p**4*q1*q2*q3**4*q4**2 + 10*p**4*q1*q2*q3**4*q4*q5**2 - 90*p**4*q1*q2*q3**4*q4*q5 + 360*p**4*q1*q2*q3**4*q4 + 10*p**4*q1*q2*q3**3*q4**4 + 10*p**4*q1*q2*q3**3*q4**3*q5 - 90*p**4*q1*q2*q3**3*q4**3 + 10*p**4*q1*q2*q3**3*q4**2*q5**2 - 90*p**4*q1*q2*q3**3*q4**2*q5 + 360*p**4*q1*q2*q3**3*q4**2 + 10*p**4*q1*q2*q3**3*q4*q5**3 - 90*p**4*q1*q2*q3**3*q4*q5**2 + 360*p**4*q1*q2*q3**3*q4*q5 - 840*p**4*q1*q2*q3**3*q4 + 10*p**4*q1*q2*q3**2*q4**5 + 10*p**4*q1*q2*q3**2*q4**4*q5 - 90*p**4*q1*q2*q3**2*q4**4 + 10*p**4*q1*q2*q3**2*q4**3*q5**2 - 90*p**4*q1*q2*q3**2*q4**3*q5 + 360*p**4*q1*q2*q3**2*q4**3 + 10*p**4*q1*q2*q3**2*q4**2*q5**3 - 90*p**4*q1*q2*q3**2*q4**2*q5**2 + 360*p**4*q1*q2*q3**2*q4**2*q5 - 840*p**4*q1*q2*q3**2*q4**2 + 10*p**4*q1*q2*q3**2*q4*q5**4 - 90*p**4*q1*q2*q3**2*q4*q5**3 + 360*p**4*q1*q2*q3**2*q4*q5**2 - 840*p**4*q1*q2*q3**2*q4*q5 + 1260*p**4*q1*q2*q3**2*q4 + 10*p**4*q1*q2*q3*q4**6 + 10*p**4*q1*q2*q3*q4**5*q5 - 90*p**4*q1*q2*q3*q4**5 + 10*p**4*q1*q2*q3*q4**4*q5**2 - 90*p**4*q1*q2*q3*q4**4*q5 + 360*p**4*q1*q2*q3*q4**4 + 10*p**4*q1*q2*q3*q4**3*q5**3 - 90*p**4*q1*q2*q3*q4**3*q5**2 + 360*p**4*q1*q2*q3*q4**3*q5 - 840*p**4*q1*q2*q3*q4**3 + 10*p**4*q1*q2*q3*q4**2*q5**4 - 90*p**4*q1*q2*q3*q4**2*q5**3 + 360*p**4*q1*q2*q3*q4**2*q5**2 - 840*p**4*q1*q2*q3*q4**2*q5 + 1260*p**4*q1*q2*q3*q4**2 + 10*p**4*q1*q2*q3*q4*q5**5 - 90*p**4*q1*q2*q3*q4*q5**4 + 360*p**4*q1*q2*q3*q4*q5**3 - 840*p**4*q1*q2*q3*q4*q5**2 + 1260*p**4*q1*q2*q3*q4*q5 - 1260*p**4*q1*q2*q3*q4 - 45*p**4*q2**6*q3*q4 - 45*p**4*q2**5*q3**2*q4 - 45*p**4*q2**5*q3*q4**2 - 45*p**4*q2**5*q3*q4*q5 + 360*p**4*q2**5*q3*q4 - 45*p**4*q2**4*q3**3*q4 - 45*p**4*q2**4*q3**2*q4**2 - 45*p**4*q2**4*q3**2*q4*q5 + 360*p**4*q2**4*q3**2*q4 - 45*p**4*q2**4*q3*q4**3 - 45*p**4*q2**4*q3*q4**2*q5 + 360*p**4*q2**4*q3*q4**2 - 45*p**4*q2**4*q3*q4*q5**2 + 360*p**4*q2**4*q3*q4*q5 - 1260*p**4*q2**4*q3*q4 - 45*p**4*q2**3*q3**4*q4 - 45*p**4*q2**3*q3**3*q4**2 - 45*p**4*q2**3*q3**3*q4*q5 + 360*p**4*q2**3*q3**3*q4 - 45*p**4*q2**3*q3**2*q4**3 - 45*p**4*q2**3*q3**2*q4**2*q5 + 360*p**4*q2**3*q3**2*q4**2 - 45*p**4*q2**3*q3**2*q4*q5**2 + 360*p**4*q2**3*q3**2*q4*q5 - 1260*p**4*q2**3*q3**2*q4 - 45*p**4*q2**3*q3*q4**4 - 45*p**4*q2**3*q3*q4**3*q5 + 360*p**4*q2**3*q3*q4**3 - 45*p**4*q2**3*q3*q4**2*q5**2 + 360*p**4*q2**3*q3*q4**2*q5 - 1260*p**4*q2**3*q3*q4**2 - 45*p**4*q2**3*q3*q4*q5**3 + 360*p**4*q2**3*q3*q4*q5**2 - 1260*p**4*q2**3*q3*q4*q5 + 2520*p**4*q2**3*q3*q4 - 45*p**4*q2**2*q3**5*q4 - 45*p**4*q2**2*q3**4*q4**2 - 45*p**4*q2**2*q3**4*q4*q5 + 360*p**4*q2**2*q3**4*q4 - 45*p**4*q2**2*q3**3*q4**3 - 45*p**4*q2**2*q3**3*q4**2*q5 + 360*p**4*q2**2*q3**3*q4**2 - 45*p**4*q2**2*q3**3*q4*q5**2 + 360*p**4*q2**2*q3**3*q4*q5 - 1260*p**4*q2**2*q3**3*q4 - 45*p**4*q2**2*q3**2*q4**4 - 45*p**4*q2**2*q3**2*q4**3*q5 + 360*p**4*q2**2*q3**2*q4**3 - 45*p**4*q2**2*q3**2*q4**2*q5**2 + 360*p**4*q2**2*q3**2*q4**2*q5 - 1260*p**4*q2**2*q3**2*q4**2 - 45*p**4*q2**2*q3**2*q4*q5**3 + 360*p**4*q2**2*q3**2*q4*q5**2 - 1260*p**4*q2**2*q3**2*q4*q5 + 2520*p**4*q2**2*q3**2*q4 - 45*p**4*q2**2*q3*q4**5 - 45*p**4*q2**2*q3*q4**4*q5 + 360*p**4*q2**2*q3*q4**4 - 45*p**4*q2**2*q3*q4**3*q5**2 + 360*p**4*q2**2*q3*q4**3*q5 - 1260*p**4*q2**2*q3*q4**3 - 45*p**4*q2**2*q3*q4**2*q5**3 + 360*p**4*q2**2*q3*q4**2*q5**2 - 1260*p**4*q2**2*q3*q4**2*q5 + 2520*p**4*q2**2*q3*q4**2 - 45*p**4*q2**2*q3*q4*q5**4 + 360*p**4*q2**2*q3*q4*q5**3 - 1260*p**4*q2**2*q3*q4*q5**2 + 2520*p**4*q2**2*q3*q4*q5 - 3150*p**4*q2**2*q3*q4 - 45*p**4*q2*q3**6*q4 - 45*p**4*q2*q3**5*q4**2 - 45*p**4*q2*q3**5*q4*q5 + 360*p**4*q2*q3**5*q4 - 45*p**4*q2*q3**4*q4**3 - 45*p**4*q2*q3**4*q4**2*q5 + 360*p**4*q2*q3**4*q4**2 - 45*p**4*q2*q3**4*q4*q5**2 + 360*p**4*q2*q3**4*q4*q5 - 1260*p**4*q2*q3**4*q4 - 45*p**4*q2*q3**3*q4**4 - 45*p**4*q2*q3**3*q4**3*q5 + 360*p**4*q2*q3**3*q4**3 - 45*p**4*q2*q3**3*q4**2*q5**2 + 360*p**4*q2*q3**3*q4**2*q5 - 1260*p**4*q2*q3**3*q4**2 - 45*p**4*q2*q3**3*q4*q5**3 + 360*p**4*q2*q3**3*q4*q5**2 - 1260*p**4*q2*q3**3*q4*q5 + 2520*p**4*q2*q3**3*q4 - 45*p**4*q2*q3**2*q4**5 - 45*p**4*q2*q3**2*q4**4*q5 + 360*p**4*q2*q3**2*q4**4 - 45*p**4*q2*q3**2*q4**3*q5**2 + 360*p**4*q2*q3**2*q4**3*q5 - 1260*p**4*q2*q3**2*q4**3 - 45*p**4*q2*q3**2*q4**2*q5**3 + 360*p**4*q2*q3**2*q4**2*q5**2 - 1260*p**4*q2*q3**2*q4**2*q5 + 2520*p**4*q2*q3**2*q4**2 - 45*p**4*q2*q3**2*q4*q5**4 + 360*p**4*q2*q3**2*q4*q5**3 - 1260*p**4*q2*q3**2*q4*q5**2 + 2520*p**4*q2*q3**2*q4*q5 - 3150*p**4*q2*q3**2*q4 - 45*p**4*q2*q3*q4**6 - 45*p**4*q2*q3*q4**5*q5 + 360*p**4*q2*q3*q4**5 - 45*p**4*q2*q3*q4**4*q5**2 + 360*p**4*q2*q3*q4**4*q5 - 1260*p**4*q2*q3*q4**4 - 45*p**4*q2*q3*q4**3*q5**3 + 360*p**4*q2*q3*q4**3*q5**2 - 1260*p**4*q2*q3*q4**3*q5 + 2520*p**4*q2*q3*q4**3 - 45*p**4*q2*q3*q4**2*q5**4 + 360*p**4*q2*q3*q4**2*q5**3 - 1260*p**4*q2*q3*q4**2*q5**2 + 2520*p**4*q2*q3*q4**2*q5 - 3150*p**4*q2*q3*q4**2 - 45*p**4*q2*q3*q4*q5**5 + 360*p**4*q2*q3*q4*q5**4 - 1260*p**4*q2*q3*q4*q5**3 + 2520*p**4*q2*q3*q4*q5**2 - 3150*p**4*q2*q3*q4*q5 + 2520*p**4*q2*q3*q4 + 120*p**4*q3**6*q4 + 120*p**4*q3**5*q4**2 + 120*p**4*q3**5*q4*q5 - 840*p**4*q3**5*q4 + 120*p**4*q3**4*q4**3 + 120*p**4*q3**4*q4**2*q5 - 840*p**4*q3**4*q4**2 + 120*p**4*q3**4*q4*q5**2 - 840*p**4*q3**4*q4*q5 + 2520*p**4*q3**4*q4 + 120*p**4*q3**3*q4**4 + 120*p**4*q3**3*q4**3*q5 - 840*p**4*q3**3*q4**3 + 120*p**4*q3**3*q4**2*q5**2 - 840*p**4*q3**3*q4**2*q5 + 2520*p**4*q3**3*q4**2 + 120*p**4*q3**3*q4*q5**3 - 840*p**4*q3**3*q4*q5**2 + 2520*p**4*q3**3*q4*q5 - 4200*p**4*q3**3*q4 + 120*p**4*q3**2*q4**5 + 120*p**4*q3**2*q4**4*q5 - 840*p**4*q3**2*q4**4 + 120*p**4*q3**2*q4**3*q5**2 - 840*p**4*q3**2*q4**3*q5 + 2520*p**4*q3**2*q4**3 + 120*p**4*q3**2*q4**2*q5**3 - 840*p**4*q3**2*q4**2*q5**2 + 2520*p**4*q3**2*q4**2*q5 - 4200*p**4*q3**2*q4**2 + 120*p**4*q3**2*q4*q5**4 - 840*p**4*q3**2*q4*q5**3 + 2520*p**4*q3**2*q4*q5**2 - 4200*p**4*q3**2*q4*q5 + 4200*p**4*q3**2*q4 + 120*p**4*q3*q4**6 + 120*p**4*q3*q4**5*q5 - 840*p**4*q3*q4**5 + 120*p**4*q3*q4**4*q5**2 - 840*p**4*q3*q4**4*q5 + 2520*p**4*q3*q4**4 + 120*p**4*q3*q4**3*q5**3 - 840*p**4*q3*q4**3*q5**2 + 2520*p**4*q3*q4**3*q5 - 4200*p**4*q3*q4**3 + 120*p**4*q3*q4**2*q5**4 - 840*p**4*q3*q4**2*q5**3 + 2520*p**4*q3*q4**2*q5**2 - 4200*p**4*q3*q4**2*q5 + 4200*p**4*q3*q4**2 + 120*p**4*q3*q4*q5**5 - 840*p**4*q3*q4*q5**4 + 2520*p**4*q3*q4*q5**3 - 4200*p**4*q3*q4*q5**2 + 4200*p**4*q3*q4*q5 - 2520*p**4*q3*q4 - 210*p**4*q4**6 - 210*p**4*q4**5*q5 + 1260*p**4*q4**5 - 210*p**4*q4**4*q5**2 + 1260*p**4*q4**4*q5 - 3150*p**4*q4**4 - 210*p**4*q4**3*q5**3 + 1260*p**4*q4**3*q5**2 - 3150*p**4*q4**3*q5 + 4200*p**4*q4**3 - 210*p**4*q4**2*q5**4 + 1260*p**4*q4**2*q5**3 - 3150*p**4*q4**2*q5**2 + 4200*p**4*q4**2*q5 - 3150*p**4*q4**2 - 210*p**4*q4*q5**5 + 1260*p**4*q4*q5**4 - 3150*p**4*q4*q5**3 + 4200*p**4*q4*q5**2 - 3150*p**4*q4*q5 + 1260*p**4*q4 + 252*p**4*q5**5 - 1260*p**4*q5**4 + 2520*p**4*q5**3 - 2520*p**4*q5**2 + 1260*p**4*q5 - 252*p**4 - 40*p**3*q1**6*q2*q3*q4 - 40*p**3*q1**5*q2**2*q3*q4 - 40*p**3*q1**5*q2*q3**2*q4 - 40*p**3*q1**5*q2*q3*q4**2 - 40*p**3*q1**5*q2*q3*q4*q5 + 360*p**3*q1**5*q2*q3*q4 - 40*p**3*q1**4*q2**3*q3*q4 - 40*p**3*q1**4*q2**2*q3**2*q4 - 40*p**3*q1**4*q2**2*q3*q4**2 - 40*p**3*q1**4*q2**2*q3*q4*q5 + 360*p**3*q1**4*q2**2*q3*q4 - 40*p**3*q1**4*q2*q3**3*q4 - 40*p**3*q1**4*q2*q3**2*q4**2 - 40*p**3*q1**4*q2*q3**2*q4*q5 + 360*p**3*q1**4*q2*q3**2*q4 - 40*p**3*q1**4*q2*q3*q4**3 - 40*p**3*q1**4*q2*q3*q4**2*q5 + 360*p**3*q1**4*q2*q3*q4**2 - 40*p**3*q1**4*q2*q3*q4*q5**2 + 360*p**3*q1**4*q2*q3*q4*q5 - 1440*p**3*q1**4*q2*q3*q4 - 40*p**3*q1**3*q2**4*q3*q4 - 40*p**3*q1**3*q2**3*q3**2*q4 - 40*p**3*q1**3*q2**3*q3*q4**2 - 40*p**3*q1**3*q2**3*q3*q4*q5 + 360*p**3*q1**3*q2**3*q3*q4 - 40*p**3*q1**3*q2**2*q3**3*q4 - 40*p**3*q1**3*q2**2*q3**2*q4**2 - 40*p**3*q1**3*q2**2*q3**2*q4*q5 + 360*p**3*q1**3*q2**2*q3**2*q4 - 40*p**3*q1**3*q2**2*q3*q4**3 - 40*p**3*q1**3*q2**2*q3*q4**2*q5 + 360*p**3*q1**3*q2**2*q3*q4**2 - 40*p**3*q1**3*q2**2*q3*q4*q5**2 + 360*p**3*q1**3*q2**2*q3*q4*q5 - 1440*p**3*q1**3*q2**2*q3*q4 - 40*p**3*q1**3*q2*q3**4*q4 - 40*p**3*q1**3*q2*q3**3*q4**2 - 40*p**3*q1**3*q2*q3**3*q4*q5 + 360*p**3*q1**3*q2*q3**3*q4 - 40*p**3*q1**3*q2*q3**2*q4**3 - 40*p**3*q1**3*q2*q3**2*q4**2*q5 + 360*p**3*q1**3*q2*q3**2*q4**2 - 40*p**3*q1**3*q2*q3**2*q4*q5**2 + 360*p**3*q1**3*q2*q3**2*q4*q5 - 1440*p**3*q1**3*q2*q3**2*q4 - 40*p**3*q1**3*q2*q3*q4**4 - 40*p**3*q1**3*q2*q3*q4**3*q5 + 360*p**3*q1**3*q2*q3*q4**3 - 40*p**3*q1**3*q2*q3*q4**2*q5**2 + 360*p**3*q1**3*q2*q3*q4**2*q5 - 1440*p**3*q1**3*q2*q3*q4**2 - 40*p**3*q1**3*q2*q3*q4*q5**3 + 360*p**3*q1**3*q2*q3*q4*q5**2 - 1440*p**3*q1**3*q2*q3*q4*q5 + 3360*p**3*q1**3*q2*q3*q4 - 40*p**3*q1**2*q2**5*q3*q4 - 40*p**3*q1**2*q2**4*q3**2*q4 - 40*p**3*q1**2*q2**4*q3*q4**2 - 40*p**3*q1**2*q2**4*q3*q4*q5 + 360*p**3*q1**2*q2**4*q3*q4 - 40*p**3*q1**2*q2**3*q3**3*q4 - 40*p**3*q1**2*q2**3*q3**2*q4**2 - 40*p**3*q1**2*q2**3*q3**2*q4*q5 + 360*p**3*q1**2*q2**3*q3**2*q4 - 40*p**3*q1**2*q2**3*q3*q4**3 - 40*p**3*q1**2*q2**3*q3*q4**2*q5 + 360*p**3*q1**2*q2**3*q3*q4**2 - 40*p**3*q1**2*q2**3*q3*q4*q5**2 + 360*p**3*q1**2*q2**3*q3*q4*q5 - 1440*p**3*q1**2*q2**3*q3*q4 - 40*p**3*q1**2*q2**2*q3**4*q4 - 40*p**3*q1**2*q2**2*q3**3*q4**2 - 40*p**3*q1**2*q2**2*q3**3*q4*q5 + 360*p**3*q1**2*q2**2*q3**3*q4 - 40*p**3*q1**2*q2**2*q3**2*q4**3 - 40*p**3*q1**2*q2**2*q3**2*q4**2*q5 + 360*p**3*q1**2*q2**2*q3**2*q4**2 - 40*p**3*q1**2*q2**2*q3**2*q4*q5**2 + 360*p**3*q1**2*q2**2*q3**2*q4*q5 - 1440*p**3*q1**2*q2**2*q3**2*q4 - 40*p**3*q1**2*q2**2*q3*q4**4 - 40*p**3*q1**2*q2**2*q3*q4**3*q5 + 360*p**3*q1**2*q2**2*q3*q4**3 - 40*p**3*q1**2*q2**2*q3*q4**2*q5**2 + 360*p**3*q1**2*q2**2*q3*q4**2*q5 - 1440*p**3*q1**2*q2**2*q3*q4**2 - 40*p**3*q1**2*q2**2*q3*q4*q5**3 + 360*p**3*q1**2*q2**2*q3*q4*q5**2 - 1440*p**3*q1**2*q2**2*q3*q4*q5 + 3360*p**3*q1**2*q2**2*q3*q4 - 40*p**3*q1**2*q2*q3**5*q4 - 40*p**3*q1**2*q2*q3**4*q4**2 - 40*p**3*q1**2*q2*q3**4*q4*q5 + 360*p**3*q1**2*q2*q3**4*q4 - 40*p**3*q1**2*q2*q3**3*q4**3 - 40*p**3*q1**2*q2*q3**3*q4**2*q5 + 360*p**3*q1**2*q2*q3**3*q4**2 - 40*p**3*q1**2*q2*q3**3*q4*q5**2 + 360*p**3*q1**2*q2*q3**3*q4*q5 - 1440*p**3*q1**2*q2*q3**3*q4 - 40*p**3*q1**2*q2*q3**2*q4**4 - 40*p**3*q1**2*q2*q3**2*q4**3*q5 + 360*p**3*q1**2*q2*q3**2*q4**3 - 40*p**3*q1**2*q2*q3**2*q4**2*q5**2 + 360*p**3*q1**2*q2*q3**2*q4**2*q5 - 1440*p**3*q1**2*q2*q3**2*q4**2 - 40*p**3*q1**2*q2*q3**2*q4*q5**3 + 360*p**3*q1**2*q2*q3**2*q4*q5**2 - 1440*p**3*q1**2*q2*q3**2*q4*q5 + 3360*p**3*q1**2*q2*q3**2*q4 - 40*p**3*q1**2*q2*q3*q4**5 - 40*p**3*q1**2*q2*q3*q4**4*q5 + 360*p**3*q1**2*q2*q3*q4**4 - 40*p**3*q1**2*q2*q3*q4**3*q5**2 + 360*p**3*q1**2*q2*q3*q4**3*q5 - 1440*p**3*q1**2*q2*q3*q4**3 - 40*p**3*q1**2*q2*q3*q4**2*q5**3 + 360*p**3*q1**2*q2*q3*q4**2*q5**2 - 1440*p**3*q1**2*q2*q3*q4**2*q5 + 3360*p**3*q1**2*q2*q3*q4**2 - 40*p**3*q1**2*q2*q3*q4*q5**4 + 360*p**3*q1**2*q2*q3*q4*q5**3 - 1440*p**3*q1**2*q2*q3*q4*q5**2 + 3360*p**3*q1**2*q2*q3*q4*q5 - 5040*p**3*q1**2*q2*q3*q4 - 40*p**3*q1*q2**6*q3*q4 - 40*p**3*q1*q2**5*q3**2*q4 - 40*p**3*q1*q2**5*q3*q4**2 - 40*p**3*q1*q2**5*q3*q4*q5 + 360*p**3*q1*q2**5*q3*q4 - 40*p**3*q1*q2**4*q3**3*q4 - 40*p**3*q1*q2**4*q3**2*q4**2 - 40*p**3*q1*q2**4*q3**2*q4*q5 + 360*p**3*q1*q2**4*q3**2*q4 - 40*p**3*q1*q2**4*q3*q4**3 - 40*p**3*q1*q2**4*q3*q4**2*q5 + 360*p**3*q1*q2**4*q3*q4**2 - 40*p**3*q1*q2**4*q3*q4*q5**2 + 360*p**3*q1*q2**4*q3*q4*q5 - 1440*p**3*q1*q2**4*q3*q4 - 40*p**3*q1*q2**3*q3**4*q4 - 40*p**3*q1*q2**3*q3**3*q4**2 - 40*p**3*q1*q2**3*q3**3*q4*q5 + 360*p**3*q1*q2**3*q3**3*q4 - 40*p**3*q1*q2**3*q3**2*q4**3 - 40*p**3*q1*q2**3*q3**2*q4**2*q5 + 360*p**3*q1*q2**3*q3**2*q4**2 - 40*p**3*q1*q2**3*q3**2*q4*q5**2 + 360*p**3*q1*q2**3*q3**2*q4*q5 - 1440*p**3*q1*q2**3*q3**2*q4 - 40*p**3*q1*q2**3*q3*q4**4 - 40*p**3*q1*q2**3*q3*q4**3*q5 + 360*p**3*q1*q2**3*q3*q4**3 - 40*p**3*q1*q2**3*q3*q4**2*q5**2 + 360*p**3*q1*q2**3*q3*q4**2*q5 - 1440*p**3*q1*q2**3*q3*q4**2 - 40*p**3*q1*q2**3*q3*q4*q5**3 + 360*p**3*q1*q2**3*q3*q4*q5**2 - 1440*p**3*q1*q2**3*q3*q4*q5 + 3360*p**3*q1*q2**3*q3*q4 - 40*p**3*q1*q2**2*q3**5*q4 - 40*p**3*q1*q2**2*q3**4*q4**2 - 40*p**3*q1*q2**2*q3**4*q4*q5 + 360*p**3*q1*q2**2*q3**4*q4 - 40*p**3*q1*q2**2*q3**3*q4**3 - 40*p**3*q1*q2**2*q3**3*q4**2*q5 + 360*p**3*q1*q2**2*q3**3*q4**2 - 40*p**3*q1*q2**2*q3**3*q4*q5**2 + 360*p**3*q1*q2**2*q3**3*q4*q5 - 1440*p**3*q1*q2**2*q3**3*q4 - 40*p**3*q1*q2**2*q3**2*q4**4 - 40*p**3*q1*q2**2*q3**2*q4**3*q5 + 360*p**3*q1*q2**2*q3**2*q4**3 - 40*p**3*q1*q2**2*q3**2*q4**2*q5**2 + 360*p**3*q1*q2**2*q3**2*q4**2*q5 - 1440*p**3*q1*q2**2*q3**2*q4**2 - 40*p**3*q1*q2**2*q3**2*q4*q5**3 + 360*p**3*q1*q2**2*q3**2*q4*q5**2 - 1440*p**3*q1*q2**2*q3**2*q4*q5 + 3360*p**3*q1*q2**2*q3**2*q4 - 40*p**3*q1*q2**2*q3*q4**5 - 40*p**3*q1*q2**2*q3*q4**4*q5 + 360*p**3*q1*q2**2*q3*q4**4 - 40*p**3*q1*q2**2*q3*q4**3*q5**2 + 360*p**3*q1*q2**2*q3*q4**3*q5 - 1440*p**3*q1*q2**2*q3*q4**3 - 40*p**3*q1*q2**2*q3*q4**2*q5**3 + 360*p**3*q1*q2**2*q3*q4**2*q5**2 - 1440*p**3*q1*q2**2*q3*q4**2*q5 + 3360*p**3*q1*q2**2*q3*q4**2 - 40*p**3*q1*q2**2*q3*q4*q5**4 + 360*p**3*q1*q2**2*q3*q4*q5**3 - 1440*p**3*q1*q2**2*q3*q4*q5**2 + 3360*p**3*q1*q2**2*q3*q4*q5 - 5040*p**3*q1*q2**2*q3*q4 - 40*p**3*q1*q2*q3**6*q4 - 40*p**3*q1*q2*q3**5*q4**2 - 40*p**3*q1*q2*q3**5*q4*q5 + 360*p**3*q1*q2*q3**5*q4 - 40*p**3*q1*q2*q3**4*q4**3 - 40*p**3*q1*q2*q3**4*q4**2*q5 + 360*p**3*q1*q2*q3**4*q4**2 - 40*p**3*q1*q2*q3**4*q4*q5**2 + 360*p**3*q1*q2*q3**4*q4*q5 - 1440*p**3*q1*q2*q3**4*q4 - 40*p**3*q1*q2*q3**3*q4**4 - 40*p**3*q1*q2*q3**3*q4**3*q5 + 360*p**3*q1*q2*q3**3*q4**3 - 40*p**3*q1*q2*q3**3*q4**2*q5**2 + 360*p**3*q1*q2*q3**3*q4**2*q5 - 1440*p**3*q1*q2*q3**3*q4**2 - 40*p**3*q1*q2*q3**3*q4*q5**3 + 360*p**3*q1*q2*q3**3*q4*q5**2 - 1440*p**3*q1*q2*q3**3*q4*q5 + 3360*p**3*q1*q2*q3**3*q4 - 40*p**3*q1*q2*q3**2*q4**5 - 40*p**3*q1*q2*q3**2*q4**4*q5 + 360*p**3*q1*q2*q3**2*q4**4 - 40*p**3*q1*q2*q3**2*q4**3*q5**2 + 360*p**3*q1*q2*q3**2*q4**3*q5 - 1440*p**3*q1*q2*q3**2*q4**3 - 40*p**3*q1*q2*q3**2*q4**2*q5**3 + 360*p**3*q1*q2*q3**2*q4**2*q5**2 - 1440*p**3*q1*q2*q3**2*q4**2*q5 + 3360*p**3*q1*q2*q3**2*q4**2 - 40*p**3*q1*q2*q3**2*q4*q5**4 + 360*p**3*q1*q2*q3**2*q4*q5**3 - 1440*p**3*q1*q2*q3**2*q4*q5**2 + 3360*p**3*q1*q2*q3**2*q4*q5 - 5040*p**3*q1*q2*q3**2*q4 - 40*p**3*q1*q2*q3*q4**6 - 40*p**3*q1*q2*q3*q4**5*q5 + 360*p**3*q1*q2*q3*q4**5 - 40*p**3*q1*q2*q3*q4**4*q5**2 + 360*p**3*q1*q2*q3*q4**4*q5 - 1440*p**3*q1*q2*q3*q4**4 - 40*p**3*q1*q2*q3*q4**3*q5**3 + 360*p**3*q1*q2*q3*q4**3*q5**2 - 1440*p**3*q1*q2*q3*q4**3*q5 + 3360*p**3*q1*q2*q3*q4**3 - 40*p**3*q1*q2*q3*q4**2*q5**4 + 360*p**3*q1*q2*q3*q4**2*q5**3 - 1440*p**3*q1*q2*q3*q4**2*q5**2 + 3360*p**3*q1*q2*q3*q4**2*q5 - 5040*p**3*q1*q2*q3*q4**2 - 40*p**3*q1*q2*q3*q4*q5**5 + 360*p**3*q1*q2*q3*q4*q5**4 - 1440*p**3*q1*q2*q3*q4*q5**3 + 3360*p**3*q1*q2*q3*q4*q5**2 - 5040*p**3*q1*q2*q3*q4*q5 + 5040*p**3*q1*q2*q3*q4 + 135*p**3*q2**6*q3*q4 + 135*p**3*q2**5*q3**2*q4 + 135*p**3*q2**5*q3*q4**2 + 135*p**3*q2**5*q3*q4*q5 - 1080*p**3*q2**5*q3*q4 + 135*p**3*q2**4*q3**3*q4 + 135*p**3*q2**4*q3**2*q4**2 + 135*p**3*q2**4*q3**2*q4*q5 - 1080*p**3*q2**4*q3**2*q4 + 135*p**3*q2**4*q3*q4**3 + 135*p**3*q2**4*q3*q4**2*q5 - 1080*p**3*q2**4*q3*q4**2 + 135*p**3*q2**4*q3*q4*q5**2 - 1080*p**3*q2**4*q3*q4*q5 + 3780*p**3*q2**4*q3*q4 + 135*p**3*q2**3*q3**4*q4 + 135*p**3*q2**3*q3**3*q4**2 + 135*p**3*q2**3*q3**3*q4*q5 - 1080*p**3*q2**3*q3**3*q4 + 135*p**3*q2**3*q3**2*q4**3 + 135*p**3*q2**3*q3**2*q4**2*q5 - 1080*p**3*q2**3*q3**2*q4**2 + 135*p**3*q2**3*q3**2*q4*q5**2 - 1080*p**3*q2**3*q3**2*q4*q5 + 3780*p**3*q2**3*q3**2*q4 + 135*p**3*q2**3*q3*q4**4 + 135*p**3*q2**3*q3*q4**3*q5 - 1080*p**3*q2**3*q3*q4**3 + 135*p**3*q2**3*q3*q4**2*q5**2 - 1080*p**3*q2**3*q3*q4**2*q5 + 3780*p**3*q2**3*q3*q4**2 + 135*p**3*q2**3*q3*q4*q5**3 - 1080*p**3*q2**3*q3*q4*q5**2 + 3780*p**3*q2**3*q3*q4*q5 - 7560*p**3*q2**3*q3*q4 + 135*p**3*q2**2*q3**5*q4 + 135*p**3*q2**2*q3**4*q4**2 + 135*p**3*q2**2*q3**4*q4*q5 - 1080*p**3*q2**2*q3**4*q4 + 135*p**3*q2**2*q3**3*q4**3 + 135*p**3*q2**2*q3**3*q4**2*q5 - 1080*p**3*q2**2*q3**3*q4**2 + 135*p**3*q2**2*q3**3*q4*q5**2 - 1080*p**3*q2**2*q3**3*q4*q5 + 3780*p**3*q2**2*q3**3*q4 + 135*p**3*q2**2*q3**2*q4**4 + 135*p**3*q2**2*q3**2*q4**3*q5 - 1080*p**3*q2**2*q3**2*q4**3 + 135*p**3*q2**2*q3**2*q4**2*q5**2 - 1080*p**3*q2**2*q3**2*q4**2*q5 + 3780*p**3*q2**2*q3**2*q4**2 + 135*p**3*q2**2*q3**2*q4*q5**3 - 1080*p**3*q2**2*q3**2*q4*q5**2 + 3780*p**3*q2**2*q3**2*q4*q5 - 7560*p**3*q2**2*q3**2*q4 + 135*p**3*q2**2*q3*q4**5 + 135*p**3*q2**2*q3*q4**4*q5 - 1080*p**3*q2**2*q3*q4**4 + 135*p**3*q2**2*q3*q4**3*q5**2 - 1080*p**3*q2**2*q3*q4**3*q5 + 3780*p**3*q2**2*q3*q4**3 + 135*p**3*q2**2*q3*q4**2*q5**3 - 1080*p**3*q2**2*q3*q4**2*q5**2 + 3780*p**3*q2**2*q3*q4**2*q5 - 7560*p**3*q2**2*q3*q4**2 + 135*p**3*q2**2*q3*q4*q5**4 - 1080*p**3*q2**2*q3*q4*q5**3 + 3780*p**3*q2**2*q3*q4*q5**2 - 7560*p**3*q2**2*q3*q4*q5 + 9450*p**3*q2**2*q3*q4 + 135*p**3*q2*q3**6*q4 + 135*p**3*q2*q3**5*q4**2 + 135*p**3*q2*q3**5*q4*q5 - 1080*p**3*q2*q3**5*q4 + 135*p**3*q2*q3**4*q4**3 + 135*p**3*q2*q3**4*q4**2*q5 - 1080*p**3*q2*q3**4*q4**2 + 135*p**3*q2*q3**4*q4*q5**2 - 1080*p**3*q2*q3**4*q4*q5 + 3780*p**3*q2*q3**4*q4 + 135*p**3*q2*q3**3*q4**4 + 135*p**3*q2*q3**3*q4**3*q5 - 1080*p**3*q2*q3**3*q4**3 + 135*p**3*q2*q3**3*q4**2*q5**2 - 1080*p**3*q2*q3**3*q4**2*q5 + 3780*p**3*q2*q3**3*q4**2 + 135*p**3*q2*q3**3*q4*q5**3 - 1080*p**3*q2*q3**3*q4*q5**2 + 3780*p**3*q2*q3**3*q4*q5 - 7560*p**3*q2*q3**3*q4 + 135*p**3*q2*q3**2*q4**5 + 135*p**3*q2*q3**2*q4**4*q5 - 1080*p**3*q2*q3**2*q4**4 + 135*p**3*q2*q3**2*q4**3*q5**2 - 1080*p**3*q2*q3**2*q4**3*q5 + 3780*p**3*q2*q3**2*q4**3 + 135*p**3*q2*q3**2*q4**2*q5**3 - 1080*p**3*q2*q3**2*q4**2*q5**2 + 3780*p**3*q2*q3**2*q4**2*q5 - 7560*p**3*q2*q3**2*q4**2 + 135*p**3*q2*q3**2*q4*q5**4 - 1080*p**3*q2*q3**2*q4*q5**3 + 3780*p**3*q2*q3**2*q4*q5**2 - 7560*p**3*q2*q3**2*q4*q5 + 9450*p**3*q2*q3**2*q4 + 135*p**3*q2*q3*q4**6 + 135*p**3*q2*q3*q4**5*q5 - 1080*p**3*q2*q3*q4**5 + 135*p**3*q2*q3*q4**4*q5**2 - 1080*p**3*q2*q3*q4**4*q5 + 3780*p**3*q2*q3*q4**4 + 135*p**3*q2*q3*q4**3*q5**3 - 1080*p**3*q2*q3*q4**3*q5**2 + 3780*p**3*q2*q3*q4**3*q5 - 7560*p**3*q2*q3*q4**3 + 135*p**3*q2*q3*q4**2*q5**4 - 1080*p**3*q2*q3*q4**2*q5**3 + 3780*p**3*q2*q3*q4**2*q5**2 - 7560*p**3*q2*q3*q4**2*q5 + 9450*p**3*q2*q3*q4**2 + 135*p**3*q2*q3*q4*q5**5 - 1080*p**3*q2*q3*q4*q5**4 + 3780*p**3*q2*q3*q4*q5**3 - 7560*p**3*q2*q3*q4*q5**2 + 9450*p**3*q2*q3*q4*q5 - 7560*p**3*q2*q3*q4 - 240*p**3*q3**6*q4 - 240*p**3*q3**5*q4**2 - 240*p**3*q3**5*q4*q5 + 1680*p**3*q3**5*q4 - 240*p**3*q3**4*q4**3 - 240*p**3*q3**4*q4**2*q5 + 1680*p**3*q3**4*q4**2 - 240*p**3*q3**4*q4*q5**2 + 1680*p**3*q3**4*q4*q5 - 5040*p**3*q3**4*q4 - 240*p**3*q3**3*q4**4 - 240*p**3*q3**3*q4**3*q5 + 1680*p**3*q3**3*q4**3 - 240*p**3*q3**3*q4**2*q5**2 + 1680*p**3*q3**3*q4**2*q5 - 5040*p**3*q3**3*q4**2 - 240*p**3*q3**3*q4*q5**3 + 1680*p**3*q3**3*q4*q5**2 - 5040*p**3*q3**3*q4*q5 + 8400*p**3*q3**3*q4 - 240*p**3*q3**2*q4**5 - 240*p**3*q3**2*q4**4*q5 + 1680*p**3*q3**2*q4**4 - 240*p**3*q3**2*q4**3*q5**2 + 1680*p**3*q3**2*q4**3*q5 - 5040*p**3*q3**2*q4**3 - 240*p**3*q3**2*q4**2*q5**3 + 1680*p**3*q3**2*q4**2*q5**2 - 5040*p**3*q3**2*q4**2*q5 + 8400*p**3*q3**2*q4**2 - 240*p**3*q3**2*q4*q5**4 + 1680*p**3*q3**2*q4*q5**3 - 5040*p**3*q3**2*q4*q5**2 + 8400*p**3*q3**2*q4*q5 - 8400*p**3*q3**2*q4 - 240*p**3*q3*q4**6 - 240*p**3*q3*q4**5*q5 + 1680*p**3*q3*q4**5 - 240*p**3*q3*q4**4*q5**2 + 1680*p**3*q3*q4**4*q5 - 5040*p**3*q3*q4**4 - 240*p**3*q3*q4**3*q5**3 + 1680*p**3*q3*q4**3*q5**2 - 5040*p**3*q3*q4**3*q5 + 8400*p**3*q3*q4**3 - 240*p**3*q3*q4**2*q5**4 + 1680*p**3*q3*q4**2*q5**3 - 5040*p**3*q3*q4**2*q5**2 + 8400*p**3*q3*q4**2*q5 - 8400*p**3*q3*q4**2 - 240*p**3*q3*q4*q5**5 + 1680*p**3*q3*q4*q5**4 - 5040*p**3*q3*q4*q5**3 + 8400*p**3*q3*q4*q5**2 - 8400*p**3*q3*q4*q5 + 5040*p**3*q3*q4 + 210*p**3*q4**6 + 210*p**3*q4**5*q5 - 1260*p**3*q4**5 + 210*p**3*q4**4*q5**2 - 1260*p**3*q4**4*q5 + 3150*p**3*q4**4 + 210*p**3*q4**3*q5**3 - 1260*p**3*q4**3*q5**2 + 3150*p**3*q4**3*q5 - 4200*p**3*q4**3 + 210*p**3*q4**2*q5**4 - 1260*p**3*q4**2*q5**3 + 3150*p**3*q4**2*q5**2 - 4200*p**3*q4**2*q5 + 3150*p**3*q4**2 + 210*p**3*q4*q5**5 - 1260*p**3*q4*q5**4 + 3150*p**3*q4*q5**3 - 4200*p**3*q4*q5**2 + 3150*p**3*q4*q5 - 1260*p**3*q4 + 60*p**2*q1**6*q2*q3*q4 + 60*p**2*q1**5*q2**2*q3*q4 + 60*p**2*q1**5*q2*q3**2*q4 + 60*p**2*q1**5*q2*q3*q4**2 + 60*p**2*q1**5*q2*q3*q4*q5 - 540*p**2*q1**5*q2*q3*q4 + 60*p**2*q1**4*q2**3*q3*q4 + 60*p**2*q1**4*q2**2*q3**2*q4 + 60*p**2*q1**4*q2**2*q3*q4**2 + 60*p**2*q1**4*q2**2*q3*q4*q5 - 540*p**2*q1**4*q2**2*q3*q4 + 60*p**2*q1**4*q2*q3**3*q4 + 60*p**2*q1**4*q2*q3**2*q4**2 + 60*p**2*q1**4*q2*q3**2*q4*q5 - 540*p**2*q1**4*q2*q3**2*q4 + 60*p**2*q1**4*q2*q3*q4**3 + 60*p**2*q1**4*q2*q3*q4**2*q5 - 540*p**2*q1**4*q2*q3*q4**2 + 60*p**2*q1**4*q2*q3*q4*q5**2 - 540*p**2*q1**4*q2*q3*q4*q5 + 2160*p**2*q1**4*q2*q3*q4 + 60*p**2*q1**3*q2**4*q3*q4 + 60*p**2*q1**3*q2**3*q3**2*q4 + 60*p**2*q1**3*q2**3*q3*q4**2 + 60*p**2*q1**3*q2**3*q3*q4*q5 - 540*p**2*q1**3*q2**3*q3*q4 + 60*p**2*q1**3*q2**2*q3**3*q4 + 60*p**2*q1**3*q2**2*q3**2*q4**2 + 60*p**2*q1**3*q2**2*q3**2*q4*q5 - 540*p**2*q1**3*q2**2*q3**2*q4 + 60*p**2*q1**3*q2**2*q3*q4**3 + 60*p**2*q1**3*q2**2*q3*q4**2*q5 - 540*p**2*q1**3*q2**2*q3*q4**2 + 60*p**2*q1**3*q2**2*q3*q4*q5**2 - 540*p**2*q1**3*q2**2*q3*q4*q5 + 2160*p**2*q1**3*q2**2*q3*q4 + 60*p**2*q1**3*q2*q3**4*q4 + 60*p**2*q1**3*q2*q3**3*q4**2 + 60*p**2*q1**3*q2*q3**3*q4*q5 - 540*p**2*q1**3*q2*q3**3*q4 + 60*p**2*q1**3*q2*q3**2*q4**3 + 60*p**2*q1**3*q2*q3**2*q4**2*q5 - 540*p**2*q1**3*q2*q3**2*q4**2 + 60*p**2*q1**3*q2*q3**2*q4*q5**2 - 540*p**2*q1**3*q2*q3**2*q4*q5 + 2160*p**2*q1**3*q2*q3**2*q4 + 60*p**2*q1**3*q2*q3*q4**4 + 60*p**2*q1**3*q2*q3*q4**3*q5 - 540*p**2*q1**3*q2*q3*q4**3 + 60*p**2*q1**3*q2*q3*q4**2*q5**2 - 540*p**2*q1**3*q2*q3*q4**2*q5 + 2160*p**2*q1**3*q2*q3*q4**2 + 60*p**2*q1**3*q2*q3*q4*q5**3 - 540*p**2*q1**3*q2*q3*q4*q5**2 + 2160*p**2*q1**3*q2*q3*q4*q5 - 5040*p**2*q1**3*q2*q3*q4 + 60*p**2*q1**2*q2**5*q3*q4 + 60*p**2*q1**2*q2**4*q3**2*q4 + 60*p**2*q1**2*q2**4*q3*q4**2 + 60*p**2*q1**2*q2**4*q3*q4*q5 - 540*p**2*q1**2*q2**4*q3*q4 + 60*p**2*q1**2*q2**3*q3**3*q4 + 60*p**2*q1**2*q2**3*q3**2*q4**2 + 60*p**2*q1**2*q2**3*q3**2*q4*q5 - 540*p**2*q1**2*q2**3*q3**2*q4 + 60*p**2*q1**2*q2**3*q3*q4**3 + 60*p**2*q1**2*q2**3*q3*q4**2*q5 - 540*p**2*q1**2*q2**3*q3*q4**2 + 60*p**2*q1**2*q2**3*q3*q4*q5**2 - 540*p**2*q1**2*q2**3*q3*q4*q5 + 2160*p**2*q1**2*q2**3*q3*q4 + 60*p**2*q1**2*q2**2*q3**4*q4 + 60*p**2*q1**2*q2**2*q3**3*q4**2 + 60*p**2*q1**2*q2**2*q3**3*q4*q5 - 540*p**2*q1**2*q2**2*q3**3*q4 + 60*p**2*q1**2*q2**2*q3**2*q4**3 + 60*p**2*q1**2*q2**2*q3**2*q4**2*q5 - 540*p**2*q1**2*q2**2*q3**2*q4**2 + 60*p**2*q1**2*q2**2*q3**2*q4*q5**2 - 540*p**2*q1**2*q2**2*q3**2*q4*q5 + 2160*p**2*q1**2*q2**2*q3**2*q4 + 60*p**2*q1**2*q2**2*q3*q4**4 + 60*p**2*q1**2*q2**2*q3*q4**3*q5 - 540*p**2*q1**2*q2**2*q3*q4**3 + 60*p**2*q1**2*q2**2*q3*q4**2*q5**2 - 540*p**2*q1**2*q2**2*q3*q4**2*q5 + 2160*p**2*q1**2*q2**2*q3*q4**2 + 60*p**2*q1**2*q2**2*q3*q4*q5**3 - 540*p**2*q1**2*q2**2*q3*q4*q5**2 + 2160*p**2*q1**2*q2**2*q3*q4*q5 - 5040*p**2*q1**2*q2**2*q3*q4 + 60*p**2*q1**2*q2*q3**5*q4 + 60*p**2*q1**2*q2*q3**4*q4**2 + 60*p**2*q1**2*q2*q3**4*q4*q5 - 540*p**2*q1**2*q2*q3**4*q4 + 60*p**2*q1**2*q2*q3**3*q4**3 + 60*p**2*q1**2*q2*q3**3*q4**2*q5 - 540*p**2*q1**2*q2*q3**3*q4**2 + 60*p**2*q1**2*q2*q3**3*q4*q5**2 - 540*p**2*q1**2*q2*q3**3*q4*q5 + 2160*p**2*q1**2*q2*q3**3*q4 + 60*p**2*q1**2*q2*q3**2*q4**4 + 60*p**2*q1**2*q2*q3**2*q4**3*q5 - 540*p**2*q1**2*q2*q3**2*q4**3 + 60*p**2*q1**2*q2*q3**2*q4**2*q5**2 - 540*p**2*q1**2*q2*q3**2*q4**2*q5 + 2160*p**2*q1**2*q2*q3**2*q4**2 + 60*p**2*q1**2*q2*q3**2*q4*q5**3 - 540*p**2*q1**2*q2*q3**2*q4*q5**2 + 2160*p**2*q1**2*q2*q3**2*q4*q5 - 5040*p**2*q1**2*q2*q3**2*q4 + 60*p**2*q1**2*q2*q3*q4**5 + 60*p**2*q1**2*q2*q3*q4**4*q5 - 540*p**2*q1**2*q2*q3*q4**4 + 60*p**2*q1**2*q2*q3*q4**3*q5**2 - 540*p**2*q1**2*q2*q3*q4**3*q5 + 2160*p**2*q1**2*q2*q3*q4**3 + 60*p**2*q1**2*q2*q3*q4**2*q5**3 - 540*p**2*q1**2*q2*q3*q4**2*q5**2 + 2160*p**2*q1**2*q2*q3*q4**2*q5 - 5040*p**2*q1**2*q2*q3*q4**2 + 60*p**2*q1**2*q2*q3*q4*q5**4 - 540*p**2*q1**2*q2*q3*q4*q5**3 + 2160*p**2*q1**2*q2*q3*q4*q5**2 - 5040*p**2*q1**2*q2*q3*q4*q5 + 7560*p**2*q1**2*q2*q3*q4 + 60*p**2*q1*q2**6*q3*q4 + 60*p**2*q1*q2**5*q3**2*q4 + 60*p**2*q1*q2**5*q3*q4**2 + 60*p**2*q1*q2**5*q3*q4*q5 - 540*p**2*q1*q2**5*q3*q4 + 60*p**2*q1*q2**4*q3**3*q4 + 60*p**2*q1*q2**4*q3**2*q4**2 + 60*p**2*q1*q2**4*q3**2*q4*q5 - 540*p**2*q1*q2**4*q3**2*q4 + 60*p**2*q1*q2**4*q3*q4**3 + 60*p**2*q1*q2**4*q3*q4**2*q5 - 540*p**2*q1*q2**4*q3*q4**2 + 60*p**2*q1*q2**4*q3*q4*q5**2 - 540*p**2*q1*q2**4*q3*q4*q5 + 2160*p**2*q1*q2**4*q3*q4 + 60*p**2*q1*q2**3*q3**4*q4 + 60*p**2*q1*q2**3*q3**3*q4**2 + 60*p**2*q1*q2**3*q3**3*q4*q5 - 540*p**2*q1*q2**3*q3**3*q4 + 60*p**2*q1*q2**3*q3**2*q4**3 + 60*p**2*q1*q2**3*q3**2*q4**2*q5 - 540*p**2*q1*q2**3*q3**2*q4**2 + 60*p**2*q1*q2**3*q3**2*q4*q5**2 - 540*p**2*q1*q2**3*q3**2*q4*q5 + 2160*p**2*q1*q2**3*q3**2*q4 + 60*p**2*q1*q2**3*q3*q4**4 + 60*p**2*q1*q2**3*q3*q4**3*q5 - 540*p**2*q1*q2**3*q3*q4**3 + 60*p**2*q1*q2**3*q3*q4**2*q5**2 - 540*p**2*q1*q2**3*q3*q4**2*q5 + 2160*p**2*q1*q2**3*q3*q4**2 + 60*p**2*q1*q2**3*q3*q4*q5**3 - 540*p**2*q1*q2**3*q3*q4*q5**2 + 2160*p**2*q1*q2**3*q3*q4*q5 - 5040*p**2*q1*q2**3*q3*q4 + 60*p**2*q1*q2**2*q3**5*q4 + 60*p**2*q1*q2**2*q3**4*q4**2 + 60*p**2*q1*q2**2*q3**4*q4*q5 - 540*p**2*q1*q2**2*q3**4*q4 + 60*p**2*q1*q2**2*q3**3*q4**3 + 60*p**2*q1*q2**2*q3**3*q4**2*q5 - 540*p**2*q1*q2**2*q3**3*q4**2 + 60*p**2*q1*q2**2*q3**3*q4*q5**2 - 540*p**2*q1*q2**2*q3**3*q4*q5 + 2160*p**2*q1*q2**2*q3**3*q4 + 60*p**2*q1*q2**2*q3**2*q4**4 + 60*p**2*q1*q2**2*q3**2*q4**3*q5 - 540*p**2*q1*q2**2*q3**2*q4**3 + 60*p**2*q1*q2**2*q3**2*q4**2*q5**2 - 540*p**2*q1*q2**2*q3**2*q4**2*q5 + 2160*p**2*q1*q2**2*q3**2*q4**2 + 60*p**2*q1*q2**2*q3**2*q4*q5**3 - 540*p**2*q1*q2**2*q3**2*q4*q5**2 + 2160*p**2*q1*q2**2*q3**2*q4*q5 - 5040*p**2*q1*q2**2*q3**2*q4 + 60*p**2*q1*q2**2*q3*q4**5 + 60*p**2*q1*q2**2*q3*q4**4*q5 - 540*p**2*q1*q2**2*q3*q4**4 + 60*p**2*q1*q2**2*q3*q4**3*q5**2 - 540*p**2*q1*q2**2*q3*q4**3*q5 + 2160*p**2*q1*q2**2*q3*q4**3 + 60*p**2*q1*q2**2*q3*q4**2*q5**3 - 540*p**2*q1*q2**2*q3*q4**2*q5**2 + 2160*p**2*q1*q2**2*q3*q4**2*q5 - 5040*p**2*q1*q2**2*q3*q4**2 + 60*p**2*q1*q2**2*q3*q4*q5**4 - 540*p**2*q1*q2**2*q3*q4*q5**3 + 2160*p**2*q1*q2**2*q3*q4*q5**2 - 5040*p**2*q1*q2**2*q3*q4*q5 + 7560*p**2*q1*q2**2*q3*q4 + 60*p**2*q1*q2*q3**6*q4 + 60*p**2*q1*q2*q3**5*q4**2 + 60*p**2*q1*q2*q3**5*q4*q5 - 540*p**2*q1*q2*q3**5*q4 + 60*p**2*q1*q2*q3**4*q4**3 + 60*p**2*q1*q2*q3**4*q4**2*q5 - 540*p**2*q1*q2*q3**4*q4**2 + 60*p**2*q1*q2*q3**4*q4*q5**2 - 540*p**2*q1*q2*q3**4*q4*q5 + 2160*p**2*q1*q2*q3**4*q4 + 60*p**2*q1*q2*q3**3*q4**4 + 60*p**2*q1*q2*q3**3*q4**3*q5 - 540*p**2*q1*q2*q3**3*q4**3 + 60*p**2*q1*q2*q3**3*q4**2*q5**2 - 540*p**2*q1*q2*q3**3*q4**2*q5 + 2160*p**2*q1*q2*q3**3*q4**2 + 60*p**2*q1*q2*q3**3*q4*q5**3 - 540*p**2*q1*q2*q3**3*q4*q5**2 + 2160*p**2*q1*q2*q3**3*q4*q5 - 5040*p**2*q1*q2*q3**3*q4 + 60*p**2*q1*q2*q3**2*q4**5 + 60*p**2*q1*q2*q3**2*q4**4*q5 - 540*p**2*q1*q2*q3**2*q4**4 + 60*p**2*q1*q2*q3**2*q4**3*q5**2 - 540*p**2*q1*q2*q3**2*q4**3*q5 + 2160*p**2*q1*q2*q3**2*q4**3 + 60*p**2*q1*q2*q3**2*q4**2*q5**3 - 540*p**2*q1*q2*q3**2*q4**2*q5**2 + 2160*p**2*q1*q2*q3**2*q4**2*q5 - 5040*p**2*q1*q2*q3**2*q4**2 + 60*p**2*q1*q2*q3**2*q4*q5**4 - 540*p**2*q1*q2*q3**2*q4*q5**3 + 2160*p**2*q1*q2*q3**2*q4*q5**2 - 5040*p**2*q1*q2*q3**2*q4*q5 + 7560*p**2*q1*q2*q3**2*q4 + 60*p**2*q1*q2*q3*q4**6 + 60*p**2*q1*q2*q3*q4**5*q5 - 540*p**2*q1*q2*q3*q4**5 + 60*p**2*q1*q2*q3*q4**4*q5**2 - 540*p**2*q1*q2*q3*q4**4*q5 + 2160*p**2*q1*q2*q3*q4**4 + 60*p**2*q1*q2*q3*q4**3*q5**3 - 540*p**2*q1*q2*q3*q4**3*q5**2 + 2160*p**2*q1*q2*q3*q4**3*q5 - 5040*p**2*q1*q2*q3*q4**3 + 60*p**2*q1*q2*q3*q4**2*q5**4 - 540*p**2*q1*q2*q3*q4**2*q5**3 + 2160*p**2*q1*q2*q3*q4**2*q5**2 - 5040*p**2*q1*q2*q3*q4**2*q5 + 7560*p**2*q1*q2*q3*q4**2 + 60*p**2*q1*q2*q3*q4*q5**5 - 540*p**2*q1*q2*q3*q4*q5**4 + 2160*p**2*q1*q2*q3*q4*q5**3 - 5040*p**2*q1*q2*q3*q4*q5**2 + 7560*p**2*q1*q2*q3*q4*q5 - 7560*p**2*q1*q2*q3*q4 - 135*p**2*q2**6*q3*q4 - 135*p**2*q2**5*q3**2*q4 - 135*p**2*q2**5*q3*q4**2 - 135*p**2*q2**5*q3*q4*q5 + 1080*p**2*q2**5*q3*q4 - 135*p**2*q2**4*q3**3*q4 - 135*p**2*q2**4*q3**2*q4**2 - 135*p**2*q2**4*q3**2*q4*q5 + 1080*p**2*q2**4*q3**2*q4 - 135*p**2*q2**4*q3*q4**3 - 135*p**2*q2**4*q3*q4**2*q5 + 1080*p**2*q2**4*q3*q4**2 - 135*p**2*q2**4*q3*q4*q5**2 + 1080*p**2*q2**4*q3*q4*q5 - 3780*p**2*q2**4*q3*q4 - 135*p**2*q2**3*q3**4*q4 - 135*p**2*q2**3*q3**3*q4**2 - 135*p**2*q2**3*q3**3*q4*q5 + 1080*p**2*q2**3*q3**3*q4 - 135*p**2*q2**3*q3**2*q4**3 - 135*p**2*q2**3*q3**2*q4**2*q5 + 1080*p**2*q2**3*q3**2*q4**2 - 135*p**2*q2**3*q3**2*q4*q5**2 + 1080*p**2*q2**3*q3**2*q4*q5 - 3780*p**2*q2**3*q3**2*q4 - 135*p**2*q2**3*q3*q4**4 - 135*p**2*q2**3*q3*q4**3*q5 + 1080*p**2*q2**3*q3*q4**3 - 135*p**2*q2**3*q3*q4**2*q5**2 + 1080*p**2*q2**3*q3*q4**2*q5 - 3780*p**2*q2**3*q3*q4**2 - 135*p**2*q2**3*q3*q4*q5**3 + 1080*p**2*q2**3*q3*q4*q5**2 - 3780*p**2*q2**3*q3*q4*q5 + 7560*p**2*q2**3*q3*q4 - 135*p**2*q2**2*q3**5*q4 - 135*p**2*q2**2*q3**4*q4**2 - 135*p**2*q2**2*q3**4*q4*q5 + 1080*p**2*q2**2*q3**4*q4 - 135*p**2*q2**2*q3**3*q4**3 - 135*p**2*q2**2*q3**3*q4**2*q5 + 1080*p**2*q2**2*q3**3*q4**2 - 135*p**2*q2**2*q3**3*q4*q5**2 + 1080*p**2*q2**2*q3**3*q4*q5 - 3780*p**2*q2**2*q3**3*q4 - 135*p**2*q2**2*q3**2*q4**4 - 135*p**2*q2**2*q3**2*q4**3*q5 + 1080*p**2*q2**2*q3**2*q4**3 - 135*p**2*q2**2*q3**2*q4**2*q5**2 + 1080*p**2*q2**2*q3**2*q4**2*q5 - 3780*p**2*q2**2*q3**2*q4**2 - 135*p**2*q2**2*q3**2*q4*q5**3 + 1080*p**2*q2**2*q3**2*q4*q5**2 - 3780*p**2*q2**2*q3**2*q4*q5 + 7560*p**2*q2**2*q3**2*q4 - 135*p**2*q2**2*q3*q4**5 - 135*p**2*q2**2*q3*q4**4*q5 + 1080*p**2*q2**2*q3*q4**4 - 135*p**2*q2**2*q3*q4**3*q5**2 + 1080*p**2*q2**2*q3*q4**3*q5 - 3780*p**2*q2**2*q3*q4**3 - 135*p**2*q2**2*q3*q4**2*q5**3 + 1080*p**2*q2**2*q3*q4**2*q5**2 - 3780*p**2*q2**2*q3*q4**2*q5 + 7560*p**2*q2**2*q3*q4**2 - 135*p**2*q2**2*q3*q4*q5**4 + 1080*p**2*q2**2*q3*q4*q5**3 - 3780*p**2*q2**2*q3*q4*q5**2 + 7560*p**2*q2**2*q3*q4*q5 - 9450*p**2*q2**2*q3*q4 - 135*p**2*q2*q3**6*q4 - 135*p**2*q2*q3**5*q4**2 - 135*p**2*q2*q3**5*q4*q5 + 1080*p**2*q2*q3**5*q4 - 135*p**2*q2*q3**4*q4**3 - 135*p**2*q2*q3**4*q4**2*q5 + 1080*p**2*q2*q3**4*q4**2 - 135*p**2*q2*q3**4*q4*q5**2 + 1080*p**2*q2*q3**4*q4*q5 - 3780*p**2*q2*q3**4*q4 - 135*p**2*q2*q3**3*q4**4 - 135*p**2*q2*q3**3*q4**3*q5 + 1080*p**2*q2*q3**3*q4**3 - 135*p**2*q2*q3**3*q4**2*q5**2 + 1080*p**2*q2*q3**3*q4**2*q5 - 3780*p**2*q2*q3**3*q4**2 - 135*p**2*q2*q3**3*q4*q5**3 + 1080*p**2*q2*q3**3*q4*q5**2 - 3780*p**2*q2*q3**3*q4*q5 + 7560*p**2*q2*q3**3*q4 - 135*p**2*q2*q3**2*q4**5 - 135*p**2*q2*q3**2*q4**4*q5 + 1080*p**2*q2*q3**2*q4**4 - 135*p**2*q2*q3**2*q4**3*q5**2 + 1080*p**2*q2*q3**2*q4**3*q5 - 3780*p**2*q2*q3**2*q4**3 - 135*p**2*q2*q3**2*q4**2*q5**3 + 1080*p**2*q2*q3**2*q4**2*q5**2 - 3780*p**2*q2*q3**2*q4**2*q5 + 7560*p**2*q2*q3**2*q4**2 - 135*p**2*q2*q3**2*q4*q5**4 + 1080*p**2*q2*q3**2*q4*q5**3 - 3780*p**2*q2*q3**2*q4*q5**2 + 7560*p**2*q2*q3**2*q4*q5 - 9450*p**2*q2*q3**2*q4 - 135*p**2*q2*q3*q4**6 - 135*p**2*q2*q3*q4**5*q5 + 1080*p**2*q2*q3*q4**5 - 135*p**2*q2*q3*q4**4*q5**2 + 1080*p**2*q2*q3*q4**4*q5 - 3780*p**2*q2*q3*q4**4 - 135*p**2*q2*q3*q4**3*q5**3 + 1080*p**2*q2*q3*q4**3*q5**2 - 3780*p**2*q2*q3*q4**3*q5 + 7560*p**2*q2*q3*q4**3 - 135*p**2*q2*q3*q4**2*q5**4 + 1080*p**2*q2*q3*q4**2*q5**3 - 3780*p**2*q2*q3*q4**2*q5**2 + 7560*p**2*q2*q3*q4**2*q5 - 9450*p**2*q2*q3*q4**2 - 135*p**2*q2*q3*q4*q5**5 + 1080*p**2*q2*q3*q4*q5**4 - 3780*p**2*q2*q3*q4*q5**3 + 7560*p**2*q2*q3*q4*q5**2 - 9450*p**2*q2*q3*q4*q5 + 7560*p**2*q2*q3*q4 + 120*p**2*q3**6*q4 + 120*p**2*q3**5*q4**2 + 120*p**2*q3**5*q4*q5 - 840*p**2*q3**5*q4 + 120*p**2*q3**4*q4**3 + 120*p**2*q3**4*q4**2*q5 - 840*p**2*q3**4*q4**2 + 120*p**2*q3**4*q4*q5**2 - 840*p**2*q3**4*q4*q5 + 2520*p**2*q3**4*q4 + 120*p**2*q3**3*q4**4 + 120*p**2*q3**3*q4**3*q5 - 840*p**2*q3**3*q4**3 + 120*p**2*q3**3*q4**2*q5**2 - 840*p**2*q3**3*q4**2*q5 + 2520*p**2*q3**3*q4**2 + 120*p**2*q3**3*q4*q5**3 - 840*p**2*q3**3*q4*q5**2 + 2520*p**2*q3**3*q4*q5 - 4200*p**2*q3**3*q4 + 120*p**2*q3**2*q4**5 + 120*p**2*q3**2*q4**4*q5 - 840*p**2*q3**2*q4**4 + 120*p**2*q3**2*q4**3*q5**2 - 840*p**2*q3**2*q4**3*q5 + 2520*p**2*q3**2*q4**3 + 120*p**2*q3**2*q4**2*q5**3 - 840*p**2*q3**2*q4**2*q5**2 + 2520*p**2*q3**2*q4**2*q5 - 4200*p**2*q3**2*q4**2 + 120*p**2*q3**2*q4*q5**4 - 840*p**2*q3**2*q4*q5**3 + 2520*p**2*q3**2*q4*q5**2 - 4200*p**2*q3**2*q4*q5 + 4200*p**2*q3**2*q4 + 120*p**2*q3*q4**6 + 120*p**2*q3*q4**5*q5 - 840*p**2*q3*q4**5 + 120*p**2*q3*q4**4*q5**2 - 840*p**2*q3*q4**4*q5 + 2520*p**2*q3*q4**4 + 120*p**2*q3*q4**3*q5**3 - 840*p**2*q3*q4**3*q5**2 + 2520*p**2*q3*q4**3*q5 - 4200*p**2*q3*q4**3 + 120*p**2*q3*q4**2*q5**4 - 840*p**2*q3*q4**2*q5**3 + 2520*p**2*q3*q4**2*q5**2 - 4200*p**2*q3*q4**2*q5 + 4200*p**2*q3*q4**2 + 120*p**2*q3*q4*q5**5 - 840*p**2*q3*q4*q5**4 + 2520*p**2*q3*q4*q5**3 - 4200*p**2*q3*q4*q5**2 + 4200*p**2*q3*q4*q5 - 2520*p**2*q3*q4 - 40*p*q1**6*q2*q3*q4 - 40*p*q1**5*q2**2*q3*q4 - 40*p*q1**5*q2*q3**2*q4 - 40*p*q1**5*q2*q3*q4**2 - 40*p*q1**5*q2*q3*q4*q5 + 360*p*q1**5*q2*q3*q4 - 40*p*q1**4*q2**3*q3*q4 - 40*p*q1**4*q2**2*q3**2*q4 - 40*p*q1**4*q2**2*q3*q4**2 - 40*p*q1**4*q2**2*q3*q4*q5 + 360*p*q1**4*q2**2*q3*q4 - 40*p*q1**4*q2*q3**3*q4 - 40*p*q1**4*q2*q3**2*q4**2 - 40*p*q1**4*q2*q3**2*q4*q5 + 360*p*q1**4*q2*q3**2*q4 - 40*p*q1**4*q2*q3*q4**3 - 40*p*q1**4*q2*q3*q4**2*q5 + 360*p*q1**4*q2*q3*q4**2 - 40*p*q1**4*q2*q3*q4*q5**2 + 360*p*q1**4*q2*q3*q4*q5 - 1440*p*q1**4*q2*q3*q4 - 40*p*q1**3*q2**4*q3*q4 - 40*p*q1**3*q2**3*q3**2*q4 - 40*p*q1**3*q2**3*q3*q4**2 - 40*p*q1**3*q2**3*q3*q4*q5 + 360*p*q1**3*q2**3*q3*q4 - 40*p*q1**3*q2**2*q3**3*q4 - 40*p*q1**3*q2**2*q3**2*q4**2 - 40*p*q1**3*q2**2*q3**2*q4*q5 + 360*p*q1**3*q2**2*q3**2*q4 - 40*p*q1**3*q2**2*q3*q4**3 - 40*p*q1**3*q2**2*q3*q4**2*q5 + 360*p*q1**3*q2**2*q3*q4**2 - 40*p*q1**3*q2**2*q3*q4*q5**2 + 360*p*q1**3*q2**2*q3*q4*q5 - 1440*p*q1**3*q2**2*q3*q4 - 40*p*q1**3*q2*q3**4*q4 - 40*p*q1**3*q2*q3**3*q4**2 - 40*p*q1**3*q2*q3**3*q4*q5 + 360*p*q1**3*q2*q3**3*q4 - 40*p*q1**3*q2*q3**2*q4**3 - 40*p*q1**3*q2*q3**2*q4**2*q5 + 360*p*q1**3*q2*q3**2*q4**2 - 40*p*q1**3*q2*q3**2*q4*q5**2 + 360*p*q1**3*q2*q3**2*q4*q5 - 1440*p*q1**3*q2*q3**2*q4 - 40*p*q1**3*q2*q3*q4**4 - 40*p*q1**3*q2*q3*q4**3*q5 + 360*p*q1**3*q2*q3*q4**3 - 40*p*q1**3*q2*q3*q4**2*q5**2 + 360*p*q1**3*q2*q3*q4**2*q5 - 1440*p*q1**3*q2*q3*q4**2 - 40*p*q1**3*q2*q3*q4*q5**3 + 360*p*q1**3*q2*q3*q4*q5**2 - 1440*p*q1**3*q2*q3*q4*q5 + 3360*p*q1**3*q2*q3*q4 - 40*p*q1**2*q2**5*q3*q4 - 40*p*q1**2*q2**4*q3**2*q4 - 40*p*q1**2*q2**4*q3*q4**2 - 40*p*q1**2*q2**4*q3*q4*q5 + 360*p*q1**2*q2**4*q3*q4 - 40*p*q1**2*q2**3*q3**3*q4 - 40*p*q1**2*q2**3*q3**2*q4**2 - 40*p*q1**2*q2**3*q3**2*q4*q5 + 360*p*q1**2*q2**3*q3**2*q4 - 40*p*q1**2*q2**3*q3*q4**3 - 40*p*q1**2*q2**3*q3*q4**2*q5 + 360*p*q1**2*q2**3*q3*q4**2 - 40*p*q1**2*q2**3*q3*q4*q5**2 + 360*p*q1**2*q2**3*q3*q4*q5 - 1440*p*q1**2*q2**3*q3*q4 - 40*p*q1**2*q2**2*q3**4*q4 - 40*p*q1**2*q2**2*q3**3*q4**2 - 40*p*q1**2*q2**2*q3**3*q4*q5 + 360*p*q1**2*q2**2*q3**3*q4 - 40*p*q1**2*q2**2*q3**2*q4**3 - 40*p*q1**2*q2**2*q3**2*q4**2*q5 + 360*p*q1**2*q2**2*q3**2*q4**2 - 40*p*q1**2*q2**2*q3**2*q4*q5**2 + 360*p*q1**2*q2**2*q3**2*q4*q5 - 1440*p*q1**2*q2**2*q3**2*q4 - 40*p*q1**2*q2**2*q3*q4**4 - 40*p*q1**2*q2**2*q3*q4**3*q5 + 360*p*q1**2*q2**2*q3*q4**3 - 40*p*q1**2*q2**2*q3*q4**2*q5**2 + 360*p*q1**2*q2**2*q3*q4**2*q5 - 1440*p*q1**2*q2**2*q3*q4**2 - 40*p*q1**2*q2**2*q3*q4*q5**3 + 360*p*q1**2*q2**2*q3*q4*q5**2 - 1440*p*q1**2*q2**2*q3*q4*q5 + 3360*p*q1**2*q2**2*q3*q4 - 40*p*q1**2*q2*q3**5*q4 - 40*p*q1**2*q2*q3**4*q4**2 - 40*p*q1**2*q2*q3**4*q4*q5 + 360*p*q1**2*q2*q3**4*q4 - 40*p*q1**2*q2*q3**3*q4**3 - 40*p*q1**2*q2*q3**3*q4**2*q5 + 360*p*q1**2*q2*q3**3*q4**2 - 40*p*q1**2*q2*q3**3*q4*q5**2 + 360*p*q1**2*q2*q3**3*q4*q5 - 1440*p*q1**2*q2*q3**3*q4 - 40*p*q1**2*q2*q3**2*q4**4 - 40*p*q1**2*q2*q3**2*q4**3*q5 + 360*p*q1**2*q2*q3**2*q4**3 - 40*p*q1**2*q2*q3**2*q4**2*q5**2 + 360*p*q1**2*q2*q3**2*q4**2*q5 - 1440*p*q1**2*q2*q3**2*q4**2 - 40*p*q1**2*q2*q3**2*q4*q5**3 + 360*p*q1**2*q2*q3**2*q4*q5**2 - 1440*p*q1**2*q2*q3**2*q4*q5 + 3360*p*q1**2*q2*q3**2*q4 - 40*p*q1**2*q2*q3*q4**5 - 40*p*q1**2*q2*q3*q4**4*q5 + 360*p*q1**2*q2*q3*q4**4 - 40*p*q1**2*q2*q3*q4**3*q5**2 + 360*p*q1**2*q2*q3*q4**3*q5 - 1440*p*q1**2*q2*q3*q4**3 - 40*p*q1**2*q2*q3*q4**2*q5**3 + 360*p*q1**2*q2*q3*q4**2*q5**2 - 1440*p*q1**2*q2*q3*q4**2*q5 + 3360*p*q1**2*q2*q3*q4**2 - 40*p*q1**2*q2*q3*q4*q5**4 + 360*p*q1**2*q2*q3*q4*q5**3 - 1440*p*q1**2*q2*q3*q4*q5**2 + 3360*p*q1**2*q2*q3*q4*q5 - 5040*p*q1**2*q2*q3*q4 - 40*p*q1*q2**6*q3*q4 - 40*p*q1*q2**5*q3**2*q4 - 40*p*q1*q2**5*q3*q4**2 - 40*p*q1*q2**5*q3*q4*q5 + 360*p*q1*q2**5*q3*q4 - 40*p*q1*q2**4*q3**3*q4 - 40*p*q1*q2**4*q3**2*q4**2 - 40*p*q1*q2**4*q3**2*q4*q5 + 360*p*q1*q2**4*q3**2*q4 - 40*p*q1*q2**4*q3*q4**3 - 40*p*q1*q2**4*q3*q4**2*q5 + 360*p*q1*q2**4*q3*q4**2 - 40*p*q1*q2**4*q3*q4*q5**2 + 360*p*q1*q2**4*q3*q4*q5 - 1440*p*q1*q2**4*q3*q4 - 40*p*q1*q2**3*q3**4*q4 - 40*p*q1*q2**3*q3**3*q4**2 - 40*p*q1*q2**3*q3**3*q4*q5 + 360*p*q1*q2**3*q3**3*q4 - 40*p*q1*q2**3*q3**2*q4**3 - 40*p*q1*q2**3*q3**2*q4**2*q5 + 360*p*q1*q2**3*q3**2*q4**2 - 40*p*q1*q2**3*q3**2*q4*q5**2 + 360*p*q1*q2**3*q3**2*q4*q5 - 1440*p*q1*q2**3*q3**2*q4 - 40*p*q1*q2**3*q3*q4**4 - 40*p*q1*q2**3*q3*q4**3*q5 + 360*p*q1*q2**3*q3*q4**3 - 40*p*q1*q2**3*q3*q4**2*q5**2 + 360*p*q1*q2**3*q3*q4**2*q5 - 1440*p*q1*q2**3*q3*q4**2 - 40*p*q1*q2**3*q3*q4*q5**3 + 360*p*q1*q2**3*q3*q4*q5**2 - 1440*p*q1*q2**3*q3*q4*q5 + 3360*p*q1*q2**3*q3*q4 - 40*p*q1*q2**2*q3**5*q4 - 40*p*q1*q2**2*q3**4*q4**2 - 40*p*q1*q2**2*q3**4*q4*q5 + 360*p*q1*q2**2*q3**4*q4 - 40*p*q1*q2**2*q3**3*q4**3 - 40*p*q1*q2**2*q3**3*q4**2*q5 + 360*p*q1*q2**2*q3**3*q4**2 - 40*p*q1*q2**2*q3**3*q4*q5**2 + 360*p*q1*q2**2*q3**3*q4*q5 - 1440*p*q1*q2**2*q3**3*q4 - 40*p*q1*q2**2*q3**2*q4**4 - 40*p*q1*q2**2*q3**2*q4**3*q5 + 360*p*q1*q2**2*q3**2*q4**3 - 40*p*q1*q2**2*q3**2*q4**2*q5**2 + 360*p*q1*q2**2*q3**2*q4**2*q5 - 1440*p*q1*q2**2*q3**2*q4**2 - 40*p*q1*q2**2*q3**2*q4*q5**3 + 360*p*q1*q2**2*q3**2*q4*q5**2 - 1440*p*q1*q2**2*q3**2*q4*q5 + 3360*p*q1*q2**2*q3**2*q4 - 40*p*q1*q2**2*q3*q4**5 - 40*p*q1*q2**2*q3*q4**4*q5 + 360*p*q1*q2**2*q3*q4**4 - 40*p*q1*q2**2*q3*q4**3*q5**2 + 360*p*q1*q2**2*q3*q4**3*q5 - 1440*p*q1*q2**2*q3*q4**3 - 40*p*q1*q2**2*q3*q4**2*q5**3 + 360*p*q1*q2**2*q3*q4**2*q5**2 - 1440*p*q1*q2**2*q3*q4**2*q5 + 3360*p*q1*q2**2*q3*q4**2 - 40*p*q1*q2**2*q3*q4*q5**4 + 360*p*q1*q2**2*q3*q4*q5**3 - 1440*p*q1*q2**2*q3*q4*q5**2 + 3360*p*q1*q2**2*q3*q4*q5 - 5040*p*q1*q2**2*q3*q4 - 40*p*q1*q2*q3**6*q4 - 40*p*q1*q2*q3**5*q4**2 - 40*p*q1*q2*q3**5*q4*q5 + 360*p*q1*q2*q3**5*q4 - 40*p*q1*q2*q3**4*q4**3 - 40*p*q1*q2*q3**4*q4**2*q5 + 360*p*q1*q2*q3**4*q4**2 - 40*p*q1*q2*q3**4*q4*q5**2 + 360*p*q1*q2*q3**4*q4*q5 - 1440*p*q1*q2*q3**4*q4 - 40*p*q1*q2*q3**3*q4**4 - 40*p*q1*q2*q3**3*q4**3*q5 + 360*p*q1*q2*q3**3*q4**3 - 40*p*q1*q2*q3**3*q4**2*q5**2 + 360*p*q1*q2*q3**3*q4**2*q5 - 1440*p*q1*q2*q3**3*q4**2 - 40*p*q1*q2*q3**3*q4*q5**3 + 360*p*q1*q2*q3**3*q4*q5**2 - 1440*p*q1*q2*q3**3*q4*q5 + 3360*p*q1*q2*q3**3*q4 - 40*p*q1*q2*q3**2*q4**5 - 40*p*q1*q2*q3**2*q4**4*q5 + 360*p*q1*q2*q3**2*q4**4 - 40*p*q1*q2*q3**2*q4**3*q5**2 + 360*p*q1*q2*q3**2*q4**3*q5 - 1440*p*q1*q2*q3**2*q4**3 - 40*p*q1*q2*q3**2*q4**2*q5**3 + 360*p*q1*q2*q3**2*q4**2*q5**2 - 1440*p*q1*q2*q3**2*q4**2*q5 + 3360*p*q1*q2*q3**2*q4**2 - 40*p*q1*q2*q3**2*q4*q5**4 + 360*p*q1*q2*q3**2*q4*q5**3 - 1440*p*q1*q2*q3**2*q4*q5**2 + 3360*p*q1*q2*q3**2*q4*q5 - 5040*p*q1*q2*q3**2*q4 - 40*p*q1*q2*q3*q4**6 - 40*p*q1*q2*q3*q4**5*q5 + 360*p*q1*q2*q3*q4**5 - 40*p*q1*q2*q3*q4**4*q5**2 + 360*p*q1*q2*q3*q4**4*q5 - 1440*p*q1*q2*q3*q4**4 - 40*p*q1*q2*q3*q4**3*q5**3 + 360*p*q1*q2*q3*q4**3*q5**2 - 1440*p*q1*q2*q3*q4**3*q5 + 3360*p*q1*q2*q3*q4**3 - 40*p*q1*q2*q3*q4**2*q5**4 + 360*p*q1*q2*q3*q4**2*q5**3 - 1440*p*q1*q2*q3*q4**2*q5**2 + 3360*p*q1*q2*q3*q4**2*q5 - 5040*p*q1*q2*q3*q4**2 - 40*p*q1*q2*q3*q4*q5**5 + 360*p*q1*q2*q3*q4*q5**4 - 1440*p*q1*q2*q3*q4*q5**3 + 3360*p*q1*q2*q3*q4*q5**2 - 5040*p*q1*q2*q3*q4*q5 + 5040*p*q1*q2*q3*q4 + 45*p*q2**6*q3*q4 + 45*p*q2**5*q3**2*q4 + 45*p*q2**5*q3*q4**2 + 45*p*q2**5*q3*q4*q5 - 360*p*q2**5*q3*q4 + 45*p*q2**4*q3**3*q4 + 45*p*q2**4*q3**2*q4**2 + 45*p*q2**4*q3**2*q4*q5 - 360*p*q2**4*q3**2*q4 + 45*p*q2**4*q3*q4**3 + 45*p*q2**4*q3*q4**2*q5 - 360*p*q2**4*q3*q4**2 + 45*p*q2**4*q3*q4*q5**2 - 360*p*q2**4*q3*q4*q5 + 1260*p*q2**4*q3*q4 + 45*p*q2**3*q3**4*q4 + 45*p*q2**3*q3**3*q4**2 + 45*p*q2**3*q3**3*q4*q5 - 360*p*q2**3*q3**3*q4 + 45*p*q2**3*q3**2*q4**3 + 45*p*q2**3*q3**2*q4**2*q5 - 360*p*q2**3*q3**2*q4**2 + 45*p*q2**3*q3**2*q4*q5**2 - 360*p*q2**3*q3**2*q4*q5 + 1260*p*q2**3*q3**2*q4 + 45*p*q2**3*q3*q4**4 + 45*p*q2**3*q3*q4**3*q5 - 360*p*q2**3*q3*q4**3 + 45*p*q2**3*q3*q4**2*q5**2 - 360*p*q2**3*q3*q4**2*q5 + 1260*p*q2**3*q3*q4**2 + 45*p*q2**3*q3*q4*q5**3 - 360*p*q2**3*q3*q4*q5**2 + 1260*p*q2**3*q3*q4*q5 - 2520*p*q2**3*q3*q4 + 45*p*q2**2*q3**5*q4 + 45*p*q2**2*q3**4*q4**2 + 45*p*q2**2*q3**4*q4*q5 - 360*p*q2**2*q3**4*q4 + 45*p*q2**2*q3**3*q4**3 + 45*p*q2**2*q3**3*q4**2*q5 - 360*p*q2**2*q3**3*q4**2 + 45*p*q2**2*q3**3*q4*q5**2 - 360*p*q2**2*q3**3*q4*q5 + 1260*p*q2**2*q3**3*q4 + 45*p*q2**2*q3**2*q4**4 + 45*p*q2**2*q3**2*q4**3*q5 - 360*p*q2**2*q3**2*q4**3 + 45*p*q2**2*q3**2*q4**2*q5**2 - 360*p*q2**2*q3**2*q4**2*q5 + 1260*p*q2**2*q3**2*q4**2 + 45*p*q2**2*q3**2*q4*q5**3 - 360*p*q2**2*q3**2*q4*q5**2 + 1260*p*q2**2*q3**2*q4*q5 - 2520*p*q2**2*q3**2*q4 + 45*p*q2**2*q3*q4**5 + 45*p*q2**2*q3*q4**4*q5 - 360*p*q2**2*q3*q4**4 + 45*p*q2**2*q3*q4**3*q5**2 - 360*p*q2**2*q3*q4**3*q5 + 1260*p*q2**2*q3*q4**3 + 45*p*q2**2*q3*q4**2*q5**3 - 360*p*q2**2*q3*q4**2*q5**2 + 1260*p*q2**2*q3*q4**2*q5 - 2520*p*q2**2*q3*q4**2 + 45*p*q2**2*q3*q4*q5**4 - 360*p*q2**2*q3*q4*q5**3 + 1260*p*q2**2*q3*q4*q5**2 - 2520*p*q2**2*q3*q4*q5 + 3150*p*q2**2*q3*q4 + 45*p*q2*q3**6*q4 + 45*p*q2*q3**5*q4**2 + 45*p*q2*q3**5*q4*q5 - 360*p*q2*q3**5*q4 + 45*p*q2*q3**4*q4**3 + 45*p*q2*q3**4*q4**2*q5 - 360*p*q2*q3**4*q4**2 + 45*p*q2*q3**4*q4*q5**2 - 360*p*q2*q3**4*q4*q5 + 1260*p*q2*q3**4*q4 + 45*p*q2*q3**3*q4**4 + 45*p*q2*q3**3*q4**3*q5 - 360*p*q2*q3**3*q4**3 + 45*p*q2*q3**3*q4**2*q5**2 - 360*p*q2*q3**3*q4**2*q5 + 1260*p*q2*q3**3*q4**2 + 45*p*q2*q3**3*q4*q5**3 - 360*p*q2*q3**3*q4*q5**2 + 1260*p*q2*q3**3*q4*q5 - 2520*p*q2*q3**3*q4 + 45*p*q2*q3**2*q4**5 + 45*p*q2*q3**2*q4**4*q5 - 360*p*q2*q3**2*q4**4 + 45*p*q2*q3**2*q4**3*q5**2 - 360*p*q2*q3**2*q4**3*q5 + 1260*p*q2*q3**2*q4**3 + 45*p*q2*q3**2*q4**2*q5**3 - 360*p*q2*q3**2*q4**2*q5**2 + 1260*p*q2*q3**2*q4**2*q5 - 2520*p*q2*q3**2*q4**2 + 45*p*q2*q3**2*q4*q5**4 - 360*p*q2*q3**2*q4*q5**3 + 1260*p*q2*q3**2*q4*q5**2 - 2520*p*q2*q3**2*q4*q5 + 3150*p*q2*q3**2*q4 + 45*p*q2*q3*q4**6 + 45*p*q2*q3*q4**5*q5 - 360*p*q2*q3*q4**5 + 45*p*q2*q3*q4**4*q5**2 - 360*p*q2*q3*q4**4*q5 + 1260*p*q2*q3*q4**4 + 45*p*q2*q3*q4**3*q5**3 - 360*p*q2*q3*q4**3*q5**2 + 1260*p*q2*q3*q4**3*q5 - 2520*p*q2*q3*q4**3 + 45*p*q2*q3*q4**2*q5**4 - 360*p*q2*q3*q4**2*q5**3 + 1260*p*q2*q3*q4**2*q5**2 - 2520*p*q2*q3*q4**2*q5 + 3150*p*q2*q3*q4**2 + 45*p*q2*q3*q4*q5**5 - 360*p*q2*q3*q4*q5**4 + 1260*p*q2*q3*q4*q5**3 - 2520*p*q2*q3*q4*q5**2 + 3150*p*q2*q3*q4*q5 - 2520*p*q2*q3*q4 + 10*q1**6*q2*q3*q4 + 10*q1**5*q2**2*q3*q4 + 10*q1**5*q2*q3**2*q4 + 10*q1**5*q2*q3*q4**2 + 10*q1**5*q2*q3*q4*q5 - 90*q1**5*q2*q3*q4 + 10*q1**4*q2**3*q3*q4 + 10*q1**4*q2**2*q3**2*q4 + 10*q1**4*q2**2*q3*q4**2 + 10*q1**4*q2**2*q3*q4*q5 - 90*q1**4*q2**2*q3*q4 + 10*q1**4*q2*q3**3*q4 + 10*q1**4*q2*q3**2*q4**2 + 10*q1**4*q2*q3**2*q4*q5 - 90*q1**4*q2*q3**2*q4 + 10*q1**4*q2*q3*q4**3 + 10*q1**4*q2*q3*q4**2*q5 - 90*q1**4*q2*q3*q4**2 + 10*q1**4*q2*q3*q4*q5**2 - 90*q1**4*q2*q3*q4*q5 + 360*q1**4*q2*q3*q4 + 10*q1**3*q2**4*q3*q4 + 10*q1**3*q2**3*q3**2*q4 + 10*q1**3*q2**3*q3*q4**2 + 10*q1**3*q2**3*q3*q4*q5 - 90*q1**3*q2**3*q3*q4 + 10*q1**3*q2**2*q3**3*q4 + 10*q1**3*q2**2*q3**2*q4**2 + 10*q1**3*q2**2*q3**2*q4*q5 - 90*q1**3*q2**2*q3**2*q4 + 10*q1**3*q2**2*q3*q4**3 + 10*q1**3*q2**2*q3*q4**2*q5 - 90*q1**3*q2**2*q3*q4**2 + 10*q1**3*q2**2*q3*q4*q5**2 - 90*q1**3*q2**2*q3*q4*q5 + 360*q1**3*q2**2*q3*q4 + 10*q1**3*q2*q3**4*q4 + 10*q1**3*q2*q3**3*q4**2 + 10*q1**3*q2*q3**3*q4*q5 - 90*q1**3*q2*q3**3*q4 + 10*q1**3*q2*q3**2*q4**3 + 10*q1**3*q2*q3**2*q4**2*q5 - 90*q1**3*q2*q3**2*q4**2 + 10*q1**3*q2*q3**2*q4*q5**2 - 90*q1**3*q2*q3**2*q4*q5 + 360*q1**3*q2*q3**2*q4 + 10*q1**3*q2*q3*q4**4 + 10*q1**3*q2*q3*q4**3*q5 - 90*q1**3*q2*q3*q4**3 + 10*q1**3*q2*q3*q4**2*q5**2 - 90*q1**3*q2*q3*q4**2*q5 + 360*q1**3*q2*q3*q4**2 + 10*q1**3*q2*q3*q4*q5**3 - 90*q1**3*q2*q3*q4*q5**2 + 360*q1**3*q2*q3*q4*q5 - 840*q1**3*q2*q3*q4 + 10*q1**2*q2**5*q3*q4 + 10*q1**2*q2**4*q3**2*q4 + 10*q1**2*q2**4*q3*q4**2 + 10*q1**2*q2**4*q3*q4*q5 - 90*q1**2*q2**4*q3*q4 + 10*q1**2*q2**3*q3**3*q4 + 10*q1**2*q2**3*q3**2*q4**2 + 10*q1**2*q2**3*q3**2*q4*q5 - 90*q1**2*q2**3*q3**2*q4 + 10*q1**2*q2**3*q3*q4**3 + 10*q1**2*q2**3*q3*q4**2*q5 - 90*q1**2*q2**3*q3*q4**2 + 10*q1**2*q2**3*q3*q4*q5**2 - 90*q1**2*q2**3*q3*q4*q5 + 360*q1**2*q2**3*q3*q4 + 10*q1**2*q2**2*q3**4*q4 + 10*q1**2*q2**2*q3**3*q4**2 + 10*q1**2*q2**2*q3**3*q4*q5 - 90*q1**2*q2**2*q3**3*q4 + 10*q1**2*q2**2*q3**2*q4**3 + 10*q1**2*q2**2*q3**2*q4**2*q5 - 90*q1**2*q2**2*q3**2*q4**2 + 10*q1**2*q2**2*q3**2*q4*q5**2 - 90*q1**2*q2**2*q3**2*q4*q5 + 360*q1**2*q2**2*q3**2*q4 + 10*q1**2*q2**2*q3*q4**4 + 10*q1**2*q2**2*q3*q4**3*q5 - 90*q1**2*q2**2*q3*q4**3 + 10*q1**2*q2**2*q3*q4**2*q5**2 - 90*q1**2*q2**2*q3*q4**2*q5 + 360*q1**2*q2**2*q3*q4**2 + 10*q1**2*q2**2*q3*q4*q5**3 - 90*q1**2*q2**2*q3*q4*q5**2 + 360*q1**2*q2**2*q3*q4*q5 - 840*q1**2*q2**2*q3*q4 + 10*q1**2*q2*q3**5*q4 + 10*q1**2*q2*q3**4*q4**2 + 10*q1**2*q2*q3**4*q4*q5 - 90*q1**2*q2*q3**4*q4 + 10*q1**2*q2*q3**3*q4**3 + 10*q1**2*q2*q3**3*q4**2*q5 - 90*q1**2*q2*q3**3*q4**2 + 10*q1**2*q2*q3**3*q4*q5**2 - 90*q1**2*q2*q3**3*q4*q5 + 360*q1**2*q2*q3**3*q4 + 10*q1**2*q2*q3**2*q4**4 + 10*q1**2*q2*q3**2*q4**3*q5 - 90*q1**2*q2*q3**2*q4**3 + 10*q1**2*q2*q3**2*q4**2*q5**2 - 90*q1**2*q2*q3**2*q4**2*q5 + 360*q1**2*q2*q3**2*q4**2 + 10*q1**2*q2*q3**2*q4*q5**3 - 90*q1**2*q2*q3**2*q4*q5**2 + 360*q1**2*q2*q3**2*q4*q5 - 840*q1**2*q2*q3**2*q4 + 10*q1**2*q2*q3*q4**5 + 10*q1**2*q2*q3*q4**4*q5 - 90*q1**2*q2*q3*q4**4 + 10*q1**2*q2*q3*q4**3*q5**2 - 90*q1**2*q2*q3*q4**3*q5 + 360*q1**2*q2*q3*q4**3 + 10*q1**2*q2*q3*q4**2*q5**3 - 90*q1**2*q2*q3*q4**2*q5**2 + 360*q1**2*q2*q3*q4**2*q5 - 840*q1**2*q2*q3*q4**2 + 10*q1**2*q2*q3*q4*q5**4 - 90*q1**2*q2*q3*q4*q5**3 + 360*q1**2*q2*q3*q4*q5**2 - 840*q1**2*q2*q3*q4*q5 + 1260*q1**2*q2*q3*q4 + 10*q1*q2**6*q3*q4 + 10*q1*q2**5*q3**2*q4 + 10*q1*q2**5*q3*q4**2 + 10*q1*q2**5*q3*q4*q5 - 90*q1*q2**5*q3*q4 + 10*q1*q2**4*q3**3*q4 + 10*q1*q2**4*q3**2*q4**2 + 10*q1*q2**4*q3**2*q4*q5 - 90*q1*q2**4*q3**2*q4 + 10*q1*q2**4*q3*q4**3 + 10*q1*q2**4*q3*q4**2*q5 - 90*q1*q2**4*q3*q4**2 + 10*q1*q2**4*q3*q4*q5**2 - 90*q1*q2**4*q3*q4*q5 + 360*q1*q2**4*q3*q4 + 10*q1*q2**3*q3**4*q4 + 10*q1*q2**3*q3**3*q4**2 + 10*q1*q2**3*q3**3*q4*q5 - 90*q1*q2**3*q3**3*q4 + 10*q1*q2**3*q3**2*q4**3 + 10*q1*q2**3*q3**2*q4**2*q5 - 90*q1*q2**3*q3**2*q4**2 + 10*q1*q2**3*q3**2*q4*q5**2 - 90*q1*q2**3*q3**2*q4*q5 + 360*q1*q2**3*q3**2*q4 + 10*q1*q2**3*q3*q4**4 + 10*q1*q2**3*q3*q4**3*q5 - 90*q1*q2**3*q3*q4**3 + 10*q1*q2**3*q3*q4**2*q5**2 - 90*q1*q2**3*q3*q4**2*q5 + 360*q1*q2**3*q3*q4**2 + 10*q1*q2**3*q3*q4*q5**3 - 90*q1*q2**3*q3*q4*q5**2 + 360*q1*q2**3*q3*q4*q5 - 840*q1*q2**3*q3*q4 + 10*q1*q2**2*q3**5*q4 + 10*q1*q2**2*q3**4*q4**2 + 10*q1*q2**2*q3**4*q4*q5 - 90*q1*q2**2*q3**4*q4 + 10*q1*q2**2*q3**3*q4**3 + 10*q1*q2**2*q3**3*q4**2*q5 - 90*q1*q2**2*q3**3*q4**2 + 10*q1*q2**2*q3**3*q4*q5**2 - 90*q1*q2**2*q3**3*q4*q5 + 360*q1*q2**2*q3**3*q4 + 10*q1*q2**2*q3**2*q4**4 + 10*q1*q2**2*q3**2*q4**3*q5 - 90*q1*q2**2*q3**2*q4**3 + 10*q1*q2**2*q3**2*q4**2*q5**2 - 90*q1*q2**2*q3**2*q4**2*q5 + 360*q1*q2**2*q3**2*q4**2 + 10*q1*q2**2*q3**2*q4*q5**3 - 90*q1*q2**2*q3**2*q4*q5**2 + 360*q1*q2**2*q3**2*q4*q5 - 840*q1*q2**2*q3**2*q4 + 10*q1*q2**2*q3*q4**5 + 10*q1*q2**2*q3*q4**4*q5 - 90*q1*q2**2*q3*q4**4 + 10*q1*q2**2*q3*q4**3*q5**2 - 90*q1*q2**2*q3*q4**3*q5 + 360*q1*q2**2*q3*q4**3 + 10*q1*q2**2*q3*q4**2*q5**3 - 90*q1*q2**2*q3*q4**2*q5**2 + 360*q1*q2**2*q3*q4**2*q5 - 840*q1*q2**2*q3*q4**2 + 10*q1*q2**2*q3*q4*q5**4 - 90*q1*q2**2*q3*q4*q5**3 + 360*q1*q2**2*q3*q4*q5**2 - 840*q1*q2**2*q3*q4*q5 + 1260*q1*q2**2*q3*q4 + 10*q1*q2*q3**6*q4 + 10*q1*q2*q3**5*q4**2 + 10*q1*q2*q3**5*q4*q5 - 90*q1*q2*q3**5*q4 + 10*q1*q2*q3**4*q4**3 + 10*q1*q2*q3**4*q4**2*q5 - 90*q1*q2*q3**4*q4**2 + 10*q1*q2*q3**4*q4*q5**2 - 90*q1*q2*q3**4*q4*q5 + 360*q1*q2*q3**4*q4 + 10*q1*q2*q3**3*q4**4 + 10*q1*q2*q3**3*q4**3*q5 - 90*q1*q2*q3**3*q4**3 + 10*q1*q2*q3**3*q4**2*q5**2 - 90*q1*q2*q3**3*q4**2*q5 + 360*q1*q2*q3**3*q4**2 + 10*q1*q2*q3**3*q4*q5**3 - 90*q1*q2*q3**3*q4*q5**2 + 360*q1*q2*q3**3*q4*q5 - 840*q1*q2*q3**3*q4 + 10*q1*q2*q3**2*q4**5 + 10*q1*q2*q3**2*q4**4*q5 - 90*q1*q2*q3**2*q4**4 + 10*q1*q2*q3**2*q4**3*q5**2 - 90*q1*q2*q3**2*q4**3*q5 + 360*q1*q2*q3**2*q4**3 + 10*q1*q2*q3**2*q4**2*q5**3 - 90*q1*q2*q3**2*q4**2*q5**2 + 360*q1*q2*q3**2*q4**2*q5 - 840*q1*q2*q3**2*q4**2 + 10*q1*q2*q3**2*q4*q5**4 - 90*q1*q2*q3**2*q4*q5**3 + 360*q1*q2*q3**2*q4*q5**2 - 840*q1*q2*q3**2*q4*q5 + 1260*q1*q2*q3**2*q4 + 10*q1*q2*q3*q4**6 + 10*q1*q2*q3*q4**5*q5 - 90*q1*q2*q3*q4**5 + 10*q1*q2*q3*q4**4*q5**2 - 90*q1*q2*q3*q4**4*q5 + 360*q1*q2*q3*q4**4 + 10*q1*q2*q3*q4**3*q5**3 - 90*q1*q2*q3*q4**3*q5**2 + 360*q1*q2*q3*q4**3*q5 - 840*q1*q2*q3*q4**3 + 10*q1*q2*q3*q4**2*q5**4 - 90*q1*q2*q3*q4**2*q5**3 + 360*q1*q2*q3*q4**2*q5**2 - 840*q1*q2*q3*q4**2*q5 + 1260*q1*q2*q3*q4**2 + 10*q1*q2*q3*q4*q5**5 - 90*q1*q2*q3*q4*q5**4 + 360*q1*q2*q3*q4*q5**3 - 840*q1*q2*q3*q4*q5**2 + 1260*q1*q2*q3*q4*q5 - 1260*q1*q2*q3*q4)'
f_multiparam_semisyn[10][6] = '-p*(p - 1)**4*(10*p**5*q1**5*q2*q3*q4*q5 + 10*p**5*q1**4*q2**2*q3*q4*q5 + 10*p**5*q1**4*q2*q3**2*q4*q5 + 10*p**5*q1**4*q2*q3*q4**2*q5 + 10*p**5*q1**4*q2*q3*q4*q5**2 + 10*p**5*q1**4*q2*q3*q4*q5*q6 - 90*p**5*q1**4*q2*q3*q4*q5 + 10*p**5*q1**3*q2**3*q3*q4*q5 + 10*p**5*q1**3*q2**2*q3**2*q4*q5 + 10*p**5*q1**3*q2**2*q3*q4**2*q5 + 10*p**5*q1**3*q2**2*q3*q4*q5**2 + 10*p**5*q1**3*q2**2*q3*q4*q5*q6 - 90*p**5*q1**3*q2**2*q3*q4*q5 + 10*p**5*q1**3*q2*q3**3*q4*q5 + 10*p**5*q1**3*q2*q3**2*q4**2*q5 + 10*p**5*q1**3*q2*q3**2*q4*q5**2 + 10*p**5*q1**3*q2*q3**2*q4*q5*q6 - 90*p**5*q1**3*q2*q3**2*q4*q5 + 10*p**5*q1**3*q2*q3*q4**3*q5 + 10*p**5*q1**3*q2*q3*q4**2*q5**2 + 10*p**5*q1**3*q2*q3*q4**2*q5*q6 - 90*p**5*q1**3*q2*q3*q4**2*q5 + 10*p**5*q1**3*q2*q3*q4*q5**3 + 10*p**5*q1**3*q2*q3*q4*q5**2*q6 - 90*p**5*q1**3*q2*q3*q4*q5**2 + 10*p**5*q1**3*q2*q3*q4*q5*q6**2 - 90*p**5*q1**3*q2*q3*q4*q5*q6 + 360*p**5*q1**3*q2*q3*q4*q5 + 10*p**5*q1**2*q2**4*q3*q4*q5 + 10*p**5*q1**2*q2**3*q3**2*q4*q5 + 10*p**5*q1**2*q2**3*q3*q4**2*q5 + 10*p**5*q1**2*q2**3*q3*q4*q5**2 + 10*p**5*q1**2*q2**3*q3*q4*q5*q6 - 90*p**5*q1**2*q2**3*q3*q4*q5 + 10*p**5*q1**2*q2**2*q3**3*q4*q5 + 10*p**5*q1**2*q2**2*q3**2*q4**2*q5 + 10*p**5*q1**2*q2**2*q3**2*q4*q5**2 + 10*p**5*q1**2*q2**2*q3**2*q4*q5*q6 - 90*p**5*q1**2*q2**2*q3**2*q4*q5 + 10*p**5*q1**2*q2**2*q3*q4**3*q5 + 10*p**5*q1**2*q2**2*q3*q4**2*q5**2 + 10*p**5*q1**2*q2**2*q3*q4**2*q5*q6 - 90*p**5*q1**2*q2**2*q3*q4**2*q5 + 10*p**5*q1**2*q2**2*q3*q4*q5**3 + 10*p**5*q1**2*q2**2*q3*q4*q5**2*q6 - 90*p**5*q1**2*q2**2*q3*q4*q5**2 + 10*p**5*q1**2*q2**2*q3*q4*q5*q6**2 - 90*p**5*q1**2*q2**2*q3*q4*q5*q6 + 360*p**5*q1**2*q2**2*q3*q4*q5 + 10*p**5*q1**2*q2*q3**4*q4*q5 + 10*p**5*q1**2*q2*q3**3*q4**2*q5 + 10*p**5*q1**2*q2*q3**3*q4*q5**2 + 10*p**5*q1**2*q2*q3**3*q4*q5*q6 - 90*p**5*q1**2*q2*q3**3*q4*q5 + 10*p**5*q1**2*q2*q3**2*q4**3*q5 + 10*p**5*q1**2*q2*q3**2*q4**2*q5**2 + 10*p**5*q1**2*q2*q3**2*q4**2*q5*q6 - 90*p**5*q1**2*q2*q3**2*q4**2*q5 + 10*p**5*q1**2*q2*q3**2*q4*q5**3 + 10*p**5*q1**2*q2*q3**2*q4*q5**2*q6 - 90*p**5*q1**2*q2*q3**2*q4*q5**2 + 10*p**5*q1**2*q2*q3**2*q4*q5*q6**2 - 90*p**5*q1**2*q2*q3**2*q4*q5*q6 + 360*p**5*q1**2*q2*q3**2*q4*q5 + 10*p**5*q1**2*q2*q3*q4**4*q5 + 10*p**5*q1**2*q2*q3*q4**3*q5**2 + 10*p**5*q1**2*q2*q3*q4**3*q5*q6 - 90*p**5*q1**2*q2*q3*q4**3*q5 + 10*p**5*q1**2*q2*q3*q4**2*q5**3 + 10*p**5*q1**2*q2*q3*q4**2*q5**2*q6 - 90*p**5*q1**2*q2*q3*q4**2*q5**2 + 10*p**5*q1**2*q2*q3*q4**2*q5*q6**2 - 90*p**5*q1**2*q2*q3*q4**2*q5*q6 + 360*p**5*q1**2*q2*q3*q4**2*q5 + 10*p**5*q1**2*q2*q3*q4*q5**4 + 10*p**5*q1**2*q2*q3*q4*q5**3*q6 - 90*p**5*q1**2*q2*q3*q4*q5**3 + 10*p**5*q1**2*q2*q3*q4*q5**2*q6**2 - 90*p**5*q1**2*q2*q3*q4*q5**2*q6 + 360*p**5*q1**2*q2*q3*q4*q5**2 + 10*p**5*q1**2*q2*q3*q4*q5*q6**3 - 90*p**5*q1**2*q2*q3*q4*q5*q6**2 + 360*p**5*q1**2*q2*q3*q4*q5*q6 - 840*p**5*q1**2*q2*q3*q4*q5 + 10*p**5*q1*q2**5*q3*q4*q5 + 10*p**5*q1*q2**4*q3**2*q4*q5 + 10*p**5*q1*q2**4*q3*q4**2*q5 + 10*p**5*q1*q2**4*q3*q4*q5**2 + 10*p**5*q1*q2**4*q3*q4*q5*q6 - 90*p**5*q1*q2**4*q3*q4*q5 + 10*p**5*q1*q2**3*q3**3*q4*q5 + 10*p**5*q1*q2**3*q3**2*q4**2*q5 + 10*p**5*q1*q2**3*q3**2*q4*q5**2 + 10*p**5*q1*q2**3*q3**2*q4*q5*q6 - 90*p**5*q1*q2**3*q3**2*q4*q5 + 10*p**5*q1*q2**3*q3*q4**3*q5 + 10*p**5*q1*q2**3*q3*q4**2*q5**2 + 10*p**5*q1*q2**3*q3*q4**2*q5*q6 - 90*p**5*q1*q2**3*q3*q4**2*q5 + 10*p**5*q1*q2**3*q3*q4*q5**3 + 10*p**5*q1*q2**3*q3*q4*q5**2*q6 - 90*p**5*q1*q2**3*q3*q4*q5**2 + 10*p**5*q1*q2**3*q3*q4*q5*q6**2 - 90*p**5*q1*q2**3*q3*q4*q5*q6 + 360*p**5*q1*q2**3*q3*q4*q5 + 10*p**5*q1*q2**2*q3**4*q4*q5 + 10*p**5*q1*q2**2*q3**3*q4**2*q5 + 10*p**5*q1*q2**2*q3**3*q4*q5**2 + 10*p**5*q1*q2**2*q3**3*q4*q5*q6 - 90*p**5*q1*q2**2*q3**3*q4*q5 + 10*p**5*q1*q2**2*q3**2*q4**3*q5 + 10*p**5*q1*q2**2*q3**2*q4**2*q5**2 + 10*p**5*q1*q2**2*q3**2*q4**2*q5*q6 - 90*p**5*q1*q2**2*q3**2*q4**2*q5 + 10*p**5*q1*q2**2*q3**2*q4*q5**3 + 10*p**5*q1*q2**2*q3**2*q4*q5**2*q6 - 90*p**5*q1*q2**2*q3**2*q4*q5**2 + 10*p**5*q1*q2**2*q3**2*q4*q5*q6**2 - 90*p**5*q1*q2**2*q3**2*q4*q5*q6 + 360*p**5*q1*q2**2*q3**2*q4*q5 + 10*p**5*q1*q2**2*q3*q4**4*q5 + 10*p**5*q1*q2**2*q3*q4**3*q5**2 + 10*p**5*q1*q2**2*q3*q4**3*q5*q6 - 90*p**5*q1*q2**2*q3*q4**3*q5 + 10*p**5*q1*q2**2*q3*q4**2*q5**3 + 10*p**5*q1*q2**2*q3*q4**2*q5**2*q6 - 90*p**5*q1*q2**2*q3*q4**2*q5**2 + 10*p**5*q1*q2**2*q3*q4**2*q5*q6**2 - 90*p**5*q1*q2**2*q3*q4**2*q5*q6 + 360*p**5*q1*q2**2*q3*q4**2*q5 + 10*p**5*q1*q2**2*q3*q4*q5**4 + 10*p**5*q1*q2**2*q3*q4*q5**3*q6 - 90*p**5*q1*q2**2*q3*q4*q5**3 + 10*p**5*q1*q2**2*q3*q4*q5**2*q6**2 - 90*p**5*q1*q2**2*q3*q4*q5**2*q6 + 360*p**5*q1*q2**2*q3*q4*q5**2 + 10*p**5*q1*q2**2*q3*q4*q5*q6**3 - 90*p**5*q1*q2**2*q3*q4*q5*q6**2 + 360*p**5*q1*q2**2*q3*q4*q5*q6 - 840*p**5*q1*q2**2*q3*q4*q5 + 10*p**5*q1*q2*q3**5*q4*q5 + 10*p**5*q1*q2*q3**4*q4**2*q5 + 10*p**5*q1*q2*q3**4*q4*q5**2 + 10*p**5*q1*q2*q3**4*q4*q5*q6 - 90*p**5*q1*q2*q3**4*q4*q5 + 10*p**5*q1*q2*q3**3*q4**3*q5 + 10*p**5*q1*q2*q3**3*q4**2*q5**2 + 10*p**5*q1*q2*q3**3*q4**2*q5*q6 - 90*p**5*q1*q2*q3**3*q4**2*q5 + 10*p**5*q1*q2*q3**3*q4*q5**3 + 10*p**5*q1*q2*q3**3*q4*q5**2*q6 - 90*p**5*q1*q2*q3**3*q4*q5**2 + 10*p**5*q1*q2*q3**3*q4*q5*q6**2 - 90*p**5*q1*q2*q3**3*q4*q5*q6 + 360*p**5*q1*q2*q3**3*q4*q5 + 10*p**5*q1*q2*q3**2*q4**4*q5 + 10*p**5*q1*q2*q3**2*q4**3*q5**2 + 10*p**5*q1*q2*q3**2*q4**3*q5*q6 - 90*p**5*q1*q2*q3**2*q4**3*q5 + 10*p**5*q1*q2*q3**2*q4**2*q5**3 + 10*p**5*q1*q2*q3**2*q4**2*q5**2*q6 - 90*p**5*q1*q2*q3**2*q4**2*q5**2 + 10*p**5*q1*q2*q3**2*q4**2*q5*q6**2 - 90*p**5*q1*q2*q3**2*q4**2*q5*q6 + 360*p**5*q1*q2*q3**2*q4**2*q5 + 10*p**5*q1*q2*q3**2*q4*q5**4 + 10*p**5*q1*q2*q3**2*q4*q5**3*q6 - 90*p**5*q1*q2*q3**2*q4*q5**3 + 10*p**5*q1*q2*q3**2*q4*q5**2*q6**2 - 90*p**5*q1*q2*q3**2*q4*q5**2*q6 + 360*p**5*q1*q2*q3**2*q4*q5**2 + 10*p**5*q1*q2*q3**2*q4*q5*q6**3 - 90*p**5*q1*q2*q3**2*q4*q5*q6**2 + 360*p**5*q1*q2*q3**2*q4*q5*q6 - 840*p**5*q1*q2*q3**2*q4*q5 + 10*p**5*q1*q2*q3*q4**5*q5 + 10*p**5*q1*q2*q3*q4**4*q5**2 + 10*p**5*q1*q2*q3*q4**4*q5*q6 - 90*p**5*q1*q2*q3*q4**4*q5 + 10*p**5*q1*q2*q3*q4**3*q5**3 + 10*p**5*q1*q2*q3*q4**3*q5**2*q6 - 90*p**5*q1*q2*q3*q4**3*q5**2 + 10*p**5*q1*q2*q3*q4**3*q5*q6**2 - 90*p**5*q1*q2*q3*q4**3*q5*q6 + 360*p**5*q1*q2*q3*q4**3*q5 + 10*p**5*q1*q2*q3*q4**2*q5**4 + 10*p**5*q1*q2*q3*q4**2*q5**3*q6 - 90*p**5*q1*q2*q3*q4**2*q5**3 + 10*p**5*q1*q2*q3*q4**2*q5**2*q6**2 - 90*p**5*q1*q2*q3*q4**2*q5**2*q6 + 360*p**5*q1*q2*q3*q4**2*q5**2 + 10*p**5*q1*q2*q3*q4**2*q5*q6**3 - 90*p**5*q1*q2*q3*q4**2*q5*q6**2 + 360*p**5*q1*q2*q3*q4**2*q5*q6 - 840*p**5*q1*q2*q3*q4**2*q5 + 10*p**5*q1*q2*q3*q4*q5**5 + 10*p**5*q1*q2*q3*q4*q5**4*q6 - 90*p**5*q1*q2*q3*q4*q5**4 + 10*p**5*q1*q2*q3*q4*q5**3*q6**2 - 90*p**5*q1*q2*q3*q4*q5**3*q6 + 360*p**5*q1*q2*q3*q4*q5**3 + 10*p**5*q1*q2*q3*q4*q5**2*q6**3 - 90*p**5*q1*q2*q3*q4*q5**2*q6**2 + 360*p**5*q1*q2*q3*q4*q5**2*q6 - 840*p**5*q1*q2*q3*q4*q5**2 + 10*p**5*q1*q2*q3*q4*q5*q6**4 - 90*p**5*q1*q2*q3*q4*q5*q6**3 + 360*p**5*q1*q2*q3*q4*q5*q6**2 - 840*p**5*q1*q2*q3*q4*q5*q6 + 1260*p**5*q1*q2*q3*q4*q5 - 45*p**5*q2**5*q3*q4*q5 - 45*p**5*q2**4*q3**2*q4*q5 - 45*p**5*q2**4*q3*q4**2*q5 - 45*p**5*q2**4*q3*q4*q5**2 - 45*p**5*q2**4*q3*q4*q5*q6 + 360*p**5*q2**4*q3*q4*q5 - 45*p**5*q2**3*q3**3*q4*q5 - 45*p**5*q2**3*q3**2*q4**2*q5 - 45*p**5*q2**3*q3**2*q4*q5**2 - 45*p**5*q2**3*q3**2*q4*q5*q6 + 360*p**5*q2**3*q3**2*q4*q5 - 45*p**5*q2**3*q3*q4**3*q5 - 45*p**5*q2**3*q3*q4**2*q5**2 - 45*p**5*q2**3*q3*q4**2*q5*q6 + 360*p**5*q2**3*q3*q4**2*q5 - 45*p**5*q2**3*q3*q4*q5**3 - 45*p**5*q2**3*q3*q4*q5**2*q6 + 360*p**5*q2**3*q3*q4*q5**2 - 45*p**5*q2**3*q3*q4*q5*q6**2 + 360*p**5*q2**3*q3*q4*q5*q6 - 1260*p**5*q2**3*q3*q4*q5 - 45*p**5*q2**2*q3**4*q4*q5 - 45*p**5*q2**2*q3**3*q4**2*q5 - 45*p**5*q2**2*q3**3*q4*q5**2 - 45*p**5*q2**2*q3**3*q4*q5*q6 + 360*p**5*q2**2*q3**3*q4*q5 - 45*p**5*q2**2*q3**2*q4**3*q5 - 45*p**5*q2**2*q3**2*q4**2*q5**2 - 45*p**5*q2**2*q3**2*q4**2*q5*q6 + 360*p**5*q2**2*q3**2*q4**2*q5 - 45*p**5*q2**2*q3**2*q4*q5**3 - 45*p**5*q2**2*q3**2*q4*q5**2*q6 + 360*p**5*q2**2*q3**2*q4*q5**2 - 45*p**5*q2**2*q3**2*q4*q5*q6**2 + 360*p**5*q2**2*q3**2*q4*q5*q6 - 1260*p**5*q2**2*q3**2*q4*q5 - 45*p**5*q2**2*q3*q4**4*q5 - 45*p**5*q2**2*q3*q4**3*q5**2 - 45*p**5*q2**2*q3*q4**3*q5*q6 + 360*p**5*q2**2*q3*q4**3*q5 - 45*p**5*q2**2*q3*q4**2*q5**3 - 45*p**5*q2**2*q3*q4**2*q5**2*q6 + 360*p**5*q2**2*q3*q4**2*q5**2 - 45*p**5*q2**2*q3*q4**2*q5*q6**2 + 360*p**5*q2**2*q3*q4**2*q5*q6 - 1260*p**5*q2**2*q3*q4**2*q5 - 45*p**5*q2**2*q3*q4*q5**4 - 45*p**5*q2**2*q3*q4*q5**3*q6 + 360*p**5*q2**2*q3*q4*q5**3 - 45*p**5*q2**2*q3*q4*q5**2*q6**2 + 360*p**5*q2**2*q3*q4*q5**2*q6 - 1260*p**5*q2**2*q3*q4*q5**2 - 45*p**5*q2**2*q3*q4*q5*q6**3 + 360*p**5*q2**2*q3*q4*q5*q6**2 - 1260*p**5*q2**2*q3*q4*q5*q6 + 2520*p**5*q2**2*q3*q4*q5 - 45*p**5*q2*q3**5*q4*q5 - 45*p**5*q2*q3**4*q4**2*q5 - 45*p**5*q2*q3**4*q4*q5**2 - 45*p**5*q2*q3**4*q4*q5*q6 + 360*p**5*q2*q3**4*q4*q5 - 45*p**5*q2*q3**3*q4**3*q5 - 45*p**5*q2*q3**3*q4**2*q5**2 - 45*p**5*q2*q3**3*q4**2*q5*q6 + 360*p**5*q2*q3**3*q4**2*q5 - 45*p**5*q2*q3**3*q4*q5**3 - 45*p**5*q2*q3**3*q4*q5**2*q6 + 360*p**5*q2*q3**3*q4*q5**2 - 45*p**5*q2*q3**3*q4*q5*q6**2 + 360*p**5*q2*q3**3*q4*q5*q6 - 1260*p**5*q2*q3**3*q4*q5 - 45*p**5*q2*q3**2*q4**4*q5 - 45*p**5*q2*q3**2*q4**3*q5**2 - 45*p**5*q2*q3**2*q4**3*q5*q6 + 360*p**5*q2*q3**2*q4**3*q5 - 45*p**5*q2*q3**2*q4**2*q5**3 - 45*p**5*q2*q3**2*q4**2*q5**2*q6 + 360*p**5*q2*q3**2*q4**2*q5**2 - 45*p**5*q2*q3**2*q4**2*q5*q6**2 + 360*p**5*q2*q3**2*q4**2*q5*q6 - 1260*p**5*q2*q3**2*q4**2*q5 - 45*p**5*q2*q3**2*q4*q5**4 - 45*p**5*q2*q3**2*q4*q5**3*q6 + 360*p**5*q2*q3**2*q4*q5**3 - 45*p**5*q2*q3**2*q4*q5**2*q6**2 + 360*p**5*q2*q3**2*q4*q5**2*q6 - 1260*p**5*q2*q3**2*q4*q5**2 - 45*p**5*q2*q3**2*q4*q5*q6**3 + 360*p**5*q2*q3**2*q4*q5*q6**2 - 1260*p**5*q2*q3**2*q4*q5*q6 + 2520*p**5*q2*q3**2*q4*q5 - 45*p**5*q2*q3*q4**5*q5 - 45*p**5*q2*q3*q4**4*q5**2 - 45*p**5*q2*q3*q4**4*q5*q6 + 360*p**5*q2*q3*q4**4*q5 - 45*p**5*q2*q3*q4**3*q5**3 - 45*p**5*q2*q3*q4**3*q5**2*q6 + 360*p**5*q2*q3*q4**3*q5**2 - 45*p**5*q2*q3*q4**3*q5*q6**2 + 360*p**5*q2*q3*q4**3*q5*q6 - 1260*p**5*q2*q3*q4**3*q5 - 45*p**5*q2*q3*q4**2*q5**4 - 45*p**5*q2*q3*q4**2*q5**3*q6 + 360*p**5*q2*q3*q4**2*q5**3 - 45*p**5*q2*q3*q4**2*q5**2*q6**2 + 360*p**5*q2*q3*q4**2*q5**2*q6 - 1260*p**5*q2*q3*q4**2*q5**2 - 45*p**5*q2*q3*q4**2*q5*q6**3 + 360*p**5*q2*q3*q4**2*q5*q6**2 - 1260*p**5*q2*q3*q4**2*q5*q6 + 2520*p**5*q2*q3*q4**2*q5 - 45*p**5*q2*q3*q4*q5**5 - 45*p**5*q2*q3*q4*q5**4*q6 + 360*p**5*q2*q3*q4*q5**4 - 45*p**5*q2*q3*q4*q5**3*q6**2 + 360*p**5*q2*q3*q4*q5**3*q6 - 1260*p**5*q2*q3*q4*q5**3 - 45*p**5*q2*q3*q4*q5**2*q6**3 + 360*p**5*q2*q3*q4*q5**2*q6**2 - 1260*p**5*q2*q3*q4*q5**2*q6 + 2520*p**5*q2*q3*q4*q5**2 - 45*p**5*q2*q3*q4*q5*q6**4 + 360*p**5*q2*q3*q4*q5*q6**3 - 1260*p**5*q2*q3*q4*q5*q6**2 + 2520*p**5*q2*q3*q4*q5*q6 - 3150*p**5*q2*q3*q4*q5 + 120*p**5*q3**5*q4*q5 + 120*p**5*q3**4*q4**2*q5 + 120*p**5*q3**4*q4*q5**2 + 120*p**5*q3**4*q4*q5*q6 - 840*p**5*q3**4*q4*q5 + 120*p**5*q3**3*q4**3*q5 + 120*p**5*q3**3*q4**2*q5**2 + 120*p**5*q3**3*q4**2*q5*q6 - 840*p**5*q3**3*q4**2*q5 + 120*p**5*q3**3*q4*q5**3 + 120*p**5*q3**3*q4*q5**2*q6 - 840*p**5*q3**3*q4*q5**2 + 120*p**5*q3**3*q4*q5*q6**2 - 840*p**5*q3**3*q4*q5*q6 + 2520*p**5*q3**3*q4*q5 + 120*p**5*q3**2*q4**4*q5 + 120*p**5*q3**2*q4**3*q5**2 + 120*p**5*q3**2*q4**3*q5*q6 - 840*p**5*q3**2*q4**3*q5 + 120*p**5*q3**2*q4**2*q5**3 + 120*p**5*q3**2*q4**2*q5**2*q6 - 840*p**5*q3**2*q4**2*q5**2 + 120*p**5*q3**2*q4**2*q5*q6**2 - 840*p**5*q3**2*q4**2*q5*q6 + 2520*p**5*q3**2*q4**2*q5 + 120*p**5*q3**2*q4*q5**4 + 120*p**5*q3**2*q4*q5**3*q6 - 840*p**5*q3**2*q4*q5**3 + 120*p**5*q3**2*q4*q5**2*q6**2 - 840*p**5*q3**2*q4*q5**2*q6 + 2520*p**5*q3**2*q4*q5**2 + 120*p**5*q3**2*q4*q5*q6**3 - 840*p**5*q3**2*q4*q5*q6**2 + 2520*p**5*q3**2*q4*q5*q6 - 4200*p**5*q3**2*q4*q5 + 120*p**5*q3*q4**5*q5 + 120*p**5*q3*q4**4*q5**2 + 120*p**5*q3*q4**4*q5*q6 - 840*p**5*q3*q4**4*q5 + 120*p**5*q3*q4**3*q5**3 + 120*p**5*q3*q4**3*q5**2*q6 - 840*p**5*q3*q4**3*q5**2 + 120*p**5*q3*q4**3*q5*q6**2 - 840*p**5*q3*q4**3*q5*q6 + 2520*p**5*q3*q4**3*q5 + 120*p**5*q3*q4**2*q5**4 + 120*p**5*q3*q4**2*q5**3*q6 - 840*p**5*q3*q4**2*q5**3 + 120*p**5*q3*q4**2*q5**2*q6**2 - 840*p**5*q3*q4**2*q5**2*q6 + 2520*p**5*q3*q4**2*q5**2 + 120*p**5*q3*q4**2*q5*q6**3 - 840*p**5*q3*q4**2*q5*q6**2 + 2520*p**5*q3*q4**2*q5*q6 - 4200*p**5*q3*q4**2*q5 + 120*p**5*q3*q4*q5**5 + 120*p**5*q3*q4*q5**4*q6 - 840*p**5*q3*q4*q5**4 + 120*p**5*q3*q4*q5**3*q6**2 - 840*p**5*q3*q4*q5**3*q6 + 2520*p**5*q3*q4*q5**3 + 120*p**5*q3*q4*q5**2*q6**3 - 840*p**5*q3*q4*q5**2*q6**2 + 2520*p**5*q3*q4*q5**2*q6 - 4200*p**5*q3*q4*q5**2 + 120*p**5*q3*q4*q5*q6**4 - 840*p**5*q3*q4*q5*q6**3 + 2520*p**5*q3*q4*q5*q6**2 - 4200*p**5*q3*q4*q5*q6 + 4200*p**5*q3*q4*q5 - 210*p**5*q4**5*q5 - 210*p**5*q4**4*q5**2 - 210*p**5*q4**4*q5*q6 + 1260*p**5*q4**4*q5 - 210*p**5*q4**3*q5**3 - 210*p**5*q4**3*q5**2*q6 + 1260*p**5*q4**3*q5**2 - 210*p**5*q4**3*q5*q6**2 + 1260*p**5*q4**3*q5*q6 - 3150*p**5*q4**3*q5 - 210*p**5*q4**2*q5**4 - 210*p**5*q4**2*q5**3*q6 + 1260*p**5*q4**2*q5**3 - 210*p**5*q4**2*q5**2*q6**2 + 1260*p**5*q4**2*q5**2*q6 - 3150*p**5*q4**2*q5**2 - 210*p**5*q4**2*q5*q6**3 + 1260*p**5*q4**2*q5*q6**2 - 3150*p**5*q4**2*q5*q6 + 4200*p**5*q4**2*q5 - 210*p**5*q4*q5**5 - 210*p**5*q4*q5**4*q6 + 1260*p**5*q4*q5**4 - 210*p**5*q4*q5**3*q6**2 + 1260*p**5*q4*q5**3*q6 - 3150*p**5*q4*q5**3 - 210*p**5*q4*q5**2*q6**3 + 1260*p**5*q4*q5**2*q6**2 - 3150*p**5*q4*q5**2*q6 + 4200*p**5*q4*q5**2 - 210*p**5*q4*q5*q6**4 + 1260*p**5*q4*q5*q6**3 - 3150*p**5*q4*q5*q6**2 + 4200*p**5*q4*q5*q6 - 3150*p**5*q4*q5 + 252*p**5*q5**5 + 252*p**5*q5**4*q6 - 1260*p**5*q5**4 + 252*p**5*q5**3*q6**2 - 1260*p**5*q5**3*q6 + 2520*p**5*q5**3 + 252*p**5*q5**2*q6**3 - 1260*p**5*q5**2*q6**2 + 2520*p**5*q5**2*q6 - 2520*p**5*q5**2 + 252*p**5*q5*q6**4 - 1260*p**5*q5*q6**3 + 2520*p**5*q5*q6**2 - 2520*p**5*q5*q6 + 1260*p**5*q5 - 210*p**5*q6**4 + 840*p**5*q6**3 - 1260*p**5*q6**2 + 840*p**5*q6 - 210*p**5 - 50*p**4*q1**5*q2*q3*q4*q5 - 50*p**4*q1**4*q2**2*q3*q4*q5 - 50*p**4*q1**4*q2*q3**2*q4*q5 - 50*p**4*q1**4*q2*q3*q4**2*q5 - 50*p**4*q1**4*q2*q3*q4*q5**2 - 50*p**4*q1**4*q2*q3*q4*q5*q6 + 450*p**4*q1**4*q2*q3*q4*q5 - 50*p**4*q1**3*q2**3*q3*q4*q5 - 50*p**4*q1**3*q2**2*q3**2*q4*q5 - 50*p**4*q1**3*q2**2*q3*q4**2*q5 - 50*p**4*q1**3*q2**2*q3*q4*q5**2 - 50*p**4*q1**3*q2**2*q3*q4*q5*q6 + 450*p**4*q1**3*q2**2*q3*q4*q5 - 50*p**4*q1**3*q2*q3**3*q4*q5 - 50*p**4*q1**3*q2*q3**2*q4**2*q5 - 50*p**4*q1**3*q2*q3**2*q4*q5**2 - 50*p**4*q1**3*q2*q3**2*q4*q5*q6 + 450*p**4*q1**3*q2*q3**2*q4*q5 - 50*p**4*q1**3*q2*q3*q4**3*q5 - 50*p**4*q1**3*q2*q3*q4**2*q5**2 - 50*p**4*q1**3*q2*q3*q4**2*q5*q6 + 450*p**4*q1**3*q2*q3*q4**2*q5 - 50*p**4*q1**3*q2*q3*q4*q5**3 - 50*p**4*q1**3*q2*q3*q4*q5**2*q6 + 450*p**4*q1**3*q2*q3*q4*q5**2 - 50*p**4*q1**3*q2*q3*q4*q5*q6**2 + 450*p**4*q1**3*q2*q3*q4*q5*q6 - 1800*p**4*q1**3*q2*q3*q4*q5 - 50*p**4*q1**2*q2**4*q3*q4*q5 - 50*p**4*q1**2*q2**3*q3**2*q4*q5 - 50*p**4*q1**2*q2**3*q3*q4**2*q5 - 50*p**4*q1**2*q2**3*q3*q4*q5**2 - 50*p**4*q1**2*q2**3*q3*q4*q5*q6 + 450*p**4*q1**2*q2**3*q3*q4*q5 - 50*p**4*q1**2*q2**2*q3**3*q4*q5 - 50*p**4*q1**2*q2**2*q3**2*q4**2*q5 - 50*p**4*q1**2*q2**2*q3**2*q4*q5**2 - 50*p**4*q1**2*q2**2*q3**2*q4*q5*q6 + 450*p**4*q1**2*q2**2*q3**2*q4*q5 - 50*p**4*q1**2*q2**2*q3*q4**3*q5 - 50*p**4*q1**2*q2**2*q3*q4**2*q5**2 - 50*p**4*q1**2*q2**2*q3*q4**2*q5*q6 + 450*p**4*q1**2*q2**2*q3*q4**2*q5 - 50*p**4*q1**2*q2**2*q3*q4*q5**3 - 50*p**4*q1**2*q2**2*q3*q4*q5**2*q6 + 450*p**4*q1**2*q2**2*q3*q4*q5**2 - 50*p**4*q1**2*q2**2*q3*q4*q5*q6**2 + 450*p**4*q1**2*q2**2*q3*q4*q5*q6 - 1800*p**4*q1**2*q2**2*q3*q4*q5 - 50*p**4*q1**2*q2*q3**4*q4*q5 - 50*p**4*q1**2*q2*q3**3*q4**2*q5 - 50*p**4*q1**2*q2*q3**3*q4*q5**2 - 50*p**4*q1**2*q2*q3**3*q4*q5*q6 + 450*p**4*q1**2*q2*q3**3*q4*q5 - 50*p**4*q1**2*q2*q3**2*q4**3*q5 - 50*p**4*q1**2*q2*q3**2*q4**2*q5**2 - 50*p**4*q1**2*q2*q3**2*q4**2*q5*q6 + 450*p**4*q1**2*q2*q3**2*q4**2*q5 - 50*p**4*q1**2*q2*q3**2*q4*q5**3 - 50*p**4*q1**2*q2*q3**2*q4*q5**2*q6 + 450*p**4*q1**2*q2*q3**2*q4*q5**2 - 50*p**4*q1**2*q2*q3**2*q4*q5*q6**2 + 450*p**4*q1**2*q2*q3**2*q4*q5*q6 - 1800*p**4*q1**2*q2*q3**2*q4*q5 - 50*p**4*q1**2*q2*q3*q4**4*q5 - 50*p**4*q1**2*q2*q3*q4**3*q5**2 - 50*p**4*q1**2*q2*q3*q4**3*q5*q6 + 450*p**4*q1**2*q2*q3*q4**3*q5 - 50*p**4*q1**2*q2*q3*q4**2*q5**3 - 50*p**4*q1**2*q2*q3*q4**2*q5**2*q6 + 450*p**4*q1**2*q2*q3*q4**2*q5**2 - 50*p**4*q1**2*q2*q3*q4**2*q5*q6**2 + 450*p**4*q1**2*q2*q3*q4**2*q5*q6 - 1800*p**4*q1**2*q2*q3*q4**2*q5 - 50*p**4*q1**2*q2*q3*q4*q5**4 - 50*p**4*q1**2*q2*q3*q4*q5**3*q6 + 450*p**4*q1**2*q2*q3*q4*q5**3 - 50*p**4*q1**2*q2*q3*q4*q5**2*q6**2 + 450*p**4*q1**2*q2*q3*q4*q5**2*q6 - 1800*p**4*q1**2*q2*q3*q4*q5**2 - 50*p**4*q1**2*q2*q3*q4*q5*q6**3 + 450*p**4*q1**2*q2*q3*q4*q5*q6**2 - 1800*p**4*q1**2*q2*q3*q4*q5*q6 + 4200*p**4*q1**2*q2*q3*q4*q5 - 50*p**4*q1*q2**5*q3*q4*q5 - 50*p**4*q1*q2**4*q3**2*q4*q5 - 50*p**4*q1*q2**4*q3*q4**2*q5 - 50*p**4*q1*q2**4*q3*q4*q5**2 - 50*p**4*q1*q2**4*q3*q4*q5*q6 + 450*p**4*q1*q2**4*q3*q4*q5 - 50*p**4*q1*q2**3*q3**3*q4*q5 - 50*p**4*q1*q2**3*q3**2*q4**2*q5 - 50*p**4*q1*q2**3*q3**2*q4*q5**2 - 50*p**4*q1*q2**3*q3**2*q4*q5*q6 + 450*p**4*q1*q2**3*q3**2*q4*q5 - 50*p**4*q1*q2**3*q3*q4**3*q5 - 50*p**4*q1*q2**3*q3*q4**2*q5**2 - 50*p**4*q1*q2**3*q3*q4**2*q5*q6 + 450*p**4*q1*q2**3*q3*q4**2*q5 - 50*p**4*q1*q2**3*q3*q4*q5**3 - 50*p**4*q1*q2**3*q3*q4*q5**2*q6 + 450*p**4*q1*q2**3*q3*q4*q5**2 - 50*p**4*q1*q2**3*q3*q4*q5*q6**2 + 450*p**4*q1*q2**3*q3*q4*q5*q6 - 1800*p**4*q1*q2**3*q3*q4*q5 - 50*p**4*q1*q2**2*q3**4*q4*q5 - 50*p**4*q1*q2**2*q3**3*q4**2*q5 - 50*p**4*q1*q2**2*q3**3*q4*q5**2 - 50*p**4*q1*q2**2*q3**3*q4*q5*q6 + 450*p**4*q1*q2**2*q3**3*q4*q5 - 50*p**4*q1*q2**2*q3**2*q4**3*q5 - 50*p**4*q1*q2**2*q3**2*q4**2*q5**2 - 50*p**4*q1*q2**2*q3**2*q4**2*q5*q6 + 450*p**4*q1*q2**2*q3**2*q4**2*q5 - 50*p**4*q1*q2**2*q3**2*q4*q5**3 - 50*p**4*q1*q2**2*q3**2*q4*q5**2*q6 + 450*p**4*q1*q2**2*q3**2*q4*q5**2 - 50*p**4*q1*q2**2*q3**2*q4*q5*q6**2 + 450*p**4*q1*q2**2*q3**2*q4*q5*q6 - 1800*p**4*q1*q2**2*q3**2*q4*q5 - 50*p**4*q1*q2**2*q3*q4**4*q5 - 50*p**4*q1*q2**2*q3*q4**3*q5**2 - 50*p**4*q1*q2**2*q3*q4**3*q5*q6 + 450*p**4*q1*q2**2*q3*q4**3*q5 - 50*p**4*q1*q2**2*q3*q4**2*q5**3 - 50*p**4*q1*q2**2*q3*q4**2*q5**2*q6 + 450*p**4*q1*q2**2*q3*q4**2*q5**2 - 50*p**4*q1*q2**2*q3*q4**2*q5*q6**2 + 450*p**4*q1*q2**2*q3*q4**2*q5*q6 - 1800*p**4*q1*q2**2*q3*q4**2*q5 - 50*p**4*q1*q2**2*q3*q4*q5**4 - 50*p**4*q1*q2**2*q3*q4*q5**3*q6 + 450*p**4*q1*q2**2*q3*q4*q5**3 - 50*p**4*q1*q2**2*q3*q4*q5**2*q6**2 + 450*p**4*q1*q2**2*q3*q4*q5**2*q6 - 1800*p**4*q1*q2**2*q3*q4*q5**2 - 50*p**4*q1*q2**2*q3*q4*q5*q6**3 + 450*p**4*q1*q2**2*q3*q4*q5*q6**2 - 1800*p**4*q1*q2**2*q3*q4*q5*q6 + 4200*p**4*q1*q2**2*q3*q4*q5 - 50*p**4*q1*q2*q3**5*q4*q5 - 50*p**4*q1*q2*q3**4*q4**2*q5 - 50*p**4*q1*q2*q3**4*q4*q5**2 - 50*p**4*q1*q2*q3**4*q4*q5*q6 + 450*p**4*q1*q2*q3**4*q4*q5 - 50*p**4*q1*q2*q3**3*q4**3*q5 - 50*p**4*q1*q2*q3**3*q4**2*q5**2 - 50*p**4*q1*q2*q3**3*q4**2*q5*q6 + 450*p**4*q1*q2*q3**3*q4**2*q5 - 50*p**4*q1*q2*q3**3*q4*q5**3 - 50*p**4*q1*q2*q3**3*q4*q5**2*q6 + 450*p**4*q1*q2*q3**3*q4*q5**2 - 50*p**4*q1*q2*q3**3*q4*q5*q6**2 + 450*p**4*q1*q2*q3**3*q4*q5*q6 - 1800*p**4*q1*q2*q3**3*q4*q5 - 50*p**4*q1*q2*q3**2*q4**4*q5 - 50*p**4*q1*q2*q3**2*q4**3*q5**2 - 50*p**4*q1*q2*q3**2*q4**3*q5*q6 + 450*p**4*q1*q2*q3**2*q4**3*q5 - 50*p**4*q1*q2*q3**2*q4**2*q5**3 - 50*p**4*q1*q2*q3**2*q4**2*q5**2*q6 + 450*p**4*q1*q2*q3**2*q4**2*q5**2 - 50*p**4*q1*q2*q3**2*q4**2*q5*q6**2 + 450*p**4*q1*q2*q3**2*q4**2*q5*q6 - 1800*p**4*q1*q2*q3**2*q4**2*q5 - 50*p**4*q1*q2*q3**2*q4*q5**4 - 50*p**4*q1*q2*q3**2*q4*q5**3*q6 + 450*p**4*q1*q2*q3**2*q4*q5**3 - 50*p**4*q1*q2*q3**2*q4*q5**2*q6**2 + 450*p**4*q1*q2*q3**2*q4*q5**2*q6 - 1800*p**4*q1*q2*q3**2*q4*q5**2 - 50*p**4*q1*q2*q3**2*q4*q5*q6**3 + 450*p**4*q1*q2*q3**2*q4*q5*q6**2 - 1800*p**4*q1*q2*q3**2*q4*q5*q6 + 4200*p**4*q1*q2*q3**2*q4*q5 - 50*p**4*q1*q2*q3*q4**5*q5 - 50*p**4*q1*q2*q3*q4**4*q5**2 - 50*p**4*q1*q2*q3*q4**4*q5*q6 + 450*p**4*q1*q2*q3*q4**4*q5 - 50*p**4*q1*q2*q3*q4**3*q5**3 - 50*p**4*q1*q2*q3*q4**3*q5**2*q6 + 450*p**4*q1*q2*q3*q4**3*q5**2 - 50*p**4*q1*q2*q3*q4**3*q5*q6**2 + 450*p**4*q1*q2*q3*q4**3*q5*q6 - 1800*p**4*q1*q2*q3*q4**3*q5 - 50*p**4*q1*q2*q3*q4**2*q5**4 - 50*p**4*q1*q2*q3*q4**2*q5**3*q6 + 450*p**4*q1*q2*q3*q4**2*q5**3 - 50*p**4*q1*q2*q3*q4**2*q5**2*q6**2 + 450*p**4*q1*q2*q3*q4**2*q5**2*q6 - 1800*p**4*q1*q2*q3*q4**2*q5**2 - 50*p**4*q1*q2*q3*q4**2*q5*q6**3 + 450*p**4*q1*q2*q3*q4**2*q5*q6**2 - 1800*p**4*q1*q2*q3*q4**2*q5*q6 + 4200*p**4*q1*q2*q3*q4**2*q5 - 50*p**4*q1*q2*q3*q4*q5**5 - 50*p**4*q1*q2*q3*q4*q5**4*q6 + 450*p**4*q1*q2*q3*q4*q5**4 - 50*p**4*q1*q2*q3*q4*q5**3*q6**2 + 450*p**4*q1*q2*q3*q4*q5**3*q6 - 1800*p**4*q1*q2*q3*q4*q5**3 - 50*p**4*q1*q2*q3*q4*q5**2*q6**3 + 450*p**4*q1*q2*q3*q4*q5**2*q6**2 - 1800*p**4*q1*q2*q3*q4*q5**2*q6 + 4200*p**4*q1*q2*q3*q4*q5**2 - 50*p**4*q1*q2*q3*q4*q5*q6**4 + 450*p**4*q1*q2*q3*q4*q5*q6**3 - 1800*p**4*q1*q2*q3*q4*q5*q6**2 + 4200*p**4*q1*q2*q3*q4*q5*q6 - 6300*p**4*q1*q2*q3*q4*q5 + 180*p**4*q2**5*q3*q4*q5 + 180*p**4*q2**4*q3**2*q4*q5 + 180*p**4*q2**4*q3*q4**2*q5 + 180*p**4*q2**4*q3*q4*q5**2 + 180*p**4*q2**4*q3*q4*q5*q6 - 1440*p**4*q2**4*q3*q4*q5 + 180*p**4*q2**3*q3**3*q4*q5 + 180*p**4*q2**3*q3**2*q4**2*q5 + 180*p**4*q2**3*q3**2*q4*q5**2 + 180*p**4*q2**3*q3**2*q4*q5*q6 - 1440*p**4*q2**3*q3**2*q4*q5 + 180*p**4*q2**3*q3*q4**3*q5 + 180*p**4*q2**3*q3*q4**2*q5**2 + 180*p**4*q2**3*q3*q4**2*q5*q6 - 1440*p**4*q2**3*q3*q4**2*q5 + 180*p**4*q2**3*q3*q4*q5**3 + 180*p**4*q2**3*q3*q4*q5**2*q6 - 1440*p**4*q2**3*q3*q4*q5**2 + 180*p**4*q2**3*q3*q4*q5*q6**2 - 1440*p**4*q2**3*q3*q4*q5*q6 + 5040*p**4*q2**3*q3*q4*q5 + 180*p**4*q2**2*q3**4*q4*q5 + 180*p**4*q2**2*q3**3*q4**2*q5 + 180*p**4*q2**2*q3**3*q4*q5**2 + 180*p**4*q2**2*q3**3*q4*q5*q6 - 1440*p**4*q2**2*q3**3*q4*q5 + 180*p**4*q2**2*q3**2*q4**3*q5 + 180*p**4*q2**2*q3**2*q4**2*q5**2 + 180*p**4*q2**2*q3**2*q4**2*q5*q6 - 1440*p**4*q2**2*q3**2*q4**2*q5 + 180*p**4*q2**2*q3**2*q4*q5**3 + 180*p**4*q2**2*q3**2*q4*q5**2*q6 - 1440*p**4*q2**2*q3**2*q4*q5**2 + 180*p**4*q2**2*q3**2*q4*q5*q6**2 - 1440*p**4*q2**2*q3**2*q4*q5*q6 + 5040*p**4*q2**2*q3**2*q4*q5 + 180*p**4*q2**2*q3*q4**4*q5 + 180*p**4*q2**2*q3*q4**3*q5**2 + 180*p**4*q2**2*q3*q4**3*q5*q6 - 1440*p**4*q2**2*q3*q4**3*q5 + 180*p**4*q2**2*q3*q4**2*q5**3 + 180*p**4*q2**2*q3*q4**2*q5**2*q6 - 1440*p**4*q2**2*q3*q4**2*q5**2 + 180*p**4*q2**2*q3*q4**2*q5*q6**2 - 1440*p**4*q2**2*q3*q4**2*q5*q6 + 5040*p**4*q2**2*q3*q4**2*q5 + 180*p**4*q2**2*q3*q4*q5**4 + 180*p**4*q2**2*q3*q4*q5**3*q6 - 1440*p**4*q2**2*q3*q4*q5**3 + 180*p**4*q2**2*q3*q4*q5**2*q6**2 - 1440*p**4*q2**2*q3*q4*q5**2*q6 + 5040*p**4*q2**2*q3*q4*q5**2 + 180*p**4*q2**2*q3*q4*q5*q6**3 - 1440*p**4*q2**2*q3*q4*q5*q6**2 + 5040*p**4*q2**2*q3*q4*q5*q6 - 10080*p**4*q2**2*q3*q4*q5 + 180*p**4*q2*q3**5*q4*q5 + 180*p**4*q2*q3**4*q4**2*q5 + 180*p**4*q2*q3**4*q4*q5**2 + 180*p**4*q2*q3**4*q4*q5*q6 - 1440*p**4*q2*q3**4*q4*q5 + 180*p**4*q2*q3**3*q4**3*q5 + 180*p**4*q2*q3**3*q4**2*q5**2 + 180*p**4*q2*q3**3*q4**2*q5*q6 - 1440*p**4*q2*q3**3*q4**2*q5 + 180*p**4*q2*q3**3*q4*q5**3 + 180*p**4*q2*q3**3*q4*q5**2*q6 - 1440*p**4*q2*q3**3*q4*q5**2 + 180*p**4*q2*q3**3*q4*q5*q6**2 - 1440*p**4*q2*q3**3*q4*q5*q6 + 5040*p**4*q2*q3**3*q4*q5 + 180*p**4*q2*q3**2*q4**4*q5 + 180*p**4*q2*q3**2*q4**3*q5**2 + 180*p**4*q2*q3**2*q4**3*q5*q6 - 1440*p**4*q2*q3**2*q4**3*q5 + 180*p**4*q2*q3**2*q4**2*q5**3 + 180*p**4*q2*q3**2*q4**2*q5**2*q6 - 1440*p**4*q2*q3**2*q4**2*q5**2 + 180*p**4*q2*q3**2*q4**2*q5*q6**2 - 1440*p**4*q2*q3**2*q4**2*q5*q6 + 5040*p**4*q2*q3**2*q4**2*q5 + 180*p**4*q2*q3**2*q4*q5**4 + 180*p**4*q2*q3**2*q4*q5**3*q6 - 1440*p**4*q2*q3**2*q4*q5**3 + 180*p**4*q2*q3**2*q4*q5**2*q6**2 - 1440*p**4*q2*q3**2*q4*q5**2*q6 + 5040*p**4*q2*q3**2*q4*q5**2 + 180*p**4*q2*q3**2*q4*q5*q6**3 - 1440*p**4*q2*q3**2*q4*q5*q6**2 + 5040*p**4*q2*q3**2*q4*q5*q6 - 10080*p**4*q2*q3**2*q4*q5 + 180*p**4*q2*q3*q4**5*q5 + 180*p**4*q2*q3*q4**4*q5**2 + 180*p**4*q2*q3*q4**4*q5*q6 - 1440*p**4*q2*q3*q4**4*q5 + 180*p**4*q2*q3*q4**3*q5**3 + 180*p**4*q2*q3*q4**3*q5**2*q6 - 1440*p**4*q2*q3*q4**3*q5**2 + 180*p**4*q2*q3*q4**3*q5*q6**2 - 1440*p**4*q2*q3*q4**3*q5*q6 + 5040*p**4*q2*q3*q4**3*q5 + 180*p**4*q2*q3*q4**2*q5**4 + 180*p**4*q2*q3*q4**2*q5**3*q6 - 1440*p**4*q2*q3*q4**2*q5**3 + 180*p**4*q2*q3*q4**2*q5**2*q6**2 - 1440*p**4*q2*q3*q4**2*q5**2*q6 + 5040*p**4*q2*q3*q4**2*q5**2 + 180*p**4*q2*q3*q4**2*q5*q6**3 - 1440*p**4*q2*q3*q4**2*q5*q6**2 + 5040*p**4*q2*q3*q4**2*q5*q6 - 10080*p**4*q2*q3*q4**2*q5 + 180*p**4*q2*q3*q4*q5**5 + 180*p**4*q2*q3*q4*q5**4*q6 - 1440*p**4*q2*q3*q4*q5**4 + 180*p**4*q2*q3*q4*q5**3*q6**2 - 1440*p**4*q2*q3*q4*q5**3*q6 + 5040*p**4*q2*q3*q4*q5**3 + 180*p**4*q2*q3*q4*q5**2*q6**3 - 1440*p**4*q2*q3*q4*q5**2*q6**2 + 5040*p**4*q2*q3*q4*q5**2*q6 - 10080*p**4*q2*q3*q4*q5**2 + 180*p**4*q2*q3*q4*q5*q6**4 - 1440*p**4*q2*q3*q4*q5*q6**3 + 5040*p**4*q2*q3*q4*q5*q6**2 - 10080*p**4*q2*q3*q4*q5*q6 + 12600*p**4*q2*q3*q4*q5 - 360*p**4*q3**5*q4*q5 - 360*p**4*q3**4*q4**2*q5 - 360*p**4*q3**4*q4*q5**2 - 360*p**4*q3**4*q4*q5*q6 + 2520*p**4*q3**4*q4*q5 - 360*p**4*q3**3*q4**3*q5 - 360*p**4*q3**3*q4**2*q5**2 - 360*p**4*q3**3*q4**2*q5*q6 + 2520*p**4*q3**3*q4**2*q5 - 360*p**4*q3**3*q4*q5**3 - 360*p**4*q3**3*q4*q5**2*q6 + 2520*p**4*q3**3*q4*q5**2 - 360*p**4*q3**3*q4*q5*q6**2 + 2520*p**4*q3**3*q4*q5*q6 - 7560*p**4*q3**3*q4*q5 - 360*p**4*q3**2*q4**4*q5 - 360*p**4*q3**2*q4**3*q5**2 - 360*p**4*q3**2*q4**3*q5*q6 + 2520*p**4*q3**2*q4**3*q5 - 360*p**4*q3**2*q4**2*q5**3 - 360*p**4*q3**2*q4**2*q5**2*q6 + 2520*p**4*q3**2*q4**2*q5**2 - 360*p**4*q3**2*q4**2*q5*q6**2 + 2520*p**4*q3**2*q4**2*q5*q6 - 7560*p**4*q3**2*q4**2*q5 - 360*p**4*q3**2*q4*q5**4 - 360*p**4*q3**2*q4*q5**3*q6 + 2520*p**4*q3**2*q4*q5**3 - 360*p**4*q3**2*q4*q5**2*q6**2 + 2520*p**4*q3**2*q4*q5**2*q6 - 7560*p**4*q3**2*q4*q5**2 - 360*p**4*q3**2*q4*q5*q6**3 + 2520*p**4*q3**2*q4*q5*q6**2 - 7560*p**4*q3**2*q4*q5*q6 + 12600*p**4*q3**2*q4*q5 - 360*p**4*q3*q4**5*q5 - 360*p**4*q3*q4**4*q5**2 - 360*p**4*q3*q4**4*q5*q6 + 2520*p**4*q3*q4**4*q5 - 360*p**4*q3*q4**3*q5**3 - 360*p**4*q3*q4**3*q5**2*q6 + 2520*p**4*q3*q4**3*q5**2 - 360*p**4*q3*q4**3*q5*q6**2 + 2520*p**4*q3*q4**3*q5*q6 - 7560*p**4*q3*q4**3*q5 - 360*p**4*q3*q4**2*q5**4 - 360*p**4*q3*q4**2*q5**3*q6 + 2520*p**4*q3*q4**2*q5**3 - 360*p**4*q3*q4**2*q5**2*q6**2 + 2520*p**4*q3*q4**2*q5**2*q6 - 7560*p**4*q3*q4**2*q5**2 - 360*p**4*q3*q4**2*q5*q6**3 + 2520*p**4*q3*q4**2*q5*q6**2 - 7560*p**4*q3*q4**2*q5*q6 + 12600*p**4*q3*q4**2*q5 - 360*p**4*q3*q4*q5**5 - 360*p**4*q3*q4*q5**4*q6 + 2520*p**4*q3*q4*q5**4 - 360*p**4*q3*q4*q5**3*q6**2 + 2520*p**4*q3*q4*q5**3*q6 - 7560*p**4*q3*q4*q5**3 - 360*p**4*q3*q4*q5**2*q6**3 + 2520*p**4*q3*q4*q5**2*q6**2 - 7560*p**4*q3*q4*q5**2*q6 + 12600*p**4*q3*q4*q5**2 - 360*p**4*q3*q4*q5*q6**4 + 2520*p**4*q3*q4*q5*q6**3 - 7560*p**4*q3*q4*q5*q6**2 + 12600*p**4*q3*q4*q5*q6 - 12600*p**4*q3*q4*q5 + 420*p**4*q4**5*q5 + 420*p**4*q4**4*q5**2 + 420*p**4*q4**4*q5*q6 - 2520*p**4*q4**4*q5 + 420*p**4*q4**3*q5**3 + 420*p**4*q4**3*q5**2*q6 - 2520*p**4*q4**3*q5**2 + 420*p**4*q4**3*q5*q6**2 - 2520*p**4*q4**3*q5*q6 + 6300*p**4*q4**3*q5 + 420*p**4*q4**2*q5**4 + 420*p**4*q4**2*q5**3*q6 - 2520*p**4*q4**2*q5**3 + 420*p**4*q4**2*q5**2*q6**2 - 2520*p**4*q4**2*q5**2*q6 + 6300*p**4*q4**2*q5**2 + 420*p**4*q4**2*q5*q6**3 - 2520*p**4*q4**2*q5*q6**2 + 6300*p**4*q4**2*q5*q6 - 8400*p**4*q4**2*q5 + 420*p**4*q4*q5**5 + 420*p**4*q4*q5**4*q6 - 2520*p**4*q4*q5**4 + 420*p**4*q4*q5**3*q6**2 - 2520*p**4*q4*q5**3*q6 + 6300*p**4*q4*q5**3 + 420*p**4*q4*q5**2*q6**3 - 2520*p**4*q4*q5**2*q6**2 + 6300*p**4*q4*q5**2*q6 - 8400*p**4*q4*q5**2 + 420*p**4*q4*q5*q6**4 - 2520*p**4*q4*q5*q6**3 + 6300*p**4*q4*q5*q6**2 - 8400*p**4*q4*q5*q6 + 6300*p**4*q4*q5 - 252*p**4*q5**5 - 252*p**4*q5**4*q6 + 1260*p**4*q5**4 - 252*p**4*q5**3*q6**2 + 1260*p**4*q5**3*q6 - 2520*p**4*q5**3 - 252*p**4*q5**2*q6**3 + 1260*p**4*q5**2*q6**2 - 2520*p**4*q5**2*q6 + 2520*p**4*q5**2 - 252*p**4*q5*q6**4 + 1260*p**4*q5*q6**3 - 2520*p**4*q5*q6**2 + 2520*p**4*q5*q6 - 1260*p**4*q5 + 100*p**3*q1**5*q2*q3*q4*q5 + 100*p**3*q1**4*q2**2*q3*q4*q5 + 100*p**3*q1**4*q2*q3**2*q4*q5 + 100*p**3*q1**4*q2*q3*q4**2*q5 + 100*p**3*q1**4*q2*q3*q4*q5**2 + 100*p**3*q1**4*q2*q3*q4*q5*q6 - 900*p**3*q1**4*q2*q3*q4*q5 + 100*p**3*q1**3*q2**3*q3*q4*q5 + 100*p**3*q1**3*q2**2*q3**2*q4*q5 + 100*p**3*q1**3*q2**2*q3*q4**2*q5 + 100*p**3*q1**3*q2**2*q3*q4*q5**2 + 100*p**3*q1**3*q2**2*q3*q4*q5*q6 - 900*p**3*q1**3*q2**2*q3*q4*q5 + 100*p**3*q1**3*q2*q3**3*q4*q5 + 100*p**3*q1**3*q2*q3**2*q4**2*q5 + 100*p**3*q1**3*q2*q3**2*q4*q5**2 + 100*p**3*q1**3*q2*q3**2*q4*q5*q6 - 900*p**3*q1**3*q2*q3**2*q4*q5 + 100*p**3*q1**3*q2*q3*q4**3*q5 + 100*p**3*q1**3*q2*q3*q4**2*q5**2 + 100*p**3*q1**3*q2*q3*q4**2*q5*q6 - 900*p**3*q1**3*q2*q3*q4**2*q5 + 100*p**3*q1**3*q2*q3*q4*q5**3 + 100*p**3*q1**3*q2*q3*q4*q5**2*q6 - 900*p**3*q1**3*q2*q3*q4*q5**2 + 100*p**3*q1**3*q2*q3*q4*q5*q6**2 - 900*p**3*q1**3*q2*q3*q4*q5*q6 + 3600*p**3*q1**3*q2*q3*q4*q5 + 100*p**3*q1**2*q2**4*q3*q4*q5 + 100*p**3*q1**2*q2**3*q3**2*q4*q5 + 100*p**3*q1**2*q2**3*q3*q4**2*q5 + 100*p**3*q1**2*q2**3*q3*q4*q5**2 + 100*p**3*q1**2*q2**3*q3*q4*q5*q6 - 900*p**3*q1**2*q2**3*q3*q4*q5 + 100*p**3*q1**2*q2**2*q3**3*q4*q5 + 100*p**3*q1**2*q2**2*q3**2*q4**2*q5 + 100*p**3*q1**2*q2**2*q3**2*q4*q5**2 + 100*p**3*q1**2*q2**2*q3**2*q4*q5*q6 - 900*p**3*q1**2*q2**2*q3**2*q4*q5 + 100*p**3*q1**2*q2**2*q3*q4**3*q5 + 100*p**3*q1**2*q2**2*q3*q4**2*q5**2 + 100*p**3*q1**2*q2**2*q3*q4**2*q5*q6 - 900*p**3*q1**2*q2**2*q3*q4**2*q5 + 100*p**3*q1**2*q2**2*q3*q4*q5**3 + 100*p**3*q1**2*q2**2*q3*q4*q5**2*q6 - 900*p**3*q1**2*q2**2*q3*q4*q5**2 + 100*p**3*q1**2*q2**2*q3*q4*q5*q6**2 - 900*p**3*q1**2*q2**2*q3*q4*q5*q6 + 3600*p**3*q1**2*q2**2*q3*q4*q5 + 100*p**3*q1**2*q2*q3**4*q4*q5 + 100*p**3*q1**2*q2*q3**3*q4**2*q5 + 100*p**3*q1**2*q2*q3**3*q4*q5**2 + 100*p**3*q1**2*q2*q3**3*q4*q5*q6 - 900*p**3*q1**2*q2*q3**3*q4*q5 + 100*p**3*q1**2*q2*q3**2*q4**3*q5 + 100*p**3*q1**2*q2*q3**2*q4**2*q5**2 + 100*p**3*q1**2*q2*q3**2*q4**2*q5*q6 - 900*p**3*q1**2*q2*q3**2*q4**2*q5 + 100*p**3*q1**2*q2*q3**2*q4*q5**3 + 100*p**3*q1**2*q2*q3**2*q4*q5**2*q6 - 900*p**3*q1**2*q2*q3**2*q4*q5**2 + 100*p**3*q1**2*q2*q3**2*q4*q5*q6**2 - 900*p**3*q1**2*q2*q3**2*q4*q5*q6 + 3600*p**3*q1**2*q2*q3**2*q4*q5 + 100*p**3*q1**2*q2*q3*q4**4*q5 + 100*p**3*q1**2*q2*q3*q4**3*q5**2 + 100*p**3*q1**2*q2*q3*q4**3*q5*q6 - 900*p**3*q1**2*q2*q3*q4**3*q5 + 100*p**3*q1**2*q2*q3*q4**2*q5**3 + 100*p**3*q1**2*q2*q3*q4**2*q5**2*q6 - 900*p**3*q1**2*q2*q3*q4**2*q5**2 + 100*p**3*q1**2*q2*q3*q4**2*q5*q6**2 - 900*p**3*q1**2*q2*q3*q4**2*q5*q6 + 3600*p**3*q1**2*q2*q3*q4**2*q5 + 100*p**3*q1**2*q2*q3*q4*q5**4 + 100*p**3*q1**2*q2*q3*q4*q5**3*q6 - 900*p**3*q1**2*q2*q3*q4*q5**3 + 100*p**3*q1**2*q2*q3*q4*q5**2*q6**2 - 900*p**3*q1**2*q2*q3*q4*q5**2*q6 + 3600*p**3*q1**2*q2*q3*q4*q5**2 + 100*p**3*q1**2*q2*q3*q4*q5*q6**3 - 900*p**3*q1**2*q2*q3*q4*q5*q6**2 + 3600*p**3*q1**2*q2*q3*q4*q5*q6 - 8400*p**3*q1**2*q2*q3*q4*q5 + 100*p**3*q1*q2**5*q3*q4*q5 + 100*p**3*q1*q2**4*q3**2*q4*q5 + 100*p**3*q1*q2**4*q3*q4**2*q5 + 100*p**3*q1*q2**4*q3*q4*q5**2 + 100*p**3*q1*q2**4*q3*q4*q5*q6 - 900*p**3*q1*q2**4*q3*q4*q5 + 100*p**3*q1*q2**3*q3**3*q4*q5 + 100*p**3*q1*q2**3*q3**2*q4**2*q5 + 100*p**3*q1*q2**3*q3**2*q4*q5**2 + 100*p**3*q1*q2**3*q3**2*q4*q5*q6 - 900*p**3*q1*q2**3*q3**2*q4*q5 + 100*p**3*q1*q2**3*q3*q4**3*q5 + 100*p**3*q1*q2**3*q3*q4**2*q5**2 + 100*p**3*q1*q2**3*q3*q4**2*q5*q6 - 900*p**3*q1*q2**3*q3*q4**2*q5 + 100*p**3*q1*q2**3*q3*q4*q5**3 + 100*p**3*q1*q2**3*q3*q4*q5**2*q6 - 900*p**3*q1*q2**3*q3*q4*q5**2 + 100*p**3*q1*q2**3*q3*q4*q5*q6**2 - 900*p**3*q1*q2**3*q3*q4*q5*q6 + 3600*p**3*q1*q2**3*q3*q4*q5 + 100*p**3*q1*q2**2*q3**4*q4*q5 + 100*p**3*q1*q2**2*q3**3*q4**2*q5 + 100*p**3*q1*q2**2*q3**3*q4*q5**2 + 100*p**3*q1*q2**2*q3**3*q4*q5*q6 - 900*p**3*q1*q2**2*q3**3*q4*q5 + 100*p**3*q1*q2**2*q3**2*q4**3*q5 + 100*p**3*q1*q2**2*q3**2*q4**2*q5**2 + 100*p**3*q1*q2**2*q3**2*q4**2*q5*q6 - 900*p**3*q1*q2**2*q3**2*q4**2*q5 + 100*p**3*q1*q2**2*q3**2*q4*q5**3 + 100*p**3*q1*q2**2*q3**2*q4*q5**2*q6 - 900*p**3*q1*q2**2*q3**2*q4*q5**2 + 100*p**3*q1*q2**2*q3**2*q4*q5*q6**2 - 900*p**3*q1*q2**2*q3**2*q4*q5*q6 + 3600*p**3*q1*q2**2*q3**2*q4*q5 + 100*p**3*q1*q2**2*q3*q4**4*q5 + 100*p**3*q1*q2**2*q3*q4**3*q5**2 + 100*p**3*q1*q2**2*q3*q4**3*q5*q6 - 900*p**3*q1*q2**2*q3*q4**3*q5 + 100*p**3*q1*q2**2*q3*q4**2*q5**3 + 100*p**3*q1*q2**2*q3*q4**2*q5**2*q6 - 900*p**3*q1*q2**2*q3*q4**2*q5**2 + 100*p**3*q1*q2**2*q3*q4**2*q5*q6**2 - 900*p**3*q1*q2**2*q3*q4**2*q5*q6 + 3600*p**3*q1*q2**2*q3*q4**2*q5 + 100*p**3*q1*q2**2*q3*q4*q5**4 + 100*p**3*q1*q2**2*q3*q4*q5**3*q6 - 900*p**3*q1*q2**2*q3*q4*q5**3 + 100*p**3*q1*q2**2*q3*q4*q5**2*q6**2 - 900*p**3*q1*q2**2*q3*q4*q5**2*q6 + 3600*p**3*q1*q2**2*q3*q4*q5**2 + 100*p**3*q1*q2**2*q3*q4*q5*q6**3 - 900*p**3*q1*q2**2*q3*q4*q5*q6**2 + 3600*p**3*q1*q2**2*q3*q4*q5*q6 - 8400*p**3*q1*q2**2*q3*q4*q5 + 100*p**3*q1*q2*q3**5*q4*q5 + 100*p**3*q1*q2*q3**4*q4**2*q5 + 100*p**3*q1*q2*q3**4*q4*q5**2 + 100*p**3*q1*q2*q3**4*q4*q5*q6 - 900*p**3*q1*q2*q3**4*q4*q5 + 100*p**3*q1*q2*q3**3*q4**3*q5 + 100*p**3*q1*q2*q3**3*q4**2*q5**2 + 100*p**3*q1*q2*q3**3*q4**2*q5*q6 - 900*p**3*q1*q2*q3**3*q4**2*q5 + 100*p**3*q1*q2*q3**3*q4*q5**3 + 100*p**3*q1*q2*q3**3*q4*q5**2*q6 - 900*p**3*q1*q2*q3**3*q4*q5**2 + 100*p**3*q1*q2*q3**3*q4*q5*q6**2 - 900*p**3*q1*q2*q3**3*q4*q5*q6 + 3600*p**3*q1*q2*q3**3*q4*q5 + 100*p**3*q1*q2*q3**2*q4**4*q5 + 100*p**3*q1*q2*q3**2*q4**3*q5**2 + 100*p**3*q1*q2*q3**2*q4**3*q5*q6 - 900*p**3*q1*q2*q3**2*q4**3*q5 + 100*p**3*q1*q2*q3**2*q4**2*q5**3 + 100*p**3*q1*q2*q3**2*q4**2*q5**2*q6 - 900*p**3*q1*q2*q3**2*q4**2*q5**2 + 100*p**3*q1*q2*q3**2*q4**2*q5*q6**2 - 900*p**3*q1*q2*q3**2*q4**2*q5*q6 + 3600*p**3*q1*q2*q3**2*q4**2*q5 + 100*p**3*q1*q2*q3**2*q4*q5**4 + 100*p**3*q1*q2*q3**2*q4*q5**3*q6 - 900*p**3*q1*q2*q3**2*q4*q5**3 + 100*p**3*q1*q2*q3**2*q4*q5**2*q6**2 - 900*p**3*q1*q2*q3**2*q4*q5**2*q6 + 3600*p**3*q1*q2*q3**2*q4*q5**2 + 100*p**3*q1*q2*q3**2*q4*q5*q6**3 - 900*p**3*q1*q2*q3**2*q4*q5*q6**2 + 3600*p**3*q1*q2*q3**2*q4*q5*q6 - 8400*p**3*q1*q2*q3**2*q4*q5 + 100*p**3*q1*q2*q3*q4**5*q5 + 100*p**3*q1*q2*q3*q4**4*q5**2 + 100*p**3*q1*q2*q3*q4**4*q5*q6 - 900*p**3*q1*q2*q3*q4**4*q5 + 100*p**3*q1*q2*q3*q4**3*q5**3 + 100*p**3*q1*q2*q3*q4**3*q5**2*q6 - 900*p**3*q1*q2*q3*q4**3*q5**2 + 100*p**3*q1*q2*q3*q4**3*q5*q6**2 - 900*p**3*q1*q2*q3*q4**3*q5*q6 + 3600*p**3*q1*q2*q3*q4**3*q5 + 100*p**3*q1*q2*q3*q4**2*q5**4 + 100*p**3*q1*q2*q3*q4**2*q5**3*q6 - 900*p**3*q1*q2*q3*q4**2*q5**3 + 100*p**3*q1*q2*q3*q4**2*q5**2*q6**2 - 900*p**3*q1*q2*q3*q4**2*q5**2*q6 + 3600*p**3*q1*q2*q3*q4**2*q5**2 + 100*p**3*q1*q2*q3*q4**2*q5*q6**3 - 900*p**3*q1*q2*q3*q4**2*q5*q6**2 + 3600*p**3*q1*q2*q3*q4**2*q5*q6 - 8400*p**3*q1*q2*q3*q4**2*q5 + 100*p**3*q1*q2*q3*q4*q5**5 + 100*p**3*q1*q2*q3*q4*q5**4*q6 - 900*p**3*q1*q2*q3*q4*q5**4 + 100*p**3*q1*q2*q3*q4*q5**3*q6**2 - 900*p**3*q1*q2*q3*q4*q5**3*q6 + 3600*p**3*q1*q2*q3*q4*q5**3 + 100*p**3*q1*q2*q3*q4*q5**2*q6**3 - 900*p**3*q1*q2*q3*q4*q5**2*q6**2 + 3600*p**3*q1*q2*q3*q4*q5**2*q6 - 8400*p**3*q1*q2*q3*q4*q5**2 + 100*p**3*q1*q2*q3*q4*q5*q6**4 - 900*p**3*q1*q2*q3*q4*q5*q6**3 + 3600*p**3*q1*q2*q3*q4*q5*q6**2 - 8400*p**3*q1*q2*q3*q4*q5*q6 + 12600*p**3*q1*q2*q3*q4*q5 - 270*p**3*q2**5*q3*q4*q5 - 270*p**3*q2**4*q3**2*q4*q5 - 270*p**3*q2**4*q3*q4**2*q5 - 270*p**3*q2**4*q3*q4*q5**2 - 270*p**3*q2**4*q3*q4*q5*q6 + 2160*p**3*q2**4*q3*q4*q5 - 270*p**3*q2**3*q3**3*q4*q5 - 270*p**3*q2**3*q3**2*q4**2*q5 - 270*p**3*q2**3*q3**2*q4*q5**2 - 270*p**3*q2**3*q3**2*q4*q5*q6 + 2160*p**3*q2**3*q3**2*q4*q5 - 270*p**3*q2**3*q3*q4**3*q5 - 270*p**3*q2**3*q3*q4**2*q5**2 - 270*p**3*q2**3*q3*q4**2*q5*q6 + 2160*p**3*q2**3*q3*q4**2*q5 - 270*p**3*q2**3*q3*q4*q5**3 - 270*p**3*q2**3*q3*q4*q5**2*q6 + 2160*p**3*q2**3*q3*q4*q5**2 - 270*p**3*q2**3*q3*q4*q5*q6**2 + 2160*p**3*q2**3*q3*q4*q5*q6 - 7560*p**3*q2**3*q3*q4*q5 - 270*p**3*q2**2*q3**4*q4*q5 - 270*p**3*q2**2*q3**3*q4**2*q5 - 270*p**3*q2**2*q3**3*q4*q5**2 - 270*p**3*q2**2*q3**3*q4*q5*q6 + 2160*p**3*q2**2*q3**3*q4*q5 - 270*p**3*q2**2*q3**2*q4**3*q5 - 270*p**3*q2**2*q3**2*q4**2*q5**2 - 270*p**3*q2**2*q3**2*q4**2*q5*q6 + 2160*p**3*q2**2*q3**2*q4**2*q5 - 270*p**3*q2**2*q3**2*q4*q5**3 - 270*p**3*q2**2*q3**2*q4*q5**2*q6 + 2160*p**3*q2**2*q3**2*q4*q5**2 - 270*p**3*q2**2*q3**2*q4*q5*q6**2 + 2160*p**3*q2**2*q3**2*q4*q5*q6 - 7560*p**3*q2**2*q3**2*q4*q5 - 270*p**3*q2**2*q3*q4**4*q5 - 270*p**3*q2**2*q3*q4**3*q5**2 - 270*p**3*q2**2*q3*q4**3*q5*q6 + 2160*p**3*q2**2*q3*q4**3*q5 - 270*p**3*q2**2*q3*q4**2*q5**3 - 270*p**3*q2**2*q3*q4**2*q5**2*q6 + 2160*p**3*q2**2*q3*q4**2*q5**2 - 270*p**3*q2**2*q3*q4**2*q5*q6**2 + 2160*p**3*q2**2*q3*q4**2*q5*q6 - 7560*p**3*q2**2*q3*q4**2*q5 - 270*p**3*q2**2*q3*q4*q5**4 - 270*p**3*q2**2*q3*q4*q5**3*q6 + 2160*p**3*q2**2*q3*q4*q5**3 - 270*p**3*q2**2*q3*q4*q5**2*q6**2 + 2160*p**3*q2**2*q3*q4*q5**2*q6 - 7560*p**3*q2**2*q3*q4*q5**2 - 270*p**3*q2**2*q3*q4*q5*q6**3 + 2160*p**3*q2**2*q3*q4*q5*q6**2 - 7560*p**3*q2**2*q3*q4*q5*q6 + 15120*p**3*q2**2*q3*q4*q5 - 270*p**3*q2*q3**5*q4*q5 - 270*p**3*q2*q3**4*q4**2*q5 - 270*p**3*q2*q3**4*q4*q5**2 - 270*p**3*q2*q3**4*q4*q5*q6 + 2160*p**3*q2*q3**4*q4*q5 - 270*p**3*q2*q3**3*q4**3*q5 - 270*p**3*q2*q3**3*q4**2*q5**2 - 270*p**3*q2*q3**3*q4**2*q5*q6 + 2160*p**3*q2*q3**3*q4**2*q5 - 270*p**3*q2*q3**3*q4*q5**3 - 270*p**3*q2*q3**3*q4*q5**2*q6 + 2160*p**3*q2*q3**3*q4*q5**2 - 270*p**3*q2*q3**3*q4*q5*q6**2 + 2160*p**3*q2*q3**3*q4*q5*q6 - 7560*p**3*q2*q3**3*q4*q5 - 270*p**3*q2*q3**2*q4**4*q5 - 270*p**3*q2*q3**2*q4**3*q5**2 - 270*p**3*q2*q3**2*q4**3*q5*q6 + 2160*p**3*q2*q3**2*q4**3*q5 - 270*p**3*q2*q3**2*q4**2*q5**3 - 270*p**3*q2*q3**2*q4**2*q5**2*q6 + 2160*p**3*q2*q3**2*q4**2*q5**2 - 270*p**3*q2*q3**2*q4**2*q5*q6**2 + 2160*p**3*q2*q3**2*q4**2*q5*q6 - 7560*p**3*q2*q3**2*q4**2*q5 - 270*p**3*q2*q3**2*q4*q5**4 - 270*p**3*q2*q3**2*q4*q5**3*q6 + 2160*p**3*q2*q3**2*q4*q5**3 - 270*p**3*q2*q3**2*q4*q5**2*q6**2 + 2160*p**3*q2*q3**2*q4*q5**2*q6 - 7560*p**3*q2*q3**2*q4*q5**2 - 270*p**3*q2*q3**2*q4*q5*q6**3 + 2160*p**3*q2*q3**2*q4*q5*q6**2 - 7560*p**3*q2*q3**2*q4*q5*q6 + 15120*p**3*q2*q3**2*q4*q5 - 270*p**3*q2*q3*q4**5*q5 - 270*p**3*q2*q3*q4**4*q5**2 - 270*p**3*q2*q3*q4**4*q5*q6 + 2160*p**3*q2*q3*q4**4*q5 - 270*p**3*q2*q3*q4**3*q5**3 - 270*p**3*q2*q3*q4**3*q5**2*q6 + 2160*p**3*q2*q3*q4**3*q5**2 - 270*p**3*q2*q3*q4**3*q5*q6**2 + 2160*p**3*q2*q3*q4**3*q5*q6 - 7560*p**3*q2*q3*q4**3*q5 - 270*p**3*q2*q3*q4**2*q5**4 - 270*p**3*q2*q3*q4**2*q5**3*q6 + 2160*p**3*q2*q3*q4**2*q5**3 - 270*p**3*q2*q3*q4**2*q5**2*q6**2 + 2160*p**3*q2*q3*q4**2*q5**2*q6 - 7560*p**3*q2*q3*q4**2*q5**2 - 270*p**3*q2*q3*q4**2*q5*q6**3 + 2160*p**3*q2*q3*q4**2*q5*q6**2 - 7560*p**3*q2*q3*q4**2*q5*q6 + 15120*p**3*q2*q3*q4**2*q5 - 270*p**3*q2*q3*q4*q5**5 - 270*p**3*q2*q3*q4*q5**4*q6 + 2160*p**3*q2*q3*q4*q5**4 - 270*p**3*q2*q3*q4*q5**3*q6**2 + 2160*p**3*q2*q3*q4*q5**3*q6 - 7560*p**3*q2*q3*q4*q5**3 - 270*p**3*q2*q3*q4*q5**2*q6**3 + 2160*p**3*q2*q3*q4*q5**2*q6**2 - 7560*p**3*q2*q3*q4*q5**2*q6 + 15120*p**3*q2*q3*q4*q5**2 - 270*p**3*q2*q3*q4*q5*q6**4 + 2160*p**3*q2*q3*q4*q5*q6**3 - 7560*p**3*q2*q3*q4*q5*q6**2 + 15120*p**3*q2*q3*q4*q5*q6 - 18900*p**3*q2*q3*q4*q5 + 360*p**3*q3**5*q4*q5 + 360*p**3*q3**4*q4**2*q5 + 360*p**3*q3**4*q4*q5**2 + 360*p**3*q3**4*q4*q5*q6 - 2520*p**3*q3**4*q4*q5 + 360*p**3*q3**3*q4**3*q5 + 360*p**3*q3**3*q4**2*q5**2 + 360*p**3*q3**3*q4**2*q5*q6 - 2520*p**3*q3**3*q4**2*q5 + 360*p**3*q3**3*q4*q5**3 + 360*p**3*q3**3*q4*q5**2*q6 - 2520*p**3*q3**3*q4*q5**2 + 360*p**3*q3**3*q4*q5*q6**2 - 2520*p**3*q3**3*q4*q5*q6 + 7560*p**3*q3**3*q4*q5 + 360*p**3*q3**2*q4**4*q5 + 360*p**3*q3**2*q4**3*q5**2 + 360*p**3*q3**2*q4**3*q5*q6 - 2520*p**3*q3**2*q4**3*q5 + 360*p**3*q3**2*q4**2*q5**3 + 360*p**3*q3**2*q4**2*q5**2*q6 - 2520*p**3*q3**2*q4**2*q5**2 + 360*p**3*q3**2*q4**2*q5*q6**2 - 2520*p**3*q3**2*q4**2*q5*q6 + 7560*p**3*q3**2*q4**2*q5 + 360*p**3*q3**2*q4*q5**4 + 360*p**3*q3**2*q4*q5**3*q6 - 2520*p**3*q3**2*q4*q5**3 + 360*p**3*q3**2*q4*q5**2*q6**2 - 2520*p**3*q3**2*q4*q5**2*q6 + 7560*p**3*q3**2*q4*q5**2 + 360*p**3*q3**2*q4*q5*q6**3 - 2520*p**3*q3**2*q4*q5*q6**2 + 7560*p**3*q3**2*q4*q5*q6 - 12600*p**3*q3**2*q4*q5 + 360*p**3*q3*q4**5*q5 + 360*p**3*q3*q4**4*q5**2 + 360*p**3*q3*q4**4*q5*q6 - 2520*p**3*q3*q4**4*q5 + 360*p**3*q3*q4**3*q5**3 + 360*p**3*q3*q4**3*q5**2*q6 - 2520*p**3*q3*q4**3*q5**2 + 360*p**3*q3*q4**3*q5*q6**2 - 2520*p**3*q3*q4**3*q5*q6 + 7560*p**3*q3*q4**3*q5 + 360*p**3*q3*q4**2*q5**4 + 360*p**3*q3*q4**2*q5**3*q6 - 2520*p**3*q3*q4**2*q5**3 + 360*p**3*q3*q4**2*q5**2*q6**2 - 2520*p**3*q3*q4**2*q5**2*q6 + 7560*p**3*q3*q4**2*q5**2 + 360*p**3*q3*q4**2*q5*q6**3 - 2520*p**3*q3*q4**2*q5*q6**2 + 7560*p**3*q3*q4**2*q5*q6 - 12600*p**3*q3*q4**2*q5 + 360*p**3*q3*q4*q5**5 + 360*p**3*q3*q4*q5**4*q6 - 2520*p**3*q3*q4*q5**4 + 360*p**3*q3*q4*q5**3*q6**2 - 2520*p**3*q3*q4*q5**3*q6 + 7560*p**3*q3*q4*q5**3 + 360*p**3*q3*q4*q5**2*q6**3 - 2520*p**3*q3*q4*q5**2*q6**2 + 7560*p**3*q3*q4*q5**2*q6 - 12600*p**3*q3*q4*q5**2 + 360*p**3*q3*q4*q5*q6**4 - 2520*p**3*q3*q4*q5*q6**3 + 7560*p**3*q3*q4*q5*q6**2 - 12600*p**3*q3*q4*q5*q6 + 12600*p**3*q3*q4*q5 - 210*p**3*q4**5*q5 - 210*p**3*q4**4*q5**2 - 210*p**3*q4**4*q5*q6 + 1260*p**3*q4**4*q5 - 210*p**3*q4**3*q5**3 - 210*p**3*q4**3*q5**2*q6 + 1260*p**3*q4**3*q5**2 - 210*p**3*q4**3*q5*q6**2 + 1260*p**3*q4**3*q5*q6 - 3150*p**3*q4**3*q5 - 210*p**3*q4**2*q5**4 - 210*p**3*q4**2*q5**3*q6 + 1260*p**3*q4**2*q5**3 - 210*p**3*q4**2*q5**2*q6**2 + 1260*p**3*q4**2*q5**2*q6 - 3150*p**3*q4**2*q5**2 - 210*p**3*q4**2*q5*q6**3 + 1260*p**3*q4**2*q5*q6**2 - 3150*p**3*q4**2*q5*q6 + 4200*p**3*q4**2*q5 - 210*p**3*q4*q5**5 - 210*p**3*q4*q5**4*q6 + 1260*p**3*q4*q5**4 - 210*p**3*q4*q5**3*q6**2 + 1260*p**3*q4*q5**3*q6 - 3150*p**3*q4*q5**3 - 210*p**3*q4*q5**2*q6**3 + 1260*p**3*q4*q5**2*q6**2 - 3150*p**3*q4*q5**2*q6 + 4200*p**3*q4*q5**2 - 210*p**3*q4*q5*q6**4 + 1260*p**3*q4*q5*q6**3 - 3150*p**3*q4*q5*q6**2 + 4200*p**3*q4*q5*q6 - 3150*p**3*q4*q5 - 100*p**2*q1**5*q2*q3*q4*q5 - 100*p**2*q1**4*q2**2*q3*q4*q5 - 100*p**2*q1**4*q2*q3**2*q4*q5 - 100*p**2*q1**4*q2*q3*q4**2*q5 - 100*p**2*q1**4*q2*q3*q4*q5**2 - 100*p**2*q1**4*q2*q3*q4*q5*q6 + 900*p**2*q1**4*q2*q3*q4*q5 - 100*p**2*q1**3*q2**3*q3*q4*q5 - 100*p**2*q1**3*q2**2*q3**2*q4*q5 - 100*p**2*q1**3*q2**2*q3*q4**2*q5 - 100*p**2*q1**3*q2**2*q3*q4*q5**2 - 100*p**2*q1**3*q2**2*q3*q4*q5*q6 + 900*p**2*q1**3*q2**2*q3*q4*q5 - 100*p**2*q1**3*q2*q3**3*q4*q5 - 100*p**2*q1**3*q2*q3**2*q4**2*q5 - 100*p**2*q1**3*q2*q3**2*q4*q5**2 - 100*p**2*q1**3*q2*q3**2*q4*q5*q6 + 900*p**2*q1**3*q2*q3**2*q4*q5 - 100*p**2*q1**3*q2*q3*q4**3*q5 - 100*p**2*q1**3*q2*q3*q4**2*q5**2 - 100*p**2*q1**3*q2*q3*q4**2*q5*q6 + 900*p**2*q1**3*q2*q3*q4**2*q5 - 100*p**2*q1**3*q2*q3*q4*q5**3 - 100*p**2*q1**3*q2*q3*q4*q5**2*q6 + 900*p**2*q1**3*q2*q3*q4*q5**2 - 100*p**2*q1**3*q2*q3*q4*q5*q6**2 + 900*p**2*q1**3*q2*q3*q4*q5*q6 - 3600*p**2*q1**3*q2*q3*q4*q5 - 100*p**2*q1**2*q2**4*q3*q4*q5 - 100*p**2*q1**2*q2**3*q3**2*q4*q5 - 100*p**2*q1**2*q2**3*q3*q4**2*q5 - 100*p**2*q1**2*q2**3*q3*q4*q5**2 - 100*p**2*q1**2*q2**3*q3*q4*q5*q6 + 900*p**2*q1**2*q2**3*q3*q4*q5 - 100*p**2*q1**2*q2**2*q3**3*q4*q5 - 100*p**2*q1**2*q2**2*q3**2*q4**2*q5 - 100*p**2*q1**2*q2**2*q3**2*q4*q5**2 - 100*p**2*q1**2*q2**2*q3**2*q4*q5*q6 + 900*p**2*q1**2*q2**2*q3**2*q4*q5 - 100*p**2*q1**2*q2**2*q3*q4**3*q5 - 100*p**2*q1**2*q2**2*q3*q4**2*q5**2 - 100*p**2*q1**2*q2**2*q3*q4**2*q5*q6 + 900*p**2*q1**2*q2**2*q3*q4**2*q5 - 100*p**2*q1**2*q2**2*q3*q4*q5**3 - 100*p**2*q1**2*q2**2*q3*q4*q5**2*q6 + 900*p**2*q1**2*q2**2*q3*q4*q5**2 - 100*p**2*q1**2*q2**2*q3*q4*q5*q6**2 + 900*p**2*q1**2*q2**2*q3*q4*q5*q6 - 3600*p**2*q1**2*q2**2*q3*q4*q5 - 100*p**2*q1**2*q2*q3**4*q4*q5 - 100*p**2*q1**2*q2*q3**3*q4**2*q5 - 100*p**2*q1**2*q2*q3**3*q4*q5**2 - 100*p**2*q1**2*q2*q3**3*q4*q5*q6 + 900*p**2*q1**2*q2*q3**3*q4*q5 - 100*p**2*q1**2*q2*q3**2*q4**3*q5 - 100*p**2*q1**2*q2*q3**2*q4**2*q5**2 - 100*p**2*q1**2*q2*q3**2*q4**2*q5*q6 + 900*p**2*q1**2*q2*q3**2*q4**2*q5 - 100*p**2*q1**2*q2*q3**2*q4*q5**3 - 100*p**2*q1**2*q2*q3**2*q4*q5**2*q6 + 900*p**2*q1**2*q2*q3**2*q4*q5**2 - 100*p**2*q1**2*q2*q3**2*q4*q5*q6**2 + 900*p**2*q1**2*q2*q3**2*q4*q5*q6 - 3600*p**2*q1**2*q2*q3**2*q4*q5 - 100*p**2*q1**2*q2*q3*q4**4*q5 - 100*p**2*q1**2*q2*q3*q4**3*q5**2 - 100*p**2*q1**2*q2*q3*q4**3*q5*q6 + 900*p**2*q1**2*q2*q3*q4**3*q5 - 100*p**2*q1**2*q2*q3*q4**2*q5**3 - 100*p**2*q1**2*q2*q3*q4**2*q5**2*q6 + 900*p**2*q1**2*q2*q3*q4**2*q5**2 - 100*p**2*q1**2*q2*q3*q4**2*q5*q6**2 + 900*p**2*q1**2*q2*q3*q4**2*q5*q6 - 3600*p**2*q1**2*q2*q3*q4**2*q5 - 100*p**2*q1**2*q2*q3*q4*q5**4 - 100*p**2*q1**2*q2*q3*q4*q5**3*q6 + 900*p**2*q1**2*q2*q3*q4*q5**3 - 100*p**2*q1**2*q2*q3*q4*q5**2*q6**2 + 900*p**2*q1**2*q2*q3*q4*q5**2*q6 - 3600*p**2*q1**2*q2*q3*q4*q5**2 - 100*p**2*q1**2*q2*q3*q4*q5*q6**3 + 900*p**2*q1**2*q2*q3*q4*q5*q6**2 - 3600*p**2*q1**2*q2*q3*q4*q5*q6 + 8400*p**2*q1**2*q2*q3*q4*q5 - 100*p**2*q1*q2**5*q3*q4*q5 - 100*p**2*q1*q2**4*q3**2*q4*q5 - 100*p**2*q1*q2**4*q3*q4**2*q5 - 100*p**2*q1*q2**4*q3*q4*q5**2 - 100*p**2*q1*q2**4*q3*q4*q5*q6 + 900*p**2*q1*q2**4*q3*q4*q5 - 100*p**2*q1*q2**3*q3**3*q4*q5 - 100*p**2*q1*q2**3*q3**2*q4**2*q5 - 100*p**2*q1*q2**3*q3**2*q4*q5**2 - 100*p**2*q1*q2**3*q3**2*q4*q5*q6 + 900*p**2*q1*q2**3*q3**2*q4*q5 - 100*p**2*q1*q2**3*q3*q4**3*q5 - 100*p**2*q1*q2**3*q3*q4**2*q5**2 - 100*p**2*q1*q2**3*q3*q4**2*q5*q6 + 900*p**2*q1*q2**3*q3*q4**2*q5 - 100*p**2*q1*q2**3*q3*q4*q5**3 - 100*p**2*q1*q2**3*q3*q4*q5**2*q6 + 900*p**2*q1*q2**3*q3*q4*q5**2 - 100*p**2*q1*q2**3*q3*q4*q5*q6**2 + 900*p**2*q1*q2**3*q3*q4*q5*q6 - 3600*p**2*q1*q2**3*q3*q4*q5 - 100*p**2*q1*q2**2*q3**4*q4*q5 - 100*p**2*q1*q2**2*q3**3*q4**2*q5 - 100*p**2*q1*q2**2*q3**3*q4*q5**2 - 100*p**2*q1*q2**2*q3**3*q4*q5*q6 + 900*p**2*q1*q2**2*q3**3*q4*q5 - 100*p**2*q1*q2**2*q3**2*q4**3*q5 - 100*p**2*q1*q2**2*q3**2*q4**2*q5**2 - 100*p**2*q1*q2**2*q3**2*q4**2*q5*q6 + 900*p**2*q1*q2**2*q3**2*q4**2*q5 - 100*p**2*q1*q2**2*q3**2*q4*q5**3 - 100*p**2*q1*q2**2*q3**2*q4*q5**2*q6 + 900*p**2*q1*q2**2*q3**2*q4*q5**2 - 100*p**2*q1*q2**2*q3**2*q4*q5*q6**2 + 900*p**2*q1*q2**2*q3**2*q4*q5*q6 - 3600*p**2*q1*q2**2*q3**2*q4*q5 - 100*p**2*q1*q2**2*q3*q4**4*q5 - 100*p**2*q1*q2**2*q3*q4**3*q5**2 - 100*p**2*q1*q2**2*q3*q4**3*q5*q6 + 900*p**2*q1*q2**2*q3*q4**3*q5 - 100*p**2*q1*q2**2*q3*q4**2*q5**3 - 100*p**2*q1*q2**2*q3*q4**2*q5**2*q6 + 900*p**2*q1*q2**2*q3*q4**2*q5**2 - 100*p**2*q1*q2**2*q3*q4**2*q5*q6**2 + 900*p**2*q1*q2**2*q3*q4**2*q5*q6 - 3600*p**2*q1*q2**2*q3*q4**2*q5 - 100*p**2*q1*q2**2*q3*q4*q5**4 - 100*p**2*q1*q2**2*q3*q4*q5**3*q6 + 900*p**2*q1*q2**2*q3*q4*q5**3 - 100*p**2*q1*q2**2*q3*q4*q5**2*q6**2 + 900*p**2*q1*q2**2*q3*q4*q5**2*q6 - 3600*p**2*q1*q2**2*q3*q4*q5**2 - 100*p**2*q1*q2**2*q3*q4*q5*q6**3 + 900*p**2*q1*q2**2*q3*q4*q5*q6**2 - 3600*p**2*q1*q2**2*q3*q4*q5*q6 + 8400*p**2*q1*q2**2*q3*q4*q5 - 100*p**2*q1*q2*q3**5*q4*q5 - 100*p**2*q1*q2*q3**4*q4**2*q5 - 100*p**2*q1*q2*q3**4*q4*q5**2 - 100*p**2*q1*q2*q3**4*q4*q5*q6 + 900*p**2*q1*q2*q3**4*q4*q5 - 100*p**2*q1*q2*q3**3*q4**3*q5 - 100*p**2*q1*q2*q3**3*q4**2*q5**2 - 100*p**2*q1*q2*q3**3*q4**2*q5*q6 + 900*p**2*q1*q2*q3**3*q4**2*q5 - 100*p**2*q1*q2*q3**3*q4*q5**3 - 100*p**2*q1*q2*q3**3*q4*q5**2*q6 + 900*p**2*q1*q2*q3**3*q4*q5**2 - 100*p**2*q1*q2*q3**3*q4*q5*q6**2 + 900*p**2*q1*q2*q3**3*q4*q5*q6 - 3600*p**2*q1*q2*q3**3*q4*q5 - 100*p**2*q1*q2*q3**2*q4**4*q5 - 100*p**2*q1*q2*q3**2*q4**3*q5**2 - 100*p**2*q1*q2*q3**2*q4**3*q5*q6 + 900*p**2*q1*q2*q3**2*q4**3*q5 - 100*p**2*q1*q2*q3**2*q4**2*q5**3 - 100*p**2*q1*q2*q3**2*q4**2*q5**2*q6 + 900*p**2*q1*q2*q3**2*q4**2*q5**2 - 100*p**2*q1*q2*q3**2*q4**2*q5*q6**2 + 900*p**2*q1*q2*q3**2*q4**2*q5*q6 - 3600*p**2*q1*q2*q3**2*q4**2*q5 - 100*p**2*q1*q2*q3**2*q4*q5**4 - 100*p**2*q1*q2*q3**2*q4*q5**3*q6 + 900*p**2*q1*q2*q3**2*q4*q5**3 - 100*p**2*q1*q2*q3**2*q4*q5**2*q6**2 + 900*p**2*q1*q2*q3**2*q4*q5**2*q6 - 3600*p**2*q1*q2*q3**2*q4*q5**2 - 100*p**2*q1*q2*q3**2*q4*q5*q6**3 + 900*p**2*q1*q2*q3**2*q4*q5*q6**2 - 3600*p**2*q1*q2*q3**2*q4*q5*q6 + 8400*p**2*q1*q2*q3**2*q4*q5 - 100*p**2*q1*q2*q3*q4**5*q5 - 100*p**2*q1*q2*q3*q4**4*q5**2 - 100*p**2*q1*q2*q3*q4**4*q5*q6 + 900*p**2*q1*q2*q3*q4**4*q5 - 100*p**2*q1*q2*q3*q4**3*q5**3 - 100*p**2*q1*q2*q3*q4**3*q5**2*q6 + 900*p**2*q1*q2*q3*q4**3*q5**2 - 100*p**2*q1*q2*q3*q4**3*q5*q6**2 + 900*p**2*q1*q2*q3*q4**3*q5*q6 - 3600*p**2*q1*q2*q3*q4**3*q5 - 100*p**2*q1*q2*q3*q4**2*q5**4 - 100*p**2*q1*q2*q3*q4**2*q5**3*q6 + 900*p**2*q1*q2*q3*q4**2*q5**3 - 100*p**2*q1*q2*q3*q4**2*q5**2*q6**2 + 900*p**2*q1*q2*q3*q4**2*q5**2*q6 - 3600*p**2*q1*q2*q3*q4**2*q5**2 - 100*p**2*q1*q2*q3*q4**2*q5*q6**3 + 900*p**2*q1*q2*q3*q4**2*q5*q6**2 - 3600*p**2*q1*q2*q3*q4**2*q5*q6 + 8400*p**2*q1*q2*q3*q4**2*q5 - 100*p**2*q1*q2*q3*q4*q5**5 - 100*p**2*q1*q2*q3*q4*q5**4*q6 + 900*p**2*q1*q2*q3*q4*q5**4 - 100*p**2*q1*q2*q3*q4*q5**3*q6**2 + 900*p**2*q1*q2*q3*q4*q5**3*q6 - 3600*p**2*q1*q2*q3*q4*q5**3 - 100*p**2*q1*q2*q3*q4*q5**2*q6**3 + 900*p**2*q1*q2*q3*q4*q5**2*q6**2 - 3600*p**2*q1*q2*q3*q4*q5**2*q6 + 8400*p**2*q1*q2*q3*q4*q5**2 - 100*p**2*q1*q2*q3*q4*q5*q6**4 + 900*p**2*q1*q2*q3*q4*q5*q6**3 - 3600*p**2*q1*q2*q3*q4*q5*q6**2 + 8400*p**2*q1*q2*q3*q4*q5*q6 - 12600*p**2*q1*q2*q3*q4*q5 + 180*p**2*q2**5*q3*q4*q5 + 180*p**2*q2**4*q3**2*q4*q5 + 180*p**2*q2**4*q3*q4**2*q5 + 180*p**2*q2**4*q3*q4*q5**2 + 180*p**2*q2**4*q3*q4*q5*q6 - 1440*p**2*q2**4*q3*q4*q5 + 180*p**2*q2**3*q3**3*q4*q5 + 180*p**2*q2**3*q3**2*q4**2*q5 + 180*p**2*q2**3*q3**2*q4*q5**2 + 180*p**2*q2**3*q3**2*q4*q5*q6 - 1440*p**2*q2**3*q3**2*q4*q5 + 180*p**2*q2**3*q3*q4**3*q5 + 180*p**2*q2**3*q3*q4**2*q5**2 + 180*p**2*q2**3*q3*q4**2*q5*q6 - 1440*p**2*q2**3*q3*q4**2*q5 + 180*p**2*q2**3*q3*q4*q5**3 + 180*p**2*q2**3*q3*q4*q5**2*q6 - 1440*p**2*q2**3*q3*q4*q5**2 + 180*p**2*q2**3*q3*q4*q5*q6**2 - 1440*p**2*q2**3*q3*q4*q5*q6 + 5040*p**2*q2**3*q3*q4*q5 + 180*p**2*q2**2*q3**4*q4*q5 + 180*p**2*q2**2*q3**3*q4**2*q5 + 180*p**2*q2**2*q3**3*q4*q5**2 + 180*p**2*q2**2*q3**3*q4*q5*q6 - 1440*p**2*q2**2*q3**3*q4*q5 + 180*p**2*q2**2*q3**2*q4**3*q5 + 180*p**2*q2**2*q3**2*q4**2*q5**2 + 180*p**2*q2**2*q3**2*q4**2*q5*q6 - 1440*p**2*q2**2*q3**2*q4**2*q5 + 180*p**2*q2**2*q3**2*q4*q5**3 + 180*p**2*q2**2*q3**2*q4*q5**2*q6 - 1440*p**2*q2**2*q3**2*q4*q5**2 + 180*p**2*q2**2*q3**2*q4*q5*q6**2 - 1440*p**2*q2**2*q3**2*q4*q5*q6 + 5040*p**2*q2**2*q3**2*q4*q5 + 180*p**2*q2**2*q3*q4**4*q5 + 180*p**2*q2**2*q3*q4**3*q5**2 + 180*p**2*q2**2*q3*q4**3*q5*q6 - 1440*p**2*q2**2*q3*q4**3*q5 + 180*p**2*q2**2*q3*q4**2*q5**3 + 180*p**2*q2**2*q3*q4**2*q5**2*q6 - 1440*p**2*q2**2*q3*q4**2*q5**2 + 180*p**2*q2**2*q3*q4**2*q5*q6**2 - 1440*p**2*q2**2*q3*q4**2*q5*q6 + 5040*p**2*q2**2*q3*q4**2*q5 + 180*p**2*q2**2*q3*q4*q5**4 + 180*p**2*q2**2*q3*q4*q5**3*q6 - 1440*p**2*q2**2*q3*q4*q5**3 + 180*p**2*q2**2*q3*q4*q5**2*q6**2 - 1440*p**2*q2**2*q3*q4*q5**2*q6 + 5040*p**2*q2**2*q3*q4*q5**2 + 180*p**2*q2**2*q3*q4*q5*q6**3 - 1440*p**2*q2**2*q3*q4*q5*q6**2 + 5040*p**2*q2**2*q3*q4*q5*q6 - 10080*p**2*q2**2*q3*q4*q5 + 180*p**2*q2*q3**5*q4*q5 + 180*p**2*q2*q3**4*q4**2*q5 + 180*p**2*q2*q3**4*q4*q5**2 + 180*p**2*q2*q3**4*q4*q5*q6 - 1440*p**2*q2*q3**4*q4*q5 + 180*p**2*q2*q3**3*q4**3*q5 + 180*p**2*q2*q3**3*q4**2*q5**2 + 180*p**2*q2*q3**3*q4**2*q5*q6 - 1440*p**2*q2*q3**3*q4**2*q5 + 180*p**2*q2*q3**3*q4*q5**3 + 180*p**2*q2*q3**3*q4*q5**2*q6 - 1440*p**2*q2*q3**3*q4*q5**2 + 180*p**2*q2*q3**3*q4*q5*q6**2 - 1440*p**2*q2*q3**3*q4*q5*q6 + 5040*p**2*q2*q3**3*q4*q5 + 180*p**2*q2*q3**2*q4**4*q5 + 180*p**2*q2*q3**2*q4**3*q5**2 + 180*p**2*q2*q3**2*q4**3*q5*q6 - 1440*p**2*q2*q3**2*q4**3*q5 + 180*p**2*q2*q3**2*q4**2*q5**3 + 180*p**2*q2*q3**2*q4**2*q5**2*q6 - 1440*p**2*q2*q3**2*q4**2*q5**2 + 180*p**2*q2*q3**2*q4**2*q5*q6**2 - 1440*p**2*q2*q3**2*q4**2*q5*q6 + 5040*p**2*q2*q3**2*q4**2*q5 + 180*p**2*q2*q3**2*q4*q5**4 + 180*p**2*q2*q3**2*q4*q5**3*q6 - 1440*p**2*q2*q3**2*q4*q5**3 + 180*p**2*q2*q3**2*q4*q5**2*q6**2 - 1440*p**2*q2*q3**2*q4*q5**2*q6 + 5040*p**2*q2*q3**2*q4*q5**2 + 180*p**2*q2*q3**2*q4*q5*q6**3 - 1440*p**2*q2*q3**2*q4*q5*q6**2 + 5040*p**2*q2*q3**2*q4*q5*q6 - 10080*p**2*q2*q3**2*q4*q5 + 180*p**2*q2*q3*q4**5*q5 + 180*p**2*q2*q3*q4**4*q5**2 + 180*p**2*q2*q3*q4**4*q5*q6 - 1440*p**2*q2*q3*q4**4*q5 + 180*p**2*q2*q3*q4**3*q5**3 + 180*p**2*q2*q3*q4**3*q5**2*q6 - 1440*p**2*q2*q3*q4**3*q5**2 + 180*p**2*q2*q3*q4**3*q5*q6**2 - 1440*p**2*q2*q3*q4**3*q5*q6 + 5040*p**2*q2*q3*q4**3*q5 + 180*p**2*q2*q3*q4**2*q5**4 + 180*p**2*q2*q3*q4**2*q5**3*q6 - 1440*p**2*q2*q3*q4**2*q5**3 + 180*p**2*q2*q3*q4**2*q5**2*q6**2 - 1440*p**2*q2*q3*q4**2*q5**2*q6 + 5040*p**2*q2*q3*q4**2*q5**2 + 180*p**2*q2*q3*q4**2*q5*q6**3 - 1440*p**2*q2*q3*q4**2*q5*q6**2 + 5040*p**2*q2*q3*q4**2*q5*q6 - 10080*p**2*q2*q3*q4**2*q5 + 180*p**2*q2*q3*q4*q5**5 + 180*p**2*q2*q3*q4*q5**4*q6 - 1440*p**2*q2*q3*q4*q5**4 + 180*p**2*q2*q3*q4*q5**3*q6**2 - 1440*p**2*q2*q3*q4*q5**3*q6 + 5040*p**2*q2*q3*q4*q5**3 + 180*p**2*q2*q3*q4*q5**2*q6**3 - 1440*p**2*q2*q3*q4*q5**2*q6**2 + 5040*p**2*q2*q3*q4*q5**2*q6 - 10080*p**2*q2*q3*q4*q5**2 + 180*p**2*q2*q3*q4*q5*q6**4 - 1440*p**2*q2*q3*q4*q5*q6**3 + 5040*p**2*q2*q3*q4*q5*q6**2 - 10080*p**2*q2*q3*q4*q5*q6 + 12600*p**2*q2*q3*q4*q5 - 120*p**2*q3**5*q4*q5 - 120*p**2*q3**4*q4**2*q5 - 120*p**2*q3**4*q4*q5**2 - 120*p**2*q3**4*q4*q5*q6 + 840*p**2*q3**4*q4*q5 - 120*p**2*q3**3*q4**3*q5 - 120*p**2*q3**3*q4**2*q5**2 - 120*p**2*q3**3*q4**2*q5*q6 + 840*p**2*q3**3*q4**2*q5 - 120*p**2*q3**3*q4*q5**3 - 120*p**2*q3**3*q4*q5**2*q6 + 840*p**2*q3**3*q4*q5**2 - 120*p**2*q3**3*q4*q5*q6**2 + 840*p**2*q3**3*q4*q5*q6 - 2520*p**2*q3**3*q4*q5 - 120*p**2*q3**2*q4**4*q5 - 120*p**2*q3**2*q4**3*q5**2 - 120*p**2*q3**2*q4**3*q5*q6 + 840*p**2*q3**2*q4**3*q5 - 120*p**2*q3**2*q4**2*q5**3 - 120*p**2*q3**2*q4**2*q5**2*q6 + 840*p**2*q3**2*q4**2*q5**2 - 120*p**2*q3**2*q4**2*q5*q6**2 + 840*p**2*q3**2*q4**2*q5*q6 - 2520*p**2*q3**2*q4**2*q5 - 120*p**2*q3**2*q4*q5**4 - 120*p**2*q3**2*q4*q5**3*q6 + 840*p**2*q3**2*q4*q5**3 - 120*p**2*q3**2*q4*q5**2*q6**2 + 840*p**2*q3**2*q4*q5**2*q6 - 2520*p**2*q3**2*q4*q5**2 - 120*p**2*q3**2*q4*q5*q6**3 + 840*p**2*q3**2*q4*q5*q6**2 - 2520*p**2*q3**2*q4*q5*q6 + 4200*p**2*q3**2*q4*q5 - 120*p**2*q3*q4**5*q5 - 120*p**2*q3*q4**4*q5**2 - 120*p**2*q3*q4**4*q5*q6 + 840*p**2*q3*q4**4*q5 - 120*p**2*q3*q4**3*q5**3 - 120*p**2*q3*q4**3*q5**2*q6 + 840*p**2*q3*q4**3*q5**2 - 120*p**2*q3*q4**3*q5*q6**2 + 840*p**2*q3*q4**3*q5*q6 - 2520*p**2*q3*q4**3*q5 - 120*p**2*q3*q4**2*q5**4 - 120*p**2*q3*q4**2*q5**3*q6 + 840*p**2*q3*q4**2*q5**3 - 120*p**2*q3*q4**2*q5**2*q6**2 + 840*p**2*q3*q4**2*q5**2*q6 - 2520*p**2*q3*q4**2*q5**2 - 120*p**2*q3*q4**2*q5*q6**3 + 840*p**2*q3*q4**2*q5*q6**2 - 2520*p**2*q3*q4**2*q5*q6 + 4200*p**2*q3*q4**2*q5 - 120*p**2*q3*q4*q5**5 - 120*p**2*q3*q4*q5**4*q6 + 840*p**2*q3*q4*q5**4 - 120*p**2*q3*q4*q5**3*q6**2 + 840*p**2*q3*q4*q5**3*q6 - 2520*p**2*q3*q4*q5**3 - 120*p**2*q3*q4*q5**2*q6**3 + 840*p**2*q3*q4*q5**2*q6**2 - 2520*p**2*q3*q4*q5**2*q6 + 4200*p**2*q3*q4*q5**2 - 120*p**2*q3*q4*q5*q6**4 + 840*p**2*q3*q4*q5*q6**3 - 2520*p**2*q3*q4*q5*q6**2 + 4200*p**2*q3*q4*q5*q6 - 4200*p**2*q3*q4*q5 + 50*p*q1**5*q2*q3*q4*q5 + 50*p*q1**4*q2**2*q3*q4*q5 + 50*p*q1**4*q2*q3**2*q4*q5 + 50*p*q1**4*q2*q3*q4**2*q5 + 50*p*q1**4*q2*q3*q4*q5**2 + 50*p*q1**4*q2*q3*q4*q5*q6 - 450*p*q1**4*q2*q3*q4*q5 + 50*p*q1**3*q2**3*q3*q4*q5 + 50*p*q1**3*q2**2*q3**2*q4*q5 + 50*p*q1**3*q2**2*q3*q4**2*q5 + 50*p*q1**3*q2**2*q3*q4*q5**2 + 50*p*q1**3*q2**2*q3*q4*q5*q6 - 450*p*q1**3*q2**2*q3*q4*q5 + 50*p*q1**3*q2*q3**3*q4*q5 + 50*p*q1**3*q2*q3**2*q4**2*q5 + 50*p*q1**3*q2*q3**2*q4*q5**2 + 50*p*q1**3*q2*q3**2*q4*q5*q6 - 450*p*q1**3*q2*q3**2*q4*q5 + 50*p*q1**3*q2*q3*q4**3*q5 + 50*p*q1**3*q2*q3*q4**2*q5**2 + 50*p*q1**3*q2*q3*q4**2*q5*q6 - 450*p*q1**3*q2*q3*q4**2*q5 + 50*p*q1**3*q2*q3*q4*q5**3 + 50*p*q1**3*q2*q3*q4*q5**2*q6 - 450*p*q1**3*q2*q3*q4*q5**2 + 50*p*q1**3*q2*q3*q4*q5*q6**2 - 450*p*q1**3*q2*q3*q4*q5*q6 + 1800*p*q1**3*q2*q3*q4*q5 + 50*p*q1**2*q2**4*q3*q4*q5 + 50*p*q1**2*q2**3*q3**2*q4*q5 + 50*p*q1**2*q2**3*q3*q4**2*q5 + 50*p*q1**2*q2**3*q3*q4*q5**2 + 50*p*q1**2*q2**3*q3*q4*q5*q6 - 450*p*q1**2*q2**3*q3*q4*q5 + 50*p*q1**2*q2**2*q3**3*q4*q5 + 50*p*q1**2*q2**2*q3**2*q4**2*q5 + 50*p*q1**2*q2**2*q3**2*q4*q5**2 + 50*p*q1**2*q2**2*q3**2*q4*q5*q6 - 450*p*q1**2*q2**2*q3**2*q4*q5 + 50*p*q1**2*q2**2*q3*q4**3*q5 + 50*p*q1**2*q2**2*q3*q4**2*q5**2 + 50*p*q1**2*q2**2*q3*q4**2*q5*q6 - 450*p*q1**2*q2**2*q3*q4**2*q5 + 50*p*q1**2*q2**2*q3*q4*q5**3 + 50*p*q1**2*q2**2*q3*q4*q5**2*q6 - 450*p*q1**2*q2**2*q3*q4*q5**2 + 50*p*q1**2*q2**2*q3*q4*q5*q6**2 - 450*p*q1**2*q2**2*q3*q4*q5*q6 + 1800*p*q1**2*q2**2*q3*q4*q5 + 50*p*q1**2*q2*q3**4*q4*q5 + 50*p*q1**2*q2*q3**3*q4**2*q5 + 50*p*q1**2*q2*q3**3*q4*q5**2 + 50*p*q1**2*q2*q3**3*q4*q5*q6 - 450*p*q1**2*q2*q3**3*q4*q5 + 50*p*q1**2*q2*q3**2*q4**3*q5 + 50*p*q1**2*q2*q3**2*q4**2*q5**2 + 50*p*q1**2*q2*q3**2*q4**2*q5*q6 - 450*p*q1**2*q2*q3**2*q4**2*q5 + 50*p*q1**2*q2*q3**2*q4*q5**3 + 50*p*q1**2*q2*q3**2*q4*q5**2*q6 - 450*p*q1**2*q2*q3**2*q4*q5**2 + 50*p*q1**2*q2*q3**2*q4*q5*q6**2 - 450*p*q1**2*q2*q3**2*q4*q5*q6 + 1800*p*q1**2*q2*q3**2*q4*q5 + 50*p*q1**2*q2*q3*q4**4*q5 + 50*p*q1**2*q2*q3*q4**3*q5**2 + 50*p*q1**2*q2*q3*q4**3*q5*q6 - 450*p*q1**2*q2*q3*q4**3*q5 + 50*p*q1**2*q2*q3*q4**2*q5**3 + 50*p*q1**2*q2*q3*q4**2*q5**2*q6 - 450*p*q1**2*q2*q3*q4**2*q5**2 + 50*p*q1**2*q2*q3*q4**2*q5*q6**2 - 450*p*q1**2*q2*q3*q4**2*q5*q6 + 1800*p*q1**2*q2*q3*q4**2*q5 + 50*p*q1**2*q2*q3*q4*q5**4 + 50*p*q1**2*q2*q3*q4*q5**3*q6 - 450*p*q1**2*q2*q3*q4*q5**3 + 50*p*q1**2*q2*q3*q4*q5**2*q6**2 - 450*p*q1**2*q2*q3*q4*q5**2*q6 + 1800*p*q1**2*q2*q3*q4*q5**2 + 50*p*q1**2*q2*q3*q4*q5*q6**3 - 450*p*q1**2*q2*q3*q4*q5*q6**2 + 1800*p*q1**2*q2*q3*q4*q5*q6 - 4200*p*q1**2*q2*q3*q4*q5 + 50*p*q1*q2**5*q3*q4*q5 + 50*p*q1*q2**4*q3**2*q4*q5 + 50*p*q1*q2**4*q3*q4**2*q5 + 50*p*q1*q2**4*q3*q4*q5**2 + 50*p*q1*q2**4*q3*q4*q5*q6 - 450*p*q1*q2**4*q3*q4*q5 + 50*p*q1*q2**3*q3**3*q4*q5 + 50*p*q1*q2**3*q3**2*q4**2*q5 + 50*p*q1*q2**3*q3**2*q4*q5**2 + 50*p*q1*q2**3*q3**2*q4*q5*q6 - 450*p*q1*q2**3*q3**2*q4*q5 + 50*p*q1*q2**3*q3*q4**3*q5 + 50*p*q1*q2**3*q3*q4**2*q5**2 + 50*p*q1*q2**3*q3*q4**2*q5*q6 - 450*p*q1*q2**3*q3*q4**2*q5 + 50*p*q1*q2**3*q3*q4*q5**3 + 50*p*q1*q2**3*q3*q4*q5**2*q6 - 450*p*q1*q2**3*q3*q4*q5**2 + 50*p*q1*q2**3*q3*q4*q5*q6**2 - 450*p*q1*q2**3*q3*q4*q5*q6 + 1800*p*q1*q2**3*q3*q4*q5 + 50*p*q1*q2**2*q3**4*q4*q5 + 50*p*q1*q2**2*q3**3*q4**2*q5 + 50*p*q1*q2**2*q3**3*q4*q5**2 + 50*p*q1*q2**2*q3**3*q4*q5*q6 - 450*p*q1*q2**2*q3**3*q4*q5 + 50*p*q1*q2**2*q3**2*q4**3*q5 + 50*p*q1*q2**2*q3**2*q4**2*q5**2 + 50*p*q1*q2**2*q3**2*q4**2*q5*q6 - 450*p*q1*q2**2*q3**2*q4**2*q5 + 50*p*q1*q2**2*q3**2*q4*q5**3 + 50*p*q1*q2**2*q3**2*q4*q5**2*q6 - 450*p*q1*q2**2*q3**2*q4*q5**2 + 50*p*q1*q2**2*q3**2*q4*q5*q6**2 - 450*p*q1*q2**2*q3**2*q4*q5*q6 + 1800*p*q1*q2**2*q3**2*q4*q5 + 50*p*q1*q2**2*q3*q4**4*q5 + 50*p*q1*q2**2*q3*q4**3*q5**2 + 50*p*q1*q2**2*q3*q4**3*q5*q6 - 450*p*q1*q2**2*q3*q4**3*q5 + 50*p*q1*q2**2*q3*q4**2*q5**3 + 50*p*q1*q2**2*q3*q4**2*q5**2*q6 - 450*p*q1*q2**2*q3*q4**2*q5**2 + 50*p*q1*q2**2*q3*q4**2*q5*q6**2 - 450*p*q1*q2**2*q3*q4**2*q5*q6 + 1800*p*q1*q2**2*q3*q4**2*q5 + 50*p*q1*q2**2*q3*q4*q5**4 + 50*p*q1*q2**2*q3*q4*q5**3*q6 - 450*p*q1*q2**2*q3*q4*q5**3 + 50*p*q1*q2**2*q3*q4*q5**2*q6**2 - 450*p*q1*q2**2*q3*q4*q5**2*q6 + 1800*p*q1*q2**2*q3*q4*q5**2 + 50*p*q1*q2**2*q3*q4*q5*q6**3 - 450*p*q1*q2**2*q3*q4*q5*q6**2 + 1800*p*q1*q2**2*q3*q4*q5*q6 - 4200*p*q1*q2**2*q3*q4*q5 + 50*p*q1*q2*q3**5*q4*q5 + 50*p*q1*q2*q3**4*q4**2*q5 + 50*p*q1*q2*q3**4*q4*q5**2 + 50*p*q1*q2*q3**4*q4*q5*q6 - 450*p*q1*q2*q3**4*q4*q5 + 50*p*q1*q2*q3**3*q4**3*q5 + 50*p*q1*q2*q3**3*q4**2*q5**2 + 50*p*q1*q2*q3**3*q4**2*q5*q6 - 450*p*q1*q2*q3**3*q4**2*q5 + 50*p*q1*q2*q3**3*q4*q5**3 + 50*p*q1*q2*q3**3*q4*q5**2*q6 - 450*p*q1*q2*q3**3*q4*q5**2 + 50*p*q1*q2*q3**3*q4*q5*q6**2 - 450*p*q1*q2*q3**3*q4*q5*q6 + 1800*p*q1*q2*q3**3*q4*q5 + 50*p*q1*q2*q3**2*q4**4*q5 + 50*p*q1*q2*q3**2*q4**3*q5**2 + 50*p*q1*q2*q3**2*q4**3*q5*q6 - 450*p*q1*q2*q3**2*q4**3*q5 + 50*p*q1*q2*q3**2*q4**2*q5**3 + 50*p*q1*q2*q3**2*q4**2*q5**2*q6 - 450*p*q1*q2*q3**2*q4**2*q5**2 + 50*p*q1*q2*q3**2*q4**2*q5*q6**2 - 450*p*q1*q2*q3**2*q4**2*q5*q6 + 1800*p*q1*q2*q3**2*q4**2*q5 + 50*p*q1*q2*q3**2*q4*q5**4 + 50*p*q1*q2*q3**2*q4*q5**3*q6 - 450*p*q1*q2*q3**2*q4*q5**3 + 50*p*q1*q2*q3**2*q4*q5**2*q6**2 - 450*p*q1*q2*q3**2*q4*q5**2*q6 + 1800*p*q1*q2*q3**2*q4*q5**2 + 50*p*q1*q2*q3**2*q4*q5*q6**3 - 450*p*q1*q2*q3**2*q4*q5*q6**2 + 1800*p*q1*q2*q3**2*q4*q5*q6 - 4200*p*q1*q2*q3**2*q4*q5 + 50*p*q1*q2*q3*q4**5*q5 + 50*p*q1*q2*q3*q4**4*q5**2 + 50*p*q1*q2*q3*q4**4*q5*q6 - 450*p*q1*q2*q3*q4**4*q5 + 50*p*q1*q2*q3*q4**3*q5**3 + 50*p*q1*q2*q3*q4**3*q5**2*q6 - 450*p*q1*q2*q3*q4**3*q5**2 + 50*p*q1*q2*q3*q4**3*q5*q6**2 - 450*p*q1*q2*q3*q4**3*q5*q6 + 1800*p*q1*q2*q3*q4**3*q5 + 50*p*q1*q2*q3*q4**2*q5**4 + 50*p*q1*q2*q3*q4**2*q5**3*q6 - 450*p*q1*q2*q3*q4**2*q5**3 + 50*p*q1*q2*q3*q4**2*q5**2*q6**2 - 450*p*q1*q2*q3*q4**2*q5**2*q6 + 1800*p*q1*q2*q3*q4**2*q5**2 + 50*p*q1*q2*q3*q4**2*q5*q6**3 - 450*p*q1*q2*q3*q4**2*q5*q6**2 + 1800*p*q1*q2*q3*q4**2*q5*q6 - 4200*p*q1*q2*q3*q4**2*q5 + 50*p*q1*q2*q3*q4*q5**5 + 50*p*q1*q2*q3*q4*q5**4*q6 - 450*p*q1*q2*q3*q4*q5**4 + 50*p*q1*q2*q3*q4*q5**3*q6**2 - 450*p*q1*q2*q3*q4*q5**3*q6 + 1800*p*q1*q2*q3*q4*q5**3 + 50*p*q1*q2*q3*q4*q5**2*q6**3 - 450*p*q1*q2*q3*q4*q5**2*q6**2 + 1800*p*q1*q2*q3*q4*q5**2*q6 - 4200*p*q1*q2*q3*q4*q5**2 + 50*p*q1*q2*q3*q4*q5*q6**4 - 450*p*q1*q2*q3*q4*q5*q6**3 + 1800*p*q1*q2*q3*q4*q5*q6**2 - 4200*p*q1*q2*q3*q4*q5*q6 + 6300*p*q1*q2*q3*q4*q5 - 45*p*q2**5*q3*q4*q5 - 45*p*q2**4*q3**2*q4*q5 - 45*p*q2**4*q3*q4**2*q5 - 45*p*q2**4*q3*q4*q5**2 - 45*p*q2**4*q3*q4*q5*q6 + 360*p*q2**4*q3*q4*q5 - 45*p*q2**3*q3**3*q4*q5 - 45*p*q2**3*q3**2*q4**2*q5 - 45*p*q2**3*q3**2*q4*q5**2 - 45*p*q2**3*q3**2*q4*q5*q6 + 360*p*q2**3*q3**2*q4*q5 - 45*p*q2**3*q3*q4**3*q5 - 45*p*q2**3*q3*q4**2*q5**2 - 45*p*q2**3*q3*q4**2*q5*q6 + 360*p*q2**3*q3*q4**2*q5 - 45*p*q2**3*q3*q4*q5**3 - 45*p*q2**3*q3*q4*q5**2*q6 + 360*p*q2**3*q3*q4*q5**2 - 45*p*q2**3*q3*q4*q5*q6**2 + 360*p*q2**3*q3*q4*q5*q6 - 1260*p*q2**3*q3*q4*q5 - 45*p*q2**2*q3**4*q4*q5 - 45*p*q2**2*q3**3*q4**2*q5 - 45*p*q2**2*q3**3*q4*q5**2 - 45*p*q2**2*q3**3*q4*q5*q6 + 360*p*q2**2*q3**3*q4*q5 - 45*p*q2**2*q3**2*q4**3*q5 - 45*p*q2**2*q3**2*q4**2*q5**2 - 45*p*q2**2*q3**2*q4**2*q5*q6 + 360*p*q2**2*q3**2*q4**2*q5 - 45*p*q2**2*q3**2*q4*q5**3 - 45*p*q2**2*q3**2*q4*q5**2*q6 + 360*p*q2**2*q3**2*q4*q5**2 - 45*p*q2**2*q3**2*q4*q5*q6**2 + 360*p*q2**2*q3**2*q4*q5*q6 - 1260*p*q2**2*q3**2*q4*q5 - 45*p*q2**2*q3*q4**4*q5 - 45*p*q2**2*q3*q4**3*q5**2 - 45*p*q2**2*q3*q4**3*q5*q6 + 360*p*q2**2*q3*q4**3*q5 - 45*p*q2**2*q3*q4**2*q5**3 - 45*p*q2**2*q3*q4**2*q5**2*q6 + 360*p*q2**2*q3*q4**2*q5**2 - 45*p*q2**2*q3*q4**2*q5*q6**2 + 360*p*q2**2*q3*q4**2*q5*q6 - 1260*p*q2**2*q3*q4**2*q5 - 45*p*q2**2*q3*q4*q5**4 - 45*p*q2**2*q3*q4*q5**3*q6 + 360*p*q2**2*q3*q4*q5**3 - 45*p*q2**2*q3*q4*q5**2*q6**2 + 360*p*q2**2*q3*q4*q5**2*q6 - 1260*p*q2**2*q3*q4*q5**2 - 45*p*q2**2*q3*q4*q5*q6**3 + 360*p*q2**2*q3*q4*q5*q6**2 - 1260*p*q2**2*q3*q4*q5*q6 + 2520*p*q2**2*q3*q4*q5 - 45*p*q2*q3**5*q4*q5 - 45*p*q2*q3**4*q4**2*q5 - 45*p*q2*q3**4*q4*q5**2 - 45*p*q2*q3**4*q4*q5*q6 + 360*p*q2*q3**4*q4*q5 - 45*p*q2*q3**3*q4**3*q5 - 45*p*q2*q3**3*q4**2*q5**2 - 45*p*q2*q3**3*q4**2*q5*q6 + 360*p*q2*q3**3*q4**2*q5 - 45*p*q2*q3**3*q4*q5**3 - 45*p*q2*q3**3*q4*q5**2*q6 + 360*p*q2*q3**3*q4*q5**2 - 45*p*q2*q3**3*q4*q5*q6**2 + 360*p*q2*q3**3*q4*q5*q6 - 1260*p*q2*q3**3*q4*q5 - 45*p*q2*q3**2*q4**4*q5 - 45*p*q2*q3**2*q4**3*q5**2 - 45*p*q2*q3**2*q4**3*q5*q6 + 360*p*q2*q3**2*q4**3*q5 - 45*p*q2*q3**2*q4**2*q5**3 - 45*p*q2*q3**2*q4**2*q5**2*q6 + 360*p*q2*q3**2*q4**2*q5**2 - 45*p*q2*q3**2*q4**2*q5*q6**2 + 360*p*q2*q3**2*q4**2*q5*q6 - 1260*p*q2*q3**2*q4**2*q5 - 45*p*q2*q3**2*q4*q5**4 - 45*p*q2*q3**2*q4*q5**3*q6 + 360*p*q2*q3**2*q4*q5**3 - 45*p*q2*q3**2*q4*q5**2*q6**2 + 360*p*q2*q3**2*q4*q5**2*q6 - 1260*p*q2*q3**2*q4*q5**2 - 45*p*q2*q3**2*q4*q5*q6**3 + 360*p*q2*q3**2*q4*q5*q6**2 - 1260*p*q2*q3**2*q4*q5*q6 + 2520*p*q2*q3**2*q4*q5 - 45*p*q2*q3*q4**5*q5 - 45*p*q2*q3*q4**4*q5**2 - 45*p*q2*q3*q4**4*q5*q6 + 360*p*q2*q3*q4**4*q5 - 45*p*q2*q3*q4**3*q5**3 - 45*p*q2*q3*q4**3*q5**2*q6 + 360*p*q2*q3*q4**3*q5**2 - 45*p*q2*q3*q4**3*q5*q6**2 + 360*p*q2*q3*q4**3*q5*q6 - 1260*p*q2*q3*q4**3*q5 - 45*p*q2*q3*q4**2*q5**4 - 45*p*q2*q3*q4**2*q5**3*q6 + 360*p*q2*q3*q4**2*q5**3 - 45*p*q2*q3*q4**2*q5**2*q6**2 + 360*p*q2*q3*q4**2*q5**2*q6 - 1260*p*q2*q3*q4**2*q5**2 - 45*p*q2*q3*q4**2*q5*q6**3 + 360*p*q2*q3*q4**2*q5*q6**2 - 1260*p*q2*q3*q4**2*q5*q6 + 2520*p*q2*q3*q4**2*q5 - 45*p*q2*q3*q4*q5**5 - 45*p*q2*q3*q4*q5**4*q6 + 360*p*q2*q3*q4*q5**4 - 45*p*q2*q3*q4*q5**3*q6**2 + 360*p*q2*q3*q4*q5**3*q6 - 1260*p*q2*q3*q4*q5**3 - 45*p*q2*q3*q4*q5**2*q6**3 + 360*p*q2*q3*q4*q5**2*q6**2 - 1260*p*q2*q3*q4*q5**2*q6 + 2520*p*q2*q3*q4*q5**2 - 45*p*q2*q3*q4*q5*q6**4 + 360*p*q2*q3*q4*q5*q6**3 - 1260*p*q2*q3*q4*q5*q6**2 + 2520*p*q2*q3*q4*q5*q6 - 3150*p*q2*q3*q4*q5 - 10*q1**5*q2*q3*q4*q5 - 10*q1**4*q2**2*q3*q4*q5 - 10*q1**4*q2*q3**2*q4*q5 - 10*q1**4*q2*q3*q4**2*q5 - 10*q1**4*q2*q3*q4*q5**2 - 10*q1**4*q2*q3*q4*q5*q6 + 90*q1**4*q2*q3*q4*q5 - 10*q1**3*q2**3*q3*q4*q5 - 10*q1**3*q2**2*q3**2*q4*q5 - 10*q1**3*q2**2*q3*q4**2*q5 - 10*q1**3*q2**2*q3*q4*q5**2 - 10*q1**3*q2**2*q3*q4*q5*q6 + 90*q1**3*q2**2*q3*q4*q5 - 10*q1**3*q2*q3**3*q4*q5 - 10*q1**3*q2*q3**2*q4**2*q5 - 10*q1**3*q2*q3**2*q4*q5**2 - 10*q1**3*q2*q3**2*q4*q5*q6 + 90*q1**3*q2*q3**2*q4*q5 - 10*q1**3*q2*q3*q4**3*q5 - 10*q1**3*q2*q3*q4**2*q5**2 - 10*q1**3*q2*q3*q4**2*q5*q6 + 90*q1**3*q2*q3*q4**2*q5 - 10*q1**3*q2*q3*q4*q5**3 - 10*q1**3*q2*q3*q4*q5**2*q6 + 90*q1**3*q2*q3*q4*q5**2 - 10*q1**3*q2*q3*q4*q5*q6**2 + 90*q1**3*q2*q3*q4*q5*q6 - 360*q1**3*q2*q3*q4*q5 - 10*q1**2*q2**4*q3*q4*q5 - 10*q1**2*q2**3*q3**2*q4*q5 - 10*q1**2*q2**3*q3*q4**2*q5 - 10*q1**2*q2**3*q3*q4*q5**2 - 10*q1**2*q2**3*q3*q4*q5*q6 + 90*q1**2*q2**3*q3*q4*q5 - 10*q1**2*q2**2*q3**3*q4*q5 - 10*q1**2*q2**2*q3**2*q4**2*q5 - 10*q1**2*q2**2*q3**2*q4*q5**2 - 10*q1**2*q2**2*q3**2*q4*q5*q6 + 90*q1**2*q2**2*q3**2*q4*q5 - 10*q1**2*q2**2*q3*q4**3*q5 - 10*q1**2*q2**2*q3*q4**2*q5**2 - 10*q1**2*q2**2*q3*q4**2*q5*q6 + 90*q1**2*q2**2*q3*q4**2*q5 - 10*q1**2*q2**2*q3*q4*q5**3 - 10*q1**2*q2**2*q3*q4*q5**2*q6 + 90*q1**2*q2**2*q3*q4*q5**2 - 10*q1**2*q2**2*q3*q4*q5*q6**2 + 90*q1**2*q2**2*q3*q4*q5*q6 - 360*q1**2*q2**2*q3*q4*q5 - 10*q1**2*q2*q3**4*q4*q5 - 10*q1**2*q2*q3**3*q4**2*q5 - 10*q1**2*q2*q3**3*q4*q5**2 - 10*q1**2*q2*q3**3*q4*q5*q6 + 90*q1**2*q2*q3**3*q4*q5 - 10*q1**2*q2*q3**2*q4**3*q5 - 10*q1**2*q2*q3**2*q4**2*q5**2 - 10*q1**2*q2*q3**2*q4**2*q5*q6 + 90*q1**2*q2*q3**2*q4**2*q5 - 10*q1**2*q2*q3**2*q4*q5**3 - 10*q1**2*q2*q3**2*q4*q5**2*q6 + 90*q1**2*q2*q3**2*q4*q5**2 - 10*q1**2*q2*q3**2*q4*q5*q6**2 + 90*q1**2*q2*q3**2*q4*q5*q6 - 360*q1**2*q2*q3**2*q4*q5 - 10*q1**2*q2*q3*q4**4*q5 - 10*q1**2*q2*q3*q4**3*q5**2 - 10*q1**2*q2*q3*q4**3*q5*q6 + 90*q1**2*q2*q3*q4**3*q5 - 10*q1**2*q2*q3*q4**2*q5**3 - 10*q1**2*q2*q3*q4**2*q5**2*q6 + 90*q1**2*q2*q3*q4**2*q5**2 - 10*q1**2*q2*q3*q4**2*q5*q6**2 + 90*q1**2*q2*q3*q4**2*q5*q6 - 360*q1**2*q2*q3*q4**2*q5 - 10*q1**2*q2*q3*q4*q5**4 - 10*q1**2*q2*q3*q4*q5**3*q6 + 90*q1**2*q2*q3*q4*q5**3 - 10*q1**2*q2*q3*q4*q5**2*q6**2 + 90*q1**2*q2*q3*q4*q5**2*q6 - 360*q1**2*q2*q3*q4*q5**2 - 10*q1**2*q2*q3*q4*q5*q6**3 + 90*q1**2*q2*q3*q4*q5*q6**2 - 360*q1**2*q2*q3*q4*q5*q6 + 840*q1**2*q2*q3*q4*q5 - 10*q1*q2**5*q3*q4*q5 - 10*q1*q2**4*q3**2*q4*q5 - 10*q1*q2**4*q3*q4**2*q5 - 10*q1*q2**4*q3*q4*q5**2 - 10*q1*q2**4*q3*q4*q5*q6 + 90*q1*q2**4*q3*q4*q5 - 10*q1*q2**3*q3**3*q4*q5 - 10*q1*q2**3*q3**2*q4**2*q5 - 10*q1*q2**3*q3**2*q4*q5**2 - 10*q1*q2**3*q3**2*q4*q5*q6 + 90*q1*q2**3*q3**2*q4*q5 - 10*q1*q2**3*q3*q4**3*q5 - 10*q1*q2**3*q3*q4**2*q5**2 - 10*q1*q2**3*q3*q4**2*q5*q6 + 90*q1*q2**3*q3*q4**2*q5 - 10*q1*q2**3*q3*q4*q5**3 - 10*q1*q2**3*q3*q4*q5**2*q6 + 90*q1*q2**3*q3*q4*q5**2 - 10*q1*q2**3*q3*q4*q5*q6**2 + 90*q1*q2**3*q3*q4*q5*q6 - 360*q1*q2**3*q3*q4*q5 - 10*q1*q2**2*q3**4*q4*q5 - 10*q1*q2**2*q3**3*q4**2*q5 - 10*q1*q2**2*q3**3*q4*q5**2 - 10*q1*q2**2*q3**3*q4*q5*q6 + 90*q1*q2**2*q3**3*q4*q5 - 10*q1*q2**2*q3**2*q4**3*q5 - 10*q1*q2**2*q3**2*q4**2*q5**2 - 10*q1*q2**2*q3**2*q4**2*q5*q6 + 90*q1*q2**2*q3**2*q4**2*q5 - 10*q1*q2**2*q3**2*q4*q5**3 - 10*q1*q2**2*q3**2*q4*q5**2*q6 + 90*q1*q2**2*q3**2*q4*q5**2 - 10*q1*q2**2*q3**2*q4*q5*q6**2 + 90*q1*q2**2*q3**2*q4*q5*q6 - 360*q1*q2**2*q3**2*q4*q5 - 10*q1*q2**2*q3*q4**4*q5 - 10*q1*q2**2*q3*q4**3*q5**2 - 10*q1*q2**2*q3*q4**3*q5*q6 + 90*q1*q2**2*q3*q4**3*q5 - 10*q1*q2**2*q3*q4**2*q5**3 - 10*q1*q2**2*q3*q4**2*q5**2*q6 + 90*q1*q2**2*q3*q4**2*q5**2 - 10*q1*q2**2*q3*q4**2*q5*q6**2 + 90*q1*q2**2*q3*q4**2*q5*q6 - 360*q1*q2**2*q3*q4**2*q5 - 10*q1*q2**2*q3*q4*q5**4 - 10*q1*q2**2*q3*q4*q5**3*q6 + 90*q1*q2**2*q3*q4*q5**3 - 10*q1*q2**2*q3*q4*q5**2*q6**2 + 90*q1*q2**2*q3*q4*q5**2*q6 - 360*q1*q2**2*q3*q4*q5**2 - 10*q1*q2**2*q3*q4*q5*q6**3 + 90*q1*q2**2*q3*q4*q5*q6**2 - 360*q1*q2**2*q3*q4*q5*q6 + 840*q1*q2**2*q3*q4*q5 - 10*q1*q2*q3**5*q4*q5 - 10*q1*q2*q3**4*q4**2*q5 - 10*q1*q2*q3**4*q4*q5**2 - 10*q1*q2*q3**4*q4*q5*q6 + 90*q1*q2*q3**4*q4*q5 - 10*q1*q2*q3**3*q4**3*q5 - 10*q1*q2*q3**3*q4**2*q5**2 - 10*q1*q2*q3**3*q4**2*q5*q6 + 90*q1*q2*q3**3*q4**2*q5 - 10*q1*q2*q3**3*q4*q5**3 - 10*q1*q2*q3**3*q4*q5**2*q6 + 90*q1*q2*q3**3*q4*q5**2 - 10*q1*q2*q3**3*q4*q5*q6**2 + 90*q1*q2*q3**3*q4*q5*q6 - 360*q1*q2*q3**3*q4*q5 - 10*q1*q2*q3**2*q4**4*q5 - 10*q1*q2*q3**2*q4**3*q5**2 - 10*q1*q2*q3**2*q4**3*q5*q6 + 90*q1*q2*q3**2*q4**3*q5 - 10*q1*q2*q3**2*q4**2*q5**3 - 10*q1*q2*q3**2*q4**2*q5**2*q6 + 90*q1*q2*q3**2*q4**2*q5**2 - 10*q1*q2*q3**2*q4**2*q5*q6**2 + 90*q1*q2*q3**2*q4**2*q5*q6 - 360*q1*q2*q3**2*q4**2*q5 - 10*q1*q2*q3**2*q4*q5**4 - 10*q1*q2*q3**2*q4*q5**3*q6 + 90*q1*q2*q3**2*q4*q5**3 - 10*q1*q2*q3**2*q4*q5**2*q6**2 + 90*q1*q2*q3**2*q4*q5**2*q6 - 360*q1*q2*q3**2*q4*q5**2 - 10*q1*q2*q3**2*q4*q5*q6**3 + 90*q1*q2*q3**2*q4*q5*q6**2 - 360*q1*q2*q3**2*q4*q5*q6 + 840*q1*q2*q3**2*q4*q5 - 10*q1*q2*q3*q4**5*q5 - 10*q1*q2*q3*q4**4*q5**2 - 10*q1*q2*q3*q4**4*q5*q6 + 90*q1*q2*q3*q4**4*q5 - 10*q1*q2*q3*q4**3*q5**3 - 10*q1*q2*q3*q4**3*q5**2*q6 + 90*q1*q2*q3*q4**3*q5**2 - 10*q1*q2*q3*q4**3*q5*q6**2 + 90*q1*q2*q3*q4**3*q5*q6 - 360*q1*q2*q3*q4**3*q5 - 10*q1*q2*q3*q4**2*q5**4 - 10*q1*q2*q3*q4**2*q5**3*q6 + 90*q1*q2*q3*q4**2*q5**3 - 10*q1*q2*q3*q4**2*q5**2*q6**2 + 90*q1*q2*q3*q4**2*q5**2*q6 - 360*q1*q2*q3*q4**2*q5**2 - 10*q1*q2*q3*q4**2*q5*q6**3 + 90*q1*q2*q3*q4**2*q5*q6**2 - 360*q1*q2*q3*q4**2*q5*q6 + 840*q1*q2*q3*q4**2*q5 - 10*q1*q2*q3*q4*q5**5 - 10*q1*q2*q3*q4*q5**4*q6 + 90*q1*q2*q3*q4*q5**4 - 10*q1*q2*q3*q4*q5**3*q6**2 + 90*q1*q2*q3*q4*q5**3*q6 - 360*q1*q2*q3*q4*q5**3 - 10*q1*q2*q3*q4*q5**2*q6**3 + 90*q1*q2*q3*q4*q5**2*q6**2 - 360*q1*q2*q3*q4*q5**2*q6 + 840*q1*q2*q3*q4*q5**2 - 10*q1*q2*q3*q4*q5*q6**4 + 90*q1*q2*q3*q4*q5*q6**3 - 360*q1*q2*q3*q4*q5*q6**2 + 840*q1*q2*q3*q4*q5*q6 - 1260*q1*q2*q3*q4*q5)'
f_multiparam_semisyn[10][7] = 'p*(p - 1)**3*(10*p**6*q1**4*q2*q3*q4*q5*q6 + 10*p**6*q1**3*q2**2*q3*q4*q5*q6 + 10*p**6*q1**3*q2*q3**2*q4*q5*q6 + 10*p**6*q1**3*q2*q3*q4**2*q5*q6 + 10*p**6*q1**3*q2*q3*q4*q5**2*q6 + 10*p**6*q1**3*q2*q3*q4*q5*q6**2 + 10*p**6*q1**3*q2*q3*q4*q5*q6*q7 - 90*p**6*q1**3*q2*q3*q4*q5*q6 + 10*p**6*q1**2*q2**3*q3*q4*q5*q6 + 10*p**6*q1**2*q2**2*q3**2*q4*q5*q6 + 10*p**6*q1**2*q2**2*q3*q4**2*q5*q6 + 10*p**6*q1**2*q2**2*q3*q4*q5**2*q6 + 10*p**6*q1**2*q2**2*q3*q4*q5*q6**2 + 10*p**6*q1**2*q2**2*q3*q4*q5*q6*q7 - 90*p**6*q1**2*q2**2*q3*q4*q5*q6 + 10*p**6*q1**2*q2*q3**3*q4*q5*q6 + 10*p**6*q1**2*q2*q3**2*q4**2*q5*q6 + 10*p**6*q1**2*q2*q3**2*q4*q5**2*q6 + 10*p**6*q1**2*q2*q3**2*q4*q5*q6**2 + 10*p**6*q1**2*q2*q3**2*q4*q5*q6*q7 - 90*p**6*q1**2*q2*q3**2*q4*q5*q6 + 10*p**6*q1**2*q2*q3*q4**3*q5*q6 + 10*p**6*q1**2*q2*q3*q4**2*q5**2*q6 + 10*p**6*q1**2*q2*q3*q4**2*q5*q6**2 + 10*p**6*q1**2*q2*q3*q4**2*q5*q6*q7 - 90*p**6*q1**2*q2*q3*q4**2*q5*q6 + 10*p**6*q1**2*q2*q3*q4*q5**3*q6 + 10*p**6*q1**2*q2*q3*q4*q5**2*q6**2 + 10*p**6*q1**2*q2*q3*q4*q5**2*q6*q7 - 90*p**6*q1**2*q2*q3*q4*q5**2*q6 + 10*p**6*q1**2*q2*q3*q4*q5*q6**3 + 10*p**6*q1**2*q2*q3*q4*q5*q6**2*q7 - 90*p**6*q1**2*q2*q3*q4*q5*q6**2 + 10*p**6*q1**2*q2*q3*q4*q5*q6*q7**2 - 90*p**6*q1**2*q2*q3*q4*q5*q6*q7 + 360*p**6*q1**2*q2*q3*q4*q5*q6 + 10*p**6*q1*q2**4*q3*q4*q5*q6 + 10*p**6*q1*q2**3*q3**2*q4*q5*q6 + 10*p**6*q1*q2**3*q3*q4**2*q5*q6 + 10*p**6*q1*q2**3*q3*q4*q5**2*q6 + 10*p**6*q1*q2**3*q3*q4*q5*q6**2 + 10*p**6*q1*q2**3*q3*q4*q5*q6*q7 - 90*p**6*q1*q2**3*q3*q4*q5*q6 + 10*p**6*q1*q2**2*q3**3*q4*q5*q6 + 10*p**6*q1*q2**2*q3**2*q4**2*q5*q6 + 10*p**6*q1*q2**2*q3**2*q4*q5**2*q6 + 10*p**6*q1*q2**2*q3**2*q4*q5*q6**2 + 10*p**6*q1*q2**2*q3**2*q4*q5*q6*q7 - 90*p**6*q1*q2**2*q3**2*q4*q5*q6 + 10*p**6*q1*q2**2*q3*q4**3*q5*q6 + 10*p**6*q1*q2**2*q3*q4**2*q5**2*q6 + 10*p**6*q1*q2**2*q3*q4**2*q5*q6**2 + 10*p**6*q1*q2**2*q3*q4**2*q5*q6*q7 - 90*p**6*q1*q2**2*q3*q4**2*q5*q6 + 10*p**6*q1*q2**2*q3*q4*q5**3*q6 + 10*p**6*q1*q2**2*q3*q4*q5**2*q6**2 + 10*p**6*q1*q2**2*q3*q4*q5**2*q6*q7 - 90*p**6*q1*q2**2*q3*q4*q5**2*q6 + 10*p**6*q1*q2**2*q3*q4*q5*q6**3 + 10*p**6*q1*q2**2*q3*q4*q5*q6**2*q7 - 90*p**6*q1*q2**2*q3*q4*q5*q6**2 + 10*p**6*q1*q2**2*q3*q4*q5*q6*q7**2 - 90*p**6*q1*q2**2*q3*q4*q5*q6*q7 + 360*p**6*q1*q2**2*q3*q4*q5*q6 + 10*p**6*q1*q2*q3**4*q4*q5*q6 + 10*p**6*q1*q2*q3**3*q4**2*q5*q6 + 10*p**6*q1*q2*q3**3*q4*q5**2*q6 + 10*p**6*q1*q2*q3**3*q4*q5*q6**2 + 10*p**6*q1*q2*q3**3*q4*q5*q6*q7 - 90*p**6*q1*q2*q3**3*q4*q5*q6 + 10*p**6*q1*q2*q3**2*q4**3*q5*q6 + 10*p**6*q1*q2*q3**2*q4**2*q5**2*q6 + 10*p**6*q1*q2*q3**2*q4**2*q5*q6**2 + 10*p**6*q1*q2*q3**2*q4**2*q5*q6*q7 - 90*p**6*q1*q2*q3**2*q4**2*q5*q6 + 10*p**6*q1*q2*q3**2*q4*q5**3*q6 + 10*p**6*q1*q2*q3**2*q4*q5**2*q6**2 + 10*p**6*q1*q2*q3**2*q4*q5**2*q6*q7 - 90*p**6*q1*q2*q3**2*q4*q5**2*q6 + 10*p**6*q1*q2*q3**2*q4*q5*q6**3 + 10*p**6*q1*q2*q3**2*q4*q5*q6**2*q7 - 90*p**6*q1*q2*q3**2*q4*q5*q6**2 + 10*p**6*q1*q2*q3**2*q4*q5*q6*q7**2 - 90*p**6*q1*q2*q3**2*q4*q5*q6*q7 + 360*p**6*q1*q2*q3**2*q4*q5*q6 + 10*p**6*q1*q2*q3*q4**4*q5*q6 + 10*p**6*q1*q2*q3*q4**3*q5**2*q6 + 10*p**6*q1*q2*q3*q4**3*q5*q6**2 + 10*p**6*q1*q2*q3*q4**3*q5*q6*q7 - 90*p**6*q1*q2*q3*q4**3*q5*q6 + 10*p**6*q1*q2*q3*q4**2*q5**3*q6 + 10*p**6*q1*q2*q3*q4**2*q5**2*q6**2 + 10*p**6*q1*q2*q3*q4**2*q5**2*q6*q7 - 90*p**6*q1*q2*q3*q4**2*q5**2*q6 + 10*p**6*q1*q2*q3*q4**2*q5*q6**3 + 10*p**6*q1*q2*q3*q4**2*q5*q6**2*q7 - 90*p**6*q1*q2*q3*q4**2*q5*q6**2 + 10*p**6*q1*q2*q3*q4**2*q5*q6*q7**2 - 90*p**6*q1*q2*q3*q4**2*q5*q6*q7 + 360*p**6*q1*q2*q3*q4**2*q5*q6 + 10*p**6*q1*q2*q3*q4*q5**4*q6 + 10*p**6*q1*q2*q3*q4*q5**3*q6**2 + 10*p**6*q1*q2*q3*q4*q5**3*q6*q7 - 90*p**6*q1*q2*q3*q4*q5**3*q6 + 10*p**6*q1*q2*q3*q4*q5**2*q6**3 + 10*p**6*q1*q2*q3*q4*q5**2*q6**2*q7 - 90*p**6*q1*q2*q3*q4*q5**2*q6**2 + 10*p**6*q1*q2*q3*q4*q5**2*q6*q7**2 - 90*p**6*q1*q2*q3*q4*q5**2*q6*q7 + 360*p**6*q1*q2*q3*q4*q5**2*q6 + 10*p**6*q1*q2*q3*q4*q5*q6**4 + 10*p**6*q1*q2*q3*q4*q5*q6**3*q7 - 90*p**6*q1*q2*q3*q4*q5*q6**3 + 10*p**6*q1*q2*q3*q4*q5*q6**2*q7**2 - 90*p**6*q1*q2*q3*q4*q5*q6**2*q7 + 360*p**6*q1*q2*q3*q4*q5*q6**2 + 10*p**6*q1*q2*q3*q4*q5*q6*q7**3 - 90*p**6*q1*q2*q3*q4*q5*q6*q7**2 + 360*p**6*q1*q2*q3*q4*q5*q6*q7 - 840*p**6*q1*q2*q3*q4*q5*q6 - 45*p**6*q2**4*q3*q4*q5*q6 - 45*p**6*q2**3*q3**2*q4*q5*q6 - 45*p**6*q2**3*q3*q4**2*q5*q6 - 45*p**6*q2**3*q3*q4*q5**2*q6 - 45*p**6*q2**3*q3*q4*q5*q6**2 - 45*p**6*q2**3*q3*q4*q5*q6*q7 + 360*p**6*q2**3*q3*q4*q5*q6 - 45*p**6*q2**2*q3**3*q4*q5*q6 - 45*p**6*q2**2*q3**2*q4**2*q5*q6 - 45*p**6*q2**2*q3**2*q4*q5**2*q6 - 45*p**6*q2**2*q3**2*q4*q5*q6**2 - 45*p**6*q2**2*q3**2*q4*q5*q6*q7 + 360*p**6*q2**2*q3**2*q4*q5*q6 - 45*p**6*q2**2*q3*q4**3*q5*q6 - 45*p**6*q2**2*q3*q4**2*q5**2*q6 - 45*p**6*q2**2*q3*q4**2*q5*q6**2 - 45*p**6*q2**2*q3*q4**2*q5*q6*q7 + 360*p**6*q2**2*q3*q4**2*q5*q6 - 45*p**6*q2**2*q3*q4*q5**3*q6 - 45*p**6*q2**2*q3*q4*q5**2*q6**2 - 45*p**6*q2**2*q3*q4*q5**2*q6*q7 + 360*p**6*q2**2*q3*q4*q5**2*q6 - 45*p**6*q2**2*q3*q4*q5*q6**3 - 45*p**6*q2**2*q3*q4*q5*q6**2*q7 + 360*p**6*q2**2*q3*q4*q5*q6**2 - 45*p**6*q2**2*q3*q4*q5*q6*q7**2 + 360*p**6*q2**2*q3*q4*q5*q6*q7 - 1260*p**6*q2**2*q3*q4*q5*q6 - 45*p**6*q2*q3**4*q4*q5*q6 - 45*p**6*q2*q3**3*q4**2*q5*q6 - 45*p**6*q2*q3**3*q4*q5**2*q6 - 45*p**6*q2*q3**3*q4*q5*q6**2 - 45*p**6*q2*q3**3*q4*q5*q6*q7 + 360*p**6*q2*q3**3*q4*q5*q6 - 45*p**6*q2*q3**2*q4**3*q5*q6 - 45*p**6*q2*q3**2*q4**2*q5**2*q6 - 45*p**6*q2*q3**2*q4**2*q5*q6**2 - 45*p**6*q2*q3**2*q4**2*q5*q6*q7 + 360*p**6*q2*q3**2*q4**2*q5*q6 - 45*p**6*q2*q3**2*q4*q5**3*q6 - 45*p**6*q2*q3**2*q4*q5**2*q6**2 - 45*p**6*q2*q3**2*q4*q5**2*q6*q7 + 360*p**6*q2*q3**2*q4*q5**2*q6 - 45*p**6*q2*q3**2*q4*q5*q6**3 - 45*p**6*q2*q3**2*q4*q5*q6**2*q7 + 360*p**6*q2*q3**2*q4*q5*q6**2 - 45*p**6*q2*q3**2*q4*q5*q6*q7**2 + 360*p**6*q2*q3**2*q4*q5*q6*q7 - 1260*p**6*q2*q3**2*q4*q5*q6 - 45*p**6*q2*q3*q4**4*q5*q6 - 45*p**6*q2*q3*q4**3*q5**2*q6 - 45*p**6*q2*q3*q4**3*q5*q6**2 - 45*p**6*q2*q3*q4**3*q5*q6*q7 + 360*p**6*q2*q3*q4**3*q5*q6 - 45*p**6*q2*q3*q4**2*q5**3*q6 - 45*p**6*q2*q3*q4**2*q5**2*q6**2 - 45*p**6*q2*q3*q4**2*q5**2*q6*q7 + 360*p**6*q2*q3*q4**2*q5**2*q6 - 45*p**6*q2*q3*q4**2*q5*q6**3 - 45*p**6*q2*q3*q4**2*q5*q6**2*q7 + 360*p**6*q2*q3*q4**2*q5*q6**2 - 45*p**6*q2*q3*q4**2*q5*q6*q7**2 + 360*p**6*q2*q3*q4**2*q5*q6*q7 - 1260*p**6*q2*q3*q4**2*q5*q6 - 45*p**6*q2*q3*q4*q5**4*q6 - 45*p**6*q2*q3*q4*q5**3*q6**2 - 45*p**6*q2*q3*q4*q5**3*q6*q7 + 360*p**6*q2*q3*q4*q5**3*q6 - 45*p**6*q2*q3*q4*q5**2*q6**3 - 45*p**6*q2*q3*q4*q5**2*q6**2*q7 + 360*p**6*q2*q3*q4*q5**2*q6**2 - 45*p**6*q2*q3*q4*q5**2*q6*q7**2 + 360*p**6*q2*q3*q4*q5**2*q6*q7 - 1260*p**6*q2*q3*q4*q5**2*q6 - 45*p**6*q2*q3*q4*q5*q6**4 - 45*p**6*q2*q3*q4*q5*q6**3*q7 + 360*p**6*q2*q3*q4*q5*q6**3 - 45*p**6*q2*q3*q4*q5*q6**2*q7**2 + 360*p**6*q2*q3*q4*q5*q6**2*q7 - 1260*p**6*q2*q3*q4*q5*q6**2 - 45*p**6*q2*q3*q4*q5*q6*q7**3 + 360*p**6*q2*q3*q4*q5*q6*q7**2 - 1260*p**6*q2*q3*q4*q5*q6*q7 + 2520*p**6*q2*q3*q4*q5*q6 + 120*p**6*q3**4*q4*q5*q6 + 120*p**6*q3**3*q4**2*q5*q6 + 120*p**6*q3**3*q4*q5**2*q6 + 120*p**6*q3**3*q4*q5*q6**2 + 120*p**6*q3**3*q4*q5*q6*q7 - 840*p**6*q3**3*q4*q5*q6 + 120*p**6*q3**2*q4**3*q5*q6 + 120*p**6*q3**2*q4**2*q5**2*q6 + 120*p**6*q3**2*q4**2*q5*q6**2 + 120*p**6*q3**2*q4**2*q5*q6*q7 - 840*p**6*q3**2*q4**2*q5*q6 + 120*p**6*q3**2*q4*q5**3*q6 + 120*p**6*q3**2*q4*q5**2*q6**2 + 120*p**6*q3**2*q4*q5**2*q6*q7 - 840*p**6*q3**2*q4*q5**2*q6 + 120*p**6*q3**2*q4*q5*q6**3 + 120*p**6*q3**2*q4*q5*q6**2*q7 - 840*p**6*q3**2*q4*q5*q6**2 + 120*p**6*q3**2*q4*q5*q6*q7**2 - 840*p**6*q3**2*q4*q5*q6*q7 + 2520*p**6*q3**2*q4*q5*q6 + 120*p**6*q3*q4**4*q5*q6 + 120*p**6*q3*q4**3*q5**2*q6 + 120*p**6*q3*q4**3*q5*q6**2 + 120*p**6*q3*q4**3*q5*q6*q7 - 840*p**6*q3*q4**3*q5*q6 + 120*p**6*q3*q4**2*q5**3*q6 + 120*p**6*q3*q4**2*q5**2*q6**2 + 120*p**6*q3*q4**2*q5**2*q6*q7 - 840*p**6*q3*q4**2*q5**2*q6 + 120*p**6*q3*q4**2*q5*q6**3 + 120*p**6*q3*q4**2*q5*q6**2*q7 - 840*p**6*q3*q4**2*q5*q6**2 + 120*p**6*q3*q4**2*q5*q6*q7**2 - 840*p**6*q3*q4**2*q5*q6*q7 + 2520*p**6*q3*q4**2*q5*q6 + 120*p**6*q3*q4*q5**4*q6 + 120*p**6*q3*q4*q5**3*q6**2 + 120*p**6*q3*q4*q5**3*q6*q7 - 840*p**6*q3*q4*q5**3*q6 + 120*p**6*q3*q4*q5**2*q6**3 + 120*p**6*q3*q4*q5**2*q6**2*q7 - 840*p**6*q3*q4*q5**2*q6**2 + 120*p**6*q3*q4*q5**2*q6*q7**2 - 840*p**6*q3*q4*q5**2*q6*q7 + 2520*p**6*q3*q4*q5**2*q6 + 120*p**6*q3*q4*q5*q6**4 + 120*p**6*q3*q4*q5*q6**3*q7 - 840*p**6*q3*q4*q5*q6**3 + 120*p**6*q3*q4*q5*q6**2*q7**2 - 840*p**6*q3*q4*q5*q6**2*q7 + 2520*p**6*q3*q4*q5*q6**2 + 120*p**6*q3*q4*q5*q6*q7**3 - 840*p**6*q3*q4*q5*q6*q7**2 + 2520*p**6*q3*q4*q5*q6*q7 - 4200*p**6*q3*q4*q5*q6 - 210*p**6*q4**4*q5*q6 - 210*p**6*q4**3*q5**2*q6 - 210*p**6*q4**3*q5*q6**2 - 210*p**6*q4**3*q5*q6*q7 + 1260*p**6*q4**3*q5*q6 - 210*p**6*q4**2*q5**3*q6 - 210*p**6*q4**2*q5**2*q6**2 - 210*p**6*q4**2*q5**2*q6*q7 + 1260*p**6*q4**2*q5**2*q6 - 210*p**6*q4**2*q5*q6**3 - 210*p**6*q4**2*q5*q6**2*q7 + 1260*p**6*q4**2*q5*q6**2 - 210*p**6*q4**2*q5*q6*q7**2 + 1260*p**6*q4**2*q5*q6*q7 - 3150*p**6*q4**2*q5*q6 - 210*p**6*q4*q5**4*q6 - 210*p**6*q4*q5**3*q6**2 - 210*p**6*q4*q5**3*q6*q7 + 1260*p**6*q4*q5**3*q6 - 210*p**6*q4*q5**2*q6**3 - 210*p**6*q4*q5**2*q6**2*q7 + 1260*p**6*q4*q5**2*q6**2 - 210*p**6*q4*q5**2*q6*q7**2 + 1260*p**6*q4*q5**2*q6*q7 - 3150*p**6*q4*q5**2*q6 - 210*p**6*q4*q5*q6**4 - 210*p**6*q4*q5*q6**3*q7 + 1260*p**6*q4*q5*q6**3 - 210*p**6*q4*q5*q6**2*q7**2 + 1260*p**6*q4*q5*q6**2*q7 - 3150*p**6*q4*q5*q6**2 - 210*p**6*q4*q5*q6*q7**3 + 1260*p**6*q4*q5*q6*q7**2 - 3150*p**6*q4*q5*q6*q7 + 4200*p**6*q4*q5*q6 + 252*p**6*q5**4*q6 + 252*p**6*q5**3*q6**2 + 252*p**6*q5**3*q6*q7 - 1260*p**6*q5**3*q6 + 252*p**6*q5**2*q6**3 + 252*p**6*q5**2*q6**2*q7 - 1260*p**6*q5**2*q6**2 + 252*p**6*q5**2*q6*q7**2 - 1260*p**6*q5**2*q6*q7 + 2520*p**6*q5**2*q6 + 252*p**6*q5*q6**4 + 252*p**6*q5*q6**3*q7 - 1260*p**6*q5*q6**3 + 252*p**6*q5*q6**2*q7**2 - 1260*p**6*q5*q6**2*q7 + 2520*p**6*q5*q6**2 + 252*p**6*q5*q6*q7**3 - 1260*p**6*q5*q6*q7**2 + 2520*p**6*q5*q6*q7 - 2520*p**6*q5*q6 - 210*p**6*q6**4 - 210*p**6*q6**3*q7 + 840*p**6*q6**3 - 210*p**6*q6**2*q7**2 + 840*p**6*q6**2*q7 - 1260*p**6*q6**2 - 210*p**6*q6*q7**3 + 840*p**6*q6*q7**2 - 1260*p**6*q6*q7 + 840*p**6*q6 + 120*p**6*q7**3 - 360*p**6*q7**2 + 360*p**6*q7 - 120*p**6 - 60*p**5*q1**4*q2*q3*q4*q5*q6 - 60*p**5*q1**3*q2**2*q3*q4*q5*q6 - 60*p**5*q1**3*q2*q3**2*q4*q5*q6 - 60*p**5*q1**3*q2*q3*q4**2*q5*q6 - 60*p**5*q1**3*q2*q3*q4*q5**2*q6 - 60*p**5*q1**3*q2*q3*q4*q5*q6**2 - 60*p**5*q1**3*q2*q3*q4*q5*q6*q7 + 540*p**5*q1**3*q2*q3*q4*q5*q6 - 60*p**5*q1**2*q2**3*q3*q4*q5*q6 - 60*p**5*q1**2*q2**2*q3**2*q4*q5*q6 - 60*p**5*q1**2*q2**2*q3*q4**2*q5*q6 - 60*p**5*q1**2*q2**2*q3*q4*q5**2*q6 - 60*p**5*q1**2*q2**2*q3*q4*q5*q6**2 - 60*p**5*q1**2*q2**2*q3*q4*q5*q6*q7 + 540*p**5*q1**2*q2**2*q3*q4*q5*q6 - 60*p**5*q1**2*q2*q3**3*q4*q5*q6 - 60*p**5*q1**2*q2*q3**2*q4**2*q5*q6 - 60*p**5*q1**2*q2*q3**2*q4*q5**2*q6 - 60*p**5*q1**2*q2*q3**2*q4*q5*q6**2 - 60*p**5*q1**2*q2*q3**2*q4*q5*q6*q7 + 540*p**5*q1**2*q2*q3**2*q4*q5*q6 - 60*p**5*q1**2*q2*q3*q4**3*q5*q6 - 60*p**5*q1**2*q2*q3*q4**2*q5**2*q6 - 60*p**5*q1**2*q2*q3*q4**2*q5*q6**2 - 60*p**5*q1**2*q2*q3*q4**2*q5*q6*q7 + 540*p**5*q1**2*q2*q3*q4**2*q5*q6 - 60*p**5*q1**2*q2*q3*q4*q5**3*q6 - 60*p**5*q1**2*q2*q3*q4*q5**2*q6**2 - 60*p**5*q1**2*q2*q3*q4*q5**2*q6*q7 + 540*p**5*q1**2*q2*q3*q4*q5**2*q6 - 60*p**5*q1**2*q2*q3*q4*q5*q6**3 - 60*p**5*q1**2*q2*q3*q4*q5*q6**2*q7 + 540*p**5*q1**2*q2*q3*q4*q5*q6**2 - 60*p**5*q1**2*q2*q3*q4*q5*q6*q7**2 + 540*p**5*q1**2*q2*q3*q4*q5*q6*q7 - 2160*p**5*q1**2*q2*q3*q4*q5*q6 - 60*p**5*q1*q2**4*q3*q4*q5*q6 - 60*p**5*q1*q2**3*q3**2*q4*q5*q6 - 60*p**5*q1*q2**3*q3*q4**2*q5*q6 - 60*p**5*q1*q2**3*q3*q4*q5**2*q6 - 60*p**5*q1*q2**3*q3*q4*q5*q6**2 - 60*p**5*q1*q2**3*q3*q4*q5*q6*q7 + 540*p**5*q1*q2**3*q3*q4*q5*q6 - 60*p**5*q1*q2**2*q3**3*q4*q5*q6 - 60*p**5*q1*q2**2*q3**2*q4**2*q5*q6 - 60*p**5*q1*q2**2*q3**2*q4*q5**2*q6 - 60*p**5*q1*q2**2*q3**2*q4*q5*q6**2 - 60*p**5*q1*q2**2*q3**2*q4*q5*q6*q7 + 540*p**5*q1*q2**2*q3**2*q4*q5*q6 - 60*p**5*q1*q2**2*q3*q4**3*q5*q6 - 60*p**5*q1*q2**2*q3*q4**2*q5**2*q6 - 60*p**5*q1*q2**2*q3*q4**2*q5*q6**2 - 60*p**5*q1*q2**2*q3*q4**2*q5*q6*q7 + 540*p**5*q1*q2**2*q3*q4**2*q5*q6 - 60*p**5*q1*q2**2*q3*q4*q5**3*q6 - 60*p**5*q1*q2**2*q3*q4*q5**2*q6**2 - 60*p**5*q1*q2**2*q3*q4*q5**2*q6*q7 + 540*p**5*q1*q2**2*q3*q4*q5**2*q6 - 60*p**5*q1*q2**2*q3*q4*q5*q6**3 - 60*p**5*q1*q2**2*q3*q4*q5*q6**2*q7 + 540*p**5*q1*q2**2*q3*q4*q5*q6**2 - 60*p**5*q1*q2**2*q3*q4*q5*q6*q7**2 + 540*p**5*q1*q2**2*q3*q4*q5*q6*q7 - 2160*p**5*q1*q2**2*q3*q4*q5*q6 - 60*p**5*q1*q2*q3**4*q4*q5*q6 - 60*p**5*q1*q2*q3**3*q4**2*q5*q6 - 60*p**5*q1*q2*q3**3*q4*q5**2*q6 - 60*p**5*q1*q2*q3**3*q4*q5*q6**2 - 60*p**5*q1*q2*q3**3*q4*q5*q6*q7 + 540*p**5*q1*q2*q3**3*q4*q5*q6 - 60*p**5*q1*q2*q3**2*q4**3*q5*q6 - 60*p**5*q1*q2*q3**2*q4**2*q5**2*q6 - 60*p**5*q1*q2*q3**2*q4**2*q5*q6**2 - 60*p**5*q1*q2*q3**2*q4**2*q5*q6*q7 + 540*p**5*q1*q2*q3**2*q4**2*q5*q6 - 60*p**5*q1*q2*q3**2*q4*q5**3*q6 - 60*p**5*q1*q2*q3**2*q4*q5**2*q6**2 - 60*p**5*q1*q2*q3**2*q4*q5**2*q6*q7 + 540*p**5*q1*q2*q3**2*q4*q5**2*q6 - 60*p**5*q1*q2*q3**2*q4*q5*q6**3 - 60*p**5*q1*q2*q3**2*q4*q5*q6**2*q7 + 540*p**5*q1*q2*q3**2*q4*q5*q6**2 - 60*p**5*q1*q2*q3**2*q4*q5*q6*q7**2 + 540*p**5*q1*q2*q3**2*q4*q5*q6*q7 - 2160*p**5*q1*q2*q3**2*q4*q5*q6 - 60*p**5*q1*q2*q3*q4**4*q5*q6 - 60*p**5*q1*q2*q3*q4**3*q5**2*q6 - 60*p**5*q1*q2*q3*q4**3*q5*q6**2 - 60*p**5*q1*q2*q3*q4**3*q5*q6*q7 + 540*p**5*q1*q2*q3*q4**3*q5*q6 - 60*p**5*q1*q2*q3*q4**2*q5**3*q6 - 60*p**5*q1*q2*q3*q4**2*q5**2*q6**2 - 60*p**5*q1*q2*q3*q4**2*q5**2*q6*q7 + 540*p**5*q1*q2*q3*q4**2*q5**2*q6 - 60*p**5*q1*q2*q3*q4**2*q5*q6**3 - 60*p**5*q1*q2*q3*q4**2*q5*q6**2*q7 + 540*p**5*q1*q2*q3*q4**2*q5*q6**2 - 60*p**5*q1*q2*q3*q4**2*q5*q6*q7**2 + 540*p**5*q1*q2*q3*q4**2*q5*q6*q7 - 2160*p**5*q1*q2*q3*q4**2*q5*q6 - 60*p**5*q1*q2*q3*q4*q5**4*q6 - 60*p**5*q1*q2*q3*q4*q5**3*q6**2 - 60*p**5*q1*q2*q3*q4*q5**3*q6*q7 + 540*p**5*q1*q2*q3*q4*q5**3*q6 - 60*p**5*q1*q2*q3*q4*q5**2*q6**3 - 60*p**5*q1*q2*q3*q4*q5**2*q6**2*q7 + 540*p**5*q1*q2*q3*q4*q5**2*q6**2 - 60*p**5*q1*q2*q3*q4*q5**2*q6*q7**2 + 540*p**5*q1*q2*q3*q4*q5**2*q6*q7 - 2160*p**5*q1*q2*q3*q4*q5**2*q6 - 60*p**5*q1*q2*q3*q4*q5*q6**4 - 60*p**5*q1*q2*q3*q4*q5*q6**3*q7 + 540*p**5*q1*q2*q3*q4*q5*q6**3 - 60*p**5*q1*q2*q3*q4*q5*q6**2*q7**2 + 540*p**5*q1*q2*q3*q4*q5*q6**2*q7 - 2160*p**5*q1*q2*q3*q4*q5*q6**2 - 60*p**5*q1*q2*q3*q4*q5*q6*q7**3 + 540*p**5*q1*q2*q3*q4*q5*q6*q7**2 - 2160*p**5*q1*q2*q3*q4*q5*q6*q7 + 5040*p**5*q1*q2*q3*q4*q5*q6 + 225*p**5*q2**4*q3*q4*q5*q6 + 225*p**5*q2**3*q3**2*q4*q5*q6 + 225*p**5*q2**3*q3*q4**2*q5*q6 + 225*p**5*q2**3*q3*q4*q5**2*q6 + 225*p**5*q2**3*q3*q4*q5*q6**2 + 225*p**5*q2**3*q3*q4*q5*q6*q7 - 1800*p**5*q2**3*q3*q4*q5*q6 + 225*p**5*q2**2*q3**3*q4*q5*q6 + 225*p**5*q2**2*q3**2*q4**2*q5*q6 + 225*p**5*q2**2*q3**2*q4*q5**2*q6 + 225*p**5*q2**2*q3**2*q4*q5*q6**2 + 225*p**5*q2**2*q3**2*q4*q5*q6*q7 - 1800*p**5*q2**2*q3**2*q4*q5*q6 + 225*p**5*q2**2*q3*q4**3*q5*q6 + 225*p**5*q2**2*q3*q4**2*q5**2*q6 + 225*p**5*q2**2*q3*q4**2*q5*q6**2 + 225*p**5*q2**2*q3*q4**2*q5*q6*q7 - 1800*p**5*q2**2*q3*q4**2*q5*q6 + 225*p**5*q2**2*q3*q4*q5**3*q6 + 225*p**5*q2**2*q3*q4*q5**2*q6**2 + 225*p**5*q2**2*q3*q4*q5**2*q6*q7 - 1800*p**5*q2**2*q3*q4*q5**2*q6 + 225*p**5*q2**2*q3*q4*q5*q6**3 + 225*p**5*q2**2*q3*q4*q5*q6**2*q7 - 1800*p**5*q2**2*q3*q4*q5*q6**2 + 225*p**5*q2**2*q3*q4*q5*q6*q7**2 - 1800*p**5*q2**2*q3*q4*q5*q6*q7 + 6300*p**5*q2**2*q3*q4*q5*q6 + 225*p**5*q2*q3**4*q4*q5*q6 + 225*p**5*q2*q3**3*q4**2*q5*q6 + 225*p**5*q2*q3**3*q4*q5**2*q6 + 225*p**5*q2*q3**3*q4*q5*q6**2 + 225*p**5*q2*q3**3*q4*q5*q6*q7 - 1800*p**5*q2*q3**3*q4*q5*q6 + 225*p**5*q2*q3**2*q4**3*q5*q6 + 225*p**5*q2*q3**2*q4**2*q5**2*q6 + 225*p**5*q2*q3**2*q4**2*q5*q6**2 + 225*p**5*q2*q3**2*q4**2*q5*q6*q7 - 1800*p**5*q2*q3**2*q4**2*q5*q6 + 225*p**5*q2*q3**2*q4*q5**3*q6 + 225*p**5*q2*q3**2*q4*q5**2*q6**2 + 225*p**5*q2*q3**2*q4*q5**2*q6*q7 - 1800*p**5*q2*q3**2*q4*q5**2*q6 + 225*p**5*q2*q3**2*q4*q5*q6**3 + 225*p**5*q2*q3**2*q4*q5*q6**2*q7 - 1800*p**5*q2*q3**2*q4*q5*q6**2 + 225*p**5*q2*q3**2*q4*q5*q6*q7**2 - 1800*p**5*q2*q3**2*q4*q5*q6*q7 + 6300*p**5*q2*q3**2*q4*q5*q6 + 225*p**5*q2*q3*q4**4*q5*q6 + 225*p**5*q2*q3*q4**3*q5**2*q6 + 225*p**5*q2*q3*q4**3*q5*q6**2 + 225*p**5*q2*q3*q4**3*q5*q6*q7 - 1800*p**5*q2*q3*q4**3*q5*q6 + 225*p**5*q2*q3*q4**2*q5**3*q6 + 225*p**5*q2*q3*q4**2*q5**2*q6**2 + 225*p**5*q2*q3*q4**2*q5**2*q6*q7 - 1800*p**5*q2*q3*q4**2*q5**2*q6 + 225*p**5*q2*q3*q4**2*q5*q6**3 + 225*p**5*q2*q3*q4**2*q5*q6**2*q7 - 1800*p**5*q2*q3*q4**2*q5*q6**2 + 225*p**5*q2*q3*q4**2*q5*q6*q7**2 - 1800*p**5*q2*q3*q4**2*q5*q6*q7 + 6300*p**5*q2*q3*q4**2*q5*q6 + 225*p**5*q2*q3*q4*q5**4*q6 + 225*p**5*q2*q3*q4*q5**3*q6**2 + 225*p**5*q2*q3*q4*q5**3*q6*q7 - 1800*p**5*q2*q3*q4*q5**3*q6 + 225*p**5*q2*q3*q4*q5**2*q6**3 + 225*p**5*q2*q3*q4*q5**2*q6**2*q7 - 1800*p**5*q2*q3*q4*q5**2*q6**2 + 225*p**5*q2*q3*q4*q5**2*q6*q7**2 - 1800*p**5*q2*q3*q4*q5**2*q6*q7 + 6300*p**5*q2*q3*q4*q5**2*q6 + 225*p**5*q2*q3*q4*q5*q6**4 + 225*p**5*q2*q3*q4*q5*q6**3*q7 - 1800*p**5*q2*q3*q4*q5*q6**3 + 225*p**5*q2*q3*q4*q5*q6**2*q7**2 - 1800*p**5*q2*q3*q4*q5*q6**2*q7 + 6300*p**5*q2*q3*q4*q5*q6**2 + 225*p**5*q2*q3*q4*q5*q6*q7**3 - 1800*p**5*q2*q3*q4*q5*q6*q7**2 + 6300*p**5*q2*q3*q4*q5*q6*q7 - 12600*p**5*q2*q3*q4*q5*q6 - 480*p**5*q3**4*q4*q5*q6 - 480*p**5*q3**3*q4**2*q5*q6 - 480*p**5*q3**3*q4*q5**2*q6 - 480*p**5*q3**3*q4*q5*q6**2 - 480*p**5*q3**3*q4*q5*q6*q7 + 3360*p**5*q3**3*q4*q5*q6 - 480*p**5*q3**2*q4**3*q5*q6 - 480*p**5*q3**2*q4**2*q5**2*q6 - 480*p**5*q3**2*q4**2*q5*q6**2 - 480*p**5*q3**2*q4**2*q5*q6*q7 + 3360*p**5*q3**2*q4**2*q5*q6 - 480*p**5*q3**2*q4*q5**3*q6 - 480*p**5*q3**2*q4*q5**2*q6**2 - 480*p**5*q3**2*q4*q5**2*q6*q7 + 3360*p**5*q3**2*q4*q5**2*q6 - 480*p**5*q3**2*q4*q5*q6**3 - 480*p**5*q3**2*q4*q5*q6**2*q7 + 3360*p**5*q3**2*q4*q5*q6**2 - 480*p**5*q3**2*q4*q5*q6*q7**2 + 3360*p**5*q3**2*q4*q5*q6*q7 - 10080*p**5*q3**2*q4*q5*q6 - 480*p**5*q3*q4**4*q5*q6 - 480*p**5*q3*q4**3*q5**2*q6 - 480*p**5*q3*q4**3*q5*q6**2 - 480*p**5*q3*q4**3*q5*q6*q7 + 3360*p**5*q3*q4**3*q5*q6 - 480*p**5*q3*q4**2*q5**3*q6 - 480*p**5*q3*q4**2*q5**2*q6**2 - 480*p**5*q3*q4**2*q5**2*q6*q7 + 3360*p**5*q3*q4**2*q5**2*q6 - 480*p**5*q3*q4**2*q5*q6**3 - 480*p**5*q3*q4**2*q5*q6**2*q7 + 3360*p**5*q3*q4**2*q5*q6**2 - 480*p**5*q3*q4**2*q5*q6*q7**2 + 3360*p**5*q3*q4**2*q5*q6*q7 - 10080*p**5*q3*q4**2*q5*q6 - 480*p**5*q3*q4*q5**4*q6 - 480*p**5*q3*q4*q5**3*q6**2 - 480*p**5*q3*q4*q5**3*q6*q7 + 3360*p**5*q3*q4*q5**3*q6 - 480*p**5*q3*q4*q5**2*q6**3 - 480*p**5*q3*q4*q5**2*q6**2*q7 + 3360*p**5*q3*q4*q5**2*q6**2 - 480*p**5*q3*q4*q5**2*q6*q7**2 + 3360*p**5*q3*q4*q5**2*q6*q7 - 10080*p**5*q3*q4*q5**2*q6 - 480*p**5*q3*q4*q5*q6**4 - 480*p**5*q3*q4*q5*q6**3*q7 + 3360*p**5*q3*q4*q5*q6**3 - 480*p**5*q3*q4*q5*q6**2*q7**2 + 3360*p**5*q3*q4*q5*q6**2*q7 - 10080*p**5*q3*q4*q5*q6**2 - 480*p**5*q3*q4*q5*q6*q7**3 + 3360*p**5*q3*q4*q5*q6*q7**2 - 10080*p**5*q3*q4*q5*q6*q7 + 16800*p**5*q3*q4*q5*q6 + 630*p**5*q4**4*q5*q6 + 630*p**5*q4**3*q5**2*q6 + 630*p**5*q4**3*q5*q6**2 + 630*p**5*q4**3*q5*q6*q7 - 3780*p**5*q4**3*q5*q6 + 630*p**5*q4**2*q5**3*q6 + 630*p**5*q4**2*q5**2*q6**2 + 630*p**5*q4**2*q5**2*q6*q7 - 3780*p**5*q4**2*q5**2*q6 + 630*p**5*q4**2*q5*q6**3 + 630*p**5*q4**2*q5*q6**2*q7 - 3780*p**5*q4**2*q5*q6**2 + 630*p**5*q4**2*q5*q6*q7**2 - 3780*p**5*q4**2*q5*q6*q7 + 9450*p**5*q4**2*q5*q6 + 630*p**5*q4*q5**4*q6 + 630*p**5*q4*q5**3*q6**2 + 630*p**5*q4*q5**3*q6*q7 - 3780*p**5*q4*q5**3*q6 + 630*p**5*q4*q5**2*q6**3 + 630*p**5*q4*q5**2*q6**2*q7 - 3780*p**5*q4*q5**2*q6**2 + 630*p**5*q4*q5**2*q6*q7**2 - 3780*p**5*q4*q5**2*q6*q7 + 9450*p**5*q4*q5**2*q6 + 630*p**5*q4*q5*q6**4 + 630*p**5*q4*q5*q6**3*q7 - 3780*p**5*q4*q5*q6**3 + 630*p**5*q4*q5*q6**2*q7**2 - 3780*p**5*q4*q5*q6**2*q7 + 9450*p**5*q4*q5*q6**2 + 630*p**5*q4*q5*q6*q7**3 - 3780*p**5*q4*q5*q6*q7**2 + 9450*p**5*q4*q5*q6*q7 - 12600*p**5*q4*q5*q6 - 504*p**5*q5**4*q6 - 504*p**5*q5**3*q6**2 - 504*p**5*q5**3*q6*q7 + 2520*p**5*q5**3*q6 - 504*p**5*q5**2*q6**3 - 504*p**5*q5**2*q6**2*q7 + 2520*p**5*q5**2*q6**2 - 504*p**5*q5**2*q6*q7**2 + 2520*p**5*q5**2*q6*q7 - 5040*p**5*q5**2*q6 - 504*p**5*q5*q6**4 - 504*p**5*q5*q6**3*q7 + 2520*p**5*q5*q6**3 - 504*p**5*q5*q6**2*q7**2 + 2520*p**5*q5*q6**2*q7 - 5040*p**5*q5*q6**2 - 504*p**5*q5*q6*q7**3 + 2520*p**5*q5*q6*q7**2 - 5040*p**5*q5*q6*q7 + 5040*p**5*q5*q6 + 210*p**5*q6**4 + 210*p**5*q6**3*q7 - 840*p**5*q6**3 + 210*p**5*q6**2*q7**2 - 840*p**5*q6**2*q7 + 1260*p**5*q6**2 + 210*p**5*q6*q7**3 - 840*p**5*q6*q7**2 + 1260*p**5*q6*q7 - 840*p**5*q6 + 150*p**4*q1**4*q2*q3*q4*q5*q6 + 150*p**4*q1**3*q2**2*q3*q4*q5*q6 + 150*p**4*q1**3*q2*q3**2*q4*q5*q6 + 150*p**4*q1**3*q2*q3*q4**2*q5*q6 + 150*p**4*q1**3*q2*q3*q4*q5**2*q6 + 150*p**4*q1**3*q2*q3*q4*q5*q6**2 + 150*p**4*q1**3*q2*q3*q4*q5*q6*q7 - 1350*p**4*q1**3*q2*q3*q4*q5*q6 + 150*p**4*q1**2*q2**3*q3*q4*q5*q6 + 150*p**4*q1**2*q2**2*q3**2*q4*q5*q6 + 150*p**4*q1**2*q2**2*q3*q4**2*q5*q6 + 150*p**4*q1**2*q2**2*q3*q4*q5**2*q6 + 150*p**4*q1**2*q2**2*q3*q4*q5*q6**2 + 150*p**4*q1**2*q2**2*q3*q4*q5*q6*q7 - 1350*p**4*q1**2*q2**2*q3*q4*q5*q6 + 150*p**4*q1**2*q2*q3**3*q4*q5*q6 + 150*p**4*q1**2*q2*q3**2*q4**2*q5*q6 + 150*p**4*q1**2*q2*q3**2*q4*q5**2*q6 + 150*p**4*q1**2*q2*q3**2*q4*q5*q6**2 + 150*p**4*q1**2*q2*q3**2*q4*q5*q6*q7 - 1350*p**4*q1**2*q2*q3**2*q4*q5*q6 + 150*p**4*q1**2*q2*q3*q4**3*q5*q6 + 150*p**4*q1**2*q2*q3*q4**2*q5**2*q6 + 150*p**4*q1**2*q2*q3*q4**2*q5*q6**2 + 150*p**4*q1**2*q2*q3*q4**2*q5*q6*q7 - 1350*p**4*q1**2*q2*q3*q4**2*q5*q6 + 150*p**4*q1**2*q2*q3*q4*q5**3*q6 + 150*p**4*q1**2*q2*q3*q4*q5**2*q6**2 + 150*p**4*q1**2*q2*q3*q4*q5**2*q6*q7 - 1350*p**4*q1**2*q2*q3*q4*q5**2*q6 + 150*p**4*q1**2*q2*q3*q4*q5*q6**3 + 150*p**4*q1**2*q2*q3*q4*q5*q6**2*q7 - 1350*p**4*q1**2*q2*q3*q4*q5*q6**2 + 150*p**4*q1**2*q2*q3*q4*q5*q6*q7**2 - 1350*p**4*q1**2*q2*q3*q4*q5*q6*q7 + 5400*p**4*q1**2*q2*q3*q4*q5*q6 + 150*p**4*q1*q2**4*q3*q4*q5*q6 + 150*p**4*q1*q2**3*q3**2*q4*q5*q6 + 150*p**4*q1*q2**3*q3*q4**2*q5*q6 + 150*p**4*q1*q2**3*q3*q4*q5**2*q6 + 150*p**4*q1*q2**3*q3*q4*q5*q6**2 + 150*p**4*q1*q2**3*q3*q4*q5*q6*q7 - 1350*p**4*q1*q2**3*q3*q4*q5*q6 + 150*p**4*q1*q2**2*q3**3*q4*q5*q6 + 150*p**4*q1*q2**2*q3**2*q4**2*q5*q6 + 150*p**4*q1*q2**2*q3**2*q4*q5**2*q6 + 150*p**4*q1*q2**2*q3**2*q4*q5*q6**2 + 150*p**4*q1*q2**2*q3**2*q4*q5*q6*q7 - 1350*p**4*q1*q2**2*q3**2*q4*q5*q6 + 150*p**4*q1*q2**2*q3*q4**3*q5*q6 + 150*p**4*q1*q2**2*q3*q4**2*q5**2*q6 + 150*p**4*q1*q2**2*q3*q4**2*q5*q6**2 + 150*p**4*q1*q2**2*q3*q4**2*q5*q6*q7 - 1350*p**4*q1*q2**2*q3*q4**2*q5*q6 + 150*p**4*q1*q2**2*q3*q4*q5**3*q6 + 150*p**4*q1*q2**2*q3*q4*q5**2*q6**2 + 150*p**4*q1*q2**2*q3*q4*q5**2*q6*q7 - 1350*p**4*q1*q2**2*q3*q4*q5**2*q6 + 150*p**4*q1*q2**2*q3*q4*q5*q6**3 + 150*p**4*q1*q2**2*q3*q4*q5*q6**2*q7 - 1350*p**4*q1*q2**2*q3*q4*q5*q6**2 + 150*p**4*q1*q2**2*q3*q4*q5*q6*q7**2 - 1350*p**4*q1*q2**2*q3*q4*q5*q6*q7 + 5400*p**4*q1*q2**2*q3*q4*q5*q6 + 150*p**4*q1*q2*q3**4*q4*q5*q6 + 150*p**4*q1*q2*q3**3*q4**2*q5*q6 + 150*p**4*q1*q2*q3**3*q4*q5**2*q6 + 150*p**4*q1*q2*q3**3*q4*q5*q6**2 + 150*p**4*q1*q2*q3**3*q4*q5*q6*q7 - 1350*p**4*q1*q2*q3**3*q4*q5*q6 + 150*p**4*q1*q2*q3**2*q4**3*q5*q6 + 150*p**4*q1*q2*q3**2*q4**2*q5**2*q6 + 150*p**4*q1*q2*q3**2*q4**2*q5*q6**2 + 150*p**4*q1*q2*q3**2*q4**2*q5*q6*q7 - 1350*p**4*q1*q2*q3**2*q4**2*q5*q6 + 150*p**4*q1*q2*q3**2*q4*q5**3*q6 + 150*p**4*q1*q2*q3**2*q4*q5**2*q6**2 + 150*p**4*q1*q2*q3**2*q4*q5**2*q6*q7 - 1350*p**4*q1*q2*q3**2*q4*q5**2*q6 + 150*p**4*q1*q2*q3**2*q4*q5*q6**3 + 150*p**4*q1*q2*q3**2*q4*q5*q6**2*q7 - 1350*p**4*q1*q2*q3**2*q4*q5*q6**2 + 150*p**4*q1*q2*q3**2*q4*q5*q6*q7**2 - 1350*p**4*q1*q2*q3**2*q4*q5*q6*q7 + 5400*p**4*q1*q2*q3**2*q4*q5*q6 + 150*p**4*q1*q2*q3*q4**4*q5*q6 + 150*p**4*q1*q2*q3*q4**3*q5**2*q6 + 150*p**4*q1*q2*q3*q4**3*q5*q6**2 + 150*p**4*q1*q2*q3*q4**3*q5*q6*q7 - 1350*p**4*q1*q2*q3*q4**3*q5*q6 + 150*p**4*q1*q2*q3*q4**2*q5**3*q6 + 150*p**4*q1*q2*q3*q4**2*q5**2*q6**2 + 150*p**4*q1*q2*q3*q4**2*q5**2*q6*q7 - 1350*p**4*q1*q2*q3*q4**2*q5**2*q6 + 150*p**4*q1*q2*q3*q4**2*q5*q6**3 + 150*p**4*q1*q2*q3*q4**2*q5*q6**2*q7 - 1350*p**4*q1*q2*q3*q4**2*q5*q6**2 + 150*p**4*q1*q2*q3*q4**2*q5*q6*q7**2 - 1350*p**4*q1*q2*q3*q4**2*q5*q6*q7 + 5400*p**4*q1*q2*q3*q4**2*q5*q6 + 150*p**4*q1*q2*q3*q4*q5**4*q6 + 150*p**4*q1*q2*q3*q4*q5**3*q6**2 + 150*p**4*q1*q2*q3*q4*q5**3*q6*q7 - 1350*p**4*q1*q2*q3*q4*q5**3*q6 + 150*p**4*q1*q2*q3*q4*q5**2*q6**3 + 150*p**4*q1*q2*q3*q4*q5**2*q6**2*q7 - 1350*p**4*q1*q2*q3*q4*q5**2*q6**2 + 150*p**4*q1*q2*q3*q4*q5**2*q6*q7**2 - 1350*p**4*q1*q2*q3*q4*q5**2*q6*q7 + 5400*p**4*q1*q2*q3*q4*q5**2*q6 + 150*p**4*q1*q2*q3*q4*q5*q6**4 + 150*p**4*q1*q2*q3*q4*q5*q6**3*q7 - 1350*p**4*q1*q2*q3*q4*q5*q6**3 + 150*p**4*q1*q2*q3*q4*q5*q6**2*q7**2 - 1350*p**4*q1*q2*q3*q4*q5*q6**2*q7 + 5400*p**4*q1*q2*q3*q4*q5*q6**2 + 150*p**4*q1*q2*q3*q4*q5*q6*q7**3 - 1350*p**4*q1*q2*q3*q4*q5*q6*q7**2 + 5400*p**4*q1*q2*q3*q4*q5*q6*q7 - 12600*p**4*q1*q2*q3*q4*q5*q6 - 450*p**4*q2**4*q3*q4*q5*q6 - 450*p**4*q2**3*q3**2*q4*q5*q6 - 450*p**4*q2**3*q3*q4**2*q5*q6 - 450*p**4*q2**3*q3*q4*q5**2*q6 - 450*p**4*q2**3*q3*q4*q5*q6**2 - 450*p**4*q2**3*q3*q4*q5*q6*q7 + 3600*p**4*q2**3*q3*q4*q5*q6 - 450*p**4*q2**2*q3**3*q4*q5*q6 - 450*p**4*q2**2*q3**2*q4**2*q5*q6 - 450*p**4*q2**2*q3**2*q4*q5**2*q6 - 450*p**4*q2**2*q3**2*q4*q5*q6**2 - 450*p**4*q2**2*q3**2*q4*q5*q6*q7 + 3600*p**4*q2**2*q3**2*q4*q5*q6 - 450*p**4*q2**2*q3*q4**3*q5*q6 - 450*p**4*q2**2*q3*q4**2*q5**2*q6 - 450*p**4*q2**2*q3*q4**2*q5*q6**2 - 450*p**4*q2**2*q3*q4**2*q5*q6*q7 + 3600*p**4*q2**2*q3*q4**2*q5*q6 - 450*p**4*q2**2*q3*q4*q5**3*q6 - 450*p**4*q2**2*q3*q4*q5**2*q6**2 - 450*p**4*q2**2*q3*q4*q5**2*q6*q7 + 3600*p**4*q2**2*q3*q4*q5**2*q6 - 450*p**4*q2**2*q3*q4*q5*q6**3 - 450*p**4*q2**2*q3*q4*q5*q6**2*q7 + 3600*p**4*q2**2*q3*q4*q5*q6**2 - 450*p**4*q2**2*q3*q4*q5*q6*q7**2 + 3600*p**4*q2**2*q3*q4*q5*q6*q7 - 12600*p**4*q2**2*q3*q4*q5*q6 - 450*p**4*q2*q3**4*q4*q5*q6 - 450*p**4*q2*q3**3*q4**2*q5*q6 - 450*p**4*q2*q3**3*q4*q5**2*q6 - 450*p**4*q2*q3**3*q4*q5*q6**2 - 450*p**4*q2*q3**3*q4*q5*q6*q7 + 3600*p**4*q2*q3**3*q4*q5*q6 - 450*p**4*q2*q3**2*q4**3*q5*q6 - 450*p**4*q2*q3**2*q4**2*q5**2*q6 - 450*p**4*q2*q3**2*q4**2*q5*q6**2 - 450*p**4*q2*q3**2*q4**2*q5*q6*q7 + 3600*p**4*q2*q3**2*q4**2*q5*q6 - 450*p**4*q2*q3**2*q4*q5**3*q6 - 450*p**4*q2*q3**2*q4*q5**2*q6**2 - 450*p**4*q2*q3**2*q4*q5**2*q6*q7 + 3600*p**4*q2*q3**2*q4*q5**2*q6 - 450*p**4*q2*q3**2*q4*q5*q6**3 - 450*p**4*q2*q3**2*q4*q5*q6**2*q7 + 3600*p**4*q2*q3**2*q4*q5*q6**2 - 450*p**4*q2*q3**2*q4*q5*q6*q7**2 + 3600*p**4*q2*q3**2*q4*q5*q6*q7 - 12600*p**4*q2*q3**2*q4*q5*q6 - 450*p**4*q2*q3*q4**4*q5*q6 - 450*p**4*q2*q3*q4**3*q5**2*q6 - 450*p**4*q2*q3*q4**3*q5*q6**2 - 450*p**4*q2*q3*q4**3*q5*q6*q7 + 3600*p**4*q2*q3*q4**3*q5*q6 - 450*p**4*q2*q3*q4**2*q5**3*q6 - 450*p**4*q2*q3*q4**2*q5**2*q6**2 - 450*p**4*q2*q3*q4**2*q5**2*q6*q7 + 3600*p**4*q2*q3*q4**2*q5**2*q6 - 450*p**4*q2*q3*q4**2*q5*q6**3 - 450*p**4*q2*q3*q4**2*q5*q6**2*q7 + 3600*p**4*q2*q3*q4**2*q5*q6**2 - 450*p**4*q2*q3*q4**2*q5*q6*q7**2 + 3600*p**4*q2*q3*q4**2*q5*q6*q7 - 12600*p**4*q2*q3*q4**2*q5*q6 - 450*p**4*q2*q3*q4*q5**4*q6 - 450*p**4*q2*q3*q4*q5**3*q6**2 - 450*p**4*q2*q3*q4*q5**3*q6*q7 + 3600*p**4*q2*q3*q4*q5**3*q6 - 450*p**4*q2*q3*q4*q5**2*q6**3 - 450*p**4*q2*q3*q4*q5**2*q6**2*q7 + 3600*p**4*q2*q3*q4*q5**2*q6**2 - 450*p**4*q2*q3*q4*q5**2*q6*q7**2 + 3600*p**4*q2*q3*q4*q5**2*q6*q7 - 12600*p**4*q2*q3*q4*q5**2*q6 - 450*p**4*q2*q3*q4*q5*q6**4 - 450*p**4*q2*q3*q4*q5*q6**3*q7 + 3600*p**4*q2*q3*q4*q5*q6**3 - 450*p**4*q2*q3*q4*q5*q6**2*q7**2 + 3600*p**4*q2*q3*q4*q5*q6**2*q7 - 12600*p**4*q2*q3*q4*q5*q6**2 - 450*p**4*q2*q3*q4*q5*q6*q7**3 + 3600*p**4*q2*q3*q4*q5*q6*q7**2 - 12600*p**4*q2*q3*q4*q5*q6*q7 + 25200*p**4*q2*q3*q4*q5*q6 + 720*p**4*q3**4*q4*q5*q6 + 720*p**4*q3**3*q4**2*q5*q6 + 720*p**4*q3**3*q4*q5**2*q6 + 720*p**4*q3**3*q4*q5*q6**2 + 720*p**4*q3**3*q4*q5*q6*q7 - 5040*p**4*q3**3*q4*q5*q6 + 720*p**4*q3**2*q4**3*q5*q6 + 720*p**4*q3**2*q4**2*q5**2*q6 + 720*p**4*q3**2*q4**2*q5*q6**2 + 720*p**4*q3**2*q4**2*q5*q6*q7 - 5040*p**4*q3**2*q4**2*q5*q6 + 720*p**4*q3**2*q4*q5**3*q6 + 720*p**4*q3**2*q4*q5**2*q6**2 + 720*p**4*q3**2*q4*q5**2*q6*q7 - 5040*p**4*q3**2*q4*q5**2*q6 + 720*p**4*q3**2*q4*q5*q6**3 + 720*p**4*q3**2*q4*q5*q6**2*q7 - 5040*p**4*q3**2*q4*q5*q6**2 + 720*p**4*q3**2*q4*q5*q6*q7**2 - 5040*p**4*q3**2*q4*q5*q6*q7 + 15120*p**4*q3**2*q4*q5*q6 + 720*p**4*q3*q4**4*q5*q6 + 720*p**4*q3*q4**3*q5**2*q6 + 720*p**4*q3*q4**3*q5*q6**2 + 720*p**4*q3*q4**3*q5*q6*q7 - 5040*p**4*q3*q4**3*q5*q6 + 720*p**4*q3*q4**2*q5**3*q6 + 720*p**4*q3*q4**2*q5**2*q6**2 + 720*p**4*q3*q4**2*q5**2*q6*q7 - 5040*p**4*q3*q4**2*q5**2*q6 + 720*p**4*q3*q4**2*q5*q6**3 + 720*p**4*q3*q4**2*q5*q6**2*q7 - 5040*p**4*q3*q4**2*q5*q6**2 + 720*p**4*q3*q4**2*q5*q6*q7**2 - 5040*p**4*q3*q4**2*q5*q6*q7 + 15120*p**4*q3*q4**2*q5*q6 + 720*p**4*q3*q4*q5**4*q6 + 720*p**4*q3*q4*q5**3*q6**2 + 720*p**4*q3*q4*q5**3*q6*q7 - 5040*p**4*q3*q4*q5**3*q6 + 720*p**4*q3*q4*q5**2*q6**3 + 720*p**4*q3*q4*q5**2*q6**2*q7 - 5040*p**4*q3*q4*q5**2*q6**2 + 720*p**4*q3*q4*q5**2*q6*q7**2 - 5040*p**4*q3*q4*q5**2*q6*q7 + 15120*p**4*q3*q4*q5**2*q6 + 720*p**4*q3*q4*q5*q6**4 + 720*p**4*q3*q4*q5*q6**3*q7 - 5040*p**4*q3*q4*q5*q6**3 + 720*p**4*q3*q4*q5*q6**2*q7**2 - 5040*p**4*q3*q4*q5*q6**2*q7 + 15120*p**4*q3*q4*q5*q6**2 + 720*p**4*q3*q4*q5*q6*q7**3 - 5040*p**4*q3*q4*q5*q6*q7**2 + 15120*p**4*q3*q4*q5*q6*q7 - 25200*p**4*q3*q4*q5*q6 - 630*p**4*q4**4*q5*q6 - 630*p**4*q4**3*q5**2*q6 - 630*p**4*q4**3*q5*q6**2 - 630*p**4*q4**3*q5*q6*q7 + 3780*p**4*q4**3*q5*q6 - 630*p**4*q4**2*q5**3*q6 - 630*p**4*q4**2*q5**2*q6**2 - 630*p**4*q4**2*q5**2*q6*q7 + 3780*p**4*q4**2*q5**2*q6 - 630*p**4*q4**2*q5*q6**3 - 630*p**4*q4**2*q5*q6**2*q7 + 3780*p**4*q4**2*q5*q6**2 - 630*p**4*q4**2*q5*q6*q7**2 + 3780*p**4*q4**2*q5*q6*q7 - 9450*p**4*q4**2*q5*q6 - 630*p**4*q4*q5**4*q6 - 630*p**4*q4*q5**3*q6**2 - 630*p**4*q4*q5**3*q6*q7 + 3780*p**4*q4*q5**3*q6 - 630*p**4*q4*q5**2*q6**3 - 630*p**4*q4*q5**2*q6**2*q7 + 3780*p**4*q4*q5**2*q6**2 - 630*p**4*q4*q5**2*q6*q7**2 + 3780*p**4*q4*q5**2*q6*q7 - 9450*p**4*q4*q5**2*q6 - 630*p**4*q4*q5*q6**4 - 630*p**4*q4*q5*q6**3*q7 + 3780*p**4*q4*q5*q6**3 - 630*p**4*q4*q5*q6**2*q7**2 + 3780*p**4*q4*q5*q6**2*q7 - 9450*p**4*q4*q5*q6**2 - 630*p**4*q4*q5*q6*q7**3 + 3780*p**4*q4*q5*q6*q7**2 - 9450*p**4*q4*q5*q6*q7 + 12600*p**4*q4*q5*q6 + 252*p**4*q5**4*q6 + 252*p**4*q5**3*q6**2 + 252*p**4*q5**3*q6*q7 - 1260*p**4*q5**3*q6 + 252*p**4*q5**2*q6**3 + 252*p**4*q5**2*q6**2*q7 - 1260*p**4*q5**2*q6**2 + 252*p**4*q5**2*q6*q7**2 - 1260*p**4*q5**2*q6*q7 + 2520*p**4*q5**2*q6 + 252*p**4*q5*q6**4 + 252*p**4*q5*q6**3*q7 - 1260*p**4*q5*q6**3 + 252*p**4*q5*q6**2*q7**2 - 1260*p**4*q5*q6**2*q7 + 2520*p**4*q5*q6**2 + 252*p**4*q5*q6*q7**3 - 1260*p**4*q5*q6*q7**2 + 2520*p**4*q5*q6*q7 - 2520*p**4*q5*q6 - 200*p**3*q1**4*q2*q3*q4*q5*q6 - 200*p**3*q1**3*q2**2*q3*q4*q5*q6 - 200*p**3*q1**3*q2*q3**2*q4*q5*q6 - 200*p**3*q1**3*q2*q3*q4**2*q5*q6 - 200*p**3*q1**3*q2*q3*q4*q5**2*q6 - 200*p**3*q1**3*q2*q3*q4*q5*q6**2 - 200*p**3*q1**3*q2*q3*q4*q5*q6*q7 + 1800*p**3*q1**3*q2*q3*q4*q5*q6 - 200*p**3*q1**2*q2**3*q3*q4*q5*q6 - 200*p**3*q1**2*q2**2*q3**2*q4*q5*q6 - 200*p**3*q1**2*q2**2*q3*q4**2*q5*q6 - 200*p**3*q1**2*q2**2*q3*q4*q5**2*q6 - 200*p**3*q1**2*q2**2*q3*q4*q5*q6**2 - 200*p**3*q1**2*q2**2*q3*q4*q5*q6*q7 + 1800*p**3*q1**2*q2**2*q3*q4*q5*q6 - 200*p**3*q1**2*q2*q3**3*q4*q5*q6 - 200*p**3*q1**2*q2*q3**2*q4**2*q5*q6 - 200*p**3*q1**2*q2*q3**2*q4*q5**2*q6 - 200*p**3*q1**2*q2*q3**2*q4*q5*q6**2 - 200*p**3*q1**2*q2*q3**2*q4*q5*q6*q7 + 1800*p**3*q1**2*q2*q3**2*q4*q5*q6 - 200*p**3*q1**2*q2*q3*q4**3*q5*q6 - 200*p**3*q1**2*q2*q3*q4**2*q5**2*q6 - 200*p**3*q1**2*q2*q3*q4**2*q5*q6**2 - 200*p**3*q1**2*q2*q3*q4**2*q5*q6*q7 + 1800*p**3*q1**2*q2*q3*q4**2*q5*q6 - 200*p**3*q1**2*q2*q3*q4*q5**3*q6 - 200*p**3*q1**2*q2*q3*q4*q5**2*q6**2 - 200*p**3*q1**2*q2*q3*q4*q5**2*q6*q7 + 1800*p**3*q1**2*q2*q3*q4*q5**2*q6 - 200*p**3*q1**2*q2*q3*q4*q5*q6**3 - 200*p**3*q1**2*q2*q3*q4*q5*q6**2*q7 + 1800*p**3*q1**2*q2*q3*q4*q5*q6**2 - 200*p**3*q1**2*q2*q3*q4*q5*q6*q7**2 + 1800*p**3*q1**2*q2*q3*q4*q5*q6*q7 - 7200*p**3*q1**2*q2*q3*q4*q5*q6 - 200*p**3*q1*q2**4*q3*q4*q5*q6 - 200*p**3*q1*q2**3*q3**2*q4*q5*q6 - 200*p**3*q1*q2**3*q3*q4**2*q5*q6 - 200*p**3*q1*q2**3*q3*q4*q5**2*q6 - 200*p**3*q1*q2**3*q3*q4*q5*q6**2 - 200*p**3*q1*q2**3*q3*q4*q5*q6*q7 + 1800*p**3*q1*q2**3*q3*q4*q5*q6 - 200*p**3*q1*q2**2*q3**3*q4*q5*q6 - 200*p**3*q1*q2**2*q3**2*q4**2*q5*q6 - 200*p**3*q1*q2**2*q3**2*q4*q5**2*q6 - 200*p**3*q1*q2**2*q3**2*q4*q5*q6**2 - 200*p**3*q1*q2**2*q3**2*q4*q5*q6*q7 + 1800*p**3*q1*q2**2*q3**2*q4*q5*q6 - 200*p**3*q1*q2**2*q3*q4**3*q5*q6 - 200*p**3*q1*q2**2*q3*q4**2*q5**2*q6 - 200*p**3*q1*q2**2*q3*q4**2*q5*q6**2 - 200*p**3*q1*q2**2*q3*q4**2*q5*q6*q7 + 1800*p**3*q1*q2**2*q3*q4**2*q5*q6 - 200*p**3*q1*q2**2*q3*q4*q5**3*q6 - 200*p**3*q1*q2**2*q3*q4*q5**2*q6**2 - 200*p**3*q1*q2**2*q3*q4*q5**2*q6*q7 + 1800*p**3*q1*q2**2*q3*q4*q5**2*q6 - 200*p**3*q1*q2**2*q3*q4*q5*q6**3 - 200*p**3*q1*q2**2*q3*q4*q5*q6**2*q7 + 1800*p**3*q1*q2**2*q3*q4*q5*q6**2 - 200*p**3*q1*q2**2*q3*q4*q5*q6*q7**2 + 1800*p**3*q1*q2**2*q3*q4*q5*q6*q7 - 7200*p**3*q1*q2**2*q3*q4*q5*q6 - 200*p**3*q1*q2*q3**4*q4*q5*q6 - 200*p**3*q1*q2*q3**3*q4**2*q5*q6 - 200*p**3*q1*q2*q3**3*q4*q5**2*q6 - 200*p**3*q1*q2*q3**3*q4*q5*q6**2 - 200*p**3*q1*q2*q3**3*q4*q5*q6*q7 + 1800*p**3*q1*q2*q3**3*q4*q5*q6 - 200*p**3*q1*q2*q3**2*q4**3*q5*q6 - 200*p**3*q1*q2*q3**2*q4**2*q5**2*q6 - 200*p**3*q1*q2*q3**2*q4**2*q5*q6**2 - 200*p**3*q1*q2*q3**2*q4**2*q5*q6*q7 + 1800*p**3*q1*q2*q3**2*q4**2*q5*q6 - 200*p**3*q1*q2*q3**2*q4*q5**3*q6 - 200*p**3*q1*q2*q3**2*q4*q5**2*q6**2 - 200*p**3*q1*q2*q3**2*q4*q5**2*q6*q7 + 1800*p**3*q1*q2*q3**2*q4*q5**2*q6 - 200*p**3*q1*q2*q3**2*q4*q5*q6**3 - 200*p**3*q1*q2*q3**2*q4*q5*q6**2*q7 + 1800*p**3*q1*q2*q3**2*q4*q5*q6**2 - 200*p**3*q1*q2*q3**2*q4*q5*q6*q7**2 + 1800*p**3*q1*q2*q3**2*q4*q5*q6*q7 - 7200*p**3*q1*q2*q3**2*q4*q5*q6 - 200*p**3*q1*q2*q3*q4**4*q5*q6 - 200*p**3*q1*q2*q3*q4**3*q5**2*q6 - 200*p**3*q1*q2*q3*q4**3*q5*q6**2 - 200*p**3*q1*q2*q3*q4**3*q5*q6*q7 + 1800*p**3*q1*q2*q3*q4**3*q5*q6 - 200*p**3*q1*q2*q3*q4**2*q5**3*q6 - 200*p**3*q1*q2*q3*q4**2*q5**2*q6**2 - 200*p**3*q1*q2*q3*q4**2*q5**2*q6*q7 + 1800*p**3*q1*q2*q3*q4**2*q5**2*q6 - 200*p**3*q1*q2*q3*q4**2*q5*q6**3 - 200*p**3*q1*q2*q3*q4**2*q5*q6**2*q7 + 1800*p**3*q1*q2*q3*q4**2*q5*q6**2 - 200*p**3*q1*q2*q3*q4**2*q5*q6*q7**2 + 1800*p**3*q1*q2*q3*q4**2*q5*q6*q7 - 7200*p**3*q1*q2*q3*q4**2*q5*q6 - 200*p**3*q1*q2*q3*q4*q5**4*q6 - 200*p**3*q1*q2*q3*q4*q5**3*q6**2 - 200*p**3*q1*q2*q3*q4*q5**3*q6*q7 + 1800*p**3*q1*q2*q3*q4*q5**3*q6 - 200*p**3*q1*q2*q3*q4*q5**2*q6**3 - 200*p**3*q1*q2*q3*q4*q5**2*q6**2*q7 + 1800*p**3*q1*q2*q3*q4*q5**2*q6**2 - 200*p**3*q1*q2*q3*q4*q5**2*q6*q7**2 + 1800*p**3*q1*q2*q3*q4*q5**2*q6*q7 - 7200*p**3*q1*q2*q3*q4*q5**2*q6 - 200*p**3*q1*q2*q3*q4*q5*q6**4 - 200*p**3*q1*q2*q3*q4*q5*q6**3*q7 + 1800*p**3*q1*q2*q3*q4*q5*q6**3 - 200*p**3*q1*q2*q3*q4*q5*q6**2*q7**2 + 1800*p**3*q1*q2*q3*q4*q5*q6**2*q7 - 7200*p**3*q1*q2*q3*q4*q5*q6**2 - 200*p**3*q1*q2*q3*q4*q5*q6*q7**3 + 1800*p**3*q1*q2*q3*q4*q5*q6*q7**2 - 7200*p**3*q1*q2*q3*q4*q5*q6*q7 + 16800*p**3*q1*q2*q3*q4*q5*q6 + 450*p**3*q2**4*q3*q4*q5*q6 + 450*p**3*q2**3*q3**2*q4*q5*q6 + 450*p**3*q2**3*q3*q4**2*q5*q6 + 450*p**3*q2**3*q3*q4*q5**2*q6 + 450*p**3*q2**3*q3*q4*q5*q6**2 + 450*p**3*q2**3*q3*q4*q5*q6*q7 - 3600*p**3*q2**3*q3*q4*q5*q6 + 450*p**3*q2**2*q3**3*q4*q5*q6 + 450*p**3*q2**2*q3**2*q4**2*q5*q6 + 450*p**3*q2**2*q3**2*q4*q5**2*q6 + 450*p**3*q2**2*q3**2*q4*q5*q6**2 + 450*p**3*q2**2*q3**2*q4*q5*q6*q7 - 3600*p**3*q2**2*q3**2*q4*q5*q6 + 450*p**3*q2**2*q3*q4**3*q5*q6 + 450*p**3*q2**2*q3*q4**2*q5**2*q6 + 450*p**3*q2**2*q3*q4**2*q5*q6**2 + 450*p**3*q2**2*q3*q4**2*q5*q6*q7 - 3600*p**3*q2**2*q3*q4**2*q5*q6 + 450*p**3*q2**2*q3*q4*q5**3*q6 + 450*p**3*q2**2*q3*q4*q5**2*q6**2 + 450*p**3*q2**2*q3*q4*q5**2*q6*q7 - 3600*p**3*q2**2*q3*q4*q5**2*q6 + 450*p**3*q2**2*q3*q4*q5*q6**3 + 450*p**3*q2**2*q3*q4*q5*q6**2*q7 - 3600*p**3*q2**2*q3*q4*q5*q6**2 + 450*p**3*q2**2*q3*q4*q5*q6*q7**2 - 3600*p**3*q2**2*q3*q4*q5*q6*q7 + 12600*p**3*q2**2*q3*q4*q5*q6 + 450*p**3*q2*q3**4*q4*q5*q6 + 450*p**3*q2*q3**3*q4**2*q5*q6 + 450*p**3*q2*q3**3*q4*q5**2*q6 + 450*p**3*q2*q3**3*q4*q5*q6**2 + 450*p**3*q2*q3**3*q4*q5*q6*q7 - 3600*p**3*q2*q3**3*q4*q5*q6 + 450*p**3*q2*q3**2*q4**3*q5*q6 + 450*p**3*q2*q3**2*q4**2*q5**2*q6 + 450*p**3*q2*q3**2*q4**2*q5*q6**2 + 450*p**3*q2*q3**2*q4**2*q5*q6*q7 - 3600*p**3*q2*q3**2*q4**2*q5*q6 + 450*p**3*q2*q3**2*q4*q5**3*q6 + 450*p**3*q2*q3**2*q4*q5**2*q6**2 + 450*p**3*q2*q3**2*q4*q5**2*q6*q7 - 3600*p**3*q2*q3**2*q4*q5**2*q6 + 450*p**3*q2*q3**2*q4*q5*q6**3 + 450*p**3*q2*q3**2*q4*q5*q6**2*q7 - 3600*p**3*q2*q3**2*q4*q5*q6**2 + 450*p**3*q2*q3**2*q4*q5*q6*q7**2 - 3600*p**3*q2*q3**2*q4*q5*q6*q7 + 12600*p**3*q2*q3**2*q4*q5*q6 + 450*p**3*q2*q3*q4**4*q5*q6 + 450*p**3*q2*q3*q4**3*q5**2*q6 + 450*p**3*q2*q3*q4**3*q5*q6**2 + 450*p**3*q2*q3*q4**3*q5*q6*q7 - 3600*p**3*q2*q3*q4**3*q5*q6 + 450*p**3*q2*q3*q4**2*q5**3*q6 + 450*p**3*q2*q3*q4**2*q5**2*q6**2 + 450*p**3*q2*q3*q4**2*q5**2*q6*q7 - 3600*p**3*q2*q3*q4**2*q5**2*q6 + 450*p**3*q2*q3*q4**2*q5*q6**3 + 450*p**3*q2*q3*q4**2*q5*q6**2*q7 - 3600*p**3*q2*q3*q4**2*q5*q6**2 + 450*p**3*q2*q3*q4**2*q5*q6*q7**2 - 3600*p**3*q2*q3*q4**2*q5*q6*q7 + 12600*p**3*q2*q3*q4**2*q5*q6 + 450*p**3*q2*q3*q4*q5**4*q6 + 450*p**3*q2*q3*q4*q5**3*q6**2 + 450*p**3*q2*q3*q4*q5**3*q6*q7 - 3600*p**3*q2*q3*q4*q5**3*q6 + 450*p**3*q2*q3*q4*q5**2*q6**3 + 450*p**3*q2*q3*q4*q5**2*q6**2*q7 - 3600*p**3*q2*q3*q4*q5**2*q6**2 + 450*p**3*q2*q3*q4*q5**2*q6*q7**2 - 3600*p**3*q2*q3*q4*q5**2*q6*q7 + 12600*p**3*q2*q3*q4*q5**2*q6 + 450*p**3*q2*q3*q4*q5*q6**4 + 450*p**3*q2*q3*q4*q5*q6**3*q7 - 3600*p**3*q2*q3*q4*q5*q6**3 + 450*p**3*q2*q3*q4*q5*q6**2*q7**2 - 3600*p**3*q2*q3*q4*q5*q6**2*q7 + 12600*p**3*q2*q3*q4*q5*q6**2 + 450*p**3*q2*q3*q4*q5*q6*q7**3 - 3600*p**3*q2*q3*q4*q5*q6*q7**2 + 12600*p**3*q2*q3*q4*q5*q6*q7 - 25200*p**3*q2*q3*q4*q5*q6 - 480*p**3*q3**4*q4*q5*q6 - 480*p**3*q3**3*q4**2*q5*q6 - 480*p**3*q3**3*q4*q5**2*q6 - 480*p**3*q3**3*q4*q5*q6**2 - 480*p**3*q3**3*q4*q5*q6*q7 + 3360*p**3*q3**3*q4*q5*q6 - 480*p**3*q3**2*q4**3*q5*q6 - 480*p**3*q3**2*q4**2*q5**2*q6 - 480*p**3*q3**2*q4**2*q5*q6**2 - 480*p**3*q3**2*q4**2*q5*q6*q7 + 3360*p**3*q3**2*q4**2*q5*q6 - 480*p**3*q3**2*q4*q5**3*q6 - 480*p**3*q3**2*q4*q5**2*q6**2 - 480*p**3*q3**2*q4*q5**2*q6*q7 + 3360*p**3*q3**2*q4*q5**2*q6 - 480*p**3*q3**2*q4*q5*q6**3 - 480*p**3*q3**2*q4*q5*q6**2*q7 + 3360*p**3*q3**2*q4*q5*q6**2 - 480*p**3*q3**2*q4*q5*q6*q7**2 + 3360*p**3*q3**2*q4*q5*q6*q7 - 10080*p**3*q3**2*q4*q5*q6 - 480*p**3*q3*q4**4*q5*q6 - 480*p**3*q3*q4**3*q5**2*q6 - 480*p**3*q3*q4**3*q5*q6**2 - 480*p**3*q3*q4**3*q5*q6*q7 + 3360*p**3*q3*q4**3*q5*q6 - 480*p**3*q3*q4**2*q5**3*q6 - 480*p**3*q3*q4**2*q5**2*q6**2 - 480*p**3*q3*q4**2*q5**2*q6*q7 + 3360*p**3*q3*q4**2*q5**2*q6 - 480*p**3*q3*q4**2*q5*q6**3 - 480*p**3*q3*q4**2*q5*q6**2*q7 + 3360*p**3*q3*q4**2*q5*q6**2 - 480*p**3*q3*q4**2*q5*q6*q7**2 + 3360*p**3*q3*q4**2*q5*q6*q7 - 10080*p**3*q3*q4**2*q5*q6 - 480*p**3*q3*q4*q5**4*q6 - 480*p**3*q3*q4*q5**3*q6**2 - 480*p**3*q3*q4*q5**3*q6*q7 + 3360*p**3*q3*q4*q5**3*q6 - 480*p**3*q3*q4*q5**2*q6**3 - 480*p**3*q3*q4*q5**2*q6**2*q7 + 3360*p**3*q3*q4*q5**2*q6**2 - 480*p**3*q3*q4*q5**2*q6*q7**2 + 3360*p**3*q3*q4*q5**2*q6*q7 - 10080*p**3*q3*q4*q5**2*q6 - 480*p**3*q3*q4*q5*q6**4 - 480*p**3*q3*q4*q5*q6**3*q7 + 3360*p**3*q3*q4*q5*q6**3 - 480*p**3*q3*q4*q5*q6**2*q7**2 + 3360*p**3*q3*q4*q5*q6**2*q7 - 10080*p**3*q3*q4*q5*q6**2 - 480*p**3*q3*q4*q5*q6*q7**3 + 3360*p**3*q3*q4*q5*q6*q7**2 - 10080*p**3*q3*q4*q5*q6*q7 + 16800*p**3*q3*q4*q5*q6 + 210*p**3*q4**4*q5*q6 + 210*p**3*q4**3*q5**2*q6 + 210*p**3*q4**3*q5*q6**2 + 210*p**3*q4**3*q5*q6*q7 - 1260*p**3*q4**3*q5*q6 + 210*p**3*q4**2*q5**3*q6 + 210*p**3*q4**2*q5**2*q6**2 + 210*p**3*q4**2*q5**2*q6*q7 - 1260*p**3*q4**2*q5**2*q6 + 210*p**3*q4**2*q5*q6**3 + 210*p**3*q4**2*q5*q6**2*q7 - 1260*p**3*q4**2*q5*q6**2 + 210*p**3*q4**2*q5*q6*q7**2 - 1260*p**3*q4**2*q5*q6*q7 + 3150*p**3*q4**2*q5*q6 + 210*p**3*q4*q5**4*q6 + 210*p**3*q4*q5**3*q6**2 + 210*p**3*q4*q5**3*q6*q7 - 1260*p**3*q4*q5**3*q6 + 210*p**3*q4*q5**2*q6**3 + 210*p**3*q4*q5**2*q6**2*q7 - 1260*p**3*q4*q5**2*q6**2 + 210*p**3*q4*q5**2*q6*q7**2 - 1260*p**3*q4*q5**2*q6*q7 + 3150*p**3*q4*q5**2*q6 + 210*p**3*q4*q5*q6**4 + 210*p**3*q4*q5*q6**3*q7 - 1260*p**3*q4*q5*q6**3 + 210*p**3*q4*q5*q6**2*q7**2 - 1260*p**3*q4*q5*q6**2*q7 + 3150*p**3*q4*q5*q6**2 + 210*p**3*q4*q5*q6*q7**3 - 1260*p**3*q4*q5*q6*q7**2 + 3150*p**3*q4*q5*q6*q7 - 4200*p**3*q4*q5*q6 + 150*p**2*q1**4*q2*q3*q4*q5*q6 + 150*p**2*q1**3*q2**2*q3*q4*q5*q6 + 150*p**2*q1**3*q2*q3**2*q4*q5*q6 + 150*p**2*q1**3*q2*q3*q4**2*q5*q6 + 150*p**2*q1**3*q2*q3*q4*q5**2*q6 + 150*p**2*q1**3*q2*q3*q4*q5*q6**2 + 150*p**2*q1**3*q2*q3*q4*q5*q6*q7 - 1350*p**2*q1**3*q2*q3*q4*q5*q6 + 150*p**2*q1**2*q2**3*q3*q4*q5*q6 + 150*p**2*q1**2*q2**2*q3**2*q4*q5*q6 + 150*p**2*q1**2*q2**2*q3*q4**2*q5*q6 + 150*p**2*q1**2*q2**2*q3*q4*q5**2*q6 + 150*p**2*q1**2*q2**2*q3*q4*q5*q6**2 + 150*p**2*q1**2*q2**2*q3*q4*q5*q6*q7 - 1350*p**2*q1**2*q2**2*q3*q4*q5*q6 + 150*p**2*q1**2*q2*q3**3*q4*q5*q6 + 150*p**2*q1**2*q2*q3**2*q4**2*q5*q6 + 150*p**2*q1**2*q2*q3**2*q4*q5**2*q6 + 150*p**2*q1**2*q2*q3**2*q4*q5*q6**2 + 150*p**2*q1**2*q2*q3**2*q4*q5*q6*q7 - 1350*p**2*q1**2*q2*q3**2*q4*q5*q6 + 150*p**2*q1**2*q2*q3*q4**3*q5*q6 + 150*p**2*q1**2*q2*q3*q4**2*q5**2*q6 + 150*p**2*q1**2*q2*q3*q4**2*q5*q6**2 + 150*p**2*q1**2*q2*q3*q4**2*q5*q6*q7 - 1350*p**2*q1**2*q2*q3*q4**2*q5*q6 + 150*p**2*q1**2*q2*q3*q4*q5**3*q6 + 150*p**2*q1**2*q2*q3*q4*q5**2*q6**2 + 150*p**2*q1**2*q2*q3*q4*q5**2*q6*q7 - 1350*p**2*q1**2*q2*q3*q4*q5**2*q6 + 150*p**2*q1**2*q2*q3*q4*q5*q6**3 + 150*p**2*q1**2*q2*q3*q4*q5*q6**2*q7 - 1350*p**2*q1**2*q2*q3*q4*q5*q6**2 + 150*p**2*q1**2*q2*q3*q4*q5*q6*q7**2 - 1350*p**2*q1**2*q2*q3*q4*q5*q6*q7 + 5400*p**2*q1**2*q2*q3*q4*q5*q6 + 150*p**2*q1*q2**4*q3*q4*q5*q6 + 150*p**2*q1*q2**3*q3**2*q4*q5*q6 + 150*p**2*q1*q2**3*q3*q4**2*q5*q6 + 150*p**2*q1*q2**3*q3*q4*q5**2*q6 + 150*p**2*q1*q2**3*q3*q4*q5*q6**2 + 150*p**2*q1*q2**3*q3*q4*q5*q6*q7 - 1350*p**2*q1*q2**3*q3*q4*q5*q6 + 150*p**2*q1*q2**2*q3**3*q4*q5*q6 + 150*p**2*q1*q2**2*q3**2*q4**2*q5*q6 + 150*p**2*q1*q2**2*q3**2*q4*q5**2*q6 + 150*p**2*q1*q2**2*q3**2*q4*q5*q6**2 + 150*p**2*q1*q2**2*q3**2*q4*q5*q6*q7 - 1350*p**2*q1*q2**2*q3**2*q4*q5*q6 + 150*p**2*q1*q2**2*q3*q4**3*q5*q6 + 150*p**2*q1*q2**2*q3*q4**2*q5**2*q6 + 150*p**2*q1*q2**2*q3*q4**2*q5*q6**2 + 150*p**2*q1*q2**2*q3*q4**2*q5*q6*q7 - 1350*p**2*q1*q2**2*q3*q4**2*q5*q6 + 150*p**2*q1*q2**2*q3*q4*q5**3*q6 + 150*p**2*q1*q2**2*q3*q4*q5**2*q6**2 + 150*p**2*q1*q2**2*q3*q4*q5**2*q6*q7 - 1350*p**2*q1*q2**2*q3*q4*q5**2*q6 + 150*p**2*q1*q2**2*q3*q4*q5*q6**3 + 150*p**2*q1*q2**2*q3*q4*q5*q6**2*q7 - 1350*p**2*q1*q2**2*q3*q4*q5*q6**2 + 150*p**2*q1*q2**2*q3*q4*q5*q6*q7**2 - 1350*p**2*q1*q2**2*q3*q4*q5*q6*q7 + 5400*p**2*q1*q2**2*q3*q4*q5*q6 + 150*p**2*q1*q2*q3**4*q4*q5*q6 + 150*p**2*q1*q2*q3**3*q4**2*q5*q6 + 150*p**2*q1*q2*q3**3*q4*q5**2*q6 + 150*p**2*q1*q2*q3**3*q4*q5*q6**2 + 150*p**2*q1*q2*q3**3*q4*q5*q6*q7 - 1350*p**2*q1*q2*q3**3*q4*q5*q6 + 150*p**2*q1*q2*q3**2*q4**3*q5*q6 + 150*p**2*q1*q2*q3**2*q4**2*q5**2*q6 + 150*p**2*q1*q2*q3**2*q4**2*q5*q6**2 + 150*p**2*q1*q2*q3**2*q4**2*q5*q6*q7 - 1350*p**2*q1*q2*q3**2*q4**2*q5*q6 + 150*p**2*q1*q2*q3**2*q4*q5**3*q6 + 150*p**2*q1*q2*q3**2*q4*q5**2*q6**2 + 150*p**2*q1*q2*q3**2*q4*q5**2*q6*q7 - 1350*p**2*q1*q2*q3**2*q4*q5**2*q6 + 150*p**2*q1*q2*q3**2*q4*q5*q6**3 + 150*p**2*q1*q2*q3**2*q4*q5*q6**2*q7 - 1350*p**2*q1*q2*q3**2*q4*q5*q6**2 + 150*p**2*q1*q2*q3**2*q4*q5*q6*q7**2 - 1350*p**2*q1*q2*q3**2*q4*q5*q6*q7 + 5400*p**2*q1*q2*q3**2*q4*q5*q6 + 150*p**2*q1*q2*q3*q4**4*q5*q6 + 150*p**2*q1*q2*q3*q4**3*q5**2*q6 + 150*p**2*q1*q2*q3*q4**3*q5*q6**2 + 150*p**2*q1*q2*q3*q4**3*q5*q6*q7 - 1350*p**2*q1*q2*q3*q4**3*q5*q6 + 150*p**2*q1*q2*q3*q4**2*q5**3*q6 + 150*p**2*q1*q2*q3*q4**2*q5**2*q6**2 + 150*p**2*q1*q2*q3*q4**2*q5**2*q6*q7 - 1350*p**2*q1*q2*q3*q4**2*q5**2*q6 + 150*p**2*q1*q2*q3*q4**2*q5*q6**3 + 150*p**2*q1*q2*q3*q4**2*q5*q6**2*q7 - 1350*p**2*q1*q2*q3*q4**2*q5*q6**2 + 150*p**2*q1*q2*q3*q4**2*q5*q6*q7**2 - 1350*p**2*q1*q2*q3*q4**2*q5*q6*q7 + 5400*p**2*q1*q2*q3*q4**2*q5*q6 + 150*p**2*q1*q2*q3*q4*q5**4*q6 + 150*p**2*q1*q2*q3*q4*q5**3*q6**2 + 150*p**2*q1*q2*q3*q4*q5**3*q6*q7 - 1350*p**2*q1*q2*q3*q4*q5**3*q6 + 150*p**2*q1*q2*q3*q4*q5**2*q6**3 + 150*p**2*q1*q2*q3*q4*q5**2*q6**2*q7 - 1350*p**2*q1*q2*q3*q4*q5**2*q6**2 + 150*p**2*q1*q2*q3*q4*q5**2*q6*q7**2 - 1350*p**2*q1*q2*q3*q4*q5**2*q6*q7 + 5400*p**2*q1*q2*q3*q4*q5**2*q6 + 150*p**2*q1*q2*q3*q4*q5*q6**4 + 150*p**2*q1*q2*q3*q4*q5*q6**3*q7 - 1350*p**2*q1*q2*q3*q4*q5*q6**3 + 150*p**2*q1*q2*q3*q4*q5*q6**2*q7**2 - 1350*p**2*q1*q2*q3*q4*q5*q6**2*q7 + 5400*p**2*q1*q2*q3*q4*q5*q6**2 + 150*p**2*q1*q2*q3*q4*q5*q6*q7**3 - 1350*p**2*q1*q2*q3*q4*q5*q6*q7**2 + 5400*p**2*q1*q2*q3*q4*q5*q6*q7 - 12600*p**2*q1*q2*q3*q4*q5*q6 - 225*p**2*q2**4*q3*q4*q5*q6 - 225*p**2*q2**3*q3**2*q4*q5*q6 - 225*p**2*q2**3*q3*q4**2*q5*q6 - 225*p**2*q2**3*q3*q4*q5**2*q6 - 225*p**2*q2**3*q3*q4*q5*q6**2 - 225*p**2*q2**3*q3*q4*q5*q6*q7 + 1800*p**2*q2**3*q3*q4*q5*q6 - 225*p**2*q2**2*q3**3*q4*q5*q6 - 225*p**2*q2**2*q3**2*q4**2*q5*q6 - 225*p**2*q2**2*q3**2*q4*q5**2*q6 - 225*p**2*q2**2*q3**2*q4*q5*q6**2 - 225*p**2*q2**2*q3**2*q4*q5*q6*q7 + 1800*p**2*q2**2*q3**2*q4*q5*q6 - 225*p**2*q2**2*q3*q4**3*q5*q6 - 225*p**2*q2**2*q3*q4**2*q5**2*q6 - 225*p**2*q2**2*q3*q4**2*q5*q6**2 - 225*p**2*q2**2*q3*q4**2*q5*q6*q7 + 1800*p**2*q2**2*q3*q4**2*q5*q6 - 225*p**2*q2**2*q3*q4*q5**3*q6 - 225*p**2*q2**2*q3*q4*q5**2*q6**2 - 225*p**2*q2**2*q3*q4*q5**2*q6*q7 + 1800*p**2*q2**2*q3*q4*q5**2*q6 - 225*p**2*q2**2*q3*q4*q5*q6**3 - 225*p**2*q2**2*q3*q4*q5*q6**2*q7 + 1800*p**2*q2**2*q3*q4*q5*q6**2 - 225*p**2*q2**2*q3*q4*q5*q6*q7**2 + 1800*p**2*q2**2*q3*q4*q5*q6*q7 - 6300*p**2*q2**2*q3*q4*q5*q6 - 225*p**2*q2*q3**4*q4*q5*q6 - 225*p**2*q2*q3**3*q4**2*q5*q6 - 225*p**2*q2*q3**3*q4*q5**2*q6 - 225*p**2*q2*q3**3*q4*q5*q6**2 - 225*p**2*q2*q3**3*q4*q5*q6*q7 + 1800*p**2*q2*q3**3*q4*q5*q6 - 225*p**2*q2*q3**2*q4**3*q5*q6 - 225*p**2*q2*q3**2*q4**2*q5**2*q6 - 225*p**2*q2*q3**2*q4**2*q5*q6**2 - 225*p**2*q2*q3**2*q4**2*q5*q6*q7 + 1800*p**2*q2*q3**2*q4**2*q5*q6 - 225*p**2*q2*q3**2*q4*q5**3*q6 - 225*p**2*q2*q3**2*q4*q5**2*q6**2 - 225*p**2*q2*q3**2*q4*q5**2*q6*q7 + 1800*p**2*q2*q3**2*q4*q5**2*q6 - 225*p**2*q2*q3**2*q4*q5*q6**3 - 225*p**2*q2*q3**2*q4*q5*q6**2*q7 + 1800*p**2*q2*q3**2*q4*q5*q6**2 - 225*p**2*q2*q3**2*q4*q5*q6*q7**2 + 1800*p**2*q2*q3**2*q4*q5*q6*q7 - 6300*p**2*q2*q3**2*q4*q5*q6 - 225*p**2*q2*q3*q4**4*q5*q6 - 225*p**2*q2*q3*q4**3*q5**2*q6 - 225*p**2*q2*q3*q4**3*q5*q6**2 - 225*p**2*q2*q3*q4**3*q5*q6*q7 + 1800*p**2*q2*q3*q4**3*q5*q6 - 225*p**2*q2*q3*q4**2*q5**3*q6 - 225*p**2*q2*q3*q4**2*q5**2*q6**2 - 225*p**2*q2*q3*q4**2*q5**2*q6*q7 + 1800*p**2*q2*q3*q4**2*q5**2*q6 - 225*p**2*q2*q3*q4**2*q5*q6**3 - 225*p**2*q2*q3*q4**2*q5*q6**2*q7 + 1800*p**2*q2*q3*q4**2*q5*q6**2 - 225*p**2*q2*q3*q4**2*q5*q6*q7**2 + 1800*p**2*q2*q3*q4**2*q5*q6*q7 - 6300*p**2*q2*q3*q4**2*q5*q6 - 225*p**2*q2*q3*q4*q5**4*q6 - 225*p**2*q2*q3*q4*q5**3*q6**2 - 225*p**2*q2*q3*q4*q5**3*q6*q7 + 1800*p**2*q2*q3*q4*q5**3*q6 - 225*p**2*q2*q3*q4*q5**2*q6**3 - 225*p**2*q2*q3*q4*q5**2*q6**2*q7 + 1800*p**2*q2*q3*q4*q5**2*q6**2 - 225*p**2*q2*q3*q4*q5**2*q6*q7**2 + 1800*p**2*q2*q3*q4*q5**2*q6*q7 - 6300*p**2*q2*q3*q4*q5**2*q6 - 225*p**2*q2*q3*q4*q5*q6**4 - 225*p**2*q2*q3*q4*q5*q6**3*q7 + 1800*p**2*q2*q3*q4*q5*q6**3 - 225*p**2*q2*q3*q4*q5*q6**2*q7**2 + 1800*p**2*q2*q3*q4*q5*q6**2*q7 - 6300*p**2*q2*q3*q4*q5*q6**2 - 225*p**2*q2*q3*q4*q5*q6*q7**3 + 1800*p**2*q2*q3*q4*q5*q6*q7**2 - 6300*p**2*q2*q3*q4*q5*q6*q7 + 12600*p**2*q2*q3*q4*q5*q6 + 120*p**2*q3**4*q4*q5*q6 + 120*p**2*q3**3*q4**2*q5*q6 + 120*p**2*q3**3*q4*q5**2*q6 + 120*p**2*q3**3*q4*q5*q6**2 + 120*p**2*q3**3*q4*q5*q6*q7 - 840*p**2*q3**3*q4*q5*q6 + 120*p**2*q3**2*q4**3*q5*q6 + 120*p**2*q3**2*q4**2*q5**2*q6 + 120*p**2*q3**2*q4**2*q5*q6**2 + 120*p**2*q3**2*q4**2*q5*q6*q7 - 840*p**2*q3**2*q4**2*q5*q6 + 120*p**2*q3**2*q4*q5**3*q6 + 120*p**2*q3**2*q4*q5**2*q6**2 + 120*p**2*q3**2*q4*q5**2*q6*q7 - 840*p**2*q3**2*q4*q5**2*q6 + 120*p**2*q3**2*q4*q5*q6**3 + 120*p**2*q3**2*q4*q5*q6**2*q7 - 840*p**2*q3**2*q4*q5*q6**2 + 120*p**2*q3**2*q4*q5*q6*q7**2 - 840*p**2*q3**2*q4*q5*q6*q7 + 2520*p**2*q3**2*q4*q5*q6 + 120*p**2*q3*q4**4*q5*q6 + 120*p**2*q3*q4**3*q5**2*q6 + 120*p**2*q3*q4**3*q5*q6**2 + 120*p**2*q3*q4**3*q5*q6*q7 - 840*p**2*q3*q4**3*q5*q6 + 120*p**2*q3*q4**2*q5**3*q6 + 120*p**2*q3*q4**2*q5**2*q6**2 + 120*p**2*q3*q4**2*q5**2*q6*q7 - 840*p**2*q3*q4**2*q5**2*q6 + 120*p**2*q3*q4**2*q5*q6**3 + 120*p**2*q3*q4**2*q5*q6**2*q7 - 840*p**2*q3*q4**2*q5*q6**2 + 120*p**2*q3*q4**2*q5*q6*q7**2 - 840*p**2*q3*q4**2*q5*q6*q7 + 2520*p**2*q3*q4**2*q5*q6 + 120*p**2*q3*q4*q5**4*q6 + 120*p**2*q3*q4*q5**3*q6**2 + 120*p**2*q3*q4*q5**3*q6*q7 - 840*p**2*q3*q4*q5**3*q6 + 120*p**2*q3*q4*q5**2*q6**3 + 120*p**2*q3*q4*q5**2*q6**2*q7 - 840*p**2*q3*q4*q5**2*q6**2 + 120*p**2*q3*q4*q5**2*q6*q7**2 - 840*p**2*q3*q4*q5**2*q6*q7 + 2520*p**2*q3*q4*q5**2*q6 + 120*p**2*q3*q4*q5*q6**4 + 120*p**2*q3*q4*q5*q6**3*q7 - 840*p**2*q3*q4*q5*q6**3 + 120*p**2*q3*q4*q5*q6**2*q7**2 - 840*p**2*q3*q4*q5*q6**2*q7 + 2520*p**2*q3*q4*q5*q6**2 + 120*p**2*q3*q4*q5*q6*q7**3 - 840*p**2*q3*q4*q5*q6*q7**2 + 2520*p**2*q3*q4*q5*q6*q7 - 4200*p**2*q3*q4*q5*q6 - 60*p*q1**4*q2*q3*q4*q5*q6 - 60*p*q1**3*q2**2*q3*q4*q5*q6 - 60*p*q1**3*q2*q3**2*q4*q5*q6 - 60*p*q1**3*q2*q3*q4**2*q5*q6 - 60*p*q1**3*q2*q3*q4*q5**2*q6 - 60*p*q1**3*q2*q3*q4*q5*q6**2 - 60*p*q1**3*q2*q3*q4*q5*q6*q7 + 540*p*q1**3*q2*q3*q4*q5*q6 - 60*p*q1**2*q2**3*q3*q4*q5*q6 - 60*p*q1**2*q2**2*q3**2*q4*q5*q6 - 60*p*q1**2*q2**2*q3*q4**2*q5*q6 - 60*p*q1**2*q2**2*q3*q4*q5**2*q6 - 60*p*q1**2*q2**2*q3*q4*q5*q6**2 - 60*p*q1**2*q2**2*q3*q4*q5*q6*q7 + 540*p*q1**2*q2**2*q3*q4*q5*q6 - 60*p*q1**2*q2*q3**3*q4*q5*q6 - 60*p*q1**2*q2*q3**2*q4**2*q5*q6 - 60*p*q1**2*q2*q3**2*q4*q5**2*q6 - 60*p*q1**2*q2*q3**2*q4*q5*q6**2 - 60*p*q1**2*q2*q3**2*q4*q5*q6*q7 + 540*p*q1**2*q2*q3**2*q4*q5*q6 - 60*p*q1**2*q2*q3*q4**3*q5*q6 - 60*p*q1**2*q2*q3*q4**2*q5**2*q6 - 60*p*q1**2*q2*q3*q4**2*q5*q6**2 - 60*p*q1**2*q2*q3*q4**2*q5*q6*q7 + 540*p*q1**2*q2*q3*q4**2*q5*q6 - 60*p*q1**2*q2*q3*q4*q5**3*q6 - 60*p*q1**2*q2*q3*q4*q5**2*q6**2 - 60*p*q1**2*q2*q3*q4*q5**2*q6*q7 + 540*p*q1**2*q2*q3*q4*q5**2*q6 - 60*p*q1**2*q2*q3*q4*q5*q6**3 - 60*p*q1**2*q2*q3*q4*q5*q6**2*q7 + 540*p*q1**2*q2*q3*q4*q5*q6**2 - 60*p*q1**2*q2*q3*q4*q5*q6*q7**2 + 540*p*q1**2*q2*q3*q4*q5*q6*q7 - 2160*p*q1**2*q2*q3*q4*q5*q6 - 60*p*q1*q2**4*q3*q4*q5*q6 - 60*p*q1*q2**3*q3**2*q4*q5*q6 - 60*p*q1*q2**3*q3*q4**2*q5*q6 - 60*p*q1*q2**3*q3*q4*q5**2*q6 - 60*p*q1*q2**3*q3*q4*q5*q6**2 - 60*p*q1*q2**3*q3*q4*q5*q6*q7 + 540*p*q1*q2**3*q3*q4*q5*q6 - 60*p*q1*q2**2*q3**3*q4*q5*q6 - 60*p*q1*q2**2*q3**2*q4**2*q5*q6 - 60*p*q1*q2**2*q3**2*q4*q5**2*q6 - 60*p*q1*q2**2*q3**2*q4*q5*q6**2 - 60*p*q1*q2**2*q3**2*q4*q5*q6*q7 + 540*p*q1*q2**2*q3**2*q4*q5*q6 - 60*p*q1*q2**2*q3*q4**3*q5*q6 - 60*p*q1*q2**2*q3*q4**2*q5**2*q6 - 60*p*q1*q2**2*q3*q4**2*q5*q6**2 - 60*p*q1*q2**2*q3*q4**2*q5*q6*q7 + 540*p*q1*q2**2*q3*q4**2*q5*q6 - 60*p*q1*q2**2*q3*q4*q5**3*q6 - 60*p*q1*q2**2*q3*q4*q5**2*q6**2 - 60*p*q1*q2**2*q3*q4*q5**2*q6*q7 + 540*p*q1*q2**2*q3*q4*q5**2*q6 - 60*p*q1*q2**2*q3*q4*q5*q6**3 - 60*p*q1*q2**2*q3*q4*q5*q6**2*q7 + 540*p*q1*q2**2*q3*q4*q5*q6**2 - 60*p*q1*q2**2*q3*q4*q5*q6*q7**2 + 540*p*q1*q2**2*q3*q4*q5*q6*q7 - 2160*p*q1*q2**2*q3*q4*q5*q6 - 60*p*q1*q2*q3**4*q4*q5*q6 - 60*p*q1*q2*q3**3*q4**2*q5*q6 - 60*p*q1*q2*q3**3*q4*q5**2*q6 - 60*p*q1*q2*q3**3*q4*q5*q6**2 - 60*p*q1*q2*q3**3*q4*q5*q6*q7 + 540*p*q1*q2*q3**3*q4*q5*q6 - 60*p*q1*q2*q3**2*q4**3*q5*q6 - 60*p*q1*q2*q3**2*q4**2*q5**2*q6 - 60*p*q1*q2*q3**2*q4**2*q5*q6**2 - 60*p*q1*q2*q3**2*q4**2*q5*q6*q7 + 540*p*q1*q2*q3**2*q4**2*q5*q6 - 60*p*q1*q2*q3**2*q4*q5**3*q6 - 60*p*q1*q2*q3**2*q4*q5**2*q6**2 - 60*p*q1*q2*q3**2*q4*q5**2*q6*q7 + 540*p*q1*q2*q3**2*q4*q5**2*q6 - 60*p*q1*q2*q3**2*q4*q5*q6**3 - 60*p*q1*q2*q3**2*q4*q5*q6**2*q7 + 540*p*q1*q2*q3**2*q4*q5*q6**2 - 60*p*q1*q2*q3**2*q4*q5*q6*q7**2 + 540*p*q1*q2*q3**2*q4*q5*q6*q7 - 2160*p*q1*q2*q3**2*q4*q5*q6 - 60*p*q1*q2*q3*q4**4*q5*q6 - 60*p*q1*q2*q3*q4**3*q5**2*q6 - 60*p*q1*q2*q3*q4**3*q5*q6**2 - 60*p*q1*q2*q3*q4**3*q5*q6*q7 + 540*p*q1*q2*q3*q4**3*q5*q6 - 60*p*q1*q2*q3*q4**2*q5**3*q6 - 60*p*q1*q2*q3*q4**2*q5**2*q6**2 - 60*p*q1*q2*q3*q4**2*q5**2*q6*q7 + 540*p*q1*q2*q3*q4**2*q5**2*q6 - 60*p*q1*q2*q3*q4**2*q5*q6**3 - 60*p*q1*q2*q3*q4**2*q5*q6**2*q7 + 540*p*q1*q2*q3*q4**2*q5*q6**2 - 60*p*q1*q2*q3*q4**2*q5*q6*q7**2 + 540*p*q1*q2*q3*q4**2*q5*q6*q7 - 2160*p*q1*q2*q3*q4**2*q5*q6 - 60*p*q1*q2*q3*q4*q5**4*q6 - 60*p*q1*q2*q3*q4*q5**3*q6**2 - 60*p*q1*q2*q3*q4*q5**3*q6*q7 + 540*p*q1*q2*q3*q4*q5**3*q6 - 60*p*q1*q2*q3*q4*q5**2*q6**3 - 60*p*q1*q2*q3*q4*q5**2*q6**2*q7 + 540*p*q1*q2*q3*q4*q5**2*q6**2 - 60*p*q1*q2*q3*q4*q5**2*q6*q7**2 + 540*p*q1*q2*q3*q4*q5**2*q6*q7 - 2160*p*q1*q2*q3*q4*q5**2*q6 - 60*p*q1*q2*q3*q4*q5*q6**4 - 60*p*q1*q2*q3*q4*q5*q6**3*q7 + 540*p*q1*q2*q3*q4*q5*q6**3 - 60*p*q1*q2*q3*q4*q5*q6**2*q7**2 + 540*p*q1*q2*q3*q4*q5*q6**2*q7 - 2160*p*q1*q2*q3*q4*q5*q6**2 - 60*p*q1*q2*q3*q4*q5*q6*q7**3 + 540*p*q1*q2*q3*q4*q5*q6*q7**2 - 2160*p*q1*q2*q3*q4*q5*q6*q7 + 5040*p*q1*q2*q3*q4*q5*q6 + 45*p*q2**4*q3*q4*q5*q6 + 45*p*q2**3*q3**2*q4*q5*q6 + 45*p*q2**3*q3*q4**2*q5*q6 + 45*p*q2**3*q3*q4*q5**2*q6 + 45*p*q2**3*q3*q4*q5*q6**2 + 45*p*q2**3*q3*q4*q5*q6*q7 - 360*p*q2**3*q3*q4*q5*q6 + 45*p*q2**2*q3**3*q4*q5*q6 + 45*p*q2**2*q3**2*q4**2*q5*q6 + 45*p*q2**2*q3**2*q4*q5**2*q6 + 45*p*q2**2*q3**2*q4*q5*q6**2 + 45*p*q2**2*q3**2*q4*q5*q6*q7 - 360*p*q2**2*q3**2*q4*q5*q6 + 45*p*q2**2*q3*q4**3*q5*q6 + 45*p*q2**2*q3*q4**2*q5**2*q6 + 45*p*q2**2*q3*q4**2*q5*q6**2 + 45*p*q2**2*q3*q4**2*q5*q6*q7 - 360*p*q2**2*q3*q4**2*q5*q6 + 45*p*q2**2*q3*q4*q5**3*q6 + 45*p*q2**2*q3*q4*q5**2*q6**2 + 45*p*q2**2*q3*q4*q5**2*q6*q7 - 360*p*q2**2*q3*q4*q5**2*q6 + 45*p*q2**2*q3*q4*q5*q6**3 + 45*p*q2**2*q3*q4*q5*q6**2*q7 - 360*p*q2**2*q3*q4*q5*q6**2 + 45*p*q2**2*q3*q4*q5*q6*q7**2 - 360*p*q2**2*q3*q4*q5*q6*q7 + 1260*p*q2**2*q3*q4*q5*q6 + 45*p*q2*q3**4*q4*q5*q6 + 45*p*q2*q3**3*q4**2*q5*q6 + 45*p*q2*q3**3*q4*q5**2*q6 + 45*p*q2*q3**3*q4*q5*q6**2 + 45*p*q2*q3**3*q4*q5*q6*q7 - 360*p*q2*q3**3*q4*q5*q6 + 45*p*q2*q3**2*q4**3*q5*q6 + 45*p*q2*q3**2*q4**2*q5**2*q6 + 45*p*q2*q3**2*q4**2*q5*q6**2 + 45*p*q2*q3**2*q4**2*q5*q6*q7 - 360*p*q2*q3**2*q4**2*q5*q6 + 45*p*q2*q3**2*q4*q5**3*q6 + 45*p*q2*q3**2*q4*q5**2*q6**2 + 45*p*q2*q3**2*q4*q5**2*q6*q7 - 360*p*q2*q3**2*q4*q5**2*q6 + 45*p*q2*q3**2*q4*q5*q6**3 + 45*p*q2*q3**2*q4*q5*q6**2*q7 - 360*p*q2*q3**2*q4*q5*q6**2 + 45*p*q2*q3**2*q4*q5*q6*q7**2 - 360*p*q2*q3**2*q4*q5*q6*q7 + 1260*p*q2*q3**2*q4*q5*q6 + 45*p*q2*q3*q4**4*q5*q6 + 45*p*q2*q3*q4**3*q5**2*q6 + 45*p*q2*q3*q4**3*q5*q6**2 + 45*p*q2*q3*q4**3*q5*q6*q7 - 360*p*q2*q3*q4**3*q5*q6 + 45*p*q2*q3*q4**2*q5**3*q6 + 45*p*q2*q3*q4**2*q5**2*q6**2 + 45*p*q2*q3*q4**2*q5**2*q6*q7 - 360*p*q2*q3*q4**2*q5**2*q6 + 45*p*q2*q3*q4**2*q5*q6**3 + 45*p*q2*q3*q4**2*q5*q6**2*q7 - 360*p*q2*q3*q4**2*q5*q6**2 + 45*p*q2*q3*q4**2*q5*q6*q7**2 - 360*p*q2*q3*q4**2*q5*q6*q7 + 1260*p*q2*q3*q4**2*q5*q6 + 45*p*q2*q3*q4*q5**4*q6 + 45*p*q2*q3*q4*q5**3*q6**2 + 45*p*q2*q3*q4*q5**3*q6*q7 - 360*p*q2*q3*q4*q5**3*q6 + 45*p*q2*q3*q4*q5**2*q6**3 + 45*p*q2*q3*q4*q5**2*q6**2*q7 - 360*p*q2*q3*q4*q5**2*q6**2 + 45*p*q2*q3*q4*q5**2*q6*q7**2 - 360*p*q2*q3*q4*q5**2*q6*q7 + 1260*p*q2*q3*q4*q5**2*q6 + 45*p*q2*q3*q4*q5*q6**4 + 45*p*q2*q3*q4*q5*q6**3*q7 - 360*p*q2*q3*q4*q5*q6**3 + 45*p*q2*q3*q4*q5*q6**2*q7**2 - 360*p*q2*q3*q4*q5*q6**2*q7 + 1260*p*q2*q3*q4*q5*q6**2 + 45*p*q2*q3*q4*q5*q6*q7**3 - 360*p*q2*q3*q4*q5*q6*q7**2 + 1260*p*q2*q3*q4*q5*q6*q7 - 2520*p*q2*q3*q4*q5*q6 + 10*q1**4*q2*q3*q4*q5*q6 + 10*q1**3*q2**2*q3*q4*q5*q6 + 10*q1**3*q2*q3**2*q4*q5*q6 + 10*q1**3*q2*q3*q4**2*q5*q6 + 10*q1**3*q2*q3*q4*q5**2*q6 + 10*q1**3*q2*q3*q4*q5*q6**2 + 10*q1**3*q2*q3*q4*q5*q6*q7 - 90*q1**3*q2*q3*q4*q5*q6 + 10*q1**2*q2**3*q3*q4*q5*q6 + 10*q1**2*q2**2*q3**2*q4*q5*q6 + 10*q1**2*q2**2*q3*q4**2*q5*q6 + 10*q1**2*q2**2*q3*q4*q5**2*q6 + 10*q1**2*q2**2*q3*q4*q5*q6**2 + 10*q1**2*q2**2*q3*q4*q5*q6*q7 - 90*q1**2*q2**2*q3*q4*q5*q6 + 10*q1**2*q2*q3**3*q4*q5*q6 + 10*q1**2*q2*q3**2*q4**2*q5*q6 + 10*q1**2*q2*q3**2*q4*q5**2*q6 + 10*q1**2*q2*q3**2*q4*q5*q6**2 + 10*q1**2*q2*q3**2*q4*q5*q6*q7 - 90*q1**2*q2*q3**2*q4*q5*q6 + 10*q1**2*q2*q3*q4**3*q5*q6 + 10*q1**2*q2*q3*q4**2*q5**2*q6 + 10*q1**2*q2*q3*q4**2*q5*q6**2 + 10*q1**2*q2*q3*q4**2*q5*q6*q7 - 90*q1**2*q2*q3*q4**2*q5*q6 + 10*q1**2*q2*q3*q4*q5**3*q6 + 10*q1**2*q2*q3*q4*q5**2*q6**2 + 10*q1**2*q2*q3*q4*q5**2*q6*q7 - 90*q1**2*q2*q3*q4*q5**2*q6 + 10*q1**2*q2*q3*q4*q5*q6**3 + 10*q1**2*q2*q3*q4*q5*q6**2*q7 - 90*q1**2*q2*q3*q4*q5*q6**2 + 10*q1**2*q2*q3*q4*q5*q6*q7**2 - 90*q1**2*q2*q3*q4*q5*q6*q7 + 360*q1**2*q2*q3*q4*q5*q6 + 10*q1*q2**4*q3*q4*q5*q6 + 10*q1*q2**3*q3**2*q4*q5*q6 + 10*q1*q2**3*q3*q4**2*q5*q6 + 10*q1*q2**3*q3*q4*q5**2*q6 + 10*q1*q2**3*q3*q4*q5*q6**2 + 10*q1*q2**3*q3*q4*q5*q6*q7 - 90*q1*q2**3*q3*q4*q5*q6 + 10*q1*q2**2*q3**3*q4*q5*q6 + 10*q1*q2**2*q3**2*q4**2*q5*q6 + 10*q1*q2**2*q3**2*q4*q5**2*q6 + 10*q1*q2**2*q3**2*q4*q5*q6**2 + 10*q1*q2**2*q3**2*q4*q5*q6*q7 - 90*q1*q2**2*q3**2*q4*q5*q6 + 10*q1*q2**2*q3*q4**3*q5*q6 + 10*q1*q2**2*q3*q4**2*q5**2*q6 + 10*q1*q2**2*q3*q4**2*q5*q6**2 + 10*q1*q2**2*q3*q4**2*q5*q6*q7 - 90*q1*q2**2*q3*q4**2*q5*q6 + 10*q1*q2**2*q3*q4*q5**3*q6 + 10*q1*q2**2*q3*q4*q5**2*q6**2 + 10*q1*q2**2*q3*q4*q5**2*q6*q7 - 90*q1*q2**2*q3*q4*q5**2*q6 + 10*q1*q2**2*q3*q4*q5*q6**3 + 10*q1*q2**2*q3*q4*q5*q6**2*q7 - 90*q1*q2**2*q3*q4*q5*q6**2 + 10*q1*q2**2*q3*q4*q5*q6*q7**2 - 90*q1*q2**2*q3*q4*q5*q6*q7 + 360*q1*q2**2*q3*q4*q5*q6 + 10*q1*q2*q3**4*q4*q5*q6 + 10*q1*q2*q3**3*q4**2*q5*q6 + 10*q1*q2*q3**3*q4*q5**2*q6 + 10*q1*q2*q3**3*q4*q5*q6**2 + 10*q1*q2*q3**3*q4*q5*q6*q7 - 90*q1*q2*q3**3*q4*q5*q6 + 10*q1*q2*q3**2*q4**3*q5*q6 + 10*q1*q2*q3**2*q4**2*q5**2*q6 + 10*q1*q2*q3**2*q4**2*q5*q6**2 + 10*q1*q2*q3**2*q4**2*q5*q6*q7 - 90*q1*q2*q3**2*q4**2*q5*q6 + 10*q1*q2*q3**2*q4*q5**3*q6 + 10*q1*q2*q3**2*q4*q5**2*q6**2 + 10*q1*q2*q3**2*q4*q5**2*q6*q7 - 90*q1*q2*q3**2*q4*q5**2*q6 + 10*q1*q2*q3**2*q4*q5*q6**3 + 10*q1*q2*q3**2*q4*q5*q6**2*q7 - 90*q1*q2*q3**2*q4*q5*q6**2 + 10*q1*q2*q3**2*q4*q5*q6*q7**2 - 90*q1*q2*q3**2*q4*q5*q6*q7 + 360*q1*q2*q3**2*q4*q5*q6 + 10*q1*q2*q3*q4**4*q5*q6 + 10*q1*q2*q3*q4**3*q5**2*q6 + 10*q1*q2*q3*q4**3*q5*q6**2 + 10*q1*q2*q3*q4**3*q5*q6*q7 - 90*q1*q2*q3*q4**3*q5*q6 + 10*q1*q2*q3*q4**2*q5**3*q6 + 10*q1*q2*q3*q4**2*q5**2*q6**2 + 10*q1*q2*q3*q4**2*q5**2*q6*q7 - 90*q1*q2*q3*q4**2*q5**2*q6 + 10*q1*q2*q3*q4**2*q5*q6**3 + 10*q1*q2*q3*q4**2*q5*q6**2*q7 - 90*q1*q2*q3*q4**2*q5*q6**2 + 10*q1*q2*q3*q4**2*q5*q6*q7**2 - 90*q1*q2*q3*q4**2*q5*q6*q7 + 360*q1*q2*q3*q4**2*q5*q6 + 10*q1*q2*q3*q4*q5**4*q6 + 10*q1*q2*q3*q4*q5**3*q6**2 + 10*q1*q2*q3*q4*q5**3*q6*q7 - 90*q1*q2*q3*q4*q5**3*q6 + 10*q1*q2*q3*q4*q5**2*q6**3 + 10*q1*q2*q3*q4*q5**2*q6**2*q7 - 90*q1*q2*q3*q4*q5**2*q6**2 + 10*q1*q2*q3*q4*q5**2*q6*q7**2 - 90*q1*q2*q3*q4*q5**2*q6*q7 + 360*q1*q2*q3*q4*q5**2*q6 + 10*q1*q2*q3*q4*q5*q6**4 + 10*q1*q2*q3*q4*q5*q6**3*q7 - 90*q1*q2*q3*q4*q5*q6**3 + 10*q1*q2*q3*q4*q5*q6**2*q7**2 - 90*q1*q2*q3*q4*q5*q6**2*q7 + 360*q1*q2*q3*q4*q5*q6**2 + 10*q1*q2*q3*q4*q5*q6*q7**3 - 90*q1*q2*q3*q4*q5*q6*q7**2 + 360*q1*q2*q3*q4*q5*q6*q7 - 840*q1*q2*q3*q4*q5*q6)'
f_multiparam_semisyn[10][8] = '-p*(p - 1)**2*(10*p**7*q1**3*q2*q3*q4*q5*q6*q7 + 10*p**7*q1**2*q2**2*q3*q4*q5*q6*q7 + 10*p**7*q1**2*q2*q3**2*q4*q5*q6*q7 + 10*p**7*q1**2*q2*q3*q4**2*q5*q6*q7 + 10*p**7*q1**2*q2*q3*q4*q5**2*q6*q7 + 10*p**7*q1**2*q2*q3*q4*q5*q6**2*q7 + 10*p**7*q1**2*q2*q3*q4*q5*q6*q7**2 + 10*p**7*q1**2*q2*q3*q4*q5*q6*q7*q8 - 90*p**7*q1**2*q2*q3*q4*q5*q6*q7 + 10*p**7*q1*q2**3*q3*q4*q5*q6*q7 + 10*p**7*q1*q2**2*q3**2*q4*q5*q6*q7 + 10*p**7*q1*q2**2*q3*q4**2*q5*q6*q7 + 10*p**7*q1*q2**2*q3*q4*q5**2*q6*q7 + 10*p**7*q1*q2**2*q3*q4*q5*q6**2*q7 + 10*p**7*q1*q2**2*q3*q4*q5*q6*q7**2 + 10*p**7*q1*q2**2*q3*q4*q5*q6*q7*q8 - 90*p**7*q1*q2**2*q3*q4*q5*q6*q7 + 10*p**7*q1*q2*q3**3*q4*q5*q6*q7 + 10*p**7*q1*q2*q3**2*q4**2*q5*q6*q7 + 10*p**7*q1*q2*q3**2*q4*q5**2*q6*q7 + 10*p**7*q1*q2*q3**2*q4*q5*q6**2*q7 + 10*p**7*q1*q2*q3**2*q4*q5*q6*q7**2 + 10*p**7*q1*q2*q3**2*q4*q5*q6*q7*q8 - 90*p**7*q1*q2*q3**2*q4*q5*q6*q7 + 10*p**7*q1*q2*q3*q4**3*q5*q6*q7 + 10*p**7*q1*q2*q3*q4**2*q5**2*q6*q7 + 10*p**7*q1*q2*q3*q4**2*q5*q6**2*q7 + 10*p**7*q1*q2*q3*q4**2*q5*q6*q7**2 + 10*p**7*q1*q2*q3*q4**2*q5*q6*q7*q8 - 90*p**7*q1*q2*q3*q4**2*q5*q6*q7 + 10*p**7*q1*q2*q3*q4*q5**3*q6*q7 + 10*p**7*q1*q2*q3*q4*q5**2*q6**2*q7 + 10*p**7*q1*q2*q3*q4*q5**2*q6*q7**2 + 10*p**7*q1*q2*q3*q4*q5**2*q6*q7*q8 - 90*p**7*q1*q2*q3*q4*q5**2*q6*q7 + 10*p**7*q1*q2*q3*q4*q5*q6**3*q7 + 10*p**7*q1*q2*q3*q4*q5*q6**2*q7**2 + 10*p**7*q1*q2*q3*q4*q5*q6**2*q7*q8 - 90*p**7*q1*q2*q3*q4*q5*q6**2*q7 + 10*p**7*q1*q2*q3*q4*q5*q6*q7**3 + 10*p**7*q1*q2*q3*q4*q5*q6*q7**2*q8 - 90*p**7*q1*q2*q3*q4*q5*q6*q7**2 + 10*p**7*q1*q2*q3*q4*q5*q6*q7*q8**2 - 90*p**7*q1*q2*q3*q4*q5*q6*q7*q8 + 360*p**7*q1*q2*q3*q4*q5*q6*q7 - 45*p**7*q2**3*q3*q4*q5*q6*q7 - 45*p**7*q2**2*q3**2*q4*q5*q6*q7 - 45*p**7*q2**2*q3*q4**2*q5*q6*q7 - 45*p**7*q2**2*q3*q4*q5**2*q6*q7 - 45*p**7*q2**2*q3*q4*q5*q6**2*q7 - 45*p**7*q2**2*q3*q4*q5*q6*q7**2 - 45*p**7*q2**2*q3*q4*q5*q6*q7*q8 + 360*p**7*q2**2*q3*q4*q5*q6*q7 - 45*p**7*q2*q3**3*q4*q5*q6*q7 - 45*p**7*q2*q3**2*q4**2*q5*q6*q7 - 45*p**7*q2*q3**2*q4*q5**2*q6*q7 - 45*p**7*q2*q3**2*q4*q5*q6**2*q7 - 45*p**7*q2*q3**2*q4*q5*q6*q7**2 - 45*p**7*q2*q3**2*q4*q5*q6*q7*q8 + 360*p**7*q2*q3**2*q4*q5*q6*q7 - 45*p**7*q2*q3*q4**3*q5*q6*q7 - 45*p**7*q2*q3*q4**2*q5**2*q6*q7 - 45*p**7*q2*q3*q4**2*q5*q6**2*q7 - 45*p**7*q2*q3*q4**2*q5*q6*q7**2 - 45*p**7*q2*q3*q4**2*q5*q6*q7*q8 + 360*p**7*q2*q3*q4**2*q5*q6*q7 - 45*p**7*q2*q3*q4*q5**3*q6*q7 - 45*p**7*q2*q3*q4*q5**2*q6**2*q7 - 45*p**7*q2*q3*q4*q5**2*q6*q7**2 - 45*p**7*q2*q3*q4*q5**2*q6*q7*q8 + 360*p**7*q2*q3*q4*q5**2*q6*q7 - 45*p**7*q2*q3*q4*q5*q6**3*q7 - 45*p**7*q2*q3*q4*q5*q6**2*q7**2 - 45*p**7*q2*q3*q4*q5*q6**2*q7*q8 + 360*p**7*q2*q3*q4*q5*q6**2*q7 - 45*p**7*q2*q3*q4*q5*q6*q7**3 - 45*p**7*q2*q3*q4*q5*q6*q7**2*q8 + 360*p**7*q2*q3*q4*q5*q6*q7**2 - 45*p**7*q2*q3*q4*q5*q6*q7*q8**2 + 360*p**7*q2*q3*q4*q5*q6*q7*q8 - 1260*p**7*q2*q3*q4*q5*q6*q7 + 120*p**7*q3**3*q4*q5*q6*q7 + 120*p**7*q3**2*q4**2*q5*q6*q7 + 120*p**7*q3**2*q4*q5**2*q6*q7 + 120*p**7*q3**2*q4*q5*q6**2*q7 + 120*p**7*q3**2*q4*q5*q6*q7**2 + 120*p**7*q3**2*q4*q5*q6*q7*q8 - 840*p**7*q3**2*q4*q5*q6*q7 + 120*p**7*q3*q4**3*q5*q6*q7 + 120*p**7*q3*q4**2*q5**2*q6*q7 + 120*p**7*q3*q4**2*q5*q6**2*q7 + 120*p**7*q3*q4**2*q5*q6*q7**2 + 120*p**7*q3*q4**2*q5*q6*q7*q8 - 840*p**7*q3*q4**2*q5*q6*q7 + 120*p**7*q3*q4*q5**3*q6*q7 + 120*p**7*q3*q4*q5**2*q6**2*q7 + 120*p**7*q3*q4*q5**2*q6*q7**2 + 120*p**7*q3*q4*q5**2*q6*q7*q8 - 840*p**7*q3*q4*q5**2*q6*q7 + 120*p**7*q3*q4*q5*q6**3*q7 + 120*p**7*q3*q4*q5*q6**2*q7**2 + 120*p**7*q3*q4*q5*q6**2*q7*q8 - 840*p**7*q3*q4*q5*q6**2*q7 + 120*p**7*q3*q4*q5*q6*q7**3 + 120*p**7*q3*q4*q5*q6*q7**2*q8 - 840*p**7*q3*q4*q5*q6*q7**2 + 120*p**7*q3*q4*q5*q6*q7*q8**2 - 840*p**7*q3*q4*q5*q6*q7*q8 + 2520*p**7*q3*q4*q5*q6*q7 - 210*p**7*q4**3*q5*q6*q7 - 210*p**7*q4**2*q5**2*q6*q7 - 210*p**7*q4**2*q5*q6**2*q7 - 210*p**7*q4**2*q5*q6*q7**2 - 210*p**7*q4**2*q5*q6*q7*q8 + 1260*p**7*q4**2*q5*q6*q7 - 210*p**7*q4*q5**3*q6*q7 - 210*p**7*q4*q5**2*q6**2*q7 - 210*p**7*q4*q5**2*q6*q7**2 - 210*p**7*q4*q5**2*q6*q7*q8 + 1260*p**7*q4*q5**2*q6*q7 - 210*p**7*q4*q5*q6**3*q7 - 210*p**7*q4*q5*q6**2*q7**2 - 210*p**7*q4*q5*q6**2*q7*q8 + 1260*p**7*q4*q5*q6**2*q7 - 210*p**7*q4*q5*q6*q7**3 - 210*p**7*q4*q5*q6*q7**2*q8 + 1260*p**7*q4*q5*q6*q7**2 - 210*p**7*q4*q5*q6*q7*q8**2 + 1260*p**7*q4*q5*q6*q7*q8 - 3150*p**7*q4*q5*q6*q7 + 252*p**7*q5**3*q6*q7 + 252*p**7*q5**2*q6**2*q7 + 252*p**7*q5**2*q6*q7**2 + 252*p**7*q5**2*q6*q7*q8 - 1260*p**7*q5**2*q6*q7 + 252*p**7*q5*q6**3*q7 + 252*p**7*q5*q6**2*q7**2 + 252*p**7*q5*q6**2*q7*q8 - 1260*p**7*q5*q6**2*q7 + 252*p**7*q5*q6*q7**3 + 252*p**7*q5*q6*q7**2*q8 - 1260*p**7*q5*q6*q7**2 + 252*p**7*q5*q6*q7*q8**2 - 1260*p**7*q5*q6*q7*q8 + 2520*p**7*q5*q6*q7 - 210*p**7*q6**3*q7 - 210*p**7*q6**2*q7**2 - 210*p**7*q6**2*q7*q8 + 840*p**7*q6**2*q7 - 210*p**7*q6*q7**3 - 210*p**7*q6*q7**2*q8 + 840*p**7*q6*q7**2 - 210*p**7*q6*q7*q8**2 + 840*p**7*q6*q7*q8 - 1260*p**7*q6*q7 + 120*p**7*q7**3 + 120*p**7*q7**2*q8 - 360*p**7*q7**2 + 120*p**7*q7*q8**2 - 360*p**7*q7*q8 + 360*p**7*q7 - 45*p**7*q8**2 + 90*p**7*q8 - 45*p**7 - 70*p**6*q1**3*q2*q3*q4*q5*q6*q7 - 70*p**6*q1**2*q2**2*q3*q4*q5*q6*q7 - 70*p**6*q1**2*q2*q3**2*q4*q5*q6*q7 - 70*p**6*q1**2*q2*q3*q4**2*q5*q6*q7 - 70*p**6*q1**2*q2*q3*q4*q5**2*q6*q7 - 70*p**6*q1**2*q2*q3*q4*q5*q6**2*q7 - 70*p**6*q1**2*q2*q3*q4*q5*q6*q7**2 - 70*p**6*q1**2*q2*q3*q4*q5*q6*q7*q8 + 630*p**6*q1**2*q2*q3*q4*q5*q6*q7 - 70*p**6*q1*q2**3*q3*q4*q5*q6*q7 - 70*p**6*q1*q2**2*q3**2*q4*q5*q6*q7 - 70*p**6*q1*q2**2*q3*q4**2*q5*q6*q7 - 70*p**6*q1*q2**2*q3*q4*q5**2*q6*q7 - 70*p**6*q1*q2**2*q3*q4*q5*q6**2*q7 - 70*p**6*q1*q2**2*q3*q4*q5*q6*q7**2 - 70*p**6*q1*q2**2*q3*q4*q5*q6*q7*q8 + 630*p**6*q1*q2**2*q3*q4*q5*q6*q7 - 70*p**6*q1*q2*q3**3*q4*q5*q6*q7 - 70*p**6*q1*q2*q3**2*q4**2*q5*q6*q7 - 70*p**6*q1*q2*q3**2*q4*q5**2*q6*q7 - 70*p**6*q1*q2*q3**2*q4*q5*q6**2*q7 - 70*p**6*q1*q2*q3**2*q4*q5*q6*q7**2 - 70*p**6*q1*q2*q3**2*q4*q5*q6*q7*q8 + 630*p**6*q1*q2*q3**2*q4*q5*q6*q7 - 70*p**6*q1*q2*q3*q4**3*q5*q6*q7 - 70*p**6*q1*q2*q3*q4**2*q5**2*q6*q7 - 70*p**6*q1*q2*q3*q4**2*q5*q6**2*q7 - 70*p**6*q1*q2*q3*q4**2*q5*q6*q7**2 - 70*p**6*q1*q2*q3*q4**2*q5*q6*q7*q8 + 630*p**6*q1*q2*q3*q4**2*q5*q6*q7 - 70*p**6*q1*q2*q3*q4*q5**3*q6*q7 - 70*p**6*q1*q2*q3*q4*q5**2*q6**2*q7 - 70*p**6*q1*q2*q3*q4*q5**2*q6*q7**2 - 70*p**6*q1*q2*q3*q4*q5**2*q6*q7*q8 + 630*p**6*q1*q2*q3*q4*q5**2*q6*q7 - 70*p**6*q1*q2*q3*q4*q5*q6**3*q7 - 70*p**6*q1*q2*q3*q4*q5*q6**2*q7**2 - 70*p**6*q1*q2*q3*q4*q5*q6**2*q7*q8 + 630*p**6*q1*q2*q3*q4*q5*q6**2*q7 - 70*p**6*q1*q2*q3*q4*q5*q6*q7**3 - 70*p**6*q1*q2*q3*q4*q5*q6*q7**2*q8 + 630*p**6*q1*q2*q3*q4*q5*q6*q7**2 - 70*p**6*q1*q2*q3*q4*q5*q6*q7*q8**2 + 630*p**6*q1*q2*q3*q4*q5*q6*q7*q8 - 2520*p**6*q1*q2*q3*q4*q5*q6*q7 + 270*p**6*q2**3*q3*q4*q5*q6*q7 + 270*p**6*q2**2*q3**2*q4*q5*q6*q7 + 270*p**6*q2**2*q3*q4**2*q5*q6*q7 + 270*p**6*q2**2*q3*q4*q5**2*q6*q7 + 270*p**6*q2**2*q3*q4*q5*q6**2*q7 + 270*p**6*q2**2*q3*q4*q5*q6*q7**2 + 270*p**6*q2**2*q3*q4*q5*q6*q7*q8 - 2160*p**6*q2**2*q3*q4*q5*q6*q7 + 270*p**6*q2*q3**3*q4*q5*q6*q7 + 270*p**6*q2*q3**2*q4**2*q5*q6*q7 + 270*p**6*q2*q3**2*q4*q5**2*q6*q7 + 270*p**6*q2*q3**2*q4*q5*q6**2*q7 + 270*p**6*q2*q3**2*q4*q5*q6*q7**2 + 270*p**6*q2*q3**2*q4*q5*q6*q7*q8 - 2160*p**6*q2*q3**2*q4*q5*q6*q7 + 270*p**6*q2*q3*q4**3*q5*q6*q7 + 270*p**6*q2*q3*q4**2*q5**2*q6*q7 + 270*p**6*q2*q3*q4**2*q5*q6**2*q7 + 270*p**6*q2*q3*q4**2*q5*q6*q7**2 + 270*p**6*q2*q3*q4**2*q5*q6*q7*q8 - 2160*p**6*q2*q3*q4**2*q5*q6*q7 + 270*p**6*q2*q3*q4*q5**3*q6*q7 + 270*p**6*q2*q3*q4*q5**2*q6**2*q7 + 270*p**6*q2*q3*q4*q5**2*q6*q7**2 + 270*p**6*q2*q3*q4*q5**2*q6*q7*q8 - 2160*p**6*q2*q3*q4*q5**2*q6*q7 + 270*p**6*q2*q3*q4*q5*q6**3*q7 + 270*p**6*q2*q3*q4*q5*q6**2*q7**2 + 270*p**6*q2*q3*q4*q5*q6**2*q7*q8 - 2160*p**6*q2*q3*q4*q5*q6**2*q7 + 270*p**6*q2*q3*q4*q5*q6*q7**3 + 270*p**6*q2*q3*q4*q5*q6*q7**2*q8 - 2160*p**6*q2*q3*q4*q5*q6*q7**2 + 270*p**6*q2*q3*q4*q5*q6*q7*q8**2 - 2160*p**6*q2*q3*q4*q5*q6*q7*q8 + 7560*p**6*q2*q3*q4*q5*q6*q7 - 600*p**6*q3**3*q4*q5*q6*q7 - 600*p**6*q3**2*q4**2*q5*q6*q7 - 600*p**6*q3**2*q4*q5**2*q6*q7 - 600*p**6*q3**2*q4*q5*q6**2*q7 - 600*p**6*q3**2*q4*q5*q6*q7**2 - 600*p**6*q3**2*q4*q5*q6*q7*q8 + 4200*p**6*q3**2*q4*q5*q6*q7 - 600*p**6*q3*q4**3*q5*q6*q7 - 600*p**6*q3*q4**2*q5**2*q6*q7 - 600*p**6*q3*q4**2*q5*q6**2*q7 - 600*p**6*q3*q4**2*q5*q6*q7**2 - 600*p**6*q3*q4**2*q5*q6*q7*q8 + 4200*p**6*q3*q4**2*q5*q6*q7 - 600*p**6*q3*q4*q5**3*q6*q7 - 600*p**6*q3*q4*q5**2*q6**2*q7 - 600*p**6*q3*q4*q5**2*q6*q7**2 - 600*p**6*q3*q4*q5**2*q6*q7*q8 + 4200*p**6*q3*q4*q5**2*q6*q7 - 600*p**6*q3*q4*q5*q6**3*q7 - 600*p**6*q3*q4*q5*q6**2*q7**2 - 600*p**6*q3*q4*q5*q6**2*q7*q8 + 4200*p**6*q3*q4*q5*q6**2*q7 - 600*p**6*q3*q4*q5*q6*q7**3 - 600*p**6*q3*q4*q5*q6*q7**2*q8 + 4200*p**6*q3*q4*q5*q6*q7**2 - 600*p**6*q3*q4*q5*q6*q7*q8**2 + 4200*p**6*q3*q4*q5*q6*q7*q8 - 12600*p**6*q3*q4*q5*q6*q7 + 840*p**6*q4**3*q5*q6*q7 + 840*p**6*q4**2*q5**2*q6*q7 + 840*p**6*q4**2*q5*q6**2*q7 + 840*p**6*q4**2*q5*q6*q7**2 + 840*p**6*q4**2*q5*q6*q7*q8 - 5040*p**6*q4**2*q5*q6*q7 + 840*p**6*q4*q5**3*q6*q7 + 840*p**6*q4*q5**2*q6**2*q7 + 840*p**6*q4*q5**2*q6*q7**2 + 840*p**6*q4*q5**2*q6*q7*q8 - 5040*p**6*q4*q5**2*q6*q7 + 840*p**6*q4*q5*q6**3*q7 + 840*p**6*q4*q5*q6**2*q7**2 + 840*p**6*q4*q5*q6**2*q7*q8 - 5040*p**6*q4*q5*q6**2*q7 + 840*p**6*q4*q5*q6*q7**3 + 840*p**6*q4*q5*q6*q7**2*q8 - 5040*p**6*q4*q5*q6*q7**2 + 840*p**6*q4*q5*q6*q7*q8**2 - 5040*p**6*q4*q5*q6*q7*q8 + 12600*p**6*q4*q5*q6*q7 - 756*p**6*q5**3*q6*q7 - 756*p**6*q5**2*q6**2*q7 - 756*p**6*q5**2*q6*q7**2 - 756*p**6*q5**2*q6*q7*q8 + 3780*p**6*q5**2*q6*q7 - 756*p**6*q5*q6**3*q7 - 756*p**6*q5*q6**2*q7**2 - 756*p**6*q5*q6**2*q7*q8 + 3780*p**6*q5*q6**2*q7 - 756*p**6*q5*q6*q7**3 - 756*p**6*q5*q6*q7**2*q8 + 3780*p**6*q5*q6*q7**2 - 756*p**6*q5*q6*q7*q8**2 + 3780*p**6*q5*q6*q7*q8 - 7560*p**6*q5*q6*q7 + 420*p**6*q6**3*q7 + 420*p**6*q6**2*q7**2 + 420*p**6*q6**2*q7*q8 - 1680*p**6*q6**2*q7 + 420*p**6*q6*q7**3 + 420*p**6*q6*q7**2*q8 - 1680*p**6*q6*q7**2 + 420*p**6*q6*q7*q8**2 - 1680*p**6*q6*q7*q8 + 2520*p**6*q6*q7 - 120*p**6*q7**3 - 120*p**6*q7**2*q8 + 360*p**6*q7**2 - 120*p**6*q7*q8**2 + 360*p**6*q7*q8 - 360*p**6*q7 + 210*p**5*q1**3*q2*q3*q4*q5*q6*q7 + 210*p**5*q1**2*q2**2*q3*q4*q5*q6*q7 + 210*p**5*q1**2*q2*q3**2*q4*q5*q6*q7 + 210*p**5*q1**2*q2*q3*q4**2*q5*q6*q7 + 210*p**5*q1**2*q2*q3*q4*q5**2*q6*q7 + 210*p**5*q1**2*q2*q3*q4*q5*q6**2*q7 + 210*p**5*q1**2*q2*q3*q4*q5*q6*q7**2 + 210*p**5*q1**2*q2*q3*q4*q5*q6*q7*q8 - 1890*p**5*q1**2*q2*q3*q4*q5*q6*q7 + 210*p**5*q1*q2**3*q3*q4*q5*q6*q7 + 210*p**5*q1*q2**2*q3**2*q4*q5*q6*q7 + 210*p**5*q1*q2**2*q3*q4**2*q5*q6*q7 + 210*p**5*q1*q2**2*q3*q4*q5**2*q6*q7 + 210*p**5*q1*q2**2*q3*q4*q5*q6**2*q7 + 210*p**5*q1*q2**2*q3*q4*q5*q6*q7**2 + 210*p**5*q1*q2**2*q3*q4*q5*q6*q7*q8 - 1890*p**5*q1*q2**2*q3*q4*q5*q6*q7 + 210*p**5*q1*q2*q3**3*q4*q5*q6*q7 + 210*p**5*q1*q2*q3**2*q4**2*q5*q6*q7 + 210*p**5*q1*q2*q3**2*q4*q5**2*q6*q7 + 210*p**5*q1*q2*q3**2*q4*q5*q6**2*q7 + 210*p**5*q1*q2*q3**2*q4*q5*q6*q7**2 + 210*p**5*q1*q2*q3**2*q4*q5*q6*q7*q8 - 1890*p**5*q1*q2*q3**2*q4*q5*q6*q7 + 210*p**5*q1*q2*q3*q4**3*q5*q6*q7 + 210*p**5*q1*q2*q3*q4**2*q5**2*q6*q7 + 210*p**5*q1*q2*q3*q4**2*q5*q6**2*q7 + 210*p**5*q1*q2*q3*q4**2*q5*q6*q7**2 + 210*p**5*q1*q2*q3*q4**2*q5*q6*q7*q8 - 1890*p**5*q1*q2*q3*q4**2*q5*q6*q7 + 210*p**5*q1*q2*q3*q4*q5**3*q6*q7 + 210*p**5*q1*q2*q3*q4*q5**2*q6**2*q7 + 210*p**5*q1*q2*q3*q4*q5**2*q6*q7**2 + 210*p**5*q1*q2*q3*q4*q5**2*q6*q7*q8 - 1890*p**5*q1*q2*q3*q4*q5**2*q6*q7 + 210*p**5*q1*q2*q3*q4*q5*q6**3*q7 + 210*p**5*q1*q2*q3*q4*q5*q6**2*q7**2 + 210*p**5*q1*q2*q3*q4*q5*q6**2*q7*q8 - 1890*p**5*q1*q2*q3*q4*q5*q6**2*q7 + 210*p**5*q1*q2*q3*q4*q5*q6*q7**3 + 210*p**5*q1*q2*q3*q4*q5*q6*q7**2*q8 - 1890*p**5*q1*q2*q3*q4*q5*q6*q7**2 + 210*p**5*q1*q2*q3*q4*q5*q6*q7*q8**2 - 1890*p**5*q1*q2*q3*q4*q5*q6*q7*q8 + 7560*p**5*q1*q2*q3*q4*q5*q6*q7 - 675*p**5*q2**3*q3*q4*q5*q6*q7 - 675*p**5*q2**2*q3**2*q4*q5*q6*q7 - 675*p**5*q2**2*q3*q4**2*q5*q6*q7 - 675*p**5*q2**2*q3*q4*q5**2*q6*q7 - 675*p**5*q2**2*q3*q4*q5*q6**2*q7 - 675*p**5*q2**2*q3*q4*q5*q6*q7**2 - 675*p**5*q2**2*q3*q4*q5*q6*q7*q8 + 5400*p**5*q2**2*q3*q4*q5*q6*q7 - 675*p**5*q2*q3**3*q4*q5*q6*q7 - 675*p**5*q2*q3**2*q4**2*q5*q6*q7 - 675*p**5*q2*q3**2*q4*q5**2*q6*q7 - 675*p**5*q2*q3**2*q4*q5*q6**2*q7 - 675*p**5*q2*q3**2*q4*q5*q6*q7**2 - 675*p**5*q2*q3**2*q4*q5*q6*q7*q8 + 5400*p**5*q2*q3**2*q4*q5*q6*q7 - 675*p**5*q2*q3*q4**3*q5*q6*q7 - 675*p**5*q2*q3*q4**2*q5**2*q6*q7 - 675*p**5*q2*q3*q4**2*q5*q6**2*q7 - 675*p**5*q2*q3*q4**2*q5*q6*q7**2 - 675*p**5*q2*q3*q4**2*q5*q6*q7*q8 + 5400*p**5*q2*q3*q4**2*q5*q6*q7 - 675*p**5*q2*q3*q4*q5**3*q6*q7 - 675*p**5*q2*q3*q4*q5**2*q6**2*q7 - 675*p**5*q2*q3*q4*q5**2*q6*q7**2 - 675*p**5*q2*q3*q4*q5**2*q6*q7*q8 + 5400*p**5*q2*q3*q4*q5**2*q6*q7 - 675*p**5*q2*q3*q4*q5*q6**3*q7 - 675*p**5*q2*q3*q4*q5*q6**2*q7**2 - 675*p**5*q2*q3*q4*q5*q6**2*q7*q8 + 5400*p**5*q2*q3*q4*q5*q6**2*q7 - 675*p**5*q2*q3*q4*q5*q6*q7**3 - 675*p**5*q2*q3*q4*q5*q6*q7**2*q8 + 5400*p**5*q2*q3*q4*q5*q6*q7**2 - 675*p**5*q2*q3*q4*q5*q6*q7*q8**2 + 5400*p**5*q2*q3*q4*q5*q6*q7*q8 - 18900*p**5*q2*q3*q4*q5*q6*q7 + 1200*p**5*q3**3*q4*q5*q6*q7 + 1200*p**5*q3**2*q4**2*q5*q6*q7 + 1200*p**5*q3**2*q4*q5**2*q6*q7 + 1200*p**5*q3**2*q4*q5*q6**2*q7 + 1200*p**5*q3**2*q4*q5*q6*q7**2 + 1200*p**5*q3**2*q4*q5*q6*q7*q8 - 8400*p**5*q3**2*q4*q5*q6*q7 + 1200*p**5*q3*q4**3*q5*q6*q7 + 1200*p**5*q3*q4**2*q5**2*q6*q7 + 1200*p**5*q3*q4**2*q5*q6**2*q7 + 1200*p**5*q3*q4**2*q5*q6*q7**2 + 1200*p**5*q3*q4**2*q5*q6*q7*q8 - 8400*p**5*q3*q4**2*q5*q6*q7 + 1200*p**5*q3*q4*q5**3*q6*q7 + 1200*p**5*q3*q4*q5**2*q6**2*q7 + 1200*p**5*q3*q4*q5**2*q6*q7**2 + 1200*p**5*q3*q4*q5**2*q6*q7*q8 - 8400*p**5*q3*q4*q5**2*q6*q7 + 1200*p**5*q3*q4*q5*q6**3*q7 + 1200*p**5*q3*q4*q5*q6**2*q7**2 + 1200*p**5*q3*q4*q5*q6**2*q7*q8 - 8400*p**5*q3*q4*q5*q6**2*q7 + 1200*p**5*q3*q4*q5*q6*q7**3 + 1200*p**5*q3*q4*q5*q6*q7**2*q8 - 8400*p**5*q3*q4*q5*q6*q7**2 + 1200*p**5*q3*q4*q5*q6*q7*q8**2 - 8400*p**5*q3*q4*q5*q6*q7*q8 + 25200*p**5*q3*q4*q5*q6*q7 - 1260*p**5*q4**3*q5*q6*q7 - 1260*p**5*q4**2*q5**2*q6*q7 - 1260*p**5*q4**2*q5*q6**2*q7 - 1260*p**5*q4**2*q5*q6*q7**2 - 1260*p**5*q4**2*q5*q6*q7*q8 + 7560*p**5*q4**2*q5*q6*q7 - 1260*p**5*q4*q5**3*q6*q7 - 1260*p**5*q4*q5**2*q6**2*q7 - 1260*p**5*q4*q5**2*q6*q7**2 - 1260*p**5*q4*q5**2*q6*q7*q8 + 7560*p**5*q4*q5**2*q6*q7 - 1260*p**5*q4*q5*q6**3*q7 - 1260*p**5*q4*q5*q6**2*q7**2 - 1260*p**5*q4*q5*q6**2*q7*q8 + 7560*p**5*q4*q5*q6**2*q7 - 1260*p**5*q4*q5*q6*q7**3 - 1260*p**5*q4*q5*q6*q7**2*q8 + 7560*p**5*q4*q5*q6*q7**2 - 1260*p**5*q4*q5*q6*q7*q8**2 + 7560*p**5*q4*q5*q6*q7*q8 - 18900*p**5*q4*q5*q6*q7 + 756*p**5*q5**3*q6*q7 + 756*p**5*q5**2*q6**2*q7 + 756*p**5*q5**2*q6*q7**2 + 756*p**5*q5**2*q6*q7*q8 - 3780*p**5*q5**2*q6*q7 + 756*p**5*q5*q6**3*q7 + 756*p**5*q5*q6**2*q7**2 + 756*p**5*q5*q6**2*q7*q8 - 3780*p**5*q5*q6**2*q7 + 756*p**5*q5*q6*q7**3 + 756*p**5*q5*q6*q7**2*q8 - 3780*p**5*q5*q6*q7**2 + 756*p**5*q5*q6*q7*q8**2 - 3780*p**5*q5*q6*q7*q8 + 7560*p**5*q5*q6*q7 - 210*p**5*q6**3*q7 - 210*p**5*q6**2*q7**2 - 210*p**5*q6**2*q7*q8 + 840*p**5*q6**2*q7 - 210*p**5*q6*q7**3 - 210*p**5*q6*q7**2*q8 + 840*p**5*q6*q7**2 - 210*p**5*q6*q7*q8**2 + 840*p**5*q6*q7*q8 - 1260*p**5*q6*q7 - 350*p**4*q1**3*q2*q3*q4*q5*q6*q7 - 350*p**4*q1**2*q2**2*q3*q4*q5*q6*q7 - 350*p**4*q1**2*q2*q3**2*q4*q5*q6*q7 - 350*p**4*q1**2*q2*q3*q4**2*q5*q6*q7 - 350*p**4*q1**2*q2*q3*q4*q5**2*q6*q7 - 350*p**4*q1**2*q2*q3*q4*q5*q6**2*q7 - 350*p**4*q1**2*q2*q3*q4*q5*q6*q7**2 - 350*p**4*q1**2*q2*q3*q4*q5*q6*q7*q8 + 3150*p**4*q1**2*q2*q3*q4*q5*q6*q7 - 350*p**4*q1*q2**3*q3*q4*q5*q6*q7 - 350*p**4*q1*q2**2*q3**2*q4*q5*q6*q7 - 350*p**4*q1*q2**2*q3*q4**2*q5*q6*q7 - 350*p**4*q1*q2**2*q3*q4*q5**2*q6*q7 - 350*p**4*q1*q2**2*q3*q4*q5*q6**2*q7 - 350*p**4*q1*q2**2*q3*q4*q5*q6*q7**2 - 350*p**4*q1*q2**2*q3*q4*q5*q6*q7*q8 + 3150*p**4*q1*q2**2*q3*q4*q5*q6*q7 - 350*p**4*q1*q2*q3**3*q4*q5*q6*q7 - 350*p**4*q1*q2*q3**2*q4**2*q5*q6*q7 - 350*p**4*q1*q2*q3**2*q4*q5**2*q6*q7 - 350*p**4*q1*q2*q3**2*q4*q5*q6**2*q7 - 350*p**4*q1*q2*q3**2*q4*q5*q6*q7**2 - 350*p**4*q1*q2*q3**2*q4*q5*q6*q7*q8 + 3150*p**4*q1*q2*q3**2*q4*q5*q6*q7 - 350*p**4*q1*q2*q3*q4**3*q5*q6*q7 - 350*p**4*q1*q2*q3*q4**2*q5**2*q6*q7 - 350*p**4*q1*q2*q3*q4**2*q5*q6**2*q7 - 350*p**4*q1*q2*q3*q4**2*q5*q6*q7**2 - 350*p**4*q1*q2*q3*q4**2*q5*q6*q7*q8 + 3150*p**4*q1*q2*q3*q4**2*q5*q6*q7 - 350*p**4*q1*q2*q3*q4*q5**3*q6*q7 - 350*p**4*q1*q2*q3*q4*q5**2*q6**2*q7 - 350*p**4*q1*q2*q3*q4*q5**2*q6*q7**2 - 350*p**4*q1*q2*q3*q4*q5**2*q6*q7*q8 + 3150*p**4*q1*q2*q3*q4*q5**2*q6*q7 - 350*p**4*q1*q2*q3*q4*q5*q6**3*q7 - 350*p**4*q1*q2*q3*q4*q5*q6**2*q7**2 - 350*p**4*q1*q2*q3*q4*q5*q6**2*q7*q8 + 3150*p**4*q1*q2*q3*q4*q5*q6**2*q7 - 350*p**4*q1*q2*q3*q4*q5*q6*q7**3 - 350*p**4*q1*q2*q3*q4*q5*q6*q7**2*q8 + 3150*p**4*q1*q2*q3*q4*q5*q6*q7**2 - 350*p**4*q1*q2*q3*q4*q5*q6*q7*q8**2 + 3150*p**4*q1*q2*q3*q4*q5*q6*q7*q8 - 12600*p**4*q1*q2*q3*q4*q5*q6*q7 + 900*p**4*q2**3*q3*q4*q5*q6*q7 + 900*p**4*q2**2*q3**2*q4*q5*q6*q7 + 900*p**4*q2**2*q3*q4**2*q5*q6*q7 + 900*p**4*q2**2*q3*q4*q5**2*q6*q7 + 900*p**4*q2**2*q3*q4*q5*q6**2*q7 + 900*p**4*q2**2*q3*q4*q5*q6*q7**2 + 900*p**4*q2**2*q3*q4*q5*q6*q7*q8 - 7200*p**4*q2**2*q3*q4*q5*q6*q7 + 900*p**4*q2*q3**3*q4*q5*q6*q7 + 900*p**4*q2*q3**2*q4**2*q5*q6*q7 + 900*p**4*q2*q3**2*q4*q5**2*q6*q7 + 900*p**4*q2*q3**2*q4*q5*q6**2*q7 + 900*p**4*q2*q3**2*q4*q5*q6*q7**2 + 900*p**4*q2*q3**2*q4*q5*q6*q7*q8 - 7200*p**4*q2*q3**2*q4*q5*q6*q7 + 900*p**4*q2*q3*q4**3*q5*q6*q7 + 900*p**4*q2*q3*q4**2*q5**2*q6*q7 + 900*p**4*q2*q3*q4**2*q5*q6**2*q7 + 900*p**4*q2*q3*q4**2*q5*q6*q7**2 + 900*p**4*q2*q3*q4**2*q5*q6*q7*q8 - 7200*p**4*q2*q3*q4**2*q5*q6*q7 + 900*p**4*q2*q3*q4*q5**3*q6*q7 + 900*p**4*q2*q3*q4*q5**2*q6**2*q7 + 900*p**4*q2*q3*q4*q5**2*q6*q7**2 + 900*p**4*q2*q3*q4*q5**2*q6*q7*q8 - 7200*p**4*q2*q3*q4*q5**2*q6*q7 + 900*p**4*q2*q3*q4*q5*q6**3*q7 + 900*p**4*q2*q3*q4*q5*q6**2*q7**2 + 900*p**4*q2*q3*q4*q5*q6**2*q7*q8 - 7200*p**4*q2*q3*q4*q5*q6**2*q7 + 900*p**4*q2*q3*q4*q5*q6*q7**3 + 900*p**4*q2*q3*q4*q5*q6*q7**2*q8 - 7200*p**4*q2*q3*q4*q5*q6*q7**2 + 900*p**4*q2*q3*q4*q5*q6*q7*q8**2 - 7200*p**4*q2*q3*q4*q5*q6*q7*q8 + 25200*p**4*q2*q3*q4*q5*q6*q7 - 1200*p**4*q3**3*q4*q5*q6*q7 - 1200*p**4*q3**2*q4**2*q5*q6*q7 - 1200*p**4*q3**2*q4*q5**2*q6*q7 - 1200*p**4*q3**2*q4*q5*q6**2*q7 - 1200*p**4*q3**2*q4*q5*q6*q7**2 - 1200*p**4*q3**2*q4*q5*q6*q7*q8 + 8400*p**4*q3**2*q4*q5*q6*q7 - 1200*p**4*q3*q4**3*q5*q6*q7 - 1200*p**4*q3*q4**2*q5**2*q6*q7 - 1200*p**4*q3*q4**2*q5*q6**2*q7 - 1200*p**4*q3*q4**2*q5*q6*q7**2 - 1200*p**4*q3*q4**2*q5*q6*q7*q8 + 8400*p**4*q3*q4**2*q5*q6*q7 - 1200*p**4*q3*q4*q5**3*q6*q7 - 1200*p**4*q3*q4*q5**2*q6**2*q7 - 1200*p**4*q3*q4*q5**2*q6*q7**2 - 1200*p**4*q3*q4*q5**2*q6*q7*q8 + 8400*p**4*q3*q4*q5**2*q6*q7 - 1200*p**4*q3*q4*q5*q6**3*q7 - 1200*p**4*q3*q4*q5*q6**2*q7**2 - 1200*p**4*q3*q4*q5*q6**2*q7*q8 + 8400*p**4*q3*q4*q5*q6**2*q7 - 1200*p**4*q3*q4*q5*q6*q7**3 - 1200*p**4*q3*q4*q5*q6*q7**2*q8 + 8400*p**4*q3*q4*q5*q6*q7**2 - 1200*p**4*q3*q4*q5*q6*q7*q8**2 + 8400*p**4*q3*q4*q5*q6*q7*q8 - 25200*p**4*q3*q4*q5*q6*q7 + 840*p**4*q4**3*q5*q6*q7 + 840*p**4*q4**2*q5**2*q6*q7 + 840*p**4*q4**2*q5*q6**2*q7 + 840*p**4*q4**2*q5*q6*q7**2 + 840*p**4*q4**2*q5*q6*q7*q8 - 5040*p**4*q4**2*q5*q6*q7 + 840*p**4*q4*q5**3*q6*q7 + 840*p**4*q4*q5**2*q6**2*q7 + 840*p**4*q4*q5**2*q6*q7**2 + 840*p**4*q4*q5**2*q6*q7*q8 - 5040*p**4*q4*q5**2*q6*q7 + 840*p**4*q4*q5*q6**3*q7 + 840*p**4*q4*q5*q6**2*q7**2 + 840*p**4*q4*q5*q6**2*q7*q8 - 5040*p**4*q4*q5*q6**2*q7 + 840*p**4*q4*q5*q6*q7**3 + 840*p**4*q4*q5*q6*q7**2*q8 - 5040*p**4*q4*q5*q6*q7**2 + 840*p**4*q4*q5*q6*q7*q8**2 - 5040*p**4*q4*q5*q6*q7*q8 + 12600*p**4*q4*q5*q6*q7 - 252*p**4*q5**3*q6*q7 - 252*p**4*q5**2*q6**2*q7 - 252*p**4*q5**2*q6*q7**2 - 252*p**4*q5**2*q6*q7*q8 + 1260*p**4*q5**2*q6*q7 - 252*p**4*q5*q6**3*q7 - 252*p**4*q5*q6**2*q7**2 - 252*p**4*q5*q6**2*q7*q8 + 1260*p**4*q5*q6**2*q7 - 252*p**4*q5*q6*q7**3 - 252*p**4*q5*q6*q7**2*q8 + 1260*p**4*q5*q6*q7**2 - 252*p**4*q5*q6*q7*q8**2 + 1260*p**4*q5*q6*q7*q8 - 2520*p**4*q5*q6*q7 + 350*p**3*q1**3*q2*q3*q4*q5*q6*q7 + 350*p**3*q1**2*q2**2*q3*q4*q5*q6*q7 + 350*p**3*q1**2*q2*q3**2*q4*q5*q6*q7 + 350*p**3*q1**2*q2*q3*q4**2*q5*q6*q7 + 350*p**3*q1**2*q2*q3*q4*q5**2*q6*q7 + 350*p**3*q1**2*q2*q3*q4*q5*q6**2*q7 + 350*p**3*q1**2*q2*q3*q4*q5*q6*q7**2 + 350*p**3*q1**2*q2*q3*q4*q5*q6*q7*q8 - 3150*p**3*q1**2*q2*q3*q4*q5*q6*q7 + 350*p**3*q1*q2**3*q3*q4*q5*q6*q7 + 350*p**3*q1*q2**2*q3**2*q4*q5*q6*q7 + 350*p**3*q1*q2**2*q3*q4**2*q5*q6*q7 + 350*p**3*q1*q2**2*q3*q4*q5**2*q6*q7 + 350*p**3*q1*q2**2*q3*q4*q5*q6**2*q7 + 350*p**3*q1*q2**2*q3*q4*q5*q6*q7**2 + 350*p**3*q1*q2**2*q3*q4*q5*q6*q7*q8 - 3150*p**3*q1*q2**2*q3*q4*q5*q6*q7 + 350*p**3*q1*q2*q3**3*q4*q5*q6*q7 + 350*p**3*q1*q2*q3**2*q4**2*q5*q6*q7 + 350*p**3*q1*q2*q3**2*q4*q5**2*q6*q7 + 350*p**3*q1*q2*q3**2*q4*q5*q6**2*q7 + 350*p**3*q1*q2*q3**2*q4*q5*q6*q7**2 + 350*p**3*q1*q2*q3**2*q4*q5*q6*q7*q8 - 3150*p**3*q1*q2*q3**2*q4*q5*q6*q7 + 350*p**3*q1*q2*q3*q4**3*q5*q6*q7 + 350*p**3*q1*q2*q3*q4**2*q5**2*q6*q7 + 350*p**3*q1*q2*q3*q4**2*q5*q6**2*q7 + 350*p**3*q1*q2*q3*q4**2*q5*q6*q7**2 + 350*p**3*q1*q2*q3*q4**2*q5*q6*q7*q8 - 3150*p**3*q1*q2*q3*q4**2*q5*q6*q7 + 350*p**3*q1*q2*q3*q4*q5**3*q6*q7 + 350*p**3*q1*q2*q3*q4*q5**2*q6**2*q7 + 350*p**3*q1*q2*q3*q4*q5**2*q6*q7**2 + 350*p**3*q1*q2*q3*q4*q5**2*q6*q7*q8 - 3150*p**3*q1*q2*q3*q4*q5**2*q6*q7 + 350*p**3*q1*q2*q3*q4*q5*q6**3*q7 + 350*p**3*q1*q2*q3*q4*q5*q6**2*q7**2 + 350*p**3*q1*q2*q3*q4*q5*q6**2*q7*q8 - 3150*p**3*q1*q2*q3*q4*q5*q6**2*q7 + 350*p**3*q1*q2*q3*q4*q5*q6*q7**3 + 350*p**3*q1*q2*q3*q4*q5*q6*q7**2*q8 - 3150*p**3*q1*q2*q3*q4*q5*q6*q7**2 + 350*p**3*q1*q2*q3*q4*q5*q6*q7*q8**2 - 3150*p**3*q1*q2*q3*q4*q5*q6*q7*q8 + 12600*p**3*q1*q2*q3*q4*q5*q6*q7 - 675*p**3*q2**3*q3*q4*q5*q6*q7 - 675*p**3*q2**2*q3**2*q4*q5*q6*q7 - 675*p**3*q2**2*q3*q4**2*q5*q6*q7 - 675*p**3*q2**2*q3*q4*q5**2*q6*q7 - 675*p**3*q2**2*q3*q4*q5*q6**2*q7 - 675*p**3*q2**2*q3*q4*q5*q6*q7**2 - 675*p**3*q2**2*q3*q4*q5*q6*q7*q8 + 5400*p**3*q2**2*q3*q4*q5*q6*q7 - 675*p**3*q2*q3**3*q4*q5*q6*q7 - 675*p**3*q2*q3**2*q4**2*q5*q6*q7 - 675*p**3*q2*q3**2*q4*q5**2*q6*q7 - 675*p**3*q2*q3**2*q4*q5*q6**2*q7 - 675*p**3*q2*q3**2*q4*q5*q6*q7**2 - 675*p**3*q2*q3**2*q4*q5*q6*q7*q8 + 5400*p**3*q2*q3**2*q4*q5*q6*q7 - 675*p**3*q2*q3*q4**3*q5*q6*q7 - 675*p**3*q2*q3*q4**2*q5**2*q6*q7 - 675*p**3*q2*q3*q4**2*q5*q6**2*q7 - 675*p**3*q2*q3*q4**2*q5*q6*q7**2 - 675*p**3*q2*q3*q4**2*q5*q6*q7*q8 + 5400*p**3*q2*q3*q4**2*q5*q6*q7 - 675*p**3*q2*q3*q4*q5**3*q6*q7 - 675*p**3*q2*q3*q4*q5**2*q6**2*q7 - 675*p**3*q2*q3*q4*q5**2*q6*q7**2 - 675*p**3*q2*q3*q4*q5**2*q6*q7*q8 + 5400*p**3*q2*q3*q4*q5**2*q6*q7 - 675*p**3*q2*q3*q4*q5*q6**3*q7 - 675*p**3*q2*q3*q4*q5*q6**2*q7**2 - 675*p**3*q2*q3*q4*q5*q6**2*q7*q8 + 5400*p**3*q2*q3*q4*q5*q6**2*q7 - 675*p**3*q2*q3*q4*q5*q6*q7**3 - 675*p**3*q2*q3*q4*q5*q6*q7**2*q8 + 5400*p**3*q2*q3*q4*q5*q6*q7**2 - 675*p**3*q2*q3*q4*q5*q6*q7*q8**2 + 5400*p**3*q2*q3*q4*q5*q6*q7*q8 - 18900*p**3*q2*q3*q4*q5*q6*q7 + 600*p**3*q3**3*q4*q5*q6*q7 + 600*p**3*q3**2*q4**2*q5*q6*q7 + 600*p**3*q3**2*q4*q5**2*q6*q7 + 600*p**3*q3**2*q4*q5*q6**2*q7 + 600*p**3*q3**2*q4*q5*q6*q7**2 + 600*p**3*q3**2*q4*q5*q6*q7*q8 - 4200*p**3*q3**2*q4*q5*q6*q7 + 600*p**3*q3*q4**3*q5*q6*q7 + 600*p**3*q3*q4**2*q5**2*q6*q7 + 600*p**3*q3*q4**2*q5*q6**2*q7 + 600*p**3*q3*q4**2*q5*q6*q7**2 + 600*p**3*q3*q4**2*q5*q6*q7*q8 - 4200*p**3*q3*q4**2*q5*q6*q7 + 600*p**3*q3*q4*q5**3*q6*q7 + 600*p**3*q3*q4*q5**2*q6**2*q7 + 600*p**3*q3*q4*q5**2*q6*q7**2 + 600*p**3*q3*q4*q5**2*q6*q7*q8 - 4200*p**3*q3*q4*q5**2*q6*q7 + 600*p**3*q3*q4*q5*q6**3*q7 + 600*p**3*q3*q4*q5*q6**2*q7**2 + 600*p**3*q3*q4*q5*q6**2*q7*q8 - 4200*p**3*q3*q4*q5*q6**2*q7 + 600*p**3*q3*q4*q5*q6*q7**3 + 600*p**3*q3*q4*q5*q6*q7**2*q8 - 4200*p**3*q3*q4*q5*q6*q7**2 + 600*p**3*q3*q4*q5*q6*q7*q8**2 - 4200*p**3*q3*q4*q5*q6*q7*q8 + 12600*p**3*q3*q4*q5*q6*q7 - 210*p**3*q4**3*q5*q6*q7 - 210*p**3*q4**2*q5**2*q6*q7 - 210*p**3*q4**2*q5*q6**2*q7 - 210*p**3*q4**2*q5*q6*q7**2 - 210*p**3*q4**2*q5*q6*q7*q8 + 1260*p**3*q4**2*q5*q6*q7 - 210*p**3*q4*q5**3*q6*q7 - 210*p**3*q4*q5**2*q6**2*q7 - 210*p**3*q4*q5**2*q6*q7**2 - 210*p**3*q4*q5**2*q6*q7*q8 + 1260*p**3*q4*q5**2*q6*q7 - 210*p**3*q4*q5*q6**3*q7 - 210*p**3*q4*q5*q6**2*q7**2 - 210*p**3*q4*q5*q6**2*q7*q8 + 1260*p**3*q4*q5*q6**2*q7 - 210*p**3*q4*q5*q6*q7**3 - 210*p**3*q4*q5*q6*q7**2*q8 + 1260*p**3*q4*q5*q6*q7**2 - 210*p**3*q4*q5*q6*q7*q8**2 + 1260*p**3*q4*q5*q6*q7*q8 - 3150*p**3*q4*q5*q6*q7 - 210*p**2*q1**3*q2*q3*q4*q5*q6*q7 - 210*p**2*q1**2*q2**2*q3*q4*q5*q6*q7 - 210*p**2*q1**2*q2*q3**2*q4*q5*q6*q7 - 210*p**2*q1**2*q2*q3*q4**2*q5*q6*q7 - 210*p**2*q1**2*q2*q3*q4*q5**2*q6*q7 - 210*p**2*q1**2*q2*q3*q4*q5*q6**2*q7 - 210*p**2*q1**2*q2*q3*q4*q5*q6*q7**2 - 210*p**2*q1**2*q2*q3*q4*q5*q6*q7*q8 + 1890*p**2*q1**2*q2*q3*q4*q5*q6*q7 - 210*p**2*q1*q2**3*q3*q4*q5*q6*q7 - 210*p**2*q1*q2**2*q3**2*q4*q5*q6*q7 - 210*p**2*q1*q2**2*q3*q4**2*q5*q6*q7 - 210*p**2*q1*q2**2*q3*q4*q5**2*q6*q7 - 210*p**2*q1*q2**2*q3*q4*q5*q6**2*q7 - 210*p**2*q1*q2**2*q3*q4*q5*q6*q7**2 - 210*p**2*q1*q2**2*q3*q4*q5*q6*q7*q8 + 1890*p**2*q1*q2**2*q3*q4*q5*q6*q7 - 210*p**2*q1*q2*q3**3*q4*q5*q6*q7 - 210*p**2*q1*q2*q3**2*q4**2*q5*q6*q7 - 210*p**2*q1*q2*q3**2*q4*q5**2*q6*q7 - 210*p**2*q1*q2*q3**2*q4*q5*q6**2*q7 - 210*p**2*q1*q2*q3**2*q4*q5*q6*q7**2 - 210*p**2*q1*q2*q3**2*q4*q5*q6*q7*q8 + 1890*p**2*q1*q2*q3**2*q4*q5*q6*q7 - 210*p**2*q1*q2*q3*q4**3*q5*q6*q7 - 210*p**2*q1*q2*q3*q4**2*q5**2*q6*q7 - 210*p**2*q1*q2*q3*q4**2*q5*q6**2*q7 - 210*p**2*q1*q2*q3*q4**2*q5*q6*q7**2 - 210*p**2*q1*q2*q3*q4**2*q5*q6*q7*q8 + 1890*p**2*q1*q2*q3*q4**2*q5*q6*q7 - 210*p**2*q1*q2*q3*q4*q5**3*q6*q7 - 210*p**2*q1*q2*q3*q4*q5**2*q6**2*q7 - 210*p**2*q1*q2*q3*q4*q5**2*q6*q7**2 - 210*p**2*q1*q2*q3*q4*q5**2*q6*q7*q8 + 1890*p**2*q1*q2*q3*q4*q5**2*q6*q7 - 210*p**2*q1*q2*q3*q4*q5*q6**3*q7 - 210*p**2*q1*q2*q3*q4*q5*q6**2*q7**2 - 210*p**2*q1*q2*q3*q4*q5*q6**2*q7*q8 + 1890*p**2*q1*q2*q3*q4*q5*q6**2*q7 - 210*p**2*q1*q2*q3*q4*q5*q6*q7**3 - 210*p**2*q1*q2*q3*q4*q5*q6*q7**2*q8 + 1890*p**2*q1*q2*q3*q4*q5*q6*q7**2 - 210*p**2*q1*q2*q3*q4*q5*q6*q7*q8**2 + 1890*p**2*q1*q2*q3*q4*q5*q6*q7*q8 - 7560*p**2*q1*q2*q3*q4*q5*q6*q7 + 270*p**2*q2**3*q3*q4*q5*q6*q7 + 270*p**2*q2**2*q3**2*q4*q5*q6*q7 + 270*p**2*q2**2*q3*q4**2*q5*q6*q7 + 270*p**2*q2**2*q3*q4*q5**2*q6*q7 + 270*p**2*q2**2*q3*q4*q5*q6**2*q7 + 270*p**2*q2**2*q3*q4*q5*q6*q7**2 + 270*p**2*q2**2*q3*q4*q5*q6*q7*q8 - 2160*p**2*q2**2*q3*q4*q5*q6*q7 + 270*p**2*q2*q3**3*q4*q5*q6*q7 + 270*p**2*q2*q3**2*q4**2*q5*q6*q7 + 270*p**2*q2*q3**2*q4*q5**2*q6*q7 + 270*p**2*q2*q3**2*q4*q5*q6**2*q7 + 270*p**2*q2*q3**2*q4*q5*q6*q7**2 + 270*p**2*q2*q3**2*q4*q5*q6*q7*q8 - 2160*p**2*q2*q3**2*q4*q5*q6*q7 + 270*p**2*q2*q3*q4**3*q5*q6*q7 + 270*p**2*q2*q3*q4**2*q5**2*q6*q7 + 270*p**2*q2*q3*q4**2*q5*q6**2*q7 + 270*p**2*q2*q3*q4**2*q5*q6*q7**2 + 270*p**2*q2*q3*q4**2*q5*q6*q7*q8 - 2160*p**2*q2*q3*q4**2*q5*q6*q7 + 270*p**2*q2*q3*q4*q5**3*q6*q7 + 270*p**2*q2*q3*q4*q5**2*q6**2*q7 + 270*p**2*q2*q3*q4*q5**2*q6*q7**2 + 270*p**2*q2*q3*q4*q5**2*q6*q7*q8 - 2160*p**2*q2*q3*q4*q5**2*q6*q7 + 270*p**2*q2*q3*q4*q5*q6**3*q7 + 270*p**2*q2*q3*q4*q5*q6**2*q7**2 + 270*p**2*q2*q3*q4*q5*q6**2*q7*q8 - 2160*p**2*q2*q3*q4*q5*q6**2*q7 + 270*p**2*q2*q3*q4*q5*q6*q7**3 + 270*p**2*q2*q3*q4*q5*q6*q7**2*q8 - 2160*p**2*q2*q3*q4*q5*q6*q7**2 + 270*p**2*q2*q3*q4*q5*q6*q7*q8**2 - 2160*p**2*q2*q3*q4*q5*q6*q7*q8 + 7560*p**2*q2*q3*q4*q5*q6*q7 - 120*p**2*q3**3*q4*q5*q6*q7 - 120*p**2*q3**2*q4**2*q5*q6*q7 - 120*p**2*q3**2*q4*q5**2*q6*q7 - 120*p**2*q3**2*q4*q5*q6**2*q7 - 120*p**2*q3**2*q4*q5*q6*q7**2 - 120*p**2*q3**2*q4*q5*q6*q7*q8 + 840*p**2*q3**2*q4*q5*q6*q7 - 120*p**2*q3*q4**3*q5*q6*q7 - 120*p**2*q3*q4**2*q5**2*q6*q7 - 120*p**2*q3*q4**2*q5*q6**2*q7 - 120*p**2*q3*q4**2*q5*q6*q7**2 - 120*p**2*q3*q4**2*q5*q6*q7*q8 + 840*p**2*q3*q4**2*q5*q6*q7 - 120*p**2*q3*q4*q5**3*q6*q7 - 120*p**2*q3*q4*q5**2*q6**2*q7 - 120*p**2*q3*q4*q5**2*q6*q7**2 - 120*p**2*q3*q4*q5**2*q6*q7*q8 + 840*p**2*q3*q4*q5**2*q6*q7 - 120*p**2*q3*q4*q5*q6**3*q7 - 120*p**2*q3*q4*q5*q6**2*q7**2 - 120*p**2*q3*q4*q5*q6**2*q7*q8 + 840*p**2*q3*q4*q5*q6**2*q7 - 120*p**2*q3*q4*q5*q6*q7**3 - 120*p**2*q3*q4*q5*q6*q7**2*q8 + 840*p**2*q3*q4*q5*q6*q7**2 - 120*p**2*q3*q4*q5*q6*q7*q8**2 + 840*p**2*q3*q4*q5*q6*q7*q8 - 2520*p**2*q3*q4*q5*q6*q7 + 70*p*q1**3*q2*q3*q4*q5*q6*q7 + 70*p*q1**2*q2**2*q3*q4*q5*q6*q7 + 70*p*q1**2*q2*q3**2*q4*q5*q6*q7 + 70*p*q1**2*q2*q3*q4**2*q5*q6*q7 + 70*p*q1**2*q2*q3*q4*q5**2*q6*q7 + 70*p*q1**2*q2*q3*q4*q5*q6**2*q7 + 70*p*q1**2*q2*q3*q4*q5*q6*q7**2 + 70*p*q1**2*q2*q3*q4*q5*q6*q7*q8 - 630*p*q1**2*q2*q3*q4*q5*q6*q7 + 70*p*q1*q2**3*q3*q4*q5*q6*q7 + 70*p*q1*q2**2*q3**2*q4*q5*q6*q7 + 70*p*q1*q2**2*q3*q4**2*q5*q6*q7 + 70*p*q1*q2**2*q3*q4*q5**2*q6*q7 + 70*p*q1*q2**2*q3*q4*q5*q6**2*q7 + 70*p*q1*q2**2*q3*q4*q5*q6*q7**2 + 70*p*q1*q2**2*q3*q4*q5*q6*q7*q8 - 630*p*q1*q2**2*q3*q4*q5*q6*q7 + 70*p*q1*q2*q3**3*q4*q5*q6*q7 + 70*p*q1*q2*q3**2*q4**2*q5*q6*q7 + 70*p*q1*q2*q3**2*q4*q5**2*q6*q7 + 70*p*q1*q2*q3**2*q4*q5*q6**2*q7 + 70*p*q1*q2*q3**2*q4*q5*q6*q7**2 + 70*p*q1*q2*q3**2*q4*q5*q6*q7*q8 - 630*p*q1*q2*q3**2*q4*q5*q6*q7 + 70*p*q1*q2*q3*q4**3*q5*q6*q7 + 70*p*q1*q2*q3*q4**2*q5**2*q6*q7 + 70*p*q1*q2*q3*q4**2*q5*q6**2*q7 + 70*p*q1*q2*q3*q4**2*q5*q6*q7**2 + 70*p*q1*q2*q3*q4**2*q5*q6*q7*q8 - 630*p*q1*q2*q3*q4**2*q5*q6*q7 + 70*p*q1*q2*q3*q4*q5**3*q6*q7 + 70*p*q1*q2*q3*q4*q5**2*q6**2*q7 + 70*p*q1*q2*q3*q4*q5**2*q6*q7**2 + 70*p*q1*q2*q3*q4*q5**2*q6*q7*q8 - 630*p*q1*q2*q3*q4*q5**2*q6*q7 + 70*p*q1*q2*q3*q4*q5*q6**3*q7 + 70*p*q1*q2*q3*q4*q5*q6**2*q7**2 + 70*p*q1*q2*q3*q4*q5*q6**2*q7*q8 - 630*p*q1*q2*q3*q4*q5*q6**2*q7 + 70*p*q1*q2*q3*q4*q5*q6*q7**3 + 70*p*q1*q2*q3*q4*q5*q6*q7**2*q8 - 630*p*q1*q2*q3*q4*q5*q6*q7**2 + 70*p*q1*q2*q3*q4*q5*q6*q7*q8**2 - 630*p*q1*q2*q3*q4*q5*q6*q7*q8 + 2520*p*q1*q2*q3*q4*q5*q6*q7 - 45*p*q2**3*q3*q4*q5*q6*q7 - 45*p*q2**2*q3**2*q4*q5*q6*q7 - 45*p*q2**2*q3*q4**2*q5*q6*q7 - 45*p*q2**2*q3*q4*q5**2*q6*q7 - 45*p*q2**2*q3*q4*q5*q6**2*q7 - 45*p*q2**2*q3*q4*q5*q6*q7**2 - 45*p*q2**2*q3*q4*q5*q6*q7*q8 + 360*p*q2**2*q3*q4*q5*q6*q7 - 45*p*q2*q3**3*q4*q5*q6*q7 - 45*p*q2*q3**2*q4**2*q5*q6*q7 - 45*p*q2*q3**2*q4*q5**2*q6*q7 - 45*p*q2*q3**2*q4*q5*q6**2*q7 - 45*p*q2*q3**2*q4*q5*q6*q7**2 - 45*p*q2*q3**2*q4*q5*q6*q7*q8 + 360*p*q2*q3**2*q4*q5*q6*q7 - 45*p*q2*q3*q4**3*q5*q6*q7 - 45*p*q2*q3*q4**2*q5**2*q6*q7 - 45*p*q2*q3*q4**2*q5*q6**2*q7 - 45*p*q2*q3*q4**2*q5*q6*q7**2 - 45*p*q2*q3*q4**2*q5*q6*q7*q8 + 360*p*q2*q3*q4**2*q5*q6*q7 - 45*p*q2*q3*q4*q5**3*q6*q7 - 45*p*q2*q3*q4*q5**2*q6**2*q7 - 45*p*q2*q3*q4*q5**2*q6*q7**2 - 45*p*q2*q3*q4*q5**2*q6*q7*q8 + 360*p*q2*q3*q4*q5**2*q6*q7 - 45*p*q2*q3*q4*q5*q6**3*q7 - 45*p*q2*q3*q4*q5*q6**2*q7**2 - 45*p*q2*q3*q4*q5*q6**2*q7*q8 + 360*p*q2*q3*q4*q5*q6**2*q7 - 45*p*q2*q3*q4*q5*q6*q7**3 - 45*p*q2*q3*q4*q5*q6*q7**2*q8 + 360*p*q2*q3*q4*q5*q6*q7**2 - 45*p*q2*q3*q4*q5*q6*q7*q8**2 + 360*p*q2*q3*q4*q5*q6*q7*q8 - 1260*p*q2*q3*q4*q5*q6*q7 - 10*q1**3*q2*q3*q4*q5*q6*q7 - 10*q1**2*q2**2*q3*q4*q5*q6*q7 - 10*q1**2*q2*q3**2*q4*q5*q6*q7 - 10*q1**2*q2*q3*q4**2*q5*q6*q7 - 10*q1**2*q2*q3*q4*q5**2*q6*q7 - 10*q1**2*q2*q3*q4*q5*q6**2*q7 - 10*q1**2*q2*q3*q4*q5*q6*q7**2 - 10*q1**2*q2*q3*q4*q5*q6*q7*q8 + 90*q1**2*q2*q3*q4*q5*q6*q7 - 10*q1*q2**3*q3*q4*q5*q6*q7 - 10*q1*q2**2*q3**2*q4*q5*q6*q7 - 10*q1*q2**2*q3*q4**2*q5*q6*q7 - 10*q1*q2**2*q3*q4*q5**2*q6*q7 - 10*q1*q2**2*q3*q4*q5*q6**2*q7 - 10*q1*q2**2*q3*q4*q5*q6*q7**2 - 10*q1*q2**2*q3*q4*q5*q6*q7*q8 + 90*q1*q2**2*q3*q4*q5*q6*q7 - 10*q1*q2*q3**3*q4*q5*q6*q7 - 10*q1*q2*q3**2*q4**2*q5*q6*q7 - 10*q1*q2*q3**2*q4*q5**2*q6*q7 - 10*q1*q2*q3**2*q4*q5*q6**2*q7 - 10*q1*q2*q3**2*q4*q5*q6*q7**2 - 10*q1*q2*q3**2*q4*q5*q6*q7*q8 + 90*q1*q2*q3**2*q4*q5*q6*q7 - 10*q1*q2*q3*q4**3*q5*q6*q7 - 10*q1*q2*q3*q4**2*q5**2*q6*q7 - 10*q1*q2*q3*q4**2*q5*q6**2*q7 - 10*q1*q2*q3*q4**2*q5*q6*q7**2 - 10*q1*q2*q3*q4**2*q5*q6*q7*q8 + 90*q1*q2*q3*q4**2*q5*q6*q7 - 10*q1*q2*q3*q4*q5**3*q6*q7 - 10*q1*q2*q3*q4*q5**2*q6**2*q7 - 10*q1*q2*q3*q4*q5**2*q6*q7**2 - 10*q1*q2*q3*q4*q5**2*q6*q7*q8 + 90*q1*q2*q3*q4*q5**2*q6*q7 - 10*q1*q2*q3*q4*q5*q6**3*q7 - 10*q1*q2*q3*q4*q5*q6**2*q7**2 - 10*q1*q2*q3*q4*q5*q6**2*q7*q8 + 90*q1*q2*q3*q4*q5*q6**2*q7 - 10*q1*q2*q3*q4*q5*q6*q7**3 - 10*q1*q2*q3*q4*q5*q6*q7**2*q8 + 90*q1*q2*q3*q4*q5*q6*q7**2 - 10*q1*q2*q3*q4*q5*q6*q7*q8**2 + 90*q1*q2*q3*q4*q5*q6*q7*q8 - 360*q1*q2*q3*q4*q5*q6*q7)'
f_multiparam_semisyn[10][9] = 'p*(p - 1)*(10*p**8*q1**2*q2*q3*q4*q5*q6*q7*q8 + 10*p**8*q1*q2**2*q3*q4*q5*q6*q7*q8 + 10*p**8*q1*q2*q3**2*q4*q5*q6*q7*q8 + 10*p**8*q1*q2*q3*q4**2*q5*q6*q7*q8 + 10*p**8*q1*q2*q3*q4*q5**2*q6*q7*q8 + 10*p**8*q1*q2*q3*q4*q5*q6**2*q7*q8 + 10*p**8*q1*q2*q3*q4*q5*q6*q7**2*q8 + 10*p**8*q1*q2*q3*q4*q5*q6*q7*q8**2 + 10*p**8*q1*q2*q3*q4*q5*q6*q7*q8*q9 - 90*p**8*q1*q2*q3*q4*q5*q6*q7*q8 - 45*p**8*q2**2*q3*q4*q5*q6*q7*q8 - 45*p**8*q2*q3**2*q4*q5*q6*q7*q8 - 45*p**8*q2*q3*q4**2*q5*q6*q7*q8 - 45*p**8*q2*q3*q4*q5**2*q6*q7*q8 - 45*p**8*q2*q3*q4*q5*q6**2*q7*q8 - 45*p**8*q2*q3*q4*q5*q6*q7**2*q8 - 45*p**8*q2*q3*q4*q5*q6*q7*q8**2 - 45*p**8*q2*q3*q4*q5*q6*q7*q8*q9 + 360*p**8*q2*q3*q4*q5*q6*q7*q8 + 120*p**8*q3**2*q4*q5*q6*q7*q8 + 120*p**8*q3*q4**2*q5*q6*q7*q8 + 120*p**8*q3*q4*q5**2*q6*q7*q8 + 120*p**8*q3*q4*q5*q6**2*q7*q8 + 120*p**8*q3*q4*q5*q6*q7**2*q8 + 120*p**8*q3*q4*q5*q6*q7*q8**2 + 120*p**8*q3*q4*q5*q6*q7*q8*q9 - 840*p**8*q3*q4*q5*q6*q7*q8 - 210*p**8*q4**2*q5*q6*q7*q8 - 210*p**8*q4*q5**2*q6*q7*q8 - 210*p**8*q4*q5*q6**2*q7*q8 - 210*p**8*q4*q5*q6*q7**2*q8 - 210*p**8*q4*q5*q6*q7*q8**2 - 210*p**8*q4*q5*q6*q7*q8*q9 + 1260*p**8*q4*q5*q6*q7*q8 + 252*p**8*q5**2*q6*q7*q8 + 252*p**8*q5*q6**2*q7*q8 + 252*p**8*q5*q6*q7**2*q8 + 252*p**8*q5*q6*q7*q8**2 + 252*p**8*q5*q6*q7*q8*q9 - 1260*p**8*q5*q6*q7*q8 - 210*p**8*q6**2*q7*q8 - 210*p**8*q6*q7**2*q8 - 210*p**8*q6*q7*q8**2 - 210*p**8*q6*q7*q8*q9 + 840*p**8*q6*q7*q8 + 120*p**8*q7**2*q8 + 120*p**8*q7*q8**2 + 120*p**8*q7*q8*q9 - 360*p**8*q7*q8 - 45*p**8*q8**2 - 45*p**8*q8*q9 + 90*p**8*q8 + 10*p**8*q9 - 10*p**8 - 80*p**7*q1**2*q2*q3*q4*q5*q6*q7*q8 - 80*p**7*q1*q2**2*q3*q4*q5*q6*q7*q8 - 80*p**7*q1*q2*q3**2*q4*q5*q6*q7*q8 - 80*p**7*q1*q2*q3*q4**2*q5*q6*q7*q8 - 80*p**7*q1*q2*q3*q4*q5**2*q6*q7*q8 - 80*p**7*q1*q2*q3*q4*q5*q6**2*q7*q8 - 80*p**7*q1*q2*q3*q4*q5*q6*q7**2*q8 - 80*p**7*q1*q2*q3*q4*q5*q6*q7*q8**2 - 80*p**7*q1*q2*q3*q4*q5*q6*q7*q8*q9 + 720*p**7*q1*q2*q3*q4*q5*q6*q7*q8 + 315*p**7*q2**2*q3*q4*q5*q6*q7*q8 + 315*p**7*q2*q3**2*q4*q5*q6*q7*q8 + 315*p**7*q2*q3*q4**2*q5*q6*q7*q8 + 315*p**7*q2*q3*q4*q5**2*q6*q7*q8 + 315*p**7*q2*q3*q4*q5*q6**2*q7*q8 + 315*p**7*q2*q3*q4*q5*q6*q7**2*q8 + 315*p**7*q2*q3*q4*q5*q6*q7*q8**2 + 315*p**7*q2*q3*q4*q5*q6*q7*q8*q9 - 2520*p**7*q2*q3*q4*q5*q6*q7*q8 - 720*p**7*q3**2*q4*q5*q6*q7*q8 - 720*p**7*q3*q4**2*q5*q6*q7*q8 - 720*p**7*q3*q4*q5**2*q6*q7*q8 - 720*p**7*q3*q4*q5*q6**2*q7*q8 - 720*p**7*q3*q4*q5*q6*q7**2*q8 - 720*p**7*q3*q4*q5*q6*q7*q8**2 - 720*p**7*q3*q4*q5*q6*q7*q8*q9 + 5040*p**7*q3*q4*q5*q6*q7*q8 + 1050*p**7*q4**2*q5*q6*q7*q8 + 1050*p**7*q4*q5**2*q6*q7*q8 + 1050*p**7*q4*q5*q6**2*q7*q8 + 1050*p**7*q4*q5*q6*q7**2*q8 + 1050*p**7*q4*q5*q6*q7*q8**2 + 1050*p**7*q4*q5*q6*q7*q8*q9 - 6300*p**7*q4*q5*q6*q7*q8 - 1008*p**7*q5**2*q6*q7*q8 - 1008*p**7*q5*q6**2*q7*q8 - 1008*p**7*q5*q6*q7**2*q8 - 1008*p**7*q5*q6*q7*q8**2 - 1008*p**7*q5*q6*q7*q8*q9 + 5040*p**7*q5*q6*q7*q8 + 630*p**7*q6**2*q7*q8 + 630*p**7*q6*q7**2*q8 + 630*p**7*q6*q7*q8**2 + 630*p**7*q6*q7*q8*q9 - 2520*p**7*q6*q7*q8 - 240*p**7*q7**2*q8 - 240*p**7*q7*q8**2 - 240*p**7*q7*q8*q9 + 720*p**7*q7*q8 + 45*p**7*q8**2 + 45*p**7*q8*q9 - 90*p**7*q8 + 280*p**6*q1**2*q2*q3*q4*q5*q6*q7*q8 + 280*p**6*q1*q2**2*q3*q4*q5*q6*q7*q8 + 280*p**6*q1*q2*q3**2*q4*q5*q6*q7*q8 + 280*p**6*q1*q2*q3*q4**2*q5*q6*q7*q8 + 280*p**6*q1*q2*q3*q4*q5**2*q6*q7*q8 + 280*p**6*q1*q2*q3*q4*q5*q6**2*q7*q8 + 280*p**6*q1*q2*q3*q4*q5*q6*q7**2*q8 + 280*p**6*q1*q2*q3*q4*q5*q6*q7*q8**2 + 280*p**6*q1*q2*q3*q4*q5*q6*q7*q8*q9 - 2520*p**6*q1*q2*q3*q4*q5*q6*q7*q8 - 945*p**6*q2**2*q3*q4*q5*q6*q7*q8 - 945*p**6*q2*q3**2*q4*q5*q6*q7*q8 - 945*p**6*q2*q3*q4**2*q5*q6*q7*q8 - 945*p**6*q2*q3*q4*q5**2*q6*q7*q8 - 945*p**6*q2*q3*q4*q5*q6**2*q7*q8 - 945*p**6*q2*q3*q4*q5*q6*q7**2*q8 - 945*p**6*q2*q3*q4*q5*q6*q7*q8**2 - 945*p**6*q2*q3*q4*q5*q6*q7*q8*q9 + 7560*p**6*q2*q3*q4*q5*q6*q7*q8 + 1800*p**6*q3**2*q4*q5*q6*q7*q8 + 1800*p**6*q3*q4**2*q5*q6*q7*q8 + 1800*p**6*q3*q4*q5**2*q6*q7*q8 + 1800*p**6*q3*q4*q5*q6**2*q7*q8 + 1800*p**6*q3*q4*q5*q6*q7**2*q8 + 1800*p**6*q3*q4*q5*q6*q7*q8**2 + 1800*p**6*q3*q4*q5*q6*q7*q8*q9 - 12600*p**6*q3*q4*q5*q6*q7*q8 - 2100*p**6*q4**2*q5*q6*q7*q8 - 2100*p**6*q4*q5**2*q6*q7*q8 - 2100*p**6*q4*q5*q6**2*q7*q8 - 2100*p**6*q4*q5*q6*q7**2*q8 - 2100*p**6*q4*q5*q6*q7*q8**2 - 2100*p**6*q4*q5*q6*q7*q8*q9 + 12600*p**6*q4*q5*q6*q7*q8 + 1512*p**6*q5**2*q6*q7*q8 + 1512*p**6*q5*q6**2*q7*q8 + 1512*p**6*q5*q6*q7**2*q8 + 1512*p**6*q5*q6*q7*q8**2 + 1512*p**6*q5*q6*q7*q8*q9 - 7560*p**6*q5*q6*q7*q8 - 630*p**6*q6**2*q7*q8 - 630*p**6*q6*q7**2*q8 - 630*p**6*q6*q7*q8**2 - 630*p**6*q6*q7*q8*q9 + 2520*p**6*q6*q7*q8 + 120*p**6*q7**2*q8 + 120*p**6*q7*q8**2 + 120*p**6*q7*q8*q9 - 360*p**6*q7*q8 - 560*p**5*q1**2*q2*q3*q4*q5*q6*q7*q8 - 560*p**5*q1*q2**2*q3*q4*q5*q6*q7*q8 - 560*p**5*q1*q2*q3**2*q4*q5*q6*q7*q8 - 560*p**5*q1*q2*q3*q4**2*q5*q6*q7*q8 - 560*p**5*q1*q2*q3*q4*q5**2*q6*q7*q8 - 560*p**5*q1*q2*q3*q4*q5*q6**2*q7*q8 - 560*p**5*q1*q2*q3*q4*q5*q6*q7**2*q8 - 560*p**5*q1*q2*q3*q4*q5*q6*q7*q8**2 - 560*p**5*q1*q2*q3*q4*q5*q6*q7*q8*q9 + 5040*p**5*q1*q2*q3*q4*q5*q6*q7*q8 + 1575*p**5*q2**2*q3*q4*q5*q6*q7*q8 + 1575*p**5*q2*q3**2*q4*q5*q6*q7*q8 + 1575*p**5*q2*q3*q4**2*q5*q6*q7*q8 + 1575*p**5*q2*q3*q4*q5**2*q6*q7*q8 + 1575*p**5*q2*q3*q4*q5*q6**2*q7*q8 + 1575*p**5*q2*q3*q4*q5*q6*q7**2*q8 + 1575*p**5*q2*q3*q4*q5*q6*q7*q8**2 + 1575*p**5*q2*q3*q4*q5*q6*q7*q8*q9 - 12600*p**5*q2*q3*q4*q5*q6*q7*q8 - 2400*p**5*q3**2*q4*q5*q6*q7*q8 - 2400*p**5*q3*q4**2*q5*q6*q7*q8 - 2400*p**5*q3*q4*q5**2*q6*q7*q8 - 2400*p**5*q3*q4*q5*q6**2*q7*q8 - 2400*p**5*q3*q4*q5*q6*q7**2*q8 - 2400*p**5*q3*q4*q5*q6*q7*q8**2 - 2400*p**5*q3*q4*q5*q6*q7*q8*q9 + 16800*p**5*q3*q4*q5*q6*q7*q8 + 2100*p**5*q4**2*q5*q6*q7*q8 + 2100*p**5*q4*q5**2*q6*q7*q8 + 2100*p**5*q4*q5*q6**2*q7*q8 + 2100*p**5*q4*q5*q6*q7**2*q8 + 2100*p**5*q4*q5*q6*q7*q8**2 + 2100*p**5*q4*q5*q6*q7*q8*q9 - 12600*p**5*q4*q5*q6*q7*q8 - 1008*p**5*q5**2*q6*q7*q8 - 1008*p**5*q5*q6**2*q7*q8 - 1008*p**5*q5*q6*q7**2*q8 - 1008*p**5*q5*q6*q7*q8**2 - 1008*p**5*q5*q6*q7*q8*q9 + 5040*p**5*q5*q6*q7*q8 + 210*p**5*q6**2*q7*q8 + 210*p**5*q6*q7**2*q8 + 210*p**5*q6*q7*q8**2 + 210*p**5*q6*q7*q8*q9 - 840*p**5*q6*q7*q8 + 700*p**4*q1**2*q2*q3*q4*q5*q6*q7*q8 + 700*p**4*q1*q2**2*q3*q4*q5*q6*q7*q8 + 700*p**4*q1*q2*q3**2*q4*q5*q6*q7*q8 + 700*p**4*q1*q2*q3*q4**2*q5*q6*q7*q8 + 700*p**4*q1*q2*q3*q4*q5**2*q6*q7*q8 + 700*p**4*q1*q2*q3*q4*q5*q6**2*q7*q8 + 700*p**4*q1*q2*q3*q4*q5*q6*q7**2*q8 + 700*p**4*q1*q2*q3*q4*q5*q6*q7*q8**2 + 700*p**4*q1*q2*q3*q4*q5*q6*q7*q8*q9 - 6300*p**4*q1*q2*q3*q4*q5*q6*q7*q8 - 1575*p**4*q2**2*q3*q4*q5*q6*q7*q8 - 1575*p**4*q2*q3**2*q4*q5*q6*q7*q8 - 1575*p**4*q2*q3*q4**2*q5*q6*q7*q8 - 1575*p**4*q2*q3*q4*q5**2*q6*q7*q8 - 1575*p**4*q2*q3*q4*q5*q6**2*q7*q8 - 1575*p**4*q2*q3*q4*q5*q6*q7**2*q8 - 1575*p**4*q2*q3*q4*q5*q6*q7*q8**2 - 1575*p**4*q2*q3*q4*q5*q6*q7*q8*q9 + 12600*p**4*q2*q3*q4*q5*q6*q7*q8 + 1800*p**4*q3**2*q4*q5*q6*q7*q8 + 1800*p**4*q3*q4**2*q5*q6*q7*q8 + 1800*p**4*q3*q4*q5**2*q6*q7*q8 + 1800*p**4*q3*q4*q5*q6**2*q7*q8 + 1800*p**4*q3*q4*q5*q6*q7**2*q8 + 1800*p**4*q3*q4*q5*q6*q7*q8**2 + 1800*p**4*q3*q4*q5*q6*q7*q8*q9 - 12600*p**4*q3*q4*q5*q6*q7*q8 - 1050*p**4*q4**2*q5*q6*q7*q8 - 1050*p**4*q4*q5**2*q6*q7*q8 - 1050*p**4*q4*q5*q6**2*q7*q8 - 1050*p**4*q4*q5*q6*q7**2*q8 - 1050*p**4*q4*q5*q6*q7*q8**2 - 1050*p**4*q4*q5*q6*q7*q8*q9 + 6300*p**4*q4*q5*q6*q7*q8 + 252*p**4*q5**2*q6*q7*q8 + 252*p**4*q5*q6**2*q7*q8 + 252*p**4*q5*q6*q7**2*q8 + 252*p**4*q5*q6*q7*q8**2 + 252*p**4*q5*q6*q7*q8*q9 - 1260*p**4*q5*q6*q7*q8 - 560*p**3*q1**2*q2*q3*q4*q5*q6*q7*q8 - 560*p**3*q1*q2**2*q3*q4*q5*q6*q7*q8 - 560*p**3*q1*q2*q3**2*q4*q5*q6*q7*q8 - 560*p**3*q1*q2*q3*q4**2*q5*q6*q7*q8 - 560*p**3*q1*q2*q3*q4*q5**2*q6*q7*q8 - 560*p**3*q1*q2*q3*q4*q5*q6**2*q7*q8 - 560*p**3*q1*q2*q3*q4*q5*q6*q7**2*q8 - 560*p**3*q1*q2*q3*q4*q5*q6*q7*q8**2 - 560*p**3*q1*q2*q3*q4*q5*q6*q7*q8*q9 + 5040*p**3*q1*q2*q3*q4*q5*q6*q7*q8 + 945*p**3*q2**2*q3*q4*q5*q6*q7*q8 + 945*p**3*q2*q3**2*q4*q5*q6*q7*q8 + 945*p**3*q2*q3*q4**2*q5*q6*q7*q8 + 945*p**3*q2*q3*q4*q5**2*q6*q7*q8 + 945*p**3*q2*q3*q4*q5*q6**2*q7*q8 + 945*p**3*q2*q3*q4*q5*q6*q7**2*q8 + 945*p**3*q2*q3*q4*q5*q6*q7*q8**2 + 945*p**3*q2*q3*q4*q5*q6*q7*q8*q9 - 7560*p**3*q2*q3*q4*q5*q6*q7*q8 - 720*p**3*q3**2*q4*q5*q6*q7*q8 - 720*p**3*q3*q4**2*q5*q6*q7*q8 - 720*p**3*q3*q4*q5**2*q6*q7*q8 - 720*p**3*q3*q4*q5*q6**2*q7*q8 - 720*p**3*q3*q4*q5*q6*q7**2*q8 - 720*p**3*q3*q4*q5*q6*q7*q8**2 - 720*p**3*q3*q4*q5*q6*q7*q8*q9 + 5040*p**3*q3*q4*q5*q6*q7*q8 + 210*p**3*q4**2*q5*q6*q7*q8 + 210*p**3*q4*q5**2*q6*q7*q8 + 210*p**3*q4*q5*q6**2*q7*q8 + 210*p**3*q4*q5*q6*q7**2*q8 + 210*p**3*q4*q5*q6*q7*q8**2 + 210*p**3*q4*q5*q6*q7*q8*q9 - 1260*p**3*q4*q5*q6*q7*q8 + 280*p**2*q1**2*q2*q3*q4*q5*q6*q7*q8 + 280*p**2*q1*q2**2*q3*q4*q5*q6*q7*q8 + 280*p**2*q1*q2*q3**2*q4*q5*q6*q7*q8 + 280*p**2*q1*q2*q3*q4**2*q5*q6*q7*q8 + 280*p**2*q1*q2*q3*q4*q5**2*q6*q7*q8 + 280*p**2*q1*q2*q3*q4*q5*q6**2*q7*q8 + 280*p**2*q1*q2*q3*q4*q5*q6*q7**2*q8 + 280*p**2*q1*q2*q3*q4*q5*q6*q7*q8**2 + 280*p**2*q1*q2*q3*q4*q5*q6*q7*q8*q9 - 2520*p**2*q1*q2*q3*q4*q5*q6*q7*q8 - 315*p**2*q2**2*q3*q4*q5*q6*q7*q8 - 315*p**2*q2*q3**2*q4*q5*q6*q7*q8 - 315*p**2*q2*q3*q4**2*q5*q6*q7*q8 - 315*p**2*q2*q3*q4*q5**2*q6*q7*q8 - 315*p**2*q2*q3*q4*q5*q6**2*q7*q8 - 315*p**2*q2*q3*q4*q5*q6*q7**2*q8 - 315*p**2*q2*q3*q4*q5*q6*q7*q8**2 - 315*p**2*q2*q3*q4*q5*q6*q7*q8*q9 + 2520*p**2*q2*q3*q4*q5*q6*q7*q8 + 120*p**2*q3**2*q4*q5*q6*q7*q8 + 120*p**2*q3*q4**2*q5*q6*q7*q8 + 120*p**2*q3*q4*q5**2*q6*q7*q8 + 120*p**2*q3*q4*q5*q6**2*q7*q8 + 120*p**2*q3*q4*q5*q6*q7**2*q8 + 120*p**2*q3*q4*q5*q6*q7*q8**2 + 120*p**2*q3*q4*q5*q6*q7*q8*q9 - 840*p**2*q3*q4*q5*q6*q7*q8 - 80*p*q1**2*q2*q3*q4*q5*q6*q7*q8 - 80*p*q1*q2**2*q3*q4*q5*q6*q7*q8 - 80*p*q1*q2*q3**2*q4*q5*q6*q7*q8 - 80*p*q1*q2*q3*q4**2*q5*q6*q7*q8 - 80*p*q1*q2*q3*q4*q5**2*q6*q7*q8 - 80*p*q1*q2*q3*q4*q5*q6**2*q7*q8 - 80*p*q1*q2*q3*q4*q5*q6*q7**2*q8 - 80*p*q1*q2*q3*q4*q5*q6*q7*q8**2 - 80*p*q1*q2*q3*q4*q5*q6*q7*q8*q9 + 720*p*q1*q2*q3*q4*q5*q6*q7*q8 + 45*p*q2**2*q3*q4*q5*q6*q7*q8 + 45*p*q2*q3**2*q4*q5*q6*q7*q8 + 45*p*q2*q3*q4**2*q5*q6*q7*q8 + 45*p*q2*q3*q4*q5**2*q6*q7*q8 + 45*p*q2*q3*q4*q5*q6**2*q7*q8 + 45*p*q2*q3*q4*q5*q6*q7**2*q8 + 45*p*q2*q3*q4*q5*q6*q7*q8**2 + 45*p*q2*q3*q4*q5*q6*q7*q8*q9 - 360*p*q2*q3*q4*q5*q6*q7*q8 + 10*q1**2*q2*q3*q4*q5*q6*q7*q8 + 10*q1*q2**2*q3*q4*q5*q6*q7*q8 + 10*q1*q2*q3**2*q4*q5*q6*q7*q8 + 10*q1*q2*q3*q4**2*q5*q6*q7*q8 + 10*q1*q2*q3*q4*q5**2*q6*q7*q8 + 10*q1*q2*q3*q4*q5*q6**2*q7*q8 + 10*q1*q2*q3*q4*q5*q6*q7**2*q8 + 10*q1*q2*q3*q4*q5*q6*q7*q8**2 + 10*q1*q2*q3*q4*q5*q6*q7*q8*q9 - 90*q1*q2*q3*q4*q5*q6*q7*q8)'
f_multiparam_semisyn[10][10] = '-p*(10*p**9*q1*q2*q3*q4*q5*q6*q7*q8*q9 - 45*p**9*q2*q3*q4*q5*q6*q7*q8*q9 + 120*p**9*q3*q4*q5*q6*q7*q8*q9 - 210*p**9*q4*q5*q6*q7*q8*q9 + 252*p**9*q5*q6*q7*q8*q9 - 210*p**9*q6*q7*q8*q9 + 120*p**9*q7*q8*q9 - 45*p**9*q8*q9 + 10*p**9*q9 - p**9 - 90*p**8*q1*q2*q3*q4*q5*q6*q7*q8*q9 + 360*p**8*q2*q3*q4*q5*q6*q7*q8*q9 - 840*p**8*q3*q4*q5*q6*q7*q8*q9 + 1260*p**8*q4*q5*q6*q7*q8*q9 - 1260*p**8*q5*q6*q7*q8*q9 + 840*p**8*q6*q7*q8*q9 - 360*p**8*q7*q8*q9 + 90*p**8*q8*q9 - 10*p**8*q9 + 360*p**7*q1*q2*q3*q4*q5*q6*q7*q8*q9 - 1260*p**7*q2*q3*q4*q5*q6*q7*q8*q9 + 2520*p**7*q3*q4*q5*q6*q7*q8*q9 - 3150*p**7*q4*q5*q6*q7*q8*q9 + 2520*p**7*q5*q6*q7*q8*q9 - 1260*p**7*q6*q7*q8*q9 + 360*p**7*q7*q8*q9 - 45*p**7*q8*q9 - 840*p**6*q1*q2*q3*q4*q5*q6*q7*q8*q9 + 2520*p**6*q2*q3*q4*q5*q6*q7*q8*q9 - 4200*p**6*q3*q4*q5*q6*q7*q8*q9 + 4200*p**6*q4*q5*q6*q7*q8*q9 - 2520*p**6*q5*q6*q7*q8*q9 + 840*p**6*q6*q7*q8*q9 - 120*p**6*q7*q8*q9 + 1260*p**5*q1*q2*q3*q4*q5*q6*q7*q8*q9 - 3150*p**5*q2*q3*q4*q5*q6*q7*q8*q9 + 4200*p**5*q3*q4*q5*q6*q7*q8*q9 - 3150*p**5*q4*q5*q6*q7*q8*q9 + 1260*p**5*q5*q6*q7*q8*q9 - 210*p**5*q6*q7*q8*q9 - 1260*p**4*q1*q2*q3*q4*q5*q6*q7*q8*q9 + 2520*p**4*q2*q3*q4*q5*q6*q7*q8*q9 - 2520*p**4*q3*q4*q5*q6*q7*q8*q9 + 1260*p**4*q4*q5*q6*q7*q8*q9 - 252*p**4*q5*q6*q7*q8*q9 + 840*p**3*q1*q2*q3*q4*q5*q6*q7*q8*q9 - 1260*p**3*q2*q3*q4*q5*q6*q7*q8*q9 + 840*p**3*q3*q4*q5*q6*q7*q8*q9 - 210*p**3*q4*q5*q6*q7*q8*q9 - 360*p**2*q1*q2*q3*q4*q5*q6*q7*q8*q9 + 360*p**2*q2*q3*q4*q5*q6*q7*q8*q9 - 120*p**2*q3*q4*q5*q6*q7*q8*q9 + 90*p*q1*q2*q3*q4*q5*q6*q7*q8*q9 - 45*p*q2*q3*q4*q5*q6*q7*q8*q9 - 10*q1*q2*q3*q4*q5*q6*q7*q8*q9)'

In [ ]:
start_time = time.time()
rewards_multiparam_semisyn = get_rewards("./multiparam_se*[_1|_][0-9]_moments*.txt", "storm", False, agents_quantities)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

This took too long to parse

In [ ]:
# start_time = time.time()
# rewards_multiparam_semisyn.update(get_rewards("./multiparam_se*_[2-9][0-9]_moments*.txt", "storm", False, agents_quantities)) 
# print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

### Asynchronous multiparam models

This also may take a while to factorise for f_multiparam_syn[10], so I will load precomputed version for you

In [ ]:
start_time = time.time()
f_multiparam_asyn = get_f("./multiparam_asyn**[_1|_][0-9].txt", "storm", False, agents_quantities)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
start_time = time.time()
rewards_multiparam_asyn = get_rewards("./multiparam_asyn*[_1|_][0-9]_moments*.txt", "storm", False, agents_quantities)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
start_time = time.time()
rewards_multiparam_asyn = get_rewards("./multiparam_asyn*20_moments*.txt", "storm", False, agents_quantities)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

Since the rewards contain expected value (EX) and square of expected value (E(X^2)) we compute the variance as:

In [ ]:
from src.load import to_variance

In [ ]:
# TBD UNCOMMENT THIS
# rewards = to_variance(rewards)
# rewards_multiparam_syn = to_variance(rewards_multiparam_syn)
# rewards_multiparam_semisyn = to_variance(rewards_multiparam_semisyn)
# rewards_multiparam_asyn = to_variance(rewards_multiparam_asyn)

<a name="one"></a>

## Load polynomials for **three-param** models.

### Models with linear function:

- p = prob. of initial success
- q1 = initial value
- delta = slope

In [ ]:
import copy

In [ ]:
f_lin_syn = copy.deepcopy(f_multiparam_syn)
f_lin_semisyn = copy.deepcopy(f_multiparam_semisyn)
f_lin_asyn = copy.deepcopy(f_multiparam_asyn)

## Replacing the q(i) with respective function
for structure in [f_lin_syn, f_lin_semisyn, f_lin_asyn]:
    # print(type(structure))
    for dictionary in structure.keys():
        # print(type(population))
        # print(type(structure[population]))
        # print(structure[dictionary])
        for population in range(len(structure[dictionary])):
            # print(structure[dictionary][population])
            # print("structure", [f_lin_syn].index(structure))
            # print("dictionary", dictionary)
            # print("population", population)
            # print(structure[dictionary][population])
            for BSCC in range(1,dictionary):               
                # print(structure[dictionary][population])
                # print()
                
                ## THIS SHOULD WORK WELL BUT Z3 HAS PROBLEM WITH SYMBOLIC EXPRESSIONS (MIN)
                # structure[dictionary][population] = structure[dictionary][population].replace("q"+str(BSCC),(f"min(q1 + {BSCC-1}*delta,1)"))
                structure[dictionary][population] = structure[dictionary][population].replace("q"+str(BSCC),(f"If(q1 + {BSCC-1}*delta > 1, 1, q1 + {BSCC-1}*delta )"))
            # print(structure[dictionary][population])
            #break

## Showing the difference in an example:            
print(f_lin_syn[3])
print(f_multiparam_syn[3])

In [ ]:
rewards_lin_syn = copy.deepcopy(rewards_multiparam_syn)
rewards_lin_semisyn = copy.deepcopy(rewards_multiparam_semisyn)
rewards_lin_asyn = copy.deepcopy(rewards_multiparam_asyn)

## Replacing the q(i) with respective function
for structure in [rewards_lin_syn, rewards_lin_semisyn, rewards_lin_asyn]:
    # print(type(structure))
    for dictionary in structure.keys():
        # print(type(population))
        # print(type(structure[population]))
        # print(structure[dictionary])
        for population in range(len(structure[dictionary])):
            # print(structure[dictionary][population])
            # print("structure", [f_lin_syn].index(structure))
            # print("dictionary", dictionary)
            # print("population", population)
            # print(structure[dictionary][population])
            for BSCC in range(1,dictionary):               
                # print(structure[dictionary][population])
                # print()
                
                ## THIS SHOULD WORK WELL BUT Z3 HAS PROBLEM WITH SYMBOLIC EXPRESSIONS (MIN)
                # structure[dictionary][population] = structure[dictionary][population].replace("q"+str(BSCC),(f"min(q1 + {BSCC-1}*delta,1)"))  
                structure[dictionary][population] = structure[dictionary][population].replace("q"+str(BSCC),(f"If(q1 + {BSCC-1}*delta > 1, 1, q1 + {BSCC-1}*delta )"))
            # print(structure[dictionary][population])
            #break
            
## Showing the difference in an example:            
print(rewards_lin_syn[2])
print(rewards_multiparam_syn[2])

<a name="two"></a>

### Models with sigmoidal function:

- p = prob. of initial success
- K_m = effectivity
- n = Hill coeficient

In [ ]:
f_sig_syn = copy.deepcopy(f_multiparam_syn)
f_sig_semisyn = copy.deepcopy(f_multiparam_semisyn)
f_sig_asyn = copy.deepcopy(f_multiparam_asyn)

## Replacing the q(i) with respective function
for structure in [f_sig_syn, f_sig_semisyn, f_sig_asyn]:
    # print(type(structure))
    for dictionary in structure.keys():
        # print(type(population))
        # print(type(structure[population]))
        # print(structure[dictionary])
        for population in range(len(structure[dictionary])):
            # print(structure[dictionary][population])
            # print("structure", [f_sig_syn].index(structure))
            # print("dictionary", dictionary)
            # print("population", population)
            # print(structure[dictionary][population])
            for BSCC in range(1,dictionary):               
                # print(structure[dictionary][population])
                # print()
                
                structure[dictionary][population] = structure[dictionary][population].replace("q"+str(BSCC),(f" ({BSCC}**n / (K_m + {BSCC}**n))"))
            # print(structure[dictionary][population])
            #break

## Showing the difference in an example:            
print(f_sig_syn[3])
print(f_multiparam_syn[3])

In [ ]:
rewards_sig_syn = copy.deepcopy(rewards_multiparam_syn)
rewards_sig_semisyn = copy.deepcopy(rewards_multiparam_semisyn)
rewards_sig_asyn = copy.deepcopy(rewards_multiparam_asyn)

## Replacing the q(i) with respective function
for structure in [rewards_sig_syn, rewards_sig_semisyn, rewards_sig_asyn]:
    # print(type(structure))
    for dictionary in structure.keys():
        # print(type(population))
        # print(type(structure[population]))
        # print(structure[dictionary])
        for population in range(len(structure[dictionary])):
            # print(structure[dictionary][population])
            # print("structure", [f_sig_syn].index(structure))
            # print("dictionary", dictionary)
            # print("population", population)
            # print(structure[dictionary][population])
            for BSCC in range(1,dictionary):               
                # print(structure[dictionary][population])
                # print()

                structure[dictionary][population] = structure[dictionary][population].replace("q"+str(BSCC),(f" ({BSCC}**n / (K_m + {BSCC}**n))"))
            # print(structure[dictionary][population])
            #break
            
## Showing the difference in an example:            
print(rewards_sig_syn[2])
print(rewards_multiparam_syn[2])

In [ ]:
## Show the curve
K_m = 5
n = 1
for BSCC in range(1,11):
    a = f"{BSCC}**n / (K_m + {BSCC}**n)"
    print(eval(a))
    

## Load polynomials for **four-param** models.

### Models with lin_tresh function:

- p = prob. of initial success
- q1 = initial value
- delta = slope
- thresh = maximal value

In [ ]:
import copy

In [ ]:
f_lin_tresh_syn = copy.deepcopy(f_multiparam_syn)
f_lin_tresh_semisyn = copy.deepcopy(f_multiparam_semisyn)
f_lin_tresh_asyn = copy.deepcopy(f_multiparam_asyn)

## Replacing the q(i) with respective function
for structure in [f_lin_tresh_syn, f_lin_tresh_semisyn, f_lin_tresh_asyn]:
    # print(type(structure))
    for dictionary in structure.keys():
        # print(type(population))
        # print(type(structure[population]))
        # print(structure[dictionary])
        for population in range(len(structure[dictionary])):
            # print(structure[dictionary][population])
            # print("structure", [f_lin_tresh_syn].index(structure))
            # print("dictionary", dictionary)
            # print("population", population)
            # print(structure[dictionary][population])
            for BSCC in range(1,dictionary):               
                # print(structure[dictionary][population])
                # print()
                
                ## THIS SHOULD WORK WELL BUT Z3 HAS PROBLEM WITH SYMBOLIC EXPRESSIONS (MIN)
                # structure[dictionary][population] = structure[dictionary][population].replace("q"+str(BSCC),(f"min(q1 + {BSCC-1}*delta,tresh)"))  
                structure[dictionary][population] = structure[dictionary][population].replace("q"+str(BSCC),(f"If(q1 + {BSCC-1}*delta > tresh, tresh, q1 + {BSCC-1}*delta )"))
            # print(structure[dictionary][population])
            #break

## Showing the difference in an example:            
print(f_lin_tresh_syn[3])
print(f_multiparam_syn[3])

In [ ]:
rewards_lin_tresh_syn = copy.deepcopy(rewards_multiparam_syn)
rewards_lin_tresh_semisyn = copy.deepcopy(rewards_multiparam_semisyn)
rewards_lin_tresh_asyn = copy.deepcopy(rewards_multiparam_asyn)

## Replacing the q(i) with respective function
for structure in [rewards_lin_tresh_syn, rewards_lin_tresh_semisyn, rewards_lin_tresh_asyn]:
    # print(type(structure))
    for dictionary in structure.keys():
        # print(type(population))
        # print(type(structure[population]))
        # print(structure[dictionary])
        for population in range(len(structure[dictionary])):
            # print(structure[dictionary][population])
            # print("structure", [f_lin_tresh_syn].index(structure))
            # print("dictionary", dictionary)
            # print("population", population)
            # print(structure[dictionary][population])
            for BSCC in range(1,dictionary):               
                # print(structure[dictionary][population])
                # print()
                
                ## THIS SHOULD WORK WELL BUT Z3 HAS PROBLEM WITH SYMBOLIC EXPRESSIONS (MIN)
                # structure[dictionary][population] = structure[dictionary][population].replace("q"+str(BSCC),(f"min(q1 + {BSCC-1}*delta,tresh)"))  
                structure[dictionary][population] = structure[dictionary][population].replace("q"+str(BSCC),(f"If(q1 + {BSCC-1}*delta > tresh, tresh, q1 + {BSCC-1}*delta )"))
            # print(structure[dictionary][population])
            #break
            
## Showing the difference in an example:            
print(rewards_lin_tresh_syn[2])
print(rewards_multiparam_syn[2])

<a name="three"></a>

## Load polynomials for **five-param** models.

### Models with low-high-low function:

- p = prob. of initial success
- low = threshold below which the value is set to qmin
- high = threshold above which the value is set to qmin
- q_min = low value
- q_min = high value

In [ ]:
f_low_high_low_syn = copy.deepcopy(f_multiparam_syn)
f_low_high_low_semisyn = copy.deepcopy(f_multiparam_semisyn)
f_low_high_low_asyn = copy.deepcopy(f_multiparam_asyn)

## Replacing the q(i) with respective function
for structure in [f_low_high_low_syn, f_low_high_low_semisyn, f_low_high_low_asyn]:
    # print(type(structure))
    for dictionary in structure.keys():
        # print(type(population))
        # print(type(structure[population]))
        # print(structure[dictionary])
        for population in range(len(structure[dictionary])):
            # print(structure[dictionary][population])
            # print("structure", [f_low_high_low_syn].index(structure))
            # print("dictionary", dictionary)
            # print("population", population)
            # print(structure[dictionary][population])
            for BSCC in range(1,dictionary):               
                # print(structure[dictionary][population])
                # print()
                structure[dictionary][population] = structure[dictionary][population].replace("q"+str(BSCC),(f" If ( Or( {BSCC} < low  , {BSCC} >= high), qmin, qmax)"))  
            # print(structure[dictionary][population])
            #break
            
## Showing the difference in an example:            
print(f_low_high_low_syn[3])
print(f_multiparam_syn[3])

In [ ]:
rewards_low_high_low_syn = copy.deepcopy(rewards_multiparam_syn)
rewards_low_high_low_semisyn = copy.deepcopy(rewards_multiparam_semisyn)
rewards_low_high_low_asyn = copy.deepcopy(rewards_multiparam_asyn)

## Replacing the q(i) with respective function
for structure in [rewards_low_high_low_syn, rewards_low_high_low_semisyn, rewards_low_high_low_asyn]:
    # print(type(structure))
    for dictionary in structure.keys():
        # print(type(population))
        # print(type(structure[population]))
        # print(structure[dictionary])
        for population in range(len(structure[dictionary])):
            # print(structure[dictionary][population])
            # print("structure", [f_low_high_low_syn].index(structure))
            # print("dictionary", dictionary)
            # print("population", population)
            # print(structure[dictionary][population])
            for BSCC in range(1,dictionary):               
                # print(structure[dictionary][population])
                # print()
                structure[dictionary][population] = structure[dictionary][population].replace("q"+str(BSCC),(f" If ( Or( {BSCC} < low  , {BSCC} >= high), qmin, qmax)"))
            # print(structure[dictionary][population])
            #break
            
## Showing the difference in an example:            
print(rewards_low_high_low_syn[2])
print(rewards_multiparam_syn[2])

<a name="four"></a>

### Models with bell function:

- p = prob. of initial success
- q_init = starting value of 
- q_max = maximal value of the shape
- mean = point of maximal value
- spread = width of the shape

In [ ]:
f_bell_syn = copy.deepcopy(f_multiparam_syn)
f_bell_semisyn = copy.deepcopy(f_multiparam_semisyn)
f_bell_asyn = copy.deepcopy(f_multiparam_asyn)

## Replacing the q(i) with respective function
for structure in [f_bell_syn, f_bell_semisyn, f_bell_asyn]:
    # print(type(structure))
    for dictionary in structure.keys():
        # print(type(population))
        # print(type(structure[population]))
        # print(structure[dictionary])
        for population in range(len(structure[dictionary])):
            # print(structure[dictionary][population])
            # print("structure", [f_bell_syn].index(structure))
            # print("dictionary", dictionary)
            # print("population", population)
            # print(structure[dictionary][population])
            for BSCC in range(1,dictionary):               
                # print(structure[dictionary][population])
                # print()
                
                structure[dictionary][population] = structure[dictionary][population].replace("q"+str(BSCC),(f"q_init+(q_max-q_init)*np.exp(-({BSCC}-{mean})**2/(2*spread*spread))"))
            # print(structure[dictionary][population])
            #break

## Showing the difference in an example:            
print(f_bell_syn[3])
print(f_multiparam_syn[3])

In [ ]:
rewards_bell_syn = copy.deepcopy(rewards_multiparam_syn)
rewards_bell_semisyn = copy.deepcopy(rewards_multiparam_semisyn)
rewards_bell_asyn = copy.deepcopy(rewards_multiparam_asyn)

## Replacing the q(i) with respective function
for structure in [rewards_bell_syn, rewards_bell_semisyn, rewards_bell_asyn]:
    # print(type(structure))
    for dictionary in structure.keys():
        # print(type(population))
        # print(type(structure[population]))
        # print(structure[dictionary])
        for population in range(len(structure[dictionary])):
            # print(structure[dictionary][population])
            # print("structure", [f_bell_syn].index(structure))
            # print("dictionary", dictionary)
            # print("population", population)
            # print(structure[dictionary][population])
            for BSCC in range(1,dictionary):               
                # print(structure[dictionary][population])
                # print()

                structure[dictionary][population] = structure[dictionary][population].replace("q"+str(BSCC),(f"q_init+(q_max-q_init)*np.exp(-({BSCC}-{mean})**2/(2*spread*spread))"))
            # print(structure[dictionary][population])
            #break
            
## Showing the difference in an example:            
print(rewards_bell_syn[2])
print(rewards_multiparam_syn[2])

In [ ]:
## Show the curve
q_init = 0
q_max = 1
spread = 3
mean = 8
for BSCC in range(1,11):
    a = f"q_init+(q_max-q_init)*np.exp(-({BSCC}-{mean})**2/(2*spread*spread))"
    print(eval(a))


### ADITIONAL FUNCTIONALITY

In [ ]:
from src.load import find_param

Example - Finding all params of *f_multiparam_syn*

In [ ]:
parameters = set()
for N in agents_quantities:
    for polynome in f_multiparam_syn[N]:
        if len(parameters) < N:
            parameters.update(find_param(polynome))
print(sorted(parameters))

## DATA LOADING

The data we are about to load are in a form data points for given N and number of successes, hence they match already loaded f dictionaries. 

They can be seen as a points of the polynomials.

In [ ]:
from src.load import load_all_data

Load data

In [ ]:
D = load_all_data("data/data*.csv")

In [ ]:
D_multiparam = load_all_data("data/multiparametric/data*.csv")

In [ ]:
D2 = load_all_data("data_1/data*.csv")

In [ ]:
D2_multiparam = load_all_data("data_1/multiparametric/data*.csv")

## profound data loading

In [ ]:
from src.load import load_pickled_data

In [ ]:
D3 = load_pickled_data("Data_two_param")

In [ ]:
Data_one_point_step_down_10dim_param_space = load_pickled_data("Data_one_point_step_down_10dim_param_space")

In [ ]:
D4 =  load_pickled_data("Data_one_point_linear_param_space")

In [ ]:
D4

### showing loaded f and data

An example of function and data for number of agents N=2 and number of successes i=1

In [ ]:
f[2][1] 

In [ ]:
f_multiparam_syn[2][1]

In [ ]:
f[5]

In [ ]:
D[2][1] 

In [ ]:
D3[('synchronous_', 2, 100, 0.028502714675268215, 0.03259111103419188)]

Catch data errors (subzero values)

In [ ]:
from src.load import catch_data_error

In [ ]:
catch_data_error(D,0,1) 
catch_data_error(D2,0,1) 
catch_data_error(D_multiparam,0,1) 
catch_data_error(D2_multiparam,0,1) 

Now all data and f are loaded

### margin

margin to infer intervals in which we wan to search

In [ ]:
from src.load import create_interval,margin

In [ ]:
print(D[2][0])
print(margin(0.95, 3500, D[2][0]))
print(create_interval(0.95, 3500, D[2][0]))

EXPERIMENTAL margin

`0.04
0.04840729341084247
(0.028365125369386852, 0.05163487463061315)`

In [ ]:
from src.load import create_intervals

In [ ]:
D[2][0]

In [ ]:
for data_point in D[2]:
    print(data_point)
    print(create_interval(0.95, 3500,data_point))

In [ ]:
create_intervals(0.95, 3500, D[2])

## SAMPLING AND REFINEMENT

<a name="two"></a>

In [ ]:
from src.synthetise import *
import src.space
import time

In [ ]:
from importlib import reload

In [ ]:
reload(src.load)

In [ ]:
D[2]

In [ ]:
create_intervals(0.95,100,[0.5])

# LINEAR (LIN)

In [ ]:
spaces = {}

In [ ]:
# PREVIOUS SPACE LOOKS LIKE THIS
# spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1)], ["p", "q"], ["Real", "Real"], [], [])

In [ ]:
true_point = (0.4, 0.1, 0.1) # p, q1, delta
model_type = "lin"
size_q = 0

In [ ]:
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "q1", "delta"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"lin_semisyn{5} \n Dataset: lin, alpha=0.95, #samples=1500")

In [ ]:
alpha = 0.95
n_samples = 100

In [ ]:
population = 2
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "q1", "delta"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"lin_semisyn population= {population}, \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
check_deeper(spaces[(population, size_q, model_type)], ineq_to_props(f_lin_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])),15,10e-6,0.95,False,4)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
spaces[(population, size_q, model_type)].nice_print()

In [ ]:
population = 3
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "q1", "delta"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"lin_semisyn population= {population}, \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
check_deeper(spaces[(population, size_q, model_type)], ineq_to_props(f_lin_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])),15,10e-6,0.95,False,4)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
spaces[(population, size_q, model_type)].nice_print()

In [ ]:
population = 5
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "q1", "delta"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"lin_semisyn population= {population}, \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
check_deeper(spaces[(population, size_q, model_type)], ineq_to_props(f_lin_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])),2,10e-6,0.95,False,4)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
# TBD
# population = 10
# spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "q1", "delta"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"lin_semisyn population= {population}, \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
# start_time = time.time()
# check_deeper(spaces[(population, size_q, model_type)], ineq_to_props(f_lin_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])),10,10e-6,0.95,False,4)
# print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
from importlib import reload  # Python 3.4+ only.
reload(src.space)

In [ ]:
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1)], ["p", "q"], ["Real", "Real"], [], [], true_point=true_point, title=f"model: semisyn, population = {population}, size_q = {size_q},  \n Dataset: 1, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f[population],create_intervals(alpha, n_samples, D[population])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)

In [ ]:
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1)], ["p", "q"], ["Real", "Real"], [], [], true_point=[0.5,0.5], title=f"model: semisyn, population = {population}, size_q = {size_q},  \n Dataset: 1, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f[population],create_intervals(alpha, n_samples, D[population])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)

## SAMPLING LIN

In [ ]:
population = 3
size_q = 7
n_samples = 1500
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "q1", "delta"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"model: lin_semisyn, population = {population}, size_q = {size_q},  \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_lin_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)

In [ ]:
spaces[(population, size_q, model_type)].nice_print()

In [ ]:
def transform_lin(space):
    egg = []
    for sample in space.sat_samples:
        # print(sample)
        spam = []
        spam.append(sample[0])
        spam.append(sample[1])
        for agent in range(1,population):
            spam.append(sample[1] + sample[2]*agent)
        # print(spam)
        # print()
        egg.append(spam)
    space.sat_samples = egg
    
    egg = []
    for sample in space.unsat_samples:
        # print(sample)
        spam = []
        spam.append(sample[0])
        spam.append(sample[1])
        for agent in range(1,population):
            spam.append(sample[1] + sample[2]*agent)
        # print(spam)
        # print()
        egg.append(spam)
    space.unsat_samples = egg
    
    if space.true_point:
        spam = []
        spam.append(space.true_point[0])
        spam.append(space.true_point[1])
        for agent in range(1,population):
            spam.append(space.true_point[1] + space.true_point[2]*agent)
        # print(spam)
        # print()
        space.true_point = spam

In [ ]:
transform_lin(spaces[(population, size_q, model_type)])
spaces[(population, size_q, model_type)].show(sat_samples=True)

In [ ]:
spaces[(population, size_q, model_type)].nice_print()

In [ ]:
def transform_to_morgan(space):
    # takes multiparamteric space (can be obtained by transform())
    
    egg = []
    for sample in space.sat_samples:
        # print(sample)
        spam = []
        spam.append(sample[0])
        for agent in range(0,population):
            spam.append((1-spam[agent])*sample[agent+1] + spam[agent])
        # print(spam)
        # print()
        egg.append(spam)
    space.sat_samples = egg
    
    egg = []
    for sample in space.unsat_samples:
        # print(sample)
        spam = []
        spam.append(sample[0])
        for agent in range(0,population):
            spam.append((1-spam[agent])*sample[agent+1] + spam[agent])
        # print(spam)
        # print()
        egg.append(spam)
    space.unsat_samples = egg
    
    if space.true_point:
        spam = []
        spam.append(space.true_point[0])
        for agent in range(0,population):
            spam.append((1-spam[agent])*space.true_point[agent+1] + spam[agent])
        # print(spam)
        # print()
        space.true_point = spam

In [ ]:
transform_to_morgan(spaces[(population, size_q, model_type)])
spaces[(population, size_q, model_type)].show(sat_samples=True)

In [ ]:
spaces[(population, size_q, model_type)].nice_print()

In [ ]:
population = 5
size_q = 4
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "q1", "delta"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"model: lin_semisyn, population = {population}, size_q = {size_q},  \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_lin_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)

In [ ]:
population = 5
size_q = 10
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "q1", "delta"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"model: lin_semisyn, population = {population}, size_q = {size_q},  \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_lin_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)

In [ ]:
population = 5
size_q = 15
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "q1", "delta"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"model: lin_semisyn, population = {population}, size_q = {size_q},  \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_lin_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)
print(f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run")

In [ ]:
population = 5
size_q = 20
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "q1", "delta"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"model: lin_semisyn, population = {population}, size_q = {size_q},  \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_lin_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)
print(f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run")

In [ ]:
population = 10
size_q = 2
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "q1", "delta"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"model: lin_semisyn, population = {population}, size_q = {size_q},  \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_lin_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)
print(f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run")

In [ ]:
population = 10
size_q = 3
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "q1", "delta"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"model: lin_semisyn, population = {population}, size_q = {size_q},  \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_lin_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)
print(f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run")

In [ ]:
population = 10
size_q = 5
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "q1", "delta"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"model: lin_semisyn, population = {population}, size_q = {size_q},  \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_lin_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)
print(f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run")

In [ ]:
population = 10
size_q = 7
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "q1", "delta"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"model: lin_semisyn, population = {population}, size_q = {size_q},  \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_lin_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)
print(f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run")

In [ ]:
population = 10
size_q = 10
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "q1", "delta"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"model: lin_semisyn, population = {population}, size_q = {size_q},  \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_lin_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)
print(f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run")

In [ ]:
transform_lin(spaces[(population, size_q, model_type)])
transform_to_morgan(spaces[(population, size_q, model_type)])
spaces[(population, size_q, model_type)].show(sat_samples=True)

In [ ]:
population = 10
size_q = 15
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "q1", "delta"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"model: lin_semisyn, population = {population}, size_q = {size_q},  \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_lin_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)
print(f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run")

In [ ]:
transform_lin(spaces[(population, size_q, model_type)])
transform_to_morgan(spaces[(population, size_q, model_type)])
spaces[(population, size_q, model_type)].show(sat_samples=True)

# SIGMOIDAL (SIG)

In [ ]:
spaces = {}

In [ ]:
# PREVIOUS SPACE LOOKS LIKE THIS
# spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1)], ["p", "q"], ["Real", "Real"], [], [])

In [ ]:
true_point = (0.4, 0.1, 0.1) # p, q1, delta
model_type = "sig"
size_q = 0

In [ ]:
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "K_m", "n"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"sig_semisyn{5} \n Dataset: sig, alpha=0.95, #samples=1500")

In [ ]:
alpha = 0.95
n_samples = 100

In [ ]:
population = 2
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "K_m", "n"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"sig_semisyn population= {population}, \n Dataset: sig, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
check_deeper(spaces[(population, size_q, model_type)], ineq_to_props(f_sig_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])),15,10e-6,0.95,False,4)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
spaces[(population, size_q, model_type)].nice_print()

In [ ]:
population = 3
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "K_m", "n"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"sig_semisyn population= {population}, \n Dataset: sig, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
check_deeper(spaces[(population, size_q, model_type)], ineq_to_props(f_sig_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])),15,10e-6,0.95,False,4)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
spaces[(population, size_q, model_type)].nice_print()

In [ ]:
population = 5
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "K_m", "n"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"sig_semisyn population= {population}, \n Dataset: sig, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
check_deeper(spaces[(population, size_q, model_type)], ineq_to_props(f_sig_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])),2,10e-6,0.95,False,4)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
# TBD
# population = 10
# spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "K_m", "n"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"sig_semisyn population= {population}, \n Dataset: sig, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
# start_time = time.time()
# check_deeper(spaces[(population, size_q, model_type)], ineq_to_props(f_sig_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])),10,10e-6,0.95,False,4)
# print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

## SAMPLING SIG

In [ ]:
population = 3
size_q = 7
n_samples = 1500
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "K_m", "n"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"model: sig_semisyn, population = {population}, size_q = {size_q},  \n Dataset: sig, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_sig_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)

In [ ]:
spaces[(population, size_q, model_type)].nice_print()

In [ ]:
def transform_sig(space):
    egg = []
    for sample in space.sat_samples:
        # print(sample)
        spam = []
        spam.append(sample[0])
        spam.append(sample[1])
        for agent in range(1,population):
            spam.append(sample[1] + sample[2]*agent)
        # print(spam)
        # print()
        egg.append(spam)
    space.sat_samples = egg
    
    egg = []
    for sample in space.unsat_samples:
        # print(sample)
        spam = []
        spam.append(sample[0])
        spam.append(sample[1])
        for agent in range(1,population):
            spam.append(sample[1] + sample[2]*agent)
        # print(spam)
        # print()
        egg.append(spam)
    space.unsat_samples = egg
    
    if space.true_point:
        spam = []
        spam.append(space.true_point[0])
        spam.append(space.true_point[1])
        for agent in range(1,population):
            spam.append(space.true_point[1] + space.true_point[2]*agent)
        # print(spam)
        # print()
        space.true_point = spam

In [ ]:
transform_sig(spaces[(population, size_q, model_type)])
spaces[(population, size_q, model_type)].show(sat_samples=True)

In [ ]:
spaces[(population, size_q, model_type)].nice_print()

In [ ]:
def transform_to_morgan(space):
    # takes multiparamteric space (can be obtained by transform())
    
    egg = []
    for sample in space.sat_samples:
        # print(sample)
        spam = []
        spam.append(sample[0])
        for agent in range(0,population):
            spam.append((1-spam[agent])*sample[agent+1] + spam[agent])
        # print(spam)
        # print()
        egg.append(spam)
    space.sat_samples = egg
    
    egg = []
    for sample in space.unsat_samples:
        # print(sample)
        spam = []
        spam.append(sample[0])
        for agent in range(0,population):
            spam.append((1-spam[agent])*sample[agent+1] + spam[agent])
        # print(spam)
        # print()
        egg.append(spam)
    space.unsat_samples = egg
    
    if space.true_point:
        spam = []
        spam.append(space.true_point[0])
        for agent in range(0,population):
            spam.append((1-spam[agent])*space.true_point[agent+1] + spam[agent])
        # print(spam)
        # print()
        space.true_point = spam

In [ ]:
transform_to_morgan(spaces[(population, size_q, model_type)])
spaces[(population, size_q, model_type)].show(sat_samples=True)

In [ ]:
spaces[(population, size_q, model_type)].nice_print()

In [ ]:
population = 5
size_q = 4
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "K_m", "n"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"model: sig_semisyn, population = {population}, size_q = {size_q},  \n Dataset: sig, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_sig_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)

In [ ]:
population = 5
size_q = 10
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "K_m", "n"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"model: sig_semisyn, population = {population}, size_q = {size_q},  \n Dataset: sig, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_sig_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)

In [ ]:
population = 5
size_q = 15
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "K_m", "n"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"model: sig_semisyn, population = {population}, size_q = {size_q},  \n Dataset: sig, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_sig_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)
print(f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run")

In [ ]:
population = 5
size_q = 20
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "K_m", "n"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"model: sig_semisyn, population = {population}, size_q = {size_q},  \n Dataset: sig, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_sig_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)
print(f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run")

In [ ]:
population = 10
size_q = 2
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "K_m", "n"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"model: sig_semisyn, population = {population}, size_q = {size_q},  \n Dataset: sig, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_sig_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)
print(f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run")

In [ ]:
population = 10
size_q = 3
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "K_m", "n"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"model: sig_semisyn, population = {population}, size_q = {size_q},  \n Dataset: sig, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_sig_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)
print(f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run")

In [ ]:
population = 10
size_q = 5
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "K_m", "n"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"model: sig_semisyn, population = {population}, size_q = {size_q},  \n Dataset: sig, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_sig_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)
print(f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run")

In [ ]:
population = 10
size_q = 7
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "K_m", "n"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"model: sig_semisyn, population = {population}, size_q = {size_q},  \n Dataset: sig, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_sig_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)
print(f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run")

In [ ]:
population = 10
size_q = 10
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "K_m", "n"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"model: sig_semisyn, population = {population}, size_q = {size_q},  \n Dataset: sig, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_sig_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)
print(f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run")

In [ ]:
transform_sig(spaces[(population, size_q, model_type)])
transform_to_morgan(spaces[(population, size_q, model_type)])
spaces[(population, size_q, model_type)].show(sat_samples=True)

In [ ]:
population = 10
size_q = 15
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1)], ["p", "K_m", "n"], ["Real", "Real", "Real"], [], [], true_point=true_point, title=f"model: sig_semisyn, population = {population}, size_q = {size_q},  \n Dataset: sig, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_sig_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)
print(f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run")

In [ ]:
transform_sig(spaces[(population, size_q, model_type)])
transform_to_morgan(spaces[(population, size_q, model_type)])
spaces[(population, size_q, model_type)].show(sat_samples=True)

# LIN-THRESH

In [ ]:
# PREVIOUS SPACE LOOKS LIKE THIS
# spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1)], ["p", "q"], ["Real", "Real"], [], [])

In [ ]:
true_point = (0.4, 0.1, 0.1, 1) # p, q1, delta, thresh
model_type = "lin_tresh"
size_q = 0

In [ ]:
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1),(0,1)], ["p", "q1", "delta", "tresh"], ["Real", "Real", "Real", "Real"], [], [], true_point=true_point, title=f"lin_tresh_semisyn{5} \n Dataset: lin, alpha=0.95, #samples=1500")

In [ ]:
population = 2
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1),(0,1)], ["p", "q1", "delta", "tresh"], ["Real", "Real", "Real", "Real"], [], [], true_point=true_point, title=f"lin_tresh_semisyn population= {population}, \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
check_deeper(spaces[(population, size_q, model_type)], ineq_to_props(f_lin_tresh_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])),10,10e-6,0.95,False,4)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
population = 3
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1),(0,1)], ["p", "q1", "delta", "tresh"], ["Real", "Real", "Real", "Real"], [], [], true_point=true_point, title=f"lin_tresh_semisyn population= {population}, \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
check_deeper(spaces[(population, size_q, model_type)], ineq_to_props(f_lin_tresh_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])),10,10e-6,0.95,False,4)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
# TBD
# population = 5
# spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1),(0,1)], ["p", "q1", "delta", "tresh"], ["Real", "Real", "Real", "Real"], [], [], true_point=true_point, title=f"lin_tresh_semisyn population= {population}, \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
# start_time = time.time()
# check_deeper(spaces[(population, size_q, model_type)], ineq_to_props(f_lin_tresh_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])),10,10e-6,0.95,False,4)
# print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
# TBD
# population = 10
# spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1),(0,1)], ["p", "q1", "delta", "tresh"], ["Real", "Real", "Real", "Real"], [], [], true_point=true_point, title=f"lin_tresh_semisyn population= {population}, \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
# start_time = time.time()
# check_deeper(spaces[(population, size_q, model_type)], ineq_to_props(f_lin_tresh_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])),10,10e-6,0.95,False,4)
# print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

## SAMPLING LIN-THRESH

In [ ]:
def transform_lin_thresh(space):
    egg = []
    for sample in space.sat_samples:
        # print(sample)
        spam = []
        spam.append(sample[0])
        spam.append(sample[1])
        for agent in range(1,population):
            if sample[1] + sample[2]*agent < sample[3]
                spam.append(sample[1] + sample[2]*agent)
            else:
                spam.append(sample[3])
        # print(spam)
        # print()
        egg.append(spam)
    space.sat_samples = egg
    
    egg = []
    for sample in space.unsat_samples:
        # print(sample)
        spam = []
        spam.append(sample[0])
        spam.append(sample[1])
        for agent in range(1,population):
            if sample[1] + sample[2]*agent < sample[3]
                spam.append(sample[1] + sample[2]*agent)
            else:
                spam.append(sample[3])
        # print(spam)
        # print()
        egg.append(spam)
    space.unsat_samples = egg
    
    if space.true_point:
        spam = []
        spam.append(space.true_point[0])
        spam.append(space.true_point[1])
        for agent in range(1,population):
            if space.true_point[1] + space.true_point[2]*agent < sample[3]
                spam.append(space.true_point[1] + space.true_point[2]*agent)
            else:
                spam.append(space.true_point[3])
            
        # print(spam)
        # print()
        space.true_point = spam

In [ ]:
population = 5
size_q = 2
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1),(0,1)], ["p", "q1", "delta", "tresh"], ["Real", "Real", "Real", "Real"], [], [], true_point=true_point, title=f"lin_tresh_semisyn population= {population}, \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_lin_tresh_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)

In [ ]:
population = 10
size_q = 2
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1),(0,1)], ["p", "q1", "delta", "tresh"], ["Real", "Real", "Real", "Real"], [], [], true_point=true_point, title=f"lin_tresh_semisyn population= {population}, \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_lin_tresh_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
population = 10
size_q = 3
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1),(0,1)], ["p", "q1", "delta", "tresh"], ["Real", "Real", "Real", "Real"], [], [], true_point=true_point, title=f"lin_tresh_semisyn population= {population}, \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_lin_tresh_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
population = 10
size_q = 5
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1),(0,1)], ["p", "q1", "delta", "tresh"], ["Real", "Real", "Real", "Real"], [], [], true_point=true_point, title=f"lin_tresh_semisyn population= {population}, \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_lin_tresh_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
population = 10
size_q = 7
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1),(0,1)], ["p", "q1", "delta", "tresh"], ["Real", "Real", "Real", "Real"], [], [], true_point=true_point, title=f"lin_tresh_semisyn population= {population}, \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_lin_tresh_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
population = 10
size_q = 10
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1),(0,1),(0,1)], ["p", "q1", "delta", "tresh"], ["Real", "Real", "Real", "Real"], [], [], true_point=true_point, title=f"lin_tresh_semisyn population= {population}, \n Dataset: lin, alpha={alpha}, #samples={n_samples},\ntrue_point={true_point}")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_lin_tresh_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
transform_lin_thresh(spaces[(population, size_q, model_type)])
transform_to_morgan(spaces[(population, size_q, model_type)])
spaces[(population, size_q, model_type)].show(sat_samples=True)

# LOW HIGH LOW

In [ ]:
# PREVIOUS SPACE LOOKS LIKE THIS
# spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,1)], ["p", "q"], ["Real", "Real"], [], [])

In [ ]:
true_point = "" # there is no true point for low high low for D4
model_type = "low_high_low"
size_q = 0

In [ ]:
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,2),(0,2),(0,1),(0,1)], ["p", "low", "high", "qmin", "qmax"], ["Real", "Int", "Int", "Real", "Real"], [], [], true_point=true_point, title=f"lin_semisyn{5} \n Dataset: lin, alpha=0.95, #samples=1500")

In [ ]:
population = 2
start_time = time.time()
check_deeper(spaces[(population, size_q, model_type)], ineq_to_props(f_low_high_low_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])),10,10e-6,0.95,False,4)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
population = 3
start_time = time.time()
check_deeper(spaces[(population, size_q, model_type)], ineq_to_props(f_low_high_low_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])),10,10e-6,0.95,False,4)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
# TBD
# population = 5
# alpha = 0.95
# n_samples = 1500
# start_time = time.time()
# check_deeper(spaces[(population, size_q, model_type)], ineq_to_props(f_low_high_low_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])),10,10e-6,0.95,False,4)
# print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
# TBD
# population = 10
# alpha = 0.95
# n_samples = 1500
# start_time = time.time()
# check_deeper(spaces[(population, size_q, model_type)], ineq_to_props(f_low_high_low_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])),10,10e-6,0.95,False,4)
# print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

## SAMPLING LOW HIGH LOW

In [ ]:
# " If ( Or( {BSCC} < low  , {BSCC} >= high), qmin, qmax)"
def transform_low_high_low(space): # "p", "low", "high", "qmin", "qmax"
    egg = []
    for sample in space.sat_samples:
        # print(sample)
        spam = []
        spam.append(sample[0])
        for agent in range(1,population):
            if agent < sample[1] or agent >= sample[2]:
                spam.append(sample[3])
            else:
                spam.append(sample[4])
        # print(spam)
        # print()
        egg.append(spam)
    space.sat_samples = egg
    
    egg = []
    for sample in space.unsat_samples:
        # print(sample)
        spam = []
        spam.append(sample[0])
        spam.append(sample[1])
        for agent in range(1,population):
            if agent < sample[1] or agent >= sample[2]:
                spam.append(sample[3])
            else:
                spam.append(sample[4])
        # print(spam)
        # print()
        egg.append(spam)
    space.unsat_samples = egg
    
    if space.true_point:
        spam = []
        spam.append(space.true_point[0])
        spam.append(space.true_point[1])
        for agent in range(1,population):
            if agent < space.true_point[1] or agent >= space.true_point[2]:
                spam.append(space.true_point[3])
            else:
                spam.append(space.true_point[4])
        # print(spam)
        # print()
        space.true_point = spam

In [ ]:
population = 5
size_q = 2
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,population),(0,population),(0,1),(0,1)], ["p", "low", "high", "qmin", "qmax"], ["Real", "Int", "Int", "Real", "Real"], [], [], true_point=true_point, title=f"model: low_high_low_semisyn, population = {population}, size_q = {size_q},  \n Dataset: lin, alpha = 0.95, #samples = 1500")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_low_high_low_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)

In [ ]:
population = 10
size_q = 2
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,population),(0,population),(0,1),(0,1)], ["p", "low", "high", "qmin", "qmax"], ["Real", "Int", "Int", "Real", "Real"], [], [], true_point=true_point, title=f"model: low_high_low_semisyn, population = {population}, size_q = {size_q},  \n Dataset: lin, alpha = 0.95, #samples = 1500")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_low_high_low_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)

In [ ]:
population = 10
size_q = 3
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,population),(0,population),(0,1),(0,1)], ["p", "low", "high", "qmin", "qmax"], ["Real", "Int", "Int", "Real", "Real"], [], [], true_point=true_point, title=f"model: low_high_low_semisyn, population = {population}, size_q = {size_q},  \n Dataset: lin, alpha = 0.95, #samples = 1500")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_low_high_low_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)

In [ ]:
population = 10
size_q = 4
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,population),(0,population),(0,1),(0,1)], ["p", "low", "high", "qmin", "qmax"], ["Real", "Int", "Int", "Real", "Real"], [], [], true_point=true_point, title=f"model: low_high_low_semisyn, population = {population}, size_q = {size_q},  \n Dataset: lin, alpha = 0.95, #samples = 1500")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_low_high_low_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)

In [ ]:
population = 10
size_q = 5
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,population),(0,population),(0,1),(0,1)], ["p", "low", "high", "qmin", "qmax"], ["Real", "Int", "Int", "Real", "Real"], [], [], true_point=true_point, title=f"model: low_high_low_semisyn, population = {population}, size_q = {size_q},  \n Dataset: lin, alpha = 0.95, #samples = 1500")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_low_high_low_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)

In [ ]:
population = 10
size_q = 10
spaces[(population, size_q, model_type)] = src.space.RefinedSpace([(0,1),(0,population),(0,population),(0,1),(0,1)], ["p", "low", "high", "qmin", "qmax"], ["Real", "Int", "Int", "Real", "Real"], [], [], true_point=true_point, title=f"model: low_high_low_semisyn, population = {population}, size_q = {size_q},  \n Dataset: lin, alpha = 0.95, #samples = 1500")
start_time = time.time()
spaces[(population, size_q, model_type)].sample(ineq_to_props(f_low_high_low_semisyn[population],create_intervals(alpha, n_samples, D4['multiparam_semisynchronous_'][population][n_samples][(0.4,   0.1,   0.2,   0.30000000000000004,   0.4,   0.5,   0.6000000000000001,   0.7000000000000001,   0.8,   0.9)])), size_q=size_q, silent=True, save=strftime("%d-%b-%Y-%H:%M:%S", localtime()))
spaces[(population, size_q, model_type)].show(title= f"It took {socket.gethostname()} {round(time.time() - start_time, 2)} seconds to run", sat_samples=True, unsat_samples=True, save=True)

In [ ]:
transform_low_high_low(spaces[(population, size_q, model_type)])
transform_to_morgan(spaces[(population, size_q, model_type)])
spaces[(population, size_q, model_type)].show(sat_samples=True)

# step_down

In [ ]:
f_multiparam_syn[population][0]

In [ ]:
Data_one_point_step_down_10dim_param_space["multiparam_synchronous_"]f[population][3500][(0.1, 0.1, 0.1, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)][0]

In [ ]:
create_intervals(0.95, 3500, Data_one_point_step_down_10dim_param_space["multiparam_synchronous_"]f[population][3500][(0.1, 0.1, 0.1, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)])[0]

In [ ]:
start_time = time()
check_deeper([(0,1)],ineq_to_props(["(p - 1)**10"],[Interval(0.3179496053513864, 0.3494789660771851)]),14,0.01**2,0.997,False,4)
print("  It took", socket.gethostname(), time() - start_time, "seconds to run")

In [ ]:
#from matplotlib.patches import Rectangle
#greater = [(0, 1)]
#smaller = [(0, 1)]
#smaller[0][0]
#smaller[0][1]
#smaller[1][0]
#Rectangle((smaller[0][0], 0), smaller[0][1] - smaller[0][0], smaller[1][0], fc='r')

In [ ]:
start_time = time.time()
check_deeper([(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1)],ineq_to_props(f_multiparam_semisyn[population],create_intervals(0.95, 3500, D_multiparamf[population])),1,10e-14,0.95,False,3, size_q=4, save=True)
print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

In [ ]:
parameters = set()
for N in agents_quantities:
    for polynome in f_multiparam_semisyn[N]:
        if len(parameters) < N:
            parameters.update(find_param(polynome))
print(sorted(parameters))
